First all the needed modules are imported. The manipulated toysimulation module is needed.

In [1]:
import numpy as np
import networkx as nx
from collections import defaultdict
import random
import pickle
import matplotlib as mlp
import matplotlib.pyplot as plt
import pandas as pd
import osmnx as ox

import os
import scipy as sp
import scipy.stats

import route

from toysimulations2 import ZeroDetourBus, Stop, Request, Network

ModuleNotFoundError: No module named 'osmnx'

The functions for generating the requests and starting the simulations are defined. In contrast to the functions in jumps_in_request_rate more parameters can be passed to make it possible to start the simulation with an already working bus.

In [3]:
def req_generator_uniform(graph, num_reqs, req_rate,t=0,req_idxi=0):
    """
    Generates requests with rate=req_rate whose origin and
    destination are drawn uniformly randomly. The requests
    are generated in time as a Poisson process.
    """
    req_idx=req_idxi
    

    while req_idx < num_reqs+req_idxi:
        orig, dest = random.sample(graph.nodes(), k=2)
        delta_t = np.random.exponential(1/req_rate)

        t += delta_t
        req_idx += 1
        yield Request(req_idx, t, orig, dest)
        
def simulate_single_request_rate(G, nG, x,network_type, l_avg,initpos=None,time=0,remaining_time=0,next_stop=None,stoplist=[],req_data=dict(),insertion_data=[],req_idx=0):
    """
    Simulates only as single request rate x. See the docstring of
    `simulate_different_request_rates` for details on the arguments.
    Takes in a number of parameters to make it possible to start the simulation with an allready working bus
    """
    num_reqs = 5000
    req_rate = x/2/l_avg
    if initpos==None:
        initpos=random.sample(G.nodes(), k=1)[0]

    sim = ZeroDetourBus(nG,
                        req_generator_uniform(G, num_reqs, req_rate,time,req_idx),
                        network_type, 
                        initpos,
                        time,
                        remaining_time,
                        next_stop,
                        stoplist,
                        req_data,
                        insertion_data
                       )
    print(f"simulating x={x}")
    sim.simulate_all_requests()
    return sim.req_data, sim.insertion_data,sim.stoplist,sim.position,sim.time,sim.remaining_time, sim.next_stop
        

Functions are defined that calculate the wait time change all nodes individually.

In [4]:
def calculate_waiting_times_node_change_randomG(req_data,G):

    """
    Gets req_data from the simulation, where a failure occurred at 5000 requests and the graph the simulation was done on.
    Creates two arrays for every node in the graph saving all the wait times there before and after the failure and then averaging over these values.
    These arrays and the average values are saved into to seperate dictionaries, which are returned by the fuction.

    """
    waitnodes_before={}
    waitnodes_after={}
    waitn_avg_before={}
    waitn_avg_after={}

    for i in list(G.nodes):
        waitnodes_before[i]=[]
        waitnodes_after[i]=[]
        waitn_avg_before[i]=None
        waitn_avg_after[i]=None

    for key in req_data.keys():
        if key<5000:
            node=req_data[key]['origin']
            #if waitnodes[node]==None:
                #waitnodes[node]=(float(req_data[key]['pickup_epoch'])-float(req_data[key]['req_epoch']))
            #else:
            waitnodes_before[node].append((float(req_data[key]['pickup_epoch'])-float(req_data[key]['req_epoch'])))

        else:
            node=req_data[key]['origin']
            waitnodes_after[node].append((float(req_data[key]['pickup_epoch'])-float(req_data[key]['req_epoch'])))

    for node in waitnodes_before.keys():
        waitn_avg_before[node]=np.mean(waitnodes_before[node])
    for node in waitnodes_after.keys():
        waitn_avg_after[node]=np.mean(waitnodes_after[node])
    return waitn_avg_before, waitn_avg_after, waitnodes_before,waitnodes_after


def calculate_avg_waiting_times_node_change_cutedge_randomG(G,nG,x,l_avg,edge):

    """
    For one particular edge runs a simulation while removing the edge after 5000 requests.
    For every calculation the average wait time change at every single node is calculated with the function calculate_waiting_times_node_change_randomG(req_data,G).
    The values are saved into a dictionary. The mean value of node wait time of all simulations is calculated and saved into a new dictionary.
    the dictionary of all values before and after the calculation and the resulting dictionary with the average value for every node before and after the simulation is returned.
    """
    waitnodes_all_before={} #stores all the values of the before waiting time of all simulations
    waitnodes_all_after={}
    waitnodes_avg_before={} #stores the average of all simulations
    waitnodes_avg_after={}

    for i in list(G.nodes):
        waitnodes_all_before[i]=[]
        waitnodes_all_after[i]=[]
        waitnodes_avg_before[i]=None
        waitnodes_avg_after[i]=None

    for i in range(0,30):
        req_data1, insertion_data1,stoplist1,position1,time1,remaining_time1, next_stop1=simulate_single_request_rate(G, nG, x, network_type='novolcomp', l_avg=l_avg,initpos=None,time=0,remaining_time=0,next_stop=None,stoplist=[],req_data=dict(),insertion_data=[],req_idx=0)

        G.remove_edges_from(edge)
        nG = Network(G,network_type='novolcomp')
        #l_avg = nx.average_shortest_path_length(M)
        req_data, insertion_data,stoplist,position,time,remaining_time, next_stop=simulate_single_request_rate(G, nG, x, network_type='novolcomp', l_avg=l_avg,initpos=position1,time=time1,remaining_time=remaining_time1,next_stop=next_stop1,stoplist=stoplist1,req_data=req_data1,insertion_data=insertion_data1,req_idx=list(req_data1)[-1])

        waitn_avg_before, waitn_avg_after, waitnodes_before,waitnodes_after=calculate_waiting_times_node_change_randomG(req_data,G)
        for node in waitn_avg_before.keys():
            waitnodes_all_before[node].append(waitn_avg_before[node]) #goes through dictionary and appends the average value from this simulation to the array of overall values
        for node in waitn_avg_after.keys():
            waitnodes_all_after[node].append(waitn_avg_after[node])
        G.add_edges_from(edge)
        nG=Network(G,network_type='novolcomp')

    for node in waitnodes_all_before.keys():
        waitnodes_avg_before[node]=np.mean(waitnodes_all_before[node]) #saves the average of all simulation for every node into a new dictionary

    for node in waitnodes_all_after.keys():
        waitnodes_avg_after[node]=np.mean(waitnodes_all_after[node])


    return waitnodes_avg_before,waitnodes_avg_after,waitnodes_all_before,waitnodes_all_after

The following shows how the data for the comparison of wait time change an betweenness centrality change for the street network of Ingolstadt would be created and vizualized.
The same procedure was done for other networks.

In [5]:
print("doing Ingolstadt")
graph_path = 'graph_ingolstadt.gpkl'
G = nx.read_gpickle(graph_path)
nG=Network(G,network_type='novolcomp')
l_avg = nx.average_shortest_path_length(G)
x=10
nx.edge_betweenness_centrality(G, k=None, normalized=True, weight=None, seed=None)
all_edges=G.edges()
local_edges={}
for edge in all_edges:
    print("doing Ingolstadt")
    graph_path = 'graph_ingolstadt.gpkl'
    G = nx.read_gpickle(graph_path)
    nG=Network(G,network_type='novolcomp')
    l_avg = nx.average_shortest_path_length(G)
    x=10
    edgelist=[]
    edgelist.append(edge)

    try:
        waitnodes_avg_before,waitnodes_avg_after,waitnodes_all_before,waitnodes_all_after=calculate_avg_waiting_times_node_change_cutedge_randomG(G,nG,x,l_avg,edgelist)
        #print(waitnodes_avg_before)
        #print(waitnodes_avg_after)
        before=[]
        after=[]
        #print(waitnodes_all_before)
        for i in waitnodes_avg_before.keys():
            before.append(waitnodes_avg_before[i])
        for i in waitnodes_avg_after.keys():
            after.append(waitnodes_avg_after[i])
        print(np.mean(before))
        print(np.mean(after))
        change={}
        for i in waitnodes_avg_before.keys():
            change[i]=waitnodes_avg_after[i]-waitnodes_avg_before[i]

        l_avg = nx.average_shortest_path_length(G)
        bet_centr=nx.betweenness_centrality(G, k=None, normalized=True, weight=None, endpoints=False, seed=None)

        G.remove_edges_from(edgelist)
        nG = Network(G,network_type='novolcomp')
        x=10
        l_avg = nx.average_shortest_path_length(G)
        #print(l_avg)
        bet_centr_after=nx.betweenness_centrality(G, k=None, normalized=True, weight=None, endpoints=False, seed=None)
        G.add_edges_from(edgelist)
        nG = Network(G,network_type='novolcomp')
        change_betw_nodes={}
        for node in bet_centr.keys():
            change_betw_nodes[node]=bet_centr_after[node]-bet_centr[node]

        changes=[]
        for key in change.keys():
            changes.append(change[key])
        changes_betw=[]
        for key in change_betw_nodes.keys():
            changes_betw.append(change_betw_nodes[key])

        #print(changes)
        #print(changes_betw)
        local_edges[edge]={'changes':changes,'changes_betw':changes_betw}
    except KeyError:
        G.add_edges_from(edgelist)
        nG = Network(G,network_type='novolcomp')
        print(edge)
        continue

for edge in local_edges.keys():
    plt.scatter(local_edges[edge]['changes_betw'], local_edges[edge]['changes'],color='Blue')
plt.xlabel('Betweenness Centrality Change of Specific Node')
plt.ylabel('Change in Waiting Time of Specific Node')

doing Ingolstadt


ModuleNotFoundError: No module named 'shapely'

Additionally the distance from every node to the link failure is calculated and plotted with the change in wait time.

In [ ]:
graph_path = 'graph_ingolstadt.gpkl'
G = nx.read_gpickle(graph_path)
local_edges={(85, 71): {'changes': [1.616057339820964, 4.0967697588078416, -43.968109120300426, -5.461704264953397, 2.5143348837338024, -15.423084978582807, -0.14949583891711882, -1.1854134027763052, 5.5026721859000105, 12.434138865873308, -2.7469492448681763, 6.89067176279687, 18.06426954229613, 2.169265913871129, 2.31624675435161, 20.006544790387352, 8.848047344636257, -0.1064844660810138, 2.281986941072791, -24.629148618124162, -6.471865359515917, 16.38976106659547, 1.8615925474326787, 9.085315211005593, 5.422432100933463, 3.7370854982567607, 1.1594425300798363, 1.4451038451546196, 17.14434313136269, 3.1265512237254, 0.6512023104145896, 10.833989902662182, 8.209807366605133, 3.7729889014725018, 2.187276947787403, -1.3652170432752087, -2.896716641010215, -1.4785732353413437, -4.663246119543217, -0.16820445662804673, 10.532009803485664, -4.714304757365312, -6.829500311519894, -3.3836723181604498, 0.5217513125292044, 10.242452468023885, 3.2256074627650833, 10.452535376324889, -3.2973252906712602, 1.6207452317901243, -5.823725889707845, 0.9450817485262135, 20.51555418878158, 4.648123854623208, -1.6418023550102419, 3.1465917908461165, 0.6949550784699419, 4.852516245609365, -2.777319171067006, -2.1607232094830238, 11.877302525619314, 4.812179285685119, 2.3887248570269435, -1.0254142799390635, -1.304286342851924, -4.959175270894818, 5.658078359632185, -7.882055098532476, -4.26525724386039, -6.149087920024513, 5.789603398606346, -4.984773552821764, 9.854053386368179, -0.8305356009729792, -0.814587621002147, 18.392766192482213, 1.0617136035675685, 7.4342578943264925, 10.529794778727933, 2.4152738171787007, 10.617106089263785, -2.647610071730412, 12.171495512688523, -0.8352200298475623, -8.614549204765297, -3.3951710340759576, 14.234976070491228, 9.378666615367479, 7.849806101049722, 1.2140066605420259, -0.26409063705199287, 4.880683685768474, -2.0321360757039884, 6.565577898468462, 8.344388706252687, 14.883231312155829, 3.6217241528349575, 4.576645835840537, -2.15500304977715, 9.545489206468943, 15.922967654553872, 4.224029960071221, -3.953335723211694, 10.173155301815825, 0.2477691150571104, 10.402248789946498, 5.427601266351786, -2.393196174958348, 6.8157725791483585, 6.199959966590143], 'changes_betw': [-0.027862725110431474, -0.033235846156335375, 0.17791692959888683, 0.011793521350096217, -0.0018647962073037871, 0.002038735983690112, -0.002038735983690112, 0.0, -0.009288990825688098, 0.0, 0.006822282089866127, -0.0024895636134168314, 0.0, 0.0, -0.014567658527903038, 0.0, 0.0, 4.247366632687726e-05, -0.012585554099315388, 0.00424938918822711, 0.015272923644483277, -0.002773530411145084, 0.001479701632606821, 0.001479701632606821, 0.004150688477905602, 0.002959403265213642, -0.0012175784347037977, 0.0, 0.0, -0.001033525880620681, -0.001864796207303815, 0.0, 0.0006472177726000033, 0.0, 0.003373622639677748, 0.0, 0.015123861301231167, 0.002959403265213642, 0.0058532757309516514, 0.01081500898014659, 0.0, 0.006606475413814858, 0.006408264970956076, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005853275730951596, 0.0, 0.0, -0.00981020338818503, -8.413831043800526e-05, 0.0, 0.0, 0.0, -0.004981352037926934, 9.910522142938085e-05, 0.0, 0.00011204957688136508, -0.0011043153244988113, -0.002357086225587765, 0.0, -0.0008332928822224472, 0.001193307768231372, 0.005335501512871529, 6.512628836787861e-05, 0.0, 0.0, 0.004307638787761109, 0.0, 0.0, -0.0005845185508794081, 0.0023570862255877616, 0.0017074413863404601, 0.0, 0.0008494733265375504, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0008494733265375504, -0.002291959937219883, 0.0, 0.00019821044285876088, 0.0008494733265375642, 0.00018405255408313753, 0.0, 0.0008494733265375469, 0.0, 0.0, 0.0, -0.0001840525540831306, -0.001033525880620688, -0.001033525880620681, 0.0, 0.0, 0.0001840525540831358, 0.0, 0.00018405255408313753, 0.00018405255408313406, 0.0, 0.0, 0.0, 0.0, 0.0]}, (85, 88): {'changes': [0.4307275507425121, -3.9103194317386674, 34.5403444455651, -2.2199997430444256, 1.3526256269546195, 5.745371550792868, 2.518326178155746, 11.392548479493286, 1.2627875915335949, 6.902566272314459, -16.606042134469448, -54.49836618738466, -9.916516410972605, 7.71724652480205, -0.6137429099672254, 11.73325138134993, 10.950184291323069, 8.883930380989597, -2.56413859984724, 0.09261952676527585, -5.138983598862339, -1.1110495277411303, 42.35681064623901, 53.8439294598646, -38.786649308754164, 52.725332337990196, 2.6500491232701506, 7.705301463573392, 0.5657345534017395, 0.5609033886324539, 2.500543623154236, -4.371800827801394, -4.354392770845834, -11.491895457159274, 0.19183602319451865, 12.90397049191111, -0.9680822998501135, 37.71606501916801, -1.7832698880956315, 0.9227208648528702, 2.5446893118032676, -9.225085538263485, -7.5461060425879225, 9.94680237631843, 1.031805015939426, 2.5378344010410387, -4.675487946237453, -0.44525504122194093, -1.1890076675272496, -1.2617417131592106, -5.835569917316832, 0.8786057943752681, -8.68080867331247, -1.4327334418686632, 1.460782845789936, 1.0296776884350152, 0.3518163390922453, -5.060694238609813, 3.9948407678493822, -4.6857158188025565, -1.2971431961499889, -0.40368739405413123, 4.398135342479691, -0.12336930913931354, 0.6642993011413054, -2.99288796960802, -1.0324028887387158, 0.4763734200143972, 3.8126966996034355, -11.398986422267427, 7.97333605488393, 3.5493353386825675, 0.27484747463063286, 5.0001238518346725, 7.517677528769411, 17.19036865857848, -1.2166200010416333, -9.147370553080748, 3.8575392760761815, 9.969604848555122, 10.540922268706424, 14.632673908807249, 2.2826737666952113, 12.738597377015083, 18.657846655616197, 0.9412179928042974, 1.57688623199968, 3.8799243505106915, 1.9020640355822138, 1.7852328375818374, 2.940341500731222, -5.870208140289606, 2.9130221531700613, 7.221951782380302, 7.387959580489962, -5.289026867396416, -5.3665843825931745, -4.118148988104991, -5.664047077011531, 1.5426566777373978, 10.816657147277738, 5.639458396012202, 14.495811367248564, 9.323819817784226, 2.335808538908992, -4.391666811365127, 0.11177850118775723, -0.23012365303748084, -0.3904629656529437, 0.186003380215368], 'changes_betw': [-0.02050021843599814, 0.04563977476821518, -0.042298513017167486, 0.0, -0.004144216300179526, 0.0, 0.0, 0.0, -0.011012005889681709, 0.0, 0.03712683850298526, 0.024361276960665357, 0.0, 0.0, 0.003916476546446168, 0.0, 0.0, 0.0, 0.00391647654644614, 0.0, 0.003523291749591463, 0.0, -0.01743281070498195, -0.01743281070498195, 0.0077585230490429275, -0.03424267430383639, -0.0005351681957186105, 0.0, 0.0, -0.0007362102163325362, -0.0037166480591556694, 0.0, 0.003226785107519045, 0.0, 0.004044302056534471, 0.0, 6.472177725969086e-05, -0.029712149895636142, 0.003022102486934325, -0.00041219681892473714, 0.0, 0.026904842806983506, 0.026904842806983506, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00416605990000482, 0.0, 0.0, 0.003916476546446029, 0.0, 0.0, 0.0, 0.0, 0.001391113699982044, 0.0, 0.0, 0.0035042797275212922, 0.0, 0.00041219681892464693, 0.0, 0.004198016277527028, 0.0034213549504069074, 0.004638933385110705, 0.0, 0.0, 0.0, 0.012199245991294912, 0.0, 0.0, 0.0004121968189246478, -0.00041219681892463306, 0.0, 0.0, 0.0005096839959225219, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0005096839959225219, 0.0004121968189246504, 0.0, 0.0, 0.0005096839959225496, 0.00022652622041001436, 0.0, 0.0005096839959225288, 0.0, 0.0, 0.0, -0.00022652622041000742, -0.0007362102163325501, -0.0007362102163325362, 0.0, 0.0, 0.00022652622041001262, 0.0, 0.00022652622041000742, 0.0002265262204100109, 0.0, 0.0, 0.0, 0.0, 0.0]}, (85, 75): {'changes': [1.4292478788387388, -0.03353826793990322, 1.8799569041190836, 26.648251644736078, 4.345595178754152, -40.188578119021514, -70.54693353508338, -120.70513651888879, 1.07306712346994, -3.569934829775036, 5.4391497299214535, 29.228551486455018, 0.5230347032636473, 0.7758076965587861, 0.5915381634972583, -20.43204646522642, -12.415509766435406, 3.257425948589031, 0.038921188733461065, -11.91482148056599, -39.97196475067496, -0.9933916692373828, 8.922886906602145, 1.4247686095556844, -3.1139413783248813, 5.284035448392842, 0.6981861088419876, -6.471529772689962, 8.406968335068655, 5.80998076750069, -1.615020942531764, 2.410527713838917, -4.7508408590002915, -14.887422966067192, 1.6749232835941967, -2.014118783928211, -0.05182889194263751, -0.1932754662446996, -1.5465486120443188, -1.634691651727124, -8.27845925635853, 7.914269651558612, 5.817077511859722, -4.910017820230763, -14.850944671327625, 3.044872229017642, -2.5604288125076664, -2.0639432261049535, 2.892357896708795, -6.60477761777139, -10.046102672330221, 2.162811651696167, -9.089908569949927, 1.3035744283252768, 8.42090766343668, -2.646204015309877, 0.8721331947038848, -6.539299139796952, 6.73067084049228, 2.835292138022261, -11.489424396623576, -1.0648976596613267, 9.254828035588162, 0.12982914674336854, 1.8228356930129763, -6.451114220490162, -17.412402871464877, -7.06135016321241, -0.9190200426725994, 4.258193926811757, 0.9496258833987667, 6.321167723545102, -12.441941033730131, 0.7161860961936952, 1.6719974791080006, -0.4124953940035425, -0.3283334053006115, -12.942443989487344, 6.588308597083824, 0.04686697607610313, -2.63251625403484, 0.7853473762700673, 14.77507534291658, -6.888532542561137, -6.4846166215907886, -0.8441136428178737, 3.407920871521952, 0.3957303922034612, -3.2810108428754745, -1.2535409136601388, -0.9718008280385106, -8.774388372315286, -0.6804958427913732, -4.449399135052431, -1.8189404530426714, -0.4038388528132941, -1.0739158988758248, -1.0204641225141131, -5.0240160159109735, -4.302031978875306, -4.487696459452877, -5.372179737748496, 2.042268785607547, -5.500261573266982, -1.3864656814463387, -6.545530143902937, -1.4736357990190925, 6.927609241600351, -5.903926460127678, -4.022613240210092], 'changes_betw': [-0.02830242868469146, 0.040382748410271285, 0.0019007976959047879, -0.037000226526220474, 0.0, 0.008800543662928987, 0.029652282251670638, 0.029992071582285642, -0.00023643674255294234, 0.0, -0.0034583677167775023, -0.004350314709641926, 0.0, 0.0, 0.0024027959807776667, 0.0, 0.0, 0.0, 0.0024027959807776667, 0.0, 0.04317245926573143, 0.001431362555215772, 0.0006460042392764029, 0.0006460042392764029, 0.0008919469928644237, 0.0012920084785528058, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.000155736776531884, 0.0, -0.0022470592042458104, 0.0012920084785528058, 0.00015573677653185625, -0.00018202999854380342, 0.0, -0.0035390676827985468, -0.0035390676827985607, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002402795980777611, 0.0, 0.0, 0.0, 0.0, -0.00023643674255291458, 0.0, 0.0, 0.0022207659822338494, 0.0, 0.0001820299985437479, 0.0, -0.00015573677653191176, -0.000155736776531884, -0.00015573677653189788, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0001820299985437557, -0.00018202999854376178, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00018202999854376352, 0.0, 0.00014157888775625783, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (85, 82): {'changes': [0.5492096811771106, 0.538467186452948, 7.694974176052085, -2.790282474591166, 8.332444793767138, 6.812852093975295, 6.933117787014169, 15.181756464127176, -1.1696912466863765, 3.720189432561085, 6.229509472914174, 14.662092058759725, -198.96806683613232, 3.521166522548924, 1.5919163561519625, -4.2699540637444215, 16.709522513410263, 5.201261133391881, 0.9321819001413125, 10.049797879712287, -6.148615811427618, 0.8024455654361873, 10.893111912185276, 4.921759204906067, -0.8327738350400296, 9.073141739193218, 5.27616993392909, -3.5099914276328263, 2.1043649226173784, 12.275770609074286, 11.411194659743181, -6.881850219091234, -8.618001437417632, -2.2164500412097823, -3.259697337440052, 0.6665413768167241, -1.477677463830009, 8.145738431974394, -4.628782187622427, -1.2403522033003611, 8.689072069700273, -2.764507370221409, 1.5002982939635672, 4.229429342498463, -2.7251436142468606, -6.542106237944296, -4.736923153765474, 3.103567905901187, -5.504263584009479, -12.809119167889605, -6.733520428485718, 2.5711352074036, 8.1800984538705, -5.1872502209779725, 6.927926359462106, -16.19912883606594, -2.7175895756735287, -0.5991656079685299, -6.127487509005846, 1.9102816362177393, 3.422111526260892, 5.688631201563624, 0.8038240029414965, -0.9101638932099476, -0.9722037187846979, -0.9056439176327302, -2.1547574766402136, 6.294114909544248, 17.327981315570497, -5.580442470394132, -2.193883554374054, -3.975197740589749, -4.775020972364132, -1.229175579850363, 1.4084230797155328, 1.5379652782413586, -4.035248927590942, -7.635707886887843, -3.702702535061462, -6.86152857878966, -9.044686270570423, 6.725487777121543, 7.939799809917417, 4.714654597244646, -11.060754590869408, -3.2039408172819606, 5.122295573081459, 3.6901304064039664, 2.3293777839684537, -4.089902588266369, 3.0758577049375617, -3.8814802987106987, -19.012817660560273, -8.50766412252932, 7.036824925172738, 13.020859566239324, 12.470734235776945, 23.56787580617639, 19.157456684345163, -6.167771583919517, -1.314502473949858, -7.774729676997254, -0.1522337346405891, -11.514736571578368, -4.55464325137541, 2.969988730085589, 10.620678563605054, 1.8155228534287176, -6.1245902821368645, -0.36930069802360777], 'changes_betw': [-0.01872967331682912, -0.002467517758037774, -0.006397747682151386, 0.0, -0.028708153325890287, 0.0, 0.0, 0.0, 0.00987088005436626, 0.0, 0.00309936410853838, -0.005124346714560784, 0.11602389851625326, 0.0, -0.0024675177580374963, 0.0, 0.0, 0.0, -0.002467517758037524, 0.0, 0.0, 0.0, -0.0026891898451531573, -0.0026891898451531573, 0.0, -0.005378379690306315, -0.009950164231509767, 0.0, 0.0, -0.01156699512968623, -0.023537692345031766, 0.0, 0.017143989773959187, 0.0, 0.005879164441855628, 0.0, 0.0014141708331309455, -0.0050385903596913, 0.00432462825429189, 0.001092179991262543, 0.0, 0.0019667330064883637, 0.00196673300648835, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005675290843486558, 0.0, 0.0, -0.0024675177580375796, 0.0, 0.0, 0.0, 0.0, 0.019312169312169336, 0.0, 0.0, -0.0012903904341213246, -8.494733265375452e-05, -0.0004692328851350208, 0.0, 0.028855395369156878, 0.0058791644418556, 0.005879164441855544, 0.0, 0.0, 0.0, 0.008223710823099202, 0.0, 0.0, -0.00024270666472501513, 0.00046923288513503816, 0.00022652622041001436, 0.0, 0.008126628157209215, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0081266281572092, -0.0004692328851350243, 0.0, 0.0, 0.008126628157209215, 0.0034403669724770714, 0.0, 0.008126628157209194, 0.0, 0.0, 0.0, -0.0034403669724770575, -0.011566995129686279, -0.011566995129686262, 0.0, 0.0, 0.0034969985275795697, 5.663155510250272e-05, 0.0034403669724770644, 0.003440366972477061, 0.0, 0.0, 0.0, 0.0, 0.0]}, (85, '85_62_1'): {'changes': [-0.2754452435797141, -0.7692166711128827, -1.5889086480996752, -25.938805020896368, 1.9386010054065537, -82.70128312139497, 65.293879557986, -77.01423834046963, 1.3275826513981173, 7.6673878860530635, 7.927253872093388, 20.784663524570334, 1.0310004343231753, 0.7589277610730392, -0.626040626207871, 6.84375640798217, -3.086378480638473, 4.525682092828276, -0.8371695588477017, -0.42774476002162487, -23.912288918637188, -0.6789952824262286, 12.761712303668162, 8.269688932337516, -1.5684753938807035, 10.27043263760487, -1.0836071640609575, -0.5745091348001097, 17.329227737725432, 3.260547993060925, -0.7332983181830883, 10.370498069143053, 2.8798577681656354, 0.40155698298647735, 1.9393745828242643, 7.566028114895914, 2.207311622299123, 2.9419636391226334, 0.328735153681059, 3.168786092650933, -0.6651717246612066, 5.475525253625534, 3.047578415380144, -1.7719100396487306, 0.2385752177348195, -3.4553086772180563, 0.11878687151573786, 3.854483785645357, 2.3339896377877523, 2.524436697563658, 4.2414614932449695, -0.6301467991779219, 18.215016046193426, -2.1628339905077496, 3.059379866522306, -0.4797947905806268, -1.4307033747582345, 6.945377965542946, 13.600302400257021, 1.6328016223497954, -2.4354296223826566, -9.270753283211818, -0.29420551122660754, 1.1300894686333223, 6.5238507058740325, 3.168040398353625, -3.8830869014407483, 7.4762647323311455, 5.828934298103178, -0.6931789623460105, -4.654276071452159, -8.270236090773722, 8.738410065816936, -10.509880848425865, 2.7035971309512945, 8.827810748517777, -0.7784976686493934, 4.179598357031011, -12.997531159412347, -3.4815478101069175, 16.416768110951182, -14.871235332199888, -9.595044638589457, -4.0919636553338705, -4.592701334905712, -3.3812747675045784, 3.968721666452751, -5.853015898134515, -2.4466469954558647, 0.48219903139411713, 7.857561131465246, -4.409102389160864, -4.746307669690424, 9.774487827481721, 7.109122200765398, 3.859975120642275, -1.7985306222258401, 9.175400423170913, 0.009073573230068632, 7.563668056935967, -4.729980094921018, 6.851568879886514, 2.4724240566697517, 4.688046048733497, -0.9001514111296558, -2.39049258767335, -5.365668293857766, 7.260886008981458, -9.729213793608551, 4.149320880311677], 'changes_betw': [-0.005295859424299776, 7.281199941644445e-06, -0.0002313803537044995, 0.03434703816966797, 0.0, 0.03499830105334693, -0.017159361196058445, 0.01783893985728848, 0.0, 0.0, -8.73743993010101e-05, -8.737439930100663e-05, 0.0, 0.0, 0.0002621231979030303, 0.0, 0.0, 0.0, 0.00026212319790305805, 0.0, 0.004672912318172285, -5.663155510250359e-05, -8.737439930101704e-05, -8.737439930101704e-05, 0.0, -0.00017474879860203407, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00026212319790305805, -0.0001747487986020202, 0.0, -2.9124799766994114e-05, 0.0, -8.737439930099622e-05, -8.737439930099622e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00026212319790305805, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00023299839813603618, 0.0, 2.912479976700799e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9124799767000185e-05, -2.9124799766997583e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9124799767002787e-05, 0.0, 0.0001132631102050062, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (85, 47): {'changes': [5.303142816178733, 2.8354399551584955, -12.20321749464388, 6.599953063761333, -15.054135168217847, 10.013004046902353, 13.467560206809793, 26.00120598653575, 225.27105405060112, 20.791284957676908, -34.56857630891777, -75.58421047855103, 6.61634205350191, 11.126599930072103, 4.220662568543773, -4.632637666209462, 12.900299125350159, 5.63562627382899, 2.9545340356901484, -11.435523082897703, -4.085806896934685, 0.8456081038419256, 2.4188710665608255, 6.226890417024464, -140.8886149633737, 2.183401576701982, -14.854421263233426, 13.1504631010705, 8.635759586353686, -1.688352438308712, -18.70745754645499, 4.900943483930064, -45.71570682273918, 11.66693878163403, -2.13824356348659, 1.6986935527376659, -3.8023646681390844, -0.7160254615534427, -3.507456933257849, -6.066917164116354, 14.74942009267221, 1.478754090215574, 5.156244171661754, 7.83517712764791, 6.134310282042634, 0.46076886738160283, 0.9472134616469816, 5.108035999534351, -5.762850450119579, 21.62396447398686, 6.910344057730725, -0.3512988077964607, 6.321541788091906, 6.890322339909119, 4.4194505038043985, 5.244198985354387, 9.09188664578636, -1.597619878886377, 7.280376873922819, -5.629477526838478, 28.844461651029093, 4.049149970091904, -5.791942107022834, 2.0975004824806582, -4.784278396908846, -1.581527464254286, -6.574542938614144, 20.769467816106214, 10.72617266683352, -77.65823332218025, -5.773174728804747, 13.719425357196542, -7.370735802620857, 6.392387058532279, -9.69501760858634, 1.1876440563613642, 7.625199208687846, -10.003090460617102, 5.877757145218197, 7.571161636864446, 8.994894880363233, 9.786039025545278, 13.168958756491293, 3.8598102625351203, 1.345838554589875, 15.59216592979159, 7.891748033334665, -2.9578901361025487, 22.207160806348156, 3.8663415263716914, 10.120748669289952, 0.9000851982667086, 8.377325446579988, 18.676879256966856, -1.0418380462404855, 8.619326743431913, -13.896763566625353, -11.325367295114276, -9.997437498794852, 6.449010960099656, 12.704609503559936, 32.31582510946066, -19.385146443072045, 22.106912205423612, 27.328614530915473, 8.706925233976278, 4.518207933999491, 10.678177886888818, -3.7858126067882836, -5.092342452021228], 'changes_betw': [-0.08047560945114468, -0.021184467361837545, 0.022557304514491075, -0.0009483579055444974, 0.049968889418430834, 0.0, 0.0, 0.0, -0.16879500186075114, 0.0, 0.10898921132101559, 0.050067571741883685, 0.0, 0.0, -0.01859052149572038, 0.0, 0.0, 0.00013025257673575765, -0.015659838519165775, 0.0011071469022539346, -0.00021043771043771295, 0.00021043771043771208, -0.001588515120625203, -0.001588515120625203, 0.04276627495281931, -0.00255408313512287, 0.05576229750541692, 0.0, 0.0, 0.010809566467058829, 0.05445977173805926, 0.0, 0.04250341628017466, 0.0, 0.0063597236380111555, 0.0, 0.025539213306797454, 0.0014101257220523467, 0.013837920489296718, 0.01883686876041632, 0.0, 0.013437049981392585, 0.013437049981392557, 0.0, 0.0, 0.0, 0.0, 0.0, 0.013778457356439011, 0.0, 0.0, -0.004910764849602817, -0.0004437486853389001, 0.0, 0.0, 0.0, -0.05932624262134972, 0.00011892626571525632, 0.0, 0.012590408232610012, -0.0009627364367425541, -0.002344546381243616, 0.0, -0.01816317389711891, 0.0025625778683882883, 0.011079963755804723, 0.0009287575036810505, 0.0, 0.0, 0.1070279392144835, 0.0, 0.0, 0.0009004417261298005, 0.002344546381243616, 0.002316230603692373, 0.0, -0.005974629063314099, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.005974629063314099, -0.001415788877562588, 0.0, 0.0, -0.0059746290633140575, -0.0048349374037447365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.010809566467058836, 0.010809566467058815, 0.010809566467058815, 0.0, 0.0, -0.0032987880847208054, 0.0015361493190239363, -0.0048349374037447435, -0.004834937403744747, 0.0, 0.0, 0.0, 0.0, 0.0]}, (85, '85_37_1'): {'changes': [2.3351134128344153, 0.5324853683373547, -15.7120873632526, 3.8574265508316756, 1.7812245578179002, -4.607709812251045, 2.8438454783548366, 12.540288641903686, 0.24657213315889948, -7.7526523682440995, 6.922684506350677, 92.64794922091372, -13.677348525874976, -16.452401035708505, -0.05207757280830094, -6.361631947856637, 0.4096118081968996, 12.11965956001734, -0.0869831987881291, 0.9311139185226125, -1.65877743584619, -8.570060020998199, -27.447021446175555, -10.970358071364757, -44.41295552516874, -12.123116507047584, 1.8809462986286647, 2.6032141511411737, 1.810193397859905, 1.7037392686295192, -0.9137868036588657, -20.197119790176288, -10.386105294610402, -2.205403074201456, 0.5202674218799501, 4.44857649324473, 1.5672647840897937, -14.479736477857244, 0.9636791101374129, -1.276357375607816, -1.2777006790545329, 16.07482306911622, 18.148906660049377, 9.810540354036135, -4.7995164921170215, -11.370904374911532, -5.458606134896087, -1.8901587888584857, -1.7755725725021918, -6.079039979221363, -9.060937187348372, -2.803129068527774, 9.515515259856244, -15.551204269662577, -2.5085934183496192, -5.5544675103688235, -2.6163887441348983, 8.556501875286074, -12.537250810064734, -1.180735379050212, -5.727320058236415, -8.698080290041673, 6.950887827717651, -0.36786822012108544, 3.504681498286274, -2.248972661228514, 4.216338870058678, 2.5266521940788493, -6.314565643909958, -11.0615838132819, 0.26374095888161264, -0.028360116633905363, -9.654665431501826, -0.7313416163254374, -11.541516357865419, -13.884892665436212, -2.259663634011062, -9.614659974747525, -7.518775482224214, 5.989776045818843, 1.6497798564586788, -10.218705766617205, -15.175611424431992, 1.3127845050256894, -5.2793082997603165, 3.9868491655163893, -10.002048611485549, -8.462472956057297, 0.23209199021744098, -1.7261308233264643, 2.343295024331894, -3.8567981566473577, 6.811229605026, 3.315134112937727, 2.8126012906431868, -5.347830858386942, 8.730510459690521, -1.8282657982084913, 3.447895461384462, 8.097409203437934, -7.365879564159911, 10.353107011126866, 6.7335062592148915, 6.326893284362484, -9.804009045566403, 4.654655601929335, -0.09665434937537043, -2.372203031945901, -6.219154784732609, -16.964510406854032], 'changes_betw': [-0.01770909179166047, 0.0001181172434994715, 0.028130915974952694, -0.0004332313965341589, -0.005729899843049668, 0.0, 0.0, 0.0, -0.00028780965325311825, 0.0, -0.004498365775124233, -0.030626951766095502, 0.0, 0.0, -3.640599970849978e-06, 0.0, 0.0, 1.4157888775625464e-05, 0.00019456984288793167, 0.0001840525540831345, 0.0001268142323188233, 8.353154377619201e-05, 0.00606078992929146, 0.00606078992929146, 0.01677386211025355, 0.01212157985858292, -0.0002803261977573357, 0.0, 0.0, -0.00039642088571752165, -0.005557173599987059, 0.0, 0.005293836868760435, 0.0, 0.0008021455269161815, 0.0, -9.263304370357717e-05, 0.012348106078992935, 0.0002754720644629338, 0.0013191107227804033, 0.0, -0.01293424267430384, -0.013132453117162615, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0006690613724252698, 0.0, 0.0, 0.0011520476352280817, -4.449622186625344e-05, 0.0, 0.0, 0.0, 0.007632922350047727, 1.4157888775626331e-05, 0.0, 0.0023643674255294372, -5.663155510250272e-05, -1.2135333236212853e-06, 0.0, 0.005967752374480384, 0.0008021455269161815, 0.0008304613044673864, 7.92841771435045e-05, 0.0, 0.0, 0.00781596362636118, 0.0, 0.0, 0.00024230215361713877, 1.2135333236247547e-06, 0.00016423150979725902, 0.0, 0.0003397893306150146, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003397893306150146, 7.807064381988235e-05, 0.0, 0.0, 0.00033978933061504235, 5.663155510250706e-05, 0.0, 0.00033978933061502153, 0.0, 0.0, 0.0, -5.663155510250012e-05, -0.0003964208857175286, -0.00039642088571752165, 0.0, 0.0, 5.663155510250359e-05, 0.0, 5.663155510250012e-05, 5.663155510250012e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (85, 59): {'changes': [0.6549730426705445, 0.34708173904109785, -5.553386918359109, -3.544314879020149, -9.115902387855634, 9.425644948258537, 4.041493057948742, 4.087075182642053, -0.9107079769104871, -7.612230959003341, -0.24047747541531805, 4.191477042287744, -5.754330601655113, 6.864650062766316, 0.18087334173561587, 4.93235708244498, -11.615601306985155, 4.581007801454007, -0.26879764941968887, 0.3436012275624307, 8.181395750345828, 5.9469994541113635, -0.430725383901688, -8.48302774474854, 9.962834295589488, -1.9009616048960964, 1.0426360484271129, 11.247056704001096, -5.713503014178855, -1.619920625453119, -1.744024386942563, 1.5665109253376386, -6.635094747037414, 2.915707986789016, -2.4041306225751455, -3.1780303839629482, -0.8142529060378045, -2.4655684431236153, 1.3681965809984362, -0.6106965862237814, -1.4810870073930005, 3.828501047027032, 5.051095155293638, 3.6377015448001657, 1.0762721950421508, 17.485320498383913, -4.742665286029023, 5.7083782474608995, -1.8319324204372762, 6.934546919156219, -8.098870802980457, 2.397952298043357, -17.496834658707826, -4.787088576860981, 0.8314582162742568, 4.885084655268145, -1.3498828364108526, -8.266577585264741, 5.915528205101623, -0.48876409707325763, -3.041316770076122, -1.1485602846286156, 16.46754754977499, -0.017100344665415435, 1.2763155414759808, -2.3609298512244052, 9.499992141369205, 0.58196826368561, -0.9115855303430465, 9.91253664443525, -0.8507199330530284, -7.137097194972682, -0.8405207035414151, 0.3272674219410874, 8.09827262049538, 2.8778885431602816, 6.027953465308819, 10.185149011656023, 3.857690058508524, -11.292565396357844, -2.694771317558832, 8.980518020437842, -0.01755584464578419, 11.705136928061819, 15.999224810966467, 1.7866670805484546, -0.30928921303086554, 18.47686923635476, 7.228956125732765, -0.03938642124237646, -5.343519170407205, -9.492668221188893, 9.798530378773563, -10.604489048506935, 6.124905430171765, 10.691598372450414, 5.209009880309338, 0.5562487766878377, 4.394796358007426, 1.1220011166975041, 8.640740727357496, -1.2267188214955524, 7.229226468740734, -2.1070051318999674, 2.94333817852916, 18.307870975398657, 3.2016800108613666, -2.0464013391170965, -4.471017519509701, -0.7842454923838318], 'changes_betw': [-0.0018461886963415663, 0.0, -1.6989466530753505e-05, 0.0, 0.015545361875637126, 0.0, 0.0, 0.0, -0.0018207044965454777, 0.0, -8.49473326539063e-06, -8.494733265376753e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -8.494733265376753e-06, -8.494733265376753e-06, 0.0, -1.6989466530753505e-05, 0.0018461886963416219, 0.0, 0.0, 0.0001415788877562607, 0.001846188696341608, 0.0, 0.0017046098085853473, 0.0, 2.5484199796144136e-05, 0.0, -2.5484199796144136e-05, -1.6989466530753505e-05, -2.5484199796144136e-05, 0.0, 0.0, -8.494733265362875e-06, -8.494733265376753e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0018207044965454777, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00011609468796019984, 2.548419979611638e-05, 2.5484199796130258e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -8.494733265375365e-05, 0.0, 0.0, 0.0, -8.494733265373977e-05, -5.663155510249318e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0001415788877562607, 0.00014157888775625377, 0.00014157888775625377, 0.0, 0.0, 0.0, 5.663155510250272e-05, -5.663155510250012e-05, -5.663155510250359e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (71, 64): {'changes': [1.1448181038084826, 3.533909279323993, 0.6657221621126155, -3.3387759586342582, 4.1210558309860446, -0.03096053076649241, -12.90656228281614, 2.4417397204284725, 4.090206194658798, -5.658230347014012, 0.10290965461035739, 10.389455450732498, 9.638343738666094, -8.45576296844331, 4.691387510392644, -196.65223978404686, 14.507020664199388, 12.087744047941356, 3.3022988342259794, -23.491808086356315, 2.3141863912952374, 7.290887176453424, -2.8332085989126767, -6.1667587014993615, 14.98678142998537, -0.8228227381950433, 2.2515584892809315, 4.760433850301524, 5.5588717234871865, 5.42457056767492, 3.124878350916866, 10.974009144562672, 1.3151716167946859, 1.687761118903154, 2.3136464339726714, 5.2057229588018, -2.623536413954074, -2.465129206706365, -1.187128740537318, -3.2105154647632617, 10.15744558715221, 0.3792846533809495, -1.3710385443137056, 3.643029644381727, 18.32261843643846, 17.93391022095207, 3.796115708542942, -2.2328048530995943, -1.8394925312294035, 12.72598087833785, 5.800741480330657, 2.989746455969353, 6.68214982151315, 6.169762590967025, 5.321679297440085, 22.613001544841893, 1.2491505517882757, 9.040600583682902, -5.260829097622718, 1.69716535261216, 13.246072573857049, -4.466687956382344, 2.8614295699637182, 0.0529947044481105, 6.809726338559997, 0.9059012732977649, 0.7966244127605933, 2.8515159422059355, 1.7580567656621326, -8.056689697655614, 5.911635363229493, 19.034859008157127, 14.73908176147097, -1.4333329799444243, 2.5496955882370003, 8.969722888722742, 3.675988232692035, 4.632109677802987, 3.244709513456314, 9.531147507819327, 2.518219075689956, 1.8017585001586554, 7.197239666350839, -0.30879135275810654, 6.629997029945628, 6.805583641525203, 10.784173710696194, 6.2043882786712174, 1.8439007212666922, 3.2505367871358573, -1.9095542220202333, 8.843355031669319, -11.946725881393121, 5.796374918643153, 1.546720530296625, 7.412724016453694, 2.738403933108458, 1.433644563824032, 7.873473538954926, -4.758734281048191, 6.8280480420450544, -1.2730845093380765, 8.234427506093454, 0.17828118612189314, 8.520401478820702, 1.1843322758385, 3.410932158437504, -1.335000740962471, 15.99355381267955, 6.92352981416002], 'changes_betw': [-0.01127857870977128, -0.029644596540621126, 0.003136579130462891, 0.0, -0.0018647962073037871, 0.0, 0.0, 0.0, -0.008316748377910493, 0.0, 0.00641028752649539, 9.708266589001854e-05, 0.0, 0.0, -0.02207012604566108, 0.19564786499037265, 0.0, 0.0001274209989806322, -0.018609938028898154, 0.00746403896250992, -0.0009222853259550565, -8.673617379884035e-19, 0.002589275601508012, 0.002589275601508012, -0.0005015937737650276, 0.006458424348332575, -0.0012175784347037977, 0.0, 0.0, -0.001033525880620681, -0.001864796207303815, 0.0, 0.0006472177726000033, 0.0, 0.003749817970001512, 0.0, 0.022463715353623487, 0.007285245052829126, 0.0054770804006278595, 0.015377894276976889, 0.0, 0.009095432260569844, 0.00831674837791041, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005477080400627832, 0.0, 0.0, -0.012572609743863483, -0.00023138035370451338, 0.0, 0.0, 0.0, -0.004009109590149329, 0.00015573677653188314, 0.0, 0.0004558840185751589, -0.0011892626571525658, -0.003022506998042164, 0.0, -0.0004570975518986553, 0.0015695030985551361, 0.005335501512871529, 0.00020670517612413715, 0.0, 0.0, 0.004307638787761109, 0.0, 0.0, -0.0007968868825137957, 0.0030225069980421675, 0.002018914939404228, 0.0, 0.0008494733265375504, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0008494733265375504, -0.0028158018219180386, 0.0, 0.0, 0.0008494733265375642, 0.00018405255408313753, 0.0, 0.0008494733265375469, 0.0, 0.0, 0.0, -0.0001840525540831306, -0.001033525880620688, -0.001033525880620681, 0.0, 0.0, 0.0001840525540831358, 0.0, 0.00018405255408313753, 0.00018405255408313406, 0.0, 0.0, 0.0, 0.0, 0.0]}, (71, 88): {'changes': [1.4416695058877753, 2.930536188947464, 7.054142911499824, -1.9820909517081162, 0.45418892303437275, -5.581028504616114, -4.057474531447411, -1.885454460586459, 1.2156934644996582, -16.883661395156963, 1.3530444802803032, -11.974053019671175, -6.181406285331775, -2.564827771111709, 1.467133940858119, -35.74359189477059, 5.575693575555505, -16.784853795773984, -1.676063576116821, -59.78751882108307, 4.81859982769916, 4.541095167301734, 11.901564522592963, 8.913683711645717, 6.900292334721058, 9.173785372907673, -3.300509774672747, -4.0343519457319985, 5.560715253459648, 5.295948492329572, 4.849061864249954, 12.712230797693678, 1.625786848690609, 7.531588864893536, -2.8428022130673867, -10.603711681604096, -2.888408031223971, 6.014850278146568, 0.9571553388163636, -1.6466182684345654, -1.1509683246853797, -2.8146056404624034, -0.3741059920884595, 7.59592724256521, -7.402805333607148, -2.301693986589271, -3.6468447987716246, -2.586536091434965, 1.7350453892039752, -13.317072485551478, -19.403148978513883, -2.4597443581173124, -0.8921360600407695, -11.647391969411615, -11.868292168113896, -14.253239869862199, 0.7116867293509905, -2.9479585625960567, -7.206290578735974, -0.9783552521953283, -8.697744024159107, -8.65751275207478, -16.644098172849795, 0.6271993326250431, 2.723759187560809, 0.7032239278573016, -9.03060973394301, -3.681462504214892, -5.59929257553074, -10.632023955962211, -6.964477929697068, 3.2215660682511498, -7.881193199167086, -6.310958712931466, 3.3092538135765324, -4.109627892524202, 1.0936422647384063, -12.475863832846358, -15.71772430608803, -9.403770175853822, 0.9278931390644516, -1.1408990829091863, 9.138543571066265, -4.530862530197567, -5.086122260914436, 5.10621637762452, -3.8520725781952763, -2.392306656742676, -3.57194501511745, -4.536271603820246, 1.0507784193914063, -1.267181508330907, 0.3675849158946676, -11.066952887253223, -7.834022372699991, 0.748363278025181, 2.3852666341093993, -1.5304577263827923, -0.1503636048023793, -12.487014264824154, -8.206725778243225, -4.117961864665546, -5.815396477662233, 8.15917467081485, -1.8715476996521545, -5.731971768321898, 1.2779238269284008, 0.35245070453589733, -8.8515440608993, -7.417832442345656], 'changes_betw': [0.015511566811261313, -0.010438029143432004, -0.008956513339795635, 0.0020051860775611996, 0.0, 0.0, 0.0, 0.0, 0.0005929581235391557, 0.0, 0.0013328089398425697, 0.004562830136224622, 0.0, 0.0, -0.0008030158386936603, 0.004363865831755742, 0.0, 5.6631555102503155e-05, 0.00014252887596419295, 0.014593424659683586, 0.0003827349265677592, -0.00038273492656774964, -0.0018954532461158344, -0.0018954532461158344, -0.003890992346649839, -0.002764661811552757, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00010774704964308657, 0.0, 0.0023840413748670575, -0.0020385643729171066, -0.00010774704964308657, 0.004047072344727787, 0.0, 0.002450675386455242, 0.0022524649435964605, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0017243695633094025, -4.045111078750281e-05, 0.0, 0.0, 0.0, 0.0005929581235391834, 3.539472193906474e-05, 0.0, 0.003419877871967514, -7.7868388265942e-05, -0.00024351568694076092, 0.0, 0.0001077470496430033, 0.00010774704964305881, 0.00010774704964307269, 9.061048816400496e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 9.061048816400366e-05, 0.0002435156869407644, 0.0002435156869407644, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00015290519877675726, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (71, 60): {'changes': [0.4640313028840026, 2.53681541163483, 1.1573004429017288, -1.2500799996480083, 1.6093478476154175, 3.4729942429534617, -8.262949518855379, 3.4319048967105346, 1.1208552200678668, -2.225589901453219, 3.5530975262082194, 2.051027049370987, 2.1098108230714274, -0.0026008225871692048, 0.10570524320424468, -4.187129261081395, -5.160514036012216, -4.253059199780068, 1.9691257986604, 8.689956380792552, 5.314548196784045, 3.572488598061568, -0.7144024088533456, 4.699686116754037, 0.1636008206259021, 6.030480988668359, 0.5384789354266033, 1.5504625922089872, -4.824910305559797, 1.806319026880118, 0.25168562608766365, 5.121895787035953, 7.447892645052434, -1.2424037609636116, 3.7088920148935856, -1.318377854092887, 1.5715624276414317, -2.458329314092566, -0.820085545801625, 6.218630464639304, -1.242392881880562, 2.311128953033574, 0.3025836177986321, -3.8784063464571545, 9.153259077604332, -2.2414551611373668, -5.988113140671558, 0.9718546593103099, 1.119724365420744, 1.8701816742463961, -5.11487189788221, 1.262283798521885, 14.388782294848227, 2.852086827867737, -11.736885477156477, -0.054427651782077646, 0.7167236152851117, 3.5959340198590724, -4.318869007953936, 0.8116680864620207, -4.774458349234806, 1.7752562755610626, 8.496468567638175, 0.8131131404782899, 3.9999252556035074, 1.4458394781482582, 0.9836431890754795, -9.788836309521116, -3.248894589857656, 3.0147131684914257, 8.799224829334179, -0.2187231217515091, -19.445524698782066, -4.597719374395865, -1.675175505043569, 6.955822010199142, 3.888281594120798, -8.862173549550022, 0.5081966873044053, -7.662011061228128, -9.632334312096958, -0.10439658544083841, -12.532357783812728, 6.9304239198421556, 8.125940115509337, -0.9108028281277001, -2.4246110524481708, -2.8660369584637806, -4.700672689433702, 4.448922522753719, 4.30192620510212, -9.960162989473815, -5.103601029147171, -8.10903296180905, -10.19974445540484, -4.369381352620223, 1.8360529827433822, -7.6486283440465, -6.318439343049505, 1.9812105625188678, -1.0178752445772545, 3.6865475472766605, -5.339861196028863, -2.9811191252747733, 2.595726477504428, -7.073629333938783, -6.258666904744587, 0.6374807575411978, -1.1265053891670505, -2.63832957678693], 'changes_betw': [0.00025484199796133034, -0.006696681390870929, 0.005167629403103446, 0.0, 0.0, 0.0, 0.0, 0.0, -2.8315777551274346e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00014157888775623295, 0.002208630648997622, 0.0, 0.0, -0.0001415788877562607, 0.0, -0.0002831577755125145, 0.0, 8.494733265375365e-05, 8.494733265375365e-05, 0.0, 0.0001698946653075073, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -2.831577755124659e-05, 0.0, 0.0001698946653074518, 0.00016989466530749342, 2.8315777551218835e-05, 1.4157888775623295e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8315777551260468e-05, 0.0, 0.0, -0.0001415788877562052, 0.0, 0.0, 0.0, 0.0, -2.8315777551218835e-05, 0.0, 0.0, -0.0001274209989806374, 0.0, -1.4157888775630234e-05, 0.0, -2.83157775513021e-05, -2.8315777551274346e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.4157888775625897e-05, 1.4157888775626765e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.415788877562503e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (71, 65): {'changes': [-1.8290760359008313, 2.438010582514945, 2.1308403799197038, -6.806373485361576, -3.2524439994132095, -5.545152161113634, 11.927494313468827, 4.839055868769776, 1.4134814596671532, -1.3516171773513292, 0.7159701170531747, -9.509739783892002, -5.945785192538125, -14.746396518646037, 1.6843304659621552, -8.011519819728989, -8.336038707257245, -3.0433427998862044, -0.6664732326047655, 2.3073054887231024, 91.14025613437663, -16.253954313606926, 0.6203263412712374, 1.8913293479377842, 5.775588499295338, 0.53216325739902, 0.3301168568129569, 9.505807366617859, -2.02739173573201, 4.494525681686582, -0.04017306585097913, 20.213857576029852, 6.069982469458594, -5.312974620371222, -0.11134471300540838, 9.87168949736133, 1.072330104868744, 0.43635574527934295, -2.247894098072628, -0.2986224944188791, -8.23397258119428, -1.1294654401897049, -1.180983422618425, -0.7269604568727459, -1.3970764714356392, 13.086385131915108, 1.1853290805906909, -2.7292154740889885, -4.456758639533369, 5.010294371231851, -2.8224787770561193, -1.1790061486616494, -6.345201845418586, 8.734035168438197, 11.055403577677282, -11.384642548309529, 2.053188636405661, -2.3007091417608194, -1.4250452405981378, -0.5367532976563965, -0.5474043696531794, -2.970359110440725, -4.188218455229389, 1.394681461241607, -3.490804170381864, -2.1819170487724833, 2.354241184948819, -10.919419559720325, 3.322219020271916, 3.1900245878184563, -4.340241243970695, 7.425036313824421, -1.1559413190072974, 2.934832789524421, 15.039313973397725, -8.570597335911145, -1.3434231870504476, 2.762330174793135, -3.8848310656267415, -8.79647371741649, 1.4679774443312112, 4.712012293302706, 4.289085113644887, 8.631891821614118, 1.2083385515688292, 3.597550513547418, -25.69216620867877, -3.7680628713112014, -39.72483536091181, -0.40799690039457204, 7.828837554429327, -1.3866197208812991, 11.009643733541367, -4.58714060271825, -1.1843356560475229, 5.510448429824436, 4.002235643383827, 6.127894369908944, 7.262157757410932, 0.21013854046498182, 6.408757803137121, 15.994005214106721, 17.68112290139814, 5.190044966903116, -3.431643017580427, -0.7800241363800922, -0.757801026688071, -4.367780610705836, 6.629848770750357, 11.006272366458177], 'changes_betw': [0.017492273837839745, -0.010930294645890876, -5.541802177884414e-05, 0.012101758814296953, 0.0, 0.0, 0.0, 0.0, 6.371049949033258e-05, 0.0, 0.0010794378913644709, 0.0013257851560603863, 0.0, 0.0, -0.0011358671909130202, 0.0, 0.0, 0.0, -0.0011358671909129647, 0.0, -0.016811683898839858, 0.007890595925764129, -3.6405999708846726e-06, -3.6405999708846726e-06, -0.00024634726469588853, -7.281199941783223e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.247366632689764e-05, 0.0, 0.001093393524586067, -7.281199941783223e-06, -4.2473666326869886e-05, 0.00010921799912622099, 0.0, 0.0011006747245279475, 0.0011006747245279336, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.001135867190913048, 0.0, 0.0, 0.0, 0.0, 6.371049949030483e-05, 0.0, 0.0, -0.0010266491917867993, 0.0, -0.00010921799912624874, 0.0, 4.2473666326869886e-05, 4.2473666326869886e-05, 4.247366632688376e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00010921799912625568, 0.00010921799912625568, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00010921799912625915, 0.0, 0.002421403491739883, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (71, 67): {'changes': [-0.528819255198, 1.2317618035031508, 0.9514014914620077, -3.9746019514170285, 1.5068276648904089, -5.133348990658675, 1.6773629889235622, -4.743561182042583, 1.7100340001795118, -5.011859511522459, 3.128784407357628, -0.6026117681070957, -4.877750647035327, -11.108951224814462, 0.026432615675346938, -15.7189522080333, -9.469103614787969, 5.938896979760273, 0.9126482875393656, 3.9223299499497273, -17.785663860792127, 64.13163053318155, 4.121835264560531, -1.6700168729878442, -3.578950914043048, 8.45618723593293, 0.5185842422194611, -8.726814999633518, -8.31688610178307, 3.4736033169185276, -4.7485077724322196, 9.927168150033197, 8.766373197582254, 6.165961367684986, 3.099703720629222, 1.9213981749590516, -2.272144586962014, 2.733026420794687, -1.904710328968683, -0.7000837513824081, 3.4225095924358016, -0.9202362874219858, -3.7188567992395463, 0.9920881787913345, -13.551766109674674, 7.340715087291187, -8.801472756342278, 6.17815863827397, -0.2435419488032622, -1.2849639021571875, 4.110505855212921, -2.9606290754424265, 7.2410014254033115, 1.5804009703853126, 8.876560787860711, -5.584173432899718, 1.3154416590980986, 14.212614825957274, -6.7183042769697465, -3.586470616566139, 3.900678641024257, 2.3756789505281546, 5.335145203597392, -0.1123361583621687, 2.321471433502552, 1.6205485257126995, 4.328170703746707, 2.5780774067161474, -2.480582866340683, -4.147617764709651, -3.8773912506173644, -8.785316485032155, -9.844077775647293, -5.478578980939972, -2.037796320042162, -2.1628854425435406, 2.5806467944549496, 5.15214556640808, 6.080839807064393, -4.780542098374639, 6.580390497422798, -2.73702929452395, -13.806278332975978, 5.681311290687461, -2.963995100470811, 3.6471484417161406, 7.973784250111947, 0.8918905798535093, -55.935622651850366, 3.099179046274827, 8.022642020752187, -0.7553479429079744, 6.45116553886993, 1.6517145887159472, 1.0770091445166088, 2.4294081547945154, -2.597315189347455, 5.8724645218983085, -3.7502980075253447, 5.805368006460071, -5.982513404239057, 3.8086152324417526, 7.34996734103666, 3.5215478239114475, -4.677789039003613, 5.139384230955017, -0.44392043503589207, 0.8859345987353322, 10.361594285768433, 0.5270912491297111], 'changes_betw': [-0.00030297881979840513, -0.006589081436176203, -0.00015007362102162358, 0.0059378185524973925, 0.0, 0.0001698946653075073, -0.00016989466530751077, 0.0, 1.5573677653202278e-05, 0.0, -1.55736776531884e-05, 0.00013449994336844212, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024760731679691925, -0.0071483180428134506, 0.0, 0.0, -0.00015007362102163312, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5573677653202278e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01798051874504474, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (71, 70): {'changes': [0.27357283256392684, 1.1781073409177019, -0.9359971833096239, 4.43686604593519, -0.6647007212231486, 11.760276284689013, -4.40216398520846, -5.669283525914949, -0.396921200760751, -1.2412348338925199, 6.175966968067158, 18.186358038312477, 1.182125008324931, 7.308593693501479, -0.9592192021106563, 6.234698926170665, -9.506812833584775, -14.048924389452765, 2.852089382087854, -37.443344055518764, 1.9640969727502124, -4.567142446756634, 4.433537276522941, -1.5825973573523697, -14.676161705096746, 0.5679096778032289, 0.9050704954462248, -23.661136834085482, 2.6558814160273982, -4.604591229097807, -0.9516419548498405, 7.6892430347869265, 0.2535977418863382, -0.02946509406643827, 4.278842039357649, 13.150764593564503, -2.174178369192635, 2.419076933864588, -1.1439723343530943, 1.597166813072775, -1.2148665010478226, 6.730710444059582, 6.072705538051906, 5.934694672281523, -3.009796322534953, -1.8641642122733515, 5.00633605227884, 4.516805002617886, 2.1756726238718542, -10.528664820885496, 2.4253244676918655, 0.981230166928043, 13.68791473180488, 11.312001140235026, 7.4155149119344514, 2.508006755512497, 0.21917831798351983, -10.181070431070708, -3.628411791933445, 0.5130402231113322, 1.525617961085345, 4.785631675305112, -1.8343164652938526, -0.11034445747544197, -1.6663513324143366, 4.074404319335635, -12.88041438185266, 2.4071249406595143, -0.8143647098325459, 3.027056586313151, 8.307197389106989, -15.163024454794794, 9.99042424220724, 3.343199085172671, 7.399118652769573, -6.937706272198852, 3.3195180368070822, 3.0076384467702155, -9.373046463889011, -5.264585328965637, 0.01748276215249689, 13.940856334716045, -5.993397849220912, 0.3960681402155899, 3.6112415466946004, -3.254224993157564, 13.347867502780929, -1.0623676767029337, -14.021578459891714, -2.5984097582557624, 5.237621958511511, 1.894978155949957, -1.168585420948034, -2.4061122382929625, 8.167310690712952, -7.937506797905826, 9.99213358078002, 0.2602523988618799, -4.434919078317648, -10.032369351125169, -2.8021476226811046, -3.0323578760812495, -8.753525263996437, -5.455599164026182, -13.675582986772866, -9.406707864087565, 0.5781898149442952, 5.082321327555007, -12.226736446129365, -11.13098872405925], 'changes_betw': [-0.00025484199796133034, -0.0049467663382033455, 0.0029335145543095686, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0001274209989806374, 0.0, 4.247366632688376e-05, -0.00021236833163438065, 0.0, 0.0, 0.0037490089477860145, 0.0, 0.0, 0.0, 0.00032279986408434436, 0.00805017555782084, -8.494733265375365e-05, 0.0, -0.00011892626571524678, -0.00011892626571524678, 0.00025484199796126355, -0.00023785253143050744, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -8.494733265373977e-05, 0.0, 8.494733265376753e-05, -0.00023785253143050744, 8.494733265376753e-05, 0.00032279986408423333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.494733265375365e-05, 0.0, 0.0, 0.0003227998640843166, 0.0, 0.0, 0.0, 0.0, -0.0001274209989806374, 0.0, 0.0, 0.00032279986408421946, 0.0, 0.0, 0.0, -8.494733265379528e-05, -8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (88, 60): {'changes': [0.5274420809276226, -0.37541697701464827, 0.5448315610303496, 2.0235882327971666, -1.0421034019695057, -1.701752962570879, -8.592089995446685, -20.46017193512057, 0.06702016392406307, -9.777336616898879, 1.5603919311254941, -0.5590621413690542, -0.7806193923085516, 6.073560166866457, 0.44379208443189455, 1.7326048527846467, -3.8755799024666544, 6.197712917128911, 0.7451287175980319, 13.164021081055381, 2.06898334512654, 1.2884831208245942, 6.498309747684317, 9.261298124979277, -2.533894625961892, 6.6757565089182265, -0.7996137129923184, -3.500323701117736, 10.290985115721526, -2.697717106895041, 2.2178893469853094, -5.452362405343251, -0.8119373728457191, -2.4204636926442618, 3.7616467362831685, -11.771455797173587, 1.0809683375233199, 5.110826399140137, -2.3809882330391687, -3.281576351787429, 16.551806194858074, 1.8986521432311037, -6.983733644697068, 4.543301775927944, -10.925998476547079, 15.556558116505613, -2.9698989071950166, -5.269178986985011, -0.5554169298798683, -7.725993372834921, -3.6678220834846798, 0.7365348884500094, 3.024821863847393, 6.794566232207558, -12.725924770186793, -4.893475016340119, 3.2448431206437043, -0.4218040714991389, -6.956080983274916, -3.7513346994420047, 0.7557481969045625, 5.488503906746715, 4.376121952787514, 1.8103651314358018, -1.2132154004656712, -0.6044176693010783, -11.313699289702896, -1.9368859048806257, -0.565931428078926, 1.2999685911055678, -1.4284360984855766, 2.6863508115479817, -2.9332812767003986, -4.822853567458083, 3.314858186173609, -12.166891880708192, -1.631890973560715, 3.582368683827184, -6.588087831509654, -4.530723747044021, -6.791261937155724, 22.857251400362514, 2.0149169323187834, 2.6168174303457192, 3.936030273686441, -0.9152885099498747, -2.7932106901086513, -12.550370446345255, -6.176630168137223, -2.621451020050145, 1.369105293206644, -5.043234703193519, -9.228682089795825, 1.3504139294800552, -4.601212614643487, -9.401440568351035, -5.4229934880639945, -3.6101712744072643, -4.3090068268961375, 2.0402394320997814, -8.044826440182419, -3.330411743216189, 5.798158363834034, -9.973296242116106, -2.676690240045019, -1.4472989178785838, 2.15463402826083, 14.536043793072366, -4.76054374485517, -5.223918013557352], 'changes_betw': [0.00042473666326875437, 0.010159700985389042, -0.00566032393249509, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000155736776531884, 0.0, -7.078944387814423e-05, 0.00041057877449314495, 0.0, 0.0, 0.0006909049722506055, 0.00019821044285876126, 0.0, 0.0, 0.0006909049722506333, -0.0016904519198097175, 0.0001415788877562607, 0.0, -0.0003879261524521449, -0.0003879261524521449, -0.00048136821837127617, -0.0007758523049042898, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.494733265376753e-05, 0.0, -0.00048136821837130306, -0.0007758523049042898, -8.494733265373977e-05, 0.0002661683089817124, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -8.494733265375365e-05, 0.0, 0.0, 0.0006909049722506055, 0.0, 0.0, 0.0, 0.0, 0.000155736776531884, 0.0, 0.0, 0.0006625891946992202, 0.0, 2.831577755124659e-05, 0.0, 8.494733265373977e-05, 8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8315777551250927e-05, -2.831577755125006e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8315777551251795e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (88, 46): {'changes': [-0.2466508804806118, -0.22840224461752712, -3.1234711326072215, 0.40714223236163605, 5.187173145046472, 11.60171331924883, 8.421739238883589, 4.225279313807619, -5.4646230487645795, 0.27919936410376067, 3.6749389848788923, -4.5182540000568, 10.055251274115193, -4.59878198334755, 0.6351188931717502, -4.235290696561918, 10.070124870412371, -1.3845096849743186, 0.9281353131849066, 4.086778526498591, 11.672710787505054, 6.028422437835019, 137.42663668982593, -53.76959542086698, -4.5643072269148774, -5.192844305292439, 1.450410226834741, 3.4454923755856157, 12.35669247819294, -0.7636333438040595, 0.7064095541004676, 4.2339707734191165, -5.511137554342909, -5.687783580951475, 2.2774719180099297, 6.551376003983364, 1.8340704439306386, -5.645073013432196, 0.1903463249902373, 1.1491124459939215, 0.11997119221575758, -0.4376094598994058, 4.067637452619735, -4.69992209583404, 18.41447566981114, -4.578044665264827, 1.308395759044899, 2.5136459037338312, 1.107313947065741, -3.9184224349100987, 12.765214739395844, -0.35730568456811795, 1.8103482764988712, -1.2252738184275813, 20.59064860693951, -1.8257333813131709, 1.915398155469802, 10.916476071281522, 8.198989147252291, -6.063853553981517, -6.164109076117285, -6.29562461849153, -5.901468426730958, 0.6061438427020214, 1.8439571457858293, -0.47026382611282, -17.319663842776748, 7.865615887192064, 1.7870671338228874, 2.9228754190866937, -0.5772363995283172, -4.9604129697560495, 4.495764765027076, 12.924965730946553, 5.3532672078102905, -6.676860344452223, -0.41554919975807536, -5.022382618427741, 5.793726488702703, 23.010915217129934, 4.579286825789552, 9.338490542917725, -5.242085438171955, -5.043789271497928, 3.6429689674106953, 5.512639197067486, -1.7290344067564263, 6.708042900246312, 7.148914281025895, 0.05476113239392788, -6.370399408029726, 1.7691674747055117, 3.667928507940985, 8.101024845506402, -0.5160860893915356, 3.0682326373830904, 6.260708506838142, 8.758842472097385, -2.639462206940408, 9.976550484716995, 15.754133395118203, -9.551033197969673, 3.1013348304531974, 0.29223344997799927, 7.323754479558602, -4.472573935313164, -0.7606283194844821, 10.554588456469816, -7.110340516777626, 6.34828517063039], 'changes_betw': [-0.003020888953610801, -0.0001015322880766889, -0.012821788586314498, 0.0, -0.0010577965470931483, 0.0, 0.0, 0.0, -0.0012713784120511329, 0.0, 0.007335808941313443, 0.006174053039496436, 0.0, 0.0, 0.00039399381907043174, 6.795786612300405e-05, 0.0, 0.0, 0.0005072569292754181, -0.00014724204326650747, 0.00021034577609501184, 0.0, -0.03774088636473958, 0.03846819733669884, -0.00016989466530750903, 0.0027660469556493594, 8.777891040895325e-05, 0.0, 0.0, -2.8315777551232713e-05, -0.0009133860815818129, 0.0, 0.0010181544585214503, 0.0, 0.0015100399656975683, 0.0, 0.0012341633901266358, -0.0075692118505574085, 0.0010683138358980604, 0.0015925602317038712, 0.0, 0.0076755982719284715, 0.007505703606620964, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0014619031438602992, 0.0, 0.0, 0.0020646246945942304, -8.494733265375452e-05, 0.0, 0.0, 0.0, 8.009319935928239e-05, 0.0, 0.0, 0.0030795430642524546, 0.0, 8.93969548403789e-05, 0.0, 0.001012491303011398, 0.0011419348575312377, 0.0017082504085561834, 8.494733265375495e-05, 0.0, 0.0, 0.001275019012022073, 0.0, 0.0, 0.0002592916201478897, -8.939695484038238e-05, 8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00017434428749413602, 0.0, 0.0, 0.0, 2.831577755125353e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -2.831577755123965e-05, -2.8315777551267407e-05, -2.831577755124659e-05, 0.0, 0.0, 2.831577755125353e-05, 0.0, 2.831577755124659e-05, 2.831577755124659e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (88, 87): {'changes': [0.01127179698812597, 0.07081924135276907, -5.272606624813591, 0.752474283685828, 2.91128801315773, 7.29454742172959, 5.710507700733842, -2.6130711795165666, 0.17117941728974273, -7.494096359256332, -3.758010532666148, -23.824215254714943, 5.9017383523215585, 16.330438957035568, 1.4466540880952365, 22.91686334974142, 13.834589022923268, 1.728412021469893, 3.2255626944735774, 9.628512697390732, 0.21728849556058094, 7.361168205658004, -53.98436607845615, 142.6388494783461, -0.4574369368373823, -4.719894951341132, 6.102346809188525, 14.119657905979437, 10.891905091672754, 1.6983044249817425, 2.998806102285357, 7.339912677954658, -0.1191693649285952, 1.8716170070939029, 0.49435765303548607, 0.8039306092217089, 0.7707166513308792, -2.483737539288228, -0.47523160582289137, 0.5060814179952686, 8.837250407423483, -2.851500376791023, -1.0096966019614229, 15.398809223328328, 8.937163224734718, 10.038735703951033, 0.06822730650148401, -1.042831524791879, 1.1502322933488287, 3.3492247229655163, -5.599094112420914, -0.6555015538883566, 9.378049161093799, 5.605307968853026, 6.803541730048323, 4.738262164595284, 2.340464143202471, -8.66249482980902, 0.7743342989314783, -1.8652268664185172, 6.395590449730918, 7.3308872360398425, 13.106363171803963, -0.7489313377239668, -0.4186421578968833, 1.9714416785222966, 8.869462387572554, 7.1937015056839755, 0.5073843796528763, -3.85186952508424, 1.704089178656261, -6.203396367475733, 12.155453872225223, -9.685570875648182, 1.7127512151782582, -2.6625498742046148, -1.5654965270053367, 11.452503250847514, 3.65159570047598, 12.871626692717115, 4.888066884927298, -5.2585480875996495, 4.818687258470618, 10.420356640951269, 13.64878984673939, -0.23141869015177008, 21.197539073783815, 8.754469744152061, -0.9844830259174273, 1.9934130710290034, 1.1653552071661295, -1.6736088245285714, 4.401073902639439, 2.4342326712107933, 5.303051873423499, 6.860558763740954, 2.784476354069824, -3.3734659972382843, -4.2892500458384575, 3.374608530396472, -9.119761601303253, -0.5805407093227188, 8.770703025937792, 2.132562532736017, -2.6583993312134453, 2.0213871691176735, -6.6061017453914985, -2.838616071931426, 3.0639979626011495, 1.1060663435836773], 'changes_betw': [-0.003020888953610801, -0.0001015322880766889, -0.012821788586314498, 0.0, -0.0010577965470931483, 0.0, 0.0, 0.0, -0.0012713784120511329, 0.0, 0.007335808941313443, 0.006174053039496436, 0.0, 0.0, 0.00039399381907043174, 6.795786612300405e-05, 0.0, 0.0, 0.0005072569292754181, -0.00014724204326650747, 0.00021034577609501184, 0.0, 0.03846819733669884, -0.03774088636473958, -0.00016989466530750903, 0.0027660469556493594, 8.777891040895325e-05, 0.0, 0.0, -2.8315777551232713e-05, -0.0009133860815818129, 0.0, 0.0010181544585214503, 0.0, 0.0015100399656975683, 0.0, 0.0012341633901266358, -0.0075692118505574085, 0.0010683138358980604, 0.0015925602317038712, 0.0, 0.0076755982719284715, 0.007505703606620964, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0014619031438602992, 0.0, 0.0, 0.0020646246945942304, -8.494733265375452e-05, 0.0, 0.0, 0.0, 8.009319935928239e-05, 0.0, 0.0, 0.0030795430642524546, 0.0, 8.93969548403789e-05, 0.0, 0.001012491303011398, 0.0011419348575312377, 0.0017082504085561834, 8.494733265375495e-05, 0.0, 0.0, 0.001275019012022073, 0.0, 0.0, 0.0002592916201478897, -8.939695484038238e-05, 8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00017434428749413602, 0.0, 0.0, 0.0, 2.831577755125353e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -2.831577755123965e-05, -2.8315777551267407e-05, -2.831577755124659e-05, 0.0, 0.0, 2.831577755125353e-05, 0.0, 2.831577755124659e-05, 2.831577755124659e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (88, 48): {'changes': [-1.2385819113848502, 1.5439171769104227, 6.986521877429929, -0.16236116157861602, -0.7927205431990814, -4.007266993502014, 1.3148387194500515, 4.237970616617474, -0.18699865675949923, -1.660180239307408, -2.3856692247047846, -26.28152865378007, 8.053191578013013, -14.473369246934112, -0.6394407260020571, 13.52863386741646, -2.6432138170052895, -5.121650575997904, -2.190999910321736, -0.173988456083066, 4.109807136056958, -8.1961337199007, -14.165530012565483, -7.444676830364102, 4.035565173099741, -6.848455045651946, -2.944576895368783, -1.4462097627285573, 8.984897448170926, 3.1661123113323697, -0.04109617151711831, -2.2757737784723133, -3.6250848951318204, 6.488929623866682, -2.4712101572577936, 6.944406284121072, -0.4536582013122157, -6.721250916683303, 1.0703552076444467, -1.658099676194027, -10.411617026920851, -0.7061822867257774, 0.8734791766629684, -15.110741067557115, 2.5584164645613896, 16.905024355385336, 3.2474389936144235, -0.10390431091700236, -7.751009669677437, -7.143925708266693, 8.925490720092228, -1.9821037545690459, 1.0147188008176613, 12.103030692207454, 3.043808033740021, -5.596329239542911, -1.010137595019856, -6.695405818034857, 2.6603996642239736, -1.4765205510127402, 4.391947537706557, 7.240279075730399, 10.020514953652139, -0.8534932362674503, 1.801243003370935, -1.3717687318132974, 17.53905283289791, 5.052006873723315, 20.037740446885437, 8.686733311141936, 5.812417596161936, 1.2161870111018516, -7.766339165001227, -2.6269524657479337, 2.087490571261668, -2.248565725711103, -4.0055489835065, -7.16101871696992, -1.4297316695241307, -2.318027931503906, 20.084156167832447, 4.940358191815449, -3.068905055615801, -4.39257628433225, 8.074099392988927, -2.2636819506505788, 1.3276813323877832, 15.388449505737583, 0.8323735837830668, -1.9800696420512551, 8.022099141832896, 7.606905889662642, 15.09624843829485, 12.296601240082737, -1.3944656798661867, 5.03869083339535, -3.366471115183117, 3.8092319246332806, 9.563837512823994, 3.9345805035791273, 9.393549561079396, 14.127964155825822, 13.310493064279115, 5.461620791437497, 3.888385928597444, 12.840513608350562, 1.4834570168764287, 12.633481080945558, 0.9411714588548534, 6.672497473416826], 'changes_betw': [0.00877425044091723, -0.0011281814798634882, -0.013764703978771248, 0.0003256314418393774, -0.001019367991845016, 0.0, 0.0, 0.0, 0.0009212740481853721, 0.0, 0.008224722100868859, 0.012270237690726996, 0.0, 0.0, -0.0014396550329272284, 0.0, 0.0, 1.4157888775625464e-05, -0.001241444590068419, -0.0002973156642881406, 6.937365500057219e-05, -6.937365500056698e-05, -0.00014441046551138398, -0.00014441046551138398, -0.006920780544633756, 2.831577755124659e-05, -0.0001698946653075073, 0.0, 0.0, -0.00016989466530752118, -0.0010193679918450577, 0.0, 0.0008494733265375365, 0.0, 0.0, 0.0, 0.002337265181301895, 0.0005153471514327823, 0.0, 0.0014821286992540705, 0.0, 0.0020824231833406343, 0.0015161076323155914, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00028396679772832445, -4.449622186625344e-05, 0.0, 0.0, 0.0, 0.002025589372684211, 1.4157888775626331e-05, 0.0, 0.0010913709690467954, -5.663155510250272e-05, -0.00016423150979725382, 0.0, 0.0010193679918450438, 0.0, 0.0, 7.92841771435045e-05, 0.0, 0.0, 0.0011043153244988113, 0.0, 0.0, 7.928417714350364e-05, 0.00016423150979725729, 0.00016423150979725902, 0.0, 0.0001698946653075073, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0001698946653075073, -8.494733265375538e-05, 0.0, 0.0, 0.00016989466530753505, 0.0, 0.0, 0.00016989466530751077, 0.0, 0.0, 0.0, 0.0, -0.00016989466530751424, -0.0001698946653075073, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (75, 73): {'changes': [0.03986271944500075, -1.7005103506924577, -0.9394777972830326, 5.020771159729037, 2.4816926362157474, -76.31658428099225, -9.800900584572418, -9.502766339856095, 1.468775832061631, -6.571047001677755, 3.5324064852493677, 7.813259860481281, 3.0123946529566865, -1.1721321702494834, 0.15048929256943566, -10.392447862637596, -4.513796228482249, -1.622660245779599, 2.268633440634762, 13.002196971852186, -2.3376591198337735, 2.223470041621141, -7.550755491820411, -4.453837944846924, 4.868458878373673, 1.9086153982910616, 0.12019519587926197, 4.935553839942088, 4.2232128331301055, -4.671968643634457, 1.0032703810841497, -8.001698856302426, -0.6326397513729916, -4.326750367906186, -2.351631156778751, 20.127384406181392, 0.3442247983992601, 2.2792649741989948, 0.14450436501668662, -2.0554762010928513, 6.488509088745758, 1.3627864753111822, 3.33834126191762, 7.923876796665439, 8.830335461836682, 15.020936459350878, -2.4555793906807963, 4.741940005712166, -2.10227372745571, 3.191495808842461, 2.8132023284958336, 2.700503273982072, 3.7891172742045853, 11.37810631952351, 8.405295584621967, 13.50195219812241, 1.3104392991953162, 11.360892049031861, 3.388210988383662, -1.114748713268689, 6.639903405790221, 1.738345435176143, 0.21354168075515645, 0.9114423673014862, -3.0010018342246596, 2.8403371894795413, 3.1896803639876055, 7.281980629447844, 6.410882720049557, -1.241839975312928, 6.69554357447123, -3.2217999740466894, 3.135457822741756, 3.161397602762065, -0.3880447826282136, 6.916536520446584, 6.12186129851645, -8.054264512010377, 8.198182726606092, 7.570464369447592, -2.798262169652048, -2.3428882506006516, 5.987716188567589, 0.14136534988045923, 2.9152888923159708, 0.10545763071722547, 0.7362254968828097, 2.334417340858124, 5.869187664232697, -2.3508822223323236, 3.9946322932312057, -6.227193125004135, -1.4968342793663396, -2.2061062583464945, 6.195526042124328, -1.1968988866159123, 7.8560268342915975, 1.3359383484225305, 5.806407042330761, 3.710873552059468, 3.1419587196730276, 5.538628668910576, -0.8286664425738763, -2.21121227384927, 7.346693249229787, -2.6102034369517355, -1.3543040769824017, 5.842570731055304, -24.05076459654103, -7.957815037129421], 'changes_betw': [-0.00016989466530753505, 0.0, 0.0, -0.00016989466530752118, 0.0, 0.035507985049269455, 0.0, 0.00016989466530750936, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (75, 65): {'changes': [0.027237567385771655, 1.445328945625839, -1.285409113415838, 17.69629145303786, 2.378097600906038, -4.551714470682356, 0.47017330533577706, 9.812003352726293, 0.23556211731934695, 0.11316567461449267, -1.3787076380250625, 7.001102308818076, 10.191863863491506, -1.3320189805531015, 0.4734686622468871, 6.804764574196241, -1.2292829067920934, -2.2610925643959376, -0.7344739170508952, -1.989108463912487, 68.73289983573422, 6.74349401263305, 1.2144003067526512, 0.7020428955554507, 9.896704999896315, 1.6022980663151003, 0.7621433048716071, 11.99240873777211, -3.1424995756985936, 0.8380585869793151, 2.506096251817546, 5.707912469374747, -0.46398702284639626, -11.07993242851876, 3.5468392567824196, 4.409936508643739, 0.5259601863454542, 6.141608426175054, 1.5464036719814658, -2.333627674391515, 2.2125921078089164, 1.0916396943952549, -0.9658493379014743, -1.7957038194010693, -1.268745061092659, -1.6059021051068498, 0.2464705319466134, 4.762367822828821, -0.20317719380621924, 3.067013483779931, 1.9404979975602714, 1.470204501364158, -7.42639058560988, -7.394087414358353, -2.7972209323732216, 12.911210333187, 0.9448757892312045, 4.433030152617761, 11.590352286874577, -0.6008132172436689, -9.258307756980741, 1.629450504927675, 5.814786869618075, -0.09562180138696874, -1.1818279342126559, -1.6292993491834835, 9.895444934294972, 9.958926437700086, -7.2400503428671925, -5.3204526341422, -1.9211175053243608, -1.0177048660902415, 7.141770738099751, 9.936042205113097, 16.76863143510127, 5.978993202835056, 2.406585520498737, 4.970378800276649, -1.2571920167150097, 3.06750797437644, 0.22683157815549748, -8.305085570286849, -10.456062349233946, -6.406173251807871, -1.9983696831530722, 8.20136674843856, -1.9495462009734297, 0.9848688262472081, 29.688019429723965, 2.540465800672415, 2.2068978713026297, 7.469391020669633, -1.0280297867296042, -1.2616624426207181, 1.6862260762688948, 12.66833718414506, -1.0685721306385574, -1.2134657576260395, -13.943766336921101, 2.7160976853724037, 5.75904613211074, -5.917693436413572, 0.35197372799041204, 4.660424997794138, -1.7038310066888016, 5.19894566252799, 10.395435913029957, -0.3761283403201219, 0.008528002428761283, -4.984325038275756], 'changes_betw': [0.013406711648301928, 0.008703056485931171, 0.0007511771273239487, -0.0075178389398573575, 0.0, -0.00033978933061502153, 0.00033978933061501806, 0.0, -1.8405255408304733e-05, 0.0, 0.0002368412536608161, 6.411501059819594e-05, 0.0, 0.0, -0.0006553079947574925, 0.0, 0.0, 0.0, -0.0006553079947575202, 0.0, -0.013481748458812674, 0.0019410465511383009, 0.0002184359982525183, 0.0002184359982525183, 0.00017272624306263317, 0.00043687199650502273, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000655307994757437, 0.0004368719965050366, 0.0, 7.281199941747141e-05, 0.0, 0.00021843599825252524, 0.00021843599825251137, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0006553079947575202, 0.0, 0.0, 0.0, 0.0, -1.8405255408276977e-05, 0.0, 0.0, -0.0005824959953400627, 0.0, -7.281199941749222e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -7.281199941750437e-05, 7.28119994175061e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -7.281199941750437e-05, 0.0, -0.00028315777551251556, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (75, 62): {'changes': [0.4425221577773737, 1.6565168611848549, 1.5186634018437957, 6.949774846630575, -1.5451448042375873, 0.25035743758417084, -51.96772229668855, -72.91139610520352, 0.3450147624297699, 4.154554077250907, 2.836096409168505, -0.48895213376675883, 12.411561507415172, 13.195040968205916, 0.001099810792684508, -6.9374665230812695, -6.797784978078937, -4.538201858245799, -0.6455482079863408, -2.950417191195328, 6.663788615533761, -5.8940902322883915, 10.645956900680972, -1.2296775784648446, -9.667044803118586, -0.41758077589555853, 4.159727093877983, 5.0252111626039095, 2.4400139464252106, -4.59343030813794, 2.1295611482714705, 0.946539676926875, 0.6671458229363765, 10.722211181477633, 1.2122292876005645, -2.876790911469925, -0.5083044268656813, 0.5111254145277542, -1.5388351077330853, -2.44757913218681, 2.8066669886808313, -1.251729456402174, 0.19840749291911663, 1.9825025691881137, 1.767359042460896, 11.079592500381125, -4.095708933939804, -1.3498685875940453, -2.053888759751345, 1.6383981393368572, 4.638891637917936, 0.9786340580544746, -1.228633510211239, 13.416595418797897, 3.1774969312523353, -9.024422347963224, 1.0156749979508568, -5.063703810232056, 13.156292795967147, -1.7421772738773313, -3.013032789429616, 0.23858332875511223, -9.650101749125326, -0.335128659770735, 0.9364921812466989, -1.1332808539915789, -4.147806385810043, 7.485809079841317, -7.757413952535899, 6.468928589510881, 6.041049985157713, 2.879946863480086, -6.258662407944797, -3.1406865050208523, -4.357725356699689, 4.090982615478026, 0.7960646499238777, -15.532943258231512, -8.218856771888085, -66.6749035958123, 5.320571628378502, 2.332310577038413, -10.057942313657918, -1.216807367866977, 3.4329811727828883, -2.1024588568117792, -1.8227297515228997, -4.473870374250623, -10.127226232073866, 0.7156265090501961, 5.812048985734123, -2.723227494895241, 6.796127254614987, 0.6655143304433864, -11.055212643405213, 3.409779442421467, 15.67243167031475, -0.7580389557691802, 3.3776966111219053, 2.375491218283287, 4.212655051700267, 7.673880866757173, -3.508501054892861, 4.46581170427865, -0.8186652932801053, 4.996545527351969, -10.900339327028007, -2.902038846424972, 2.9399581532055095, -2.278132935677206], 'changes_betw': [0.0018913591033875576, 3.236088862995645e-05, 9.924005846534412e-05, -0.01577566353305397, 0.0, -0.0003256314418393913, 0.015959716087137088, 0.0156340846452977, 0.0, 0.0, 3.829371821217897e-05, 3.8293718212161626e-05, 0.0, 0.0, -0.00011488115463645365, 0.0, 0.0, 0.0, -0.00011488115463648141, 0.0, -0.0016082013278751403, 4.247366632687769e-05, 3.8293718212165095e-05, 3.8293718212165095e-05, 0.0, 7.658743642433019e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00011488115463645365, 7.658743642431631e-05, 0.0, 1.2135333236240609e-05, 0.0, 3.829371821217897e-05, 3.8293718212165095e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00011488115463648141, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00010274582140026856, 0.0, -1.2135333236254486e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.213533323625015e-05, 1.2135333236251017e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.019888058292747396, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.2135333236251017e-05, 0.0, -2.8315777551251523e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (82, 33): {'changes': [0.889560258150512, 2.1341119367423715, 6.479999268289504, 2.5321786774640316, 151.33206045575753, 7.368451086986141, -1.6159315862499568, 18.585200836644418, -19.792360360401723, 12.852277296107161, -4.5397101425917015, 19.898364794001196, 22.010924199264792, 30.103910687288817, 1.0174789424268162, 12.857952400393856, 17.73372095290182, 9.106922750427913, 1.6780220388030784, 2.157577055459626, 13.326921902652543, 13.920897522571295, 14.67007102085799, 15.442502497303877, 21.807639286095878, 10.102532839942, 2.2404164325525713, 15.236180895674522, 3.701245132910344, 21.983908627446908, 225.10902427003802, 9.735625587388654, -61.72971982754806, 5.1871581220481175, -3.4973927884615534, 0.4334848477469677, 0.880626421816789, 13.27664647901959, -2.5319138080997945, 2.36773010717895, 23.101504297947713, -0.12614183907733434, -0.9216202431758944, 17.55163213034018, 7.922534161092642, 7.056092756334635, 5.7766102892657045, 8.155518395083234, -3.620064559372814, 13.128953389185256, 28.218780785549995, 3.688312843954648, 7.985739633813097, 1.6512391461922107, 11.896645620850506, 11.243041579647524, -13.98363354506068, 9.360814450115356, 12.354460262189036, 4.473099772475884, 8.854482710037473, 4.100575862773525, 15.697535201733047, -7.295768841934777, -3.829228623976128, -6.3920532738994, 24.87918747703344, 11.342526281243636, 16.098167280637654, -15.503568308705198, 32.20047751239926, 13.534969702301538, 10.427440912509752, 3.753644661704527, -13.632348540105227, 10.02772026615338, -0.519207498957968, 5.721787699276888, 19.802371708821624, 4.3335738703208335, 20.26776801021464, 17.418136853349978, 9.350520401475364, -1.194231512306743, 14.21974097644383, -3.8774137992825928, 7.6444686917438105, 15.372770712850638, 24.8736020243189, -1.2551679986572708, 0.25605556483634473, 28.835864595371163, -22.83400064315913, 17.660918868501255, 12.949522452927226, 14.09475919377249, 20.72412123137383, 54.01712656742349, 72.61189815071202, 28.83622888499741, -9.532950549291968, 12.008095818874978, -3.247570968742764, 2.134497992836856, 16.290453416734067, 10.739395894523113, 9.54751450680891, 36.315993349157964, 20.343223023626052, -2.7679933713178], 'changes_betw': [-0.014411517240263239, -0.007603999805834871, -0.009948546187078328, 0.0, -0.12358016601135863, 0.0, 0.0, 0.0, 0.10617283950617273, 0.0, 0.008870119573483437, -0.005908693752730447, 0.0, 0.0, -0.007603999805834538, 0.0, 0.0, 0.0, -0.00760399980583451, 0.0, 0.0, 0.0, -0.003954905101694106, -0.003954905101694106, 0.0, -0.007909810203388212, -0.016392003624419504, 0.0, 0.0, -0.020684675501189256, -0.12083070401113212, 0.0, 0.10102948076954192, 0.0, 0.009786741743928351, 0.0, 0.008660178308496191, -0.005587916444185562, 0.008741889552287108, 0.0060247884406905705, 0.0, 0.005727068265294566, 0.005727068265294552, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009837710143520556, 0.0, 0.0, -0.007603999805834594, 0.0, 0.0, 0.0, 0.0, 0.12233912593239799, 0.0, 0.0, -0.0009987379253434403, -0.0005804734398006566, -0.0018057375855541277, 0.0, 0.12907909001181173, 0.009786741743928323, 0.009786741743928282, 0.0, 0.0, 0.0, 0.014778813326213926, 0.0, 0.0, -0.00027668559778651694, 0.0018057375855541381, 0.001529051987767576, 0.0, 0.01656472986748217, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.016564729867482156, -0.0018057375855540965, 0.0, 0.0, 0.01656472986748217, 0.0041199456337071075, 0.0, 0.016139993204213388, 0.0, 0.0, 0.0, -0.004544682296975869, -0.020684675501189277, -0.020684675501189263, 0.0, 0.0, 0.004006682523502095, -0.00011326311020500631, 0.0041199456337071005, 0.004119945633707097, 0.0, 0.0, 0.0, 0.0, 0.0]}, (82, 59): {'changes': [0.41443570451422573, 0.5505983555200267, 5.830300082721145, -5.074293385682566, -1.7167912392211662, -16.8784154599781, 9.925717696105863, -15.436671246030386, -0.0031324198318571916, -8.166185640632477, 5.780416032147031, 3.0134968303508742, 4.561478549954103, -15.341228004376944, 2.447907032308809, -2.152165183353361, -5.705650594061069, -3.113975444179829, -0.22798632740667202, -6.547696590406929, -6.978956123014555, -3.592547745833599, 8.135179605901271, 21.023036617467, 9.158669918172023, 5.919712685933703, 0.998460224470314, 8.939333440999206, 1.9543400638245316, 1.4743885444350582, -0.38745660857991737, 4.651384818267161, 4.519089004504849, 1.2782894660956003, -0.8906456218451879, -13.305459337404557, 0.27136071840394393, 1.7901206910848089, 0.28167312111538934, -0.5926348183414305, -0.6228260447954312, 0.9654687301904943, -0.6374336809333983, -7.761899781650584, -0.33007299028611214, 7.457545108919021, 0.5032530960026236, 0.7337555040697197, -3.1697200834822112, 2.1252395179216705, -4.814742600437171, -1.80466932309362, -8.398052689075143, -26.430412997510643, -1.6889504950297578, -6.372847620521441, 0.9667882854926937, -22.883424452601474, -11.048615065568242, -1.088333886954139, -0.9101673722570354, -2.182953980342077, 4.356402167186133, 0.10994687166316552, 3.845217065202789, -0.3989666002764878, -5.714194718117227, -2.588798744008784, 0.5745697594987291, 0.9005404001590449, -11.358203850317182, -8.608790909653635, -7.866165315438735, 1.1837352446151783, -9.61074697451599, -6.389184915110263, 0.11719798782448265, -2.1709741752049467, 3.7832381657753444, -12.883185636787772, 5.492845661471733, -3.495235762335966, 3.9286110739625144, 10.275182035552433, -7.079681855481994, 4.759058157020732, 3.3159786261885245, 13.191720660895015, 1.342369959023614, 2.1565570056952623, 1.8847919526372436, -1.6079389483736009, 4.039682503351685, -13.316375340820116, 14.070865051149099, 0.7727081071916189, -3.679099973137909, 3.221165138171145, -4.203776742788563, -9.015881742898245, 9.539109571510949, -3.082436934438334, 0.7523800887620951, 7.477110524265441, 3.046006888128801, -4.40545832742356, 4.9641084764514005, 7.330088934531091, -8.315619261915742, -7.576364721148764], 'changes_betw': [0.0029731566428815026, 0.0, 0.0, 0.0, -0.00025484199796124707, 0.0, 0.0, 0.0, 0.0002548419979612748, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0002548419979612748, 0.0, 0.0, -0.00016989466530752118, -0.00025484199796126095, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0002548419979612748, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0002548419979612193, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.494733265375365e-05, 0.0, 0.0, 0.0, 8.494733265376753e-05, 8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0001698946653075073, -0.00016989466530751424, -0.0001698946653075073, 0.0, 0.0, 8.494733265375365e-05, 0.0, 8.494733265375365e-05, 8.494733265375712e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (62, '85_62_2'): {'changes': [0.14789309652863203, 1.7400844094452204, -2.7346442979718546, -2.561117802909834, 2.1250299501504486, 11.262607390449574, -11.139195773746792, -4.2298848265674565, 2.1776355503816447, -7.5840015854187754, 1.8861068380609396, 8.522343609928072, -10.082864711828023, 13.686463416812273, -0.7197349749584738, 5.629372627031046, -1.243731832788285, -4.532524482040344, -0.14945057995369737, 10.884882532614682, 12.031781214251268, -3.2694104355385036, -0.24195913160525606, 5.390185151811835, 4.002930908238142, 0.4694922431353916, 0.9871121963706386, 14.847241514798952, -6.600187395246735, 4.421131037742214, 0.2100676954111691, 6.370970707299364, 2.490794636123425, -2.749342860912975, 2.4892743606665633, 4.168379924530086, -1.142577345901742, -4.460560128312807, -0.7631262337891442, 0.3441542011637324, 2.9450914210842143, 8.84469972512116, -0.7192792379011905, 6.3495582468261205, 5.269073595962766, -2.522875189608726, -4.962344450226809, 2.112988785634002, 3.7092414726410894, -2.9803475196357, -4.000035816942699, 0.05361448964180937, 9.890814398886079, -8.001247326369537, -1.5282187208587175, -5.143388017995619, -1.2667492071892, 11.449531093679184, -3.0365647245681657, -0.27939698525012346, 12.15959442517618, -0.4612522321090893, -1.8694970532197033, 1.331567132059039, 1.4773412847484408, 1.8523285845430948, -4.7975424812125596, -11.151592589483016, -1.012293861219689, 3.2151156661374927, -5.291142056932415, -3.1343401635263035, 14.998718647643472, -6.771399243457523, -3.0425508871160787, -11.355478504883479, 2.3937526567920173, -10.465047441004543, -14.433590846998413, -5.432947285256546, -0.6447308449070874, -2.3740246634337154, 0.13163397589900683, -3.8324209644526093, -4.924467707938646, -6.321928631351454, 5.733195713492279, -10.626761791261714, 10.140167496850324, -3.451313392866581, 4.564912281007167, -14.686594348452473, -18.38595385172468, -10.72348222397568, -4.5915245845278605, -2.194099538523261, -0.28499965500580515, -4.166029119531913, -0.06099306828267004, -0.9308330741483246, 6.5276034842323725, 0.8409416101304998, 0.8120757290750191, -0.018818518202266432, 1.3841255733804019, -3.836876039654811, -6.231949035260328, 6.494966540885088, 3.7256816658049843, -11.40058443719073], 'changes_betw': [0.001698946653075073, 0.00019821044285872613, 0.0, 0.0009910522142937833, 0.0, -0.0010193679918450577, 0.001189262657152565, -0.0005096839959225281, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -5.663155510250012e-05, 5.663155510250359e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (62, 73): {'changes': [0.8267608607301931, -0.33535884704605934, 1.6406208382487364, -3.3648295273387845, 0.537921016294014, -5.752018537260824, -4.925534173753761, -2.0150189624746986, 3.513169277932178, -24.57789349215963, 1.1303321486255697, 2.9544227670721455, 0.6720646819633771, -6.983381452122757, -0.19622975699634182, -3.693332836615866, -8.145269234111424, -1.154604320282914, 0.41835334574527394, 2.006769649045367, 1.4703483128095058, 4.421797013844241, 5.790400097369002, -0.3347398424111532, -0.8104556792582969, 1.4957357357624659, -1.7980878129749627, -15.95772581191352, -14.548726871187625, 2.0858593782917723, 1.9350449581771372, 3.6795970394574624, 0.5599448125212092, -8.599636260884239, 0.09439026438831633, 4.876584814144167, -1.5522244696536625, 0.730162166852395, -0.19099450565104803, 3.0086704691243114, -5.865454535054255, -0.8783542810041922, -1.799560466544179, -7.244599439540707, -6.752763255947912, -4.953974486735206, 1.2645897484238304, 1.9672798258945932, -2.0766009117145074, -6.2954149026632535, -7.766117151502101, -2.634508343973458, 9.675163404993242, -8.207266789164976, -2.6311843530681074, -0.7769756022040042, 0.7493095629916873, -1.3739493216812946, -9.469347688301923, -2.8832826088959465, -9.782113900744974, -0.03869373149146327, -6.195741748506805, -0.12344028249633965, -1.8421909732750805, -3.3274028963652, -1.7495031162376335, -9.485204406453079, -6.678055161577902, 3.5235160220773736, 0.2234480292323724, -1.6363589122108237, 2.371235374939232, 3.862704892328196, 18.818067212996937, 4.058038016361195, -3.31855367625478, -11.33374489510166, -6.091314477251558, 2.893977514636788, -12.49742871964446, -3.6398698158409957, -6.466405991505582, -1.0808072922913254, 4.326528214074131, 0.5454813744711515, -5.683704879963358, -9.83701899109306, -1.390460997675632, -1.2578235651318934, 9.3868366677533, 9.86003746936143, -2.8073822401392476, -6.654411714401078, -0.9844403988443844, -3.333221488389995, 0.4680208096379488, 6.108763949705008, 3.7646576768507884, -5.564046932917336, 4.315294094428111, -9.73506527527013, -11.320493425606628, 6.3397469762018375, -4.807808962598926, 2.8947447889597697, -1.2626930475880158, 4.468725258271604, -5.006404844525662, -5.123390664266481], 'changes_betw': [0.00016989466530753505, 0.0, 0.0, 0.001189262657152565, 0.0, -0.00016989466530751077, 0.0, -0.00016989466530750936, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, ('85_62_1', '85_62_2'): {'changes': [0.1591885803956572, 2.1435464208322514, 0.280003799550272, -17.33434196597463, 1.912753235834515, -48.8814578479913, 61.47592931297629, 7.423158749205413, -1.1205483699582715, -4.886604845188657, 3.0568373500712767, 10.477604363851441, -6.27169267978104, -5.038160832586925, -0.26412265835842064, 10.265530132279764, 4.2647223988983, 12.598419017505108, 0.37747483661017966, -8.299349760896376, -12.754716726307734, -2.9324194049874563, -1.9759578883437996, -0.06680869289283464, 12.592472121212296, 1.121354103413438, -1.6550619970934264, 1.218156962296348, -15.653493831023269, -0.2278384937413449, 0.43564648707321396, 1.2835178812321146, 8.145128275880467, -11.795359218229692, 1.6744289586322267, -4.8015946633617546, 1.1113675947818855, 4.264983398579616, -0.09698544571281076, 0.8180597845108863, 10.511693530389266, -4.482385410801058, 1.8318027823422085, 7.952160983211684, -9.504438849199175, -3.762977558904197, 0.42890458093864936, -1.2115103688183808, 0.7221676737032539, 1.6914174706669485, -6.255239102301346, -2.179438752656523, -0.1812513125725559, 3.3422895985900993, -1.1132364624648972, -0.48697034206008993, -2.099758555648158, -10.711540450801579, -7.962100362371416, 2.668331894162982, -5.0991113980749105, 1.5150186892540631, -6.061286557302424, -0.509195044355117, 0.04255917048543978, 1.1738355862056764, 9.217623745917876, -16.381259463764593, -4.904181717107235, 3.6661266419183534, 14.160389170088536, 1.8432364923550608, -3.4496352260906633, 2.8619143002112253, 3.040068864883864, 8.133519896200482, 3.7830197854489995, 7.355662919282793, 3.5693511322399445, 3.790847735755875, -5.894113634966459, -6.048065403884436, 5.392438724508821, 3.640430788674479, 4.320911660034113, 4.307241460226535, 8.598769692009654, -19.199492582622355, -2.0952100299055303, 3.1385523565034887, 1.599574925908911, -6.790620612869816, -3.9081583675611853, -7.6557116355555195, 7.813453816888739, -4.982633335671892, 3.9237451998311172, 2.590685047870238, 2.8043386173318936, -11.74443025387697, 2.711237517919585, 4.348601450088466, 9.688299118504176, 2.4020354044315866, 6.597119124533634, -6.191576898781136, 5.404691095160587, -4.080863113784744, 1.7412578075419844, 9.682838533738277], 'changes_betw': [-0.0016285617203047886, 0.00010274582140029631, -0.00011569017685224281, 0.017838939857288433, 0.0, 0.017159361196058445, -0.017159361196058445, -0.0005096839959225281, 0.0, 0.0, -4.3687199650518926e-05, -4.368719965050852e-05, 0.0, 0.0, 0.00013106159895151515, 0.0, 0.0, 0.0, 0.00013106159895151515, 0.0, 0.0023081403815348976, 0.0, -4.368719965050505e-05, -4.368719965050505e-05, 0.0, -8.73743993010101e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00013106159895151515, -8.737439930102398e-05, 0.0, -1.4562399883510935e-05, 0.0, -4.368719965050505e-05, -4.3687199650518926e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00013106159895151515, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00011649919906800421, 0.0, 1.4562399883503996e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4562399883500526e-05, -1.4562399883497057e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4562399883500526e-05, 0.0, 5.66315551025031e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (47, 58): {'changes': [3.7426730582345353, 0.6290243870931036, -16.457657469856755, 6.273463565179142, -13.850107648257712, 3.6240521150541554, 20.832137006733547, 2.3951381449061273, 237.01534401411183, 3.7993141074959453, -31.91983547309828, -79.89267530613418, 3.1609517661546533, 15.677364403037444, 2.6873470872566827, 13.465999857215024, -9.210529268382373, 13.329762119683267, 2.4741409985816922, -8.732344612469802, 1.9075027096337465, 4.256097466527052, 3.9693932540133687, 0.14742660383708994, -124.8702110324092, 0.9629201740970217, -14.111384111309736, 13.617168505161459, 6.591938515975983, 0.06921387853881811, -17.75584766989966, 14.14928523931735, -37.71584496243386, 13.669418737792881, -1.1199904782421584, 8.52841742239599, -1.258320754356081, -2.9067316185874432, -1.5851532432594837, -3.4246361800525946, 9.058789194868723, 6.4803655005862595, 6.190317097854617, 1.9820155137468873, -0.13116510660421454, 22.37297360357411, 8.76210714506361, 10.331674725189885, -4.606930751486367, 8.33370945424798, 16.69646977696314, 0.9853210025444454, 19.954397561007283, 2.151204998966193, 6.676881352365683, 5.512828921536766, 11.436422708907045, 9.290382916048884, 10.751308567698715, -1.4820320520042216, 4.198255305480757, 5.788608195707312, 3.5368071116955093, 0.7734177616921123, -2.1629888220894316, -3.7076669597484937, 4.424366882490233, 11.408052725259893, 29.038684315236225, -72.80340644286815, 14.501603994493678, 7.321543663940474, -9.82112033451483, 4.415696149288323, -9.151865976074276, -0.08076323204994651, 6.286643175334802, -4.888250906041947, 10.83040879219368, 7.951019137540726, -8.784046359382273, 2.7983197359356495, -4.120008991639224, 1.565948875029818, 5.808257141663574, 5.03758513742298, 8.777654761425794, -3.964544982386542, 16.185312024767597, 6.541267497446761, 14.972429815207335, 5.315260947951572, 17.795206826524378, -16.01016665871532, 2.3423005795836502, -9.96360287425503, -13.022579911178198, -16.053412110050857, -12.817279001312173, 2.3153441433753983, 11.960871224195387, 31.3195894733054, -27.13442767953933, 14.54658878443388, 24.12980691436556, 11.675314824773807, 8.700738495513974, 1.0292869178483102, 4.958550081857652, 7.438295429556376], 'changes_betw': [-0.06566645779184005, -0.019746025862233868, 0.023293514730823653, -0.0008973895059522508, 0.05436349809438515, 0.0, 0.0, 0.0, -0.16879500186075114, 0.0, 0.10215378262014356, 0.050141192763516904, 0.0, 0.0, -0.017242690484280787, 0.0, 0.0, 0.00013025257673575765, -0.014312007507726154, 0.001050515347151431, -0.00019911139941721293, 0.0001991113994172112, -0.000444557707554645, -0.000444557707554645, 0.040280149683819405, -0.0002661683089817679, 0.05794827553237353, 0.0, 0.0, 0.011545776683391351, 0.05851459108339853, 0.0, 0.04208434277241614, 0.0, 0.00532336617963533, 0.0, 0.024887950423118615, 0.003358251217578448, 0.014109751953788718, 0.01769291134734577, 0.0, 0.01083766160218766, 0.01083766160218766, 0.0, 0.0, 0.0, 0.0, 0.0, 0.013115868161739722, 0.0, 0.0, -0.00392537579081928, -0.00042675921880815006, 0.0, 0.0, 0.0, -0.07571541466801412, 0.00011892626571525632, 0.0, 0.012321408345873128, -0.0008777891040887996, -0.002098199116547725, 0.0, -0.02214437222082488, 0.001526220410012477, 0.010145543096613419, 0.0008862838373541733, 0.0, 0.0, 0.09853886910461826, 0.0, 0.0, 0.0008919469928644229, 0.0020981991165477283, 0.0021038622720579853, 0.0, -0.006484313059236621, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.006484313059236621, -0.0012119152791935754, 0.0, 0.0, -0.006484313059236579, -0.005061463624154744, 0.0, -0.00033978933061501806, 0.0, 0.0, 0.0, 0.01120598735277635, 0.011545776683391351, 0.011545776683391365, 0.0, 0.0, -0.0034686827500283145, 0.0015927808741264399, -0.005061463624154758, -0.005061463624154758, 0.0, 0.0, 0.0, 0.0, 0.0]}, (37, '85_37_1'): {'changes': [-0.1523153583638912, -0.13266544899948585, -10.044634529313761, -0.40217434367200156, -1.0008250843541546, -0.5121355297033574, -11.886959739416824, -0.5882475563762455, -1.4193525528896984, 8.119274553921457, 25.221699206511616, 106.45057796913451, 0.04077453995455471, 18.375697554621013, -0.21718726695266, 1.4290245884205603, -2.8573034030405324, 4.533055379769337, 0.18072422927795628, -1.4612395291010785, 5.169728198629784, -3.7685663693235654, -26.309091002381408, -33.97197656502756, 9.683159815092779, -13.719462588080404, 0.12073499677070743, -3.314267223899776, -0.6423595296475924, 0.7744126148540573, -1.0942545008571756, -9.325087512001659, 3.0521867950912736, -4.510882745095216, -0.8288573660806691, 19.643109818477654, 2.7572227310445, -17.219952456590036, -0.8816479904887942, -2.311252059155329, -0.18820396340117895, 27.25741566874997, 26.811331588102973, 11.60122210153719, 5.567361625954675, -5.24127794684091, -3.3950023120743253, 4.68151143934918, -1.8347708709683843, -12.780670744147187, -4.529038485119429, -0.4180784354372733, -11.51552521748647, -6.685644738837652, 2.617603175081598, 5.167525790873697, -0.9798625057738377, -4.105930099888667, 8.029543886336285, -1.7065691009122972, 4.254581333519752, -1.9180756555528404, -9.968640783008226, -0.5613736465076116, 3.3452092129686264, 3.7646329806321646, 4.5824119515030475, -5.939418708127334, 0.7768364697088543, -6.008172853105052, 9.007870300980755, 12.072436759025123, 11.802706526743293, -0.6000535058453238, 3.063320588011294, 6.4474495280197175, 1.8127202615760183, 8.100902940685899, 4.378041424887442, 2.9422864131262827, 7.25053208756799, 8.017909180154192, 4.406620869213697, -1.7769880389204218, -5.59850617126645, 1.93604254768249, -0.2837819041253056, 10.176830667529089, 7.229294207258647, 0.9422665303661617, 3.9252523382796767, 15.646591991565288, -4.563801581000462, 0.9394941268031403, 6.601018318811612, 17.549492193935635, -5.086530570524374, -6.379303091513421, 2.4033357178236088, 3.506807694151121, -11.06519157979477, 8.386052925312413, 3.3911276183491736, -2.7204523916772985, 9.853471816022449, 7.338785938991009, 1.1755988441771308, -3.8495798597362523, 2.740838620988484, 9.55580095863985], 'changes_betw': [-0.001569098587447193, 0.0017604323414721068, 0.026262074656570086, -0.00029165250877789817, -0.0035212691940520457, 0.0, 0.0, 0.0, 0.004186083199844648, 0.0, -0.0225072002977202, -0.030626951766095502, 0.0, 0.0, 0.0016103587204504832, 0.0, 0.0, 1.4157888775625464e-05, 0.0018085691633092094, 9.910522142937976e-05, 0.00015513000987007336, 5.521576622494022e-05, 0.0075756840282834095, 0.0075756840282834095, 0.010940811934695732, 0.015151368056566819, 0.0002293577981651862, 0.0, 0.0, 2.8315777551260468e-05, -0.003348542950989436, 0.0, 0.003594890215685334, 0.0, 0.0009437244146724422, 0.0, -0.0021880005824961857, 0.015038104946361819, 0.00013389317670670087, -2.588871090405953e-05, 0.0, -0.017719609080465375, -0.01791781952332413, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0005841140397715022, 0.0, 0.0, 0.002228047182175591, -1.6180444315001645e-05, 0.0, 0.0, 0.0, 0.0055658705888063376, 1.4157888775626331e-05, 0.0, 0.001982104428587539, 2.8315777551251795e-05, 0.00026778635341326296, 0.0, 0.00390070061323905, 0.0009437244146724422, 0.0008870928595698796, 2.265262204100135e-05, 0.0, 0.0, 0.001275019012022073, 0.0, 0.0, 0.000228144264841512, -0.0002677863534132664, -6.229471061275187e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0002904389754542682, 0.0, 0.0, 0.0, -2.831577755124659e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8315777551260468e-05, 2.831577755123965e-05, 2.831577755125353e-05, 0.0, 0.0, -2.831577755125006e-05, 0.0, -2.831577755125353e-05, -2.831577755125353e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (37, '52_37_2'): {'changes': [0.0923226513886597, 1.2137665357724714, -18.398521011053738, 3.6234446402896197, -0.0016792392779692022, 3.496286664196049, 10.414322265217208, 16.3688747086633, 2.9262475691557057, 0.7203763685237732, 116.55500592701806, 55.48967719119898, 6.84961964851874, 5.363143918984349, -1.9968418378358486, 24.575169748692616, 20.19895252576589, 0.8711750064820194, -0.2967176601485626, 13.890615981538616, 8.082282904776704, 12.430540934419753, -53.945016010682, -48.74348237886781, -20.405268616931636, -30.014401713817193, -1.6987541605741612, 13.85912666310594, 8.703096572980257, 7.45741554850045, 1.9416239040980585, -4.084052875342081, -5.60499769159253, -1.4261664876620443, -5.494159284206177, 2.619384051157624, -0.7413738483366217, -27.05155026764607, -6.099578042314491, -1.814814624303338, -0.2552468959586349, 123.45513071520004, 184.26816327768825, 5.502198323632683, 9.31267775438667, 11.000772576350641, 4.8131139368156965, 5.080255789185827, -11.162017366035002, 11.044073900887838, 15.535688511828084, 1.4787702873292616, 0.49452819364861966, -3.6954540345687406, 4.577447617713517, 11.4512014534061, 6.506755513142252, 20.194075853122257, 13.770958364230466, -3.4222808967893243, 12.487470662471452, 4.514680488356845, 8.564769958085265, -1.8517579818876442, -12.58053891793223, -10.315644607266606, 1.5365690015393625, 1.5710195424593394, 6.584738487606074, 97.99370371507432, 9.651765806456467, -2.1940615269057844, 9.044862322034675, 8.739137998839709, 20.134408470299235, 6.834937620672122, 0.9995842247473092, -8.530059982342436, 13.286937108409575, 14.38225073575174, 1.7260176581117435, 1.5993537636153405, 4.857161329898219, -7.4000809657835305, -4.862807550215649, -1.8374684357919904, 2.6524701552965837, 1.5470603193068655, 12.283423256047854, -0.2615490849222155, 5.787715135703962, 11.089431322777898, 1.2556986181471075, 10.063928215190856, -5.925918722351241, 0.7422479425653705, 6.13463230803562, 5.295692867025977, 3.794995403864732, 18.37710990769773, 6.230686730802063, 22.723219506590283, 11.30678979224325, 4.376837776341233, 11.343789911642546, -7.9703025550981295, -4.805803004247224, 6.598024984870165, 8.965526077819618, 15.165547038818744], 'changes_betw': [0.012837969030629337, 0.0075368509619276325, 0.03866155364626299, -0.00018122097632802814, 0.0015302655210912286, 0.0, 0.0, 0.0, 0.008021455269161676, 0.0, -0.07289087908353968, -0.022450770998171617, 0.0, 0.0, 0.00792477711437961, 0.0, 0.0, 0.0001274209989806322, 0.009425513324595847, 2.2652622041001132e-05, 0.00017636684303351177, 3.397893306150094e-05, 0.019958173551445697, 0.019958173551445697, 0.003983625390353216, 0.04025613643350641, 0.0002973156642882002, 0.0, 0.0, 7.64525993883769e-05, 0.0016746759866025779, 0.0, -0.0013603708557837213, 0.0, 0.0301174700257269, 0.0, 0.005845185508793815, 0.03812678996165239, 0.024706324935682727, -0.004763118295228519, 0.0, -0.04665550216008933, -0.06879844020516807, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02938975454265974, 0.0, 0.0, 0.009369286280601324, 2.9124799767000185e-05, 0.0, 0.0, 0.0, -0.023164733103570978, 4.247366632687726e-05, 0.0, 0.002684335711858707, 0.0005606523955147824, 0.0017232173195476097, 0.0, 0.004600100318754796, 0.02960778602980435, 0.03014578580327812, -7.64525993883795e-05, 0.0, 0.0, -0.037155154280536555, 0.0, 0.0, 0.00029610213096451475, -0.0017232173195475958, -0.001350662589194706, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0016467647201592068, 0.0, 0.0, 0.0, -7.645259938836996e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 7.645259938838384e-05, 7.645259938836302e-05, 7.64525993883769e-05, 0.0, 0.0, -7.645259938838037e-05, 0.0, -7.64525993883769e-05, -7.645259938838037e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (37, '58_37_1'): {'changes': [-0.9121567393081111, -2.5921724452841985, -3.7054322443133145, 1.763324491857034, 0.2849686901890749, 7.1441561197257215, 11.82726259792085, 16.05387250659058, -8.592043335403062, 8.696721754138764, 50.28960512579148, -13.4229129769883, 0.6843298127271851, -5.806697698151197, -1.072311098875666, 1.6368261819305872, -8.786955497578674, -0.2665401613853646, 0.4436032769738638, 0.9919672694910844, 9.33032625576297, 15.166470894614207, -3.764426573846606, -0.8589702984872076, 9.080533602769833, 0.7690739505101618, 0.8231669148894127, 9.256074057228744, -8.231299269907879, -0.948377426169003, 0.4661416425940672, -16.773980208820262, 4.399847572621738, 12.809484389482293, -5.803685183422317, 6.466320119222985, 1.4561519165152212, 3.190364162061755, -4.284658665001345, 1.2941570784095546, 8.964052351028784, 51.74630305320335, 47.145080542202166, -8.912006179866438, 13.061829126252746, 7.872331353638174, 1.179002786556353, 2.741629153496447, -8.72062603051554, 8.707696347678507, -3.233328058920847, -0.15503852621992564, 11.765921383859762, 13.617758255187653, 4.507991895726178, -10.363421599615037, 1.3852881015777214, 13.595572805544634, -1.17452681801592, 1.9948466261887674, -4.094421777791979, -3.1511997436824686, 9.48638959485666, -1.2824460747403421, -8.029777756119898, -8.387929894494022, 2.8915747749902607, 5.570981503678411, -2.2123507969124034, 145.96735737973452, 14.834960425780196, 4.927091327293965, 1.8244715217585679, -13.196329235099, 9.52377041634628, -0.11987085496156169, 5.729980388062927, -1.14662727811708, 12.48365411245095, 4.497035768933813, -6.57016604647373, 8.39472419791764, 3.0034863988772997, 7.06047364896969, 3.2614987341397352, 0.8882920464782131, -2.7111902717759335, 9.42167006796663, 12.142083419176885, 1.4859089033130246, 0.9778368970775375, 5.8832601975792045, 7.358324798144878, -0.018952698708346816, -3.3181306590958286, 7.285733835203729, -3.523868474610083, -3.954861857265712, 0.4120347863349423, -1.9528325141813525, 9.84467939395094, -1.7723345505269776, 20.051308939086766, 5.707635668226317, -2.2530056697598013, -0.016157893721583605, 0.34467245454996487, -6.901219016194148, 16.89627776093323, 9.74455950314794], 'changes_betw': [0.03442106370240955, 0.005190686536252709, 0.008235441645227493, 3.114735530636292e-05, 0.006159895150720851, 0.0, 0.0, 0.0, 0.029758061906379946, 0.0, -0.04312998559940464, 0.013364035726420993, 0.0, 0.0, 0.005094412892577949, 0.0, 0.0, 0.0, 0.005094412892577893, -4.2473666326877475e-05, 7.078944387815117e-06, -7.078944387812515e-06, 0.002430302736113145, 0.002430302736113145, -0.001976845784185232, 0.00486060547222629, 0.002050062294710636, 0.0, 0.0, 0.0016055045871559703, 0.006188210928272056, 0.0, -0.004138148633561468, 0.0, 0.021591589405045103, 0.0, -0.009084105949549476, 0.0031843114411921475, 0.018428717052570265, -0.004130867433619845, 0.0, -0.02930966134330054, -0.03120681843923441, 0.0, 0.0, 0.0, 0.0, 0.0, 0.022412746954031365, 0.0, 0.0, 0.004820154361438789, 1.213533323625015e-05, 0.0, 0.0, 0.0, -0.014351447340744006, 0.0, 0.0, 0.00024836982023535015, 0.00041907350775852344, 0.001127776968755552, 0.0, -0.001377360322314336, 0.020912010743815046, 0.02229948384382635, -3.3978933061501806e-05, 0.0, 0.0, -0.04979814895717038, 0.0, 0.0, 0.00012539844344125733, -0.0011277769687555728, -0.000968399592252811, 0.0, -0.001189262657152565, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.001189262657152565, 0.0010937980356940432, 0.0, 0.0, -0.001189262657152551, -0.0004162419300033915, 0.0, -0.001189262657152565, 0.0, 0.0, 0.0, 0.00041624193000340537, 0.0016055045871559495, 0.0016055045871559703, 0.0, 0.0, -0.00041624193000339843, 0.0, -0.00041624193000339843, -0.00041624193000339843, 0.0, 0.0, 0.0, 0.0, 0.0]}, (37, 48): {'changes': [-0.6499060650018116, -0.18714526140263743, -4.233057910353764, -2.270820134769423, 0.5521169979634024, 4.362615186534754, 2.715429516129973, 17.474893611579375, -2.2602146851819853, -5.311195512483806, 4.48901630248082, -17.185677286283834, 4.319001951680633, -10.639341298177726, -0.019792948553966738, 11.393241984785988, 5.992415083645199, -5.136961866556277, -0.5150785508868054, 4.635674120815196, 13.505287795257033, 3.259609435956463, -14.581211929261258, -11.47562958668236, 4.1235631349143205, 0.6244717436682947, 1.7865452189828162, 9.846536332997346, -15.122434868494793, 4.364728085789423, 4.849124260772129, 4.634587312703729, 6.883446721757281, 3.991326454533265, 0.5860332044131837, 18.294538224933405, 1.2404242822746525, 3.130405045344901, 0.633776493616601, 2.1641987043586113, -10.811278708642845, 7.638034272413265, -1.2816011346433243, 8.241760661806268, -6.020717103954439, 5.998091612090093, 3.741334848875482, 4.215831353359121, 3.2158934917166704, 8.330721780290503, 2.537523629886067, 0.10234103976535636, -5.224352948772349, -3.919635873097718, -6.800788970888021, 1.3900391727583497, -1.1739019719566386, -1.5076313507484258, 3.595738925267824, 1.5813689156691737, -2.541899324065156, 9.947466422459243, 4.083076980083604, -0.3498295093071633, 2.0076215184760997, 2.8706969481304654, -5.422279584459602, 3.081788044957932, 5.615051810518224, 5.1182447600979515, -1.375062210841918, 2.9716411370071114, 8.451823455052647, 2.308868029716109, 4.916403078080407, 6.111904961337473, -1.192182314336364, -0.6433307224797886, -6.33770699515685, 4.827558536182181, 0.343887372003735, -0.181725621471287, -4.787502058503833, -5.666652921780553, 5.027522200216367, -0.43548444430663835, 6.499390147326977, 4.704423102822716, 10.045606793295377, -4.28241105268139, -3.641657793397286, -6.4687380899353, 19.409762501151192, -3.915989159221624, -7.876496464319217, -6.366748620096018, 13.106523333904278, -8.33920967025827, 7.123706228229224, 4.564831357076827, 13.324303494093328, -4.4018949538453, -6.108355768995835, -2.6300679648299763, 7.029825709928872, -0.39422967093176453, -6.06202368669949, -7.690232284729348, 12.952975109734552, -8.574312044119807], 'changes_betw': [0.01118109153277358, 0.00014602850994283045, 0.00404592010096598, 0.00018405255408311672, 0.00033978933061504235, 0.0, 0.0, 0.0, 0.005140324903321863, 0.0, -0.009812428199278356, 0.005531082633529121, 0.0, 0.0, -0.00013712926556963545, 0.0, 0.0, 1.4157888775625464e-05, 6.108117728909068e-05, -0.0002123683316343863, 4.105787744931519e-05, -4.105787744931432e-05, 0.0016961150753199705, 0.0016961150753199705, -0.006920780544633756, 0.0036527353041114485, 0.0003397893306150146, 0.0, 0.0, 0.00025484199796126095, 0.0003397893306150146, 0.0, 0.0, 0.0, -2.831577755124659e-05, 0.0, 0.0007798974159830552, 0.0037433457922754626, 2.8315777551218835e-05, 0.00017960293189647758, 0.0, -0.002759574777923386, -0.003439153439153436, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8315777551260468e-05, 0.0, 0.0, 0.00048055919615547227, -1.6180444315001645e-05, 0.0, 0.0, 0.0, 0.0006381162726729062, 1.4157888775626331e-05, 0.0, 0.0004401080853680128, 2.8315777551251795e-05, 6.229471061276054e-05, 0.0, -0.0003681051081662612, -2.8315777551274346e-05, 0.0, 2.265262204100135e-05, 0.0, 0.0, -0.004502208630648985, 0.0, 0.0, 2.2652622041000048e-05, -6.229471061275013e-05, -6.229471061275187e-05, 0.0, -0.0001698946653075073, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0001698946653075073, 8.494733265375365e-05, 0.0, 0.0, -0.0001698946653075073, -8.494733265375365e-05, 0.0, -0.0001698946653075073, 0.0, 0.0, 0.0, 8.494733265376059e-05, 0.00025484199796126095, 0.00025484199796126095, 0.0, 0.0, -8.494733265375538e-05, 0.0, -8.494733265376059e-05, -8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (64, 70): {'changes': [-0.21437235514244968, -1.4838228528838293, -1.4039687545593935, 4.331009935076736, 0.9986598591349178, -2.7722211420362157, 4.631545512880706, -7.565771273323321, 1.2482253122954532, 9.767028113783851, 4.304298441848957, 5.273645048970366, 9.168800128029602, 5.390994550236599, -0.5508390576932101, 20.8412845467233, -6.621827345205219, -3.2986831487610004, -0.5468113397802554, -12.115593261781527, 1.3735819906804636, -1.9713065402696373, 2.491005935730726, 6.957244430346137, -2.445578048787894, -4.9338523789269715, -0.018854984485869863, 0.9304290834707558, -6.61325017843464, -1.7558066072810732, 1.492674933475783, 6.297181610753, -9.019004625744586, -5.024533213275731, 1.688005056485359, 12.865980217030227, 2.507182246024861, -3.065024829735904, -0.36977047797475393, -1.4285347019219117, 3.066197354207361, 1.2300662338180928, 3.519375767990482, 8.806970892109518, 0.6152464907996205, -5.998112675484208, 4.73081703099551, 6.814014794679082, -0.22292319205583055, 7.236225508626546, -10.359728682389033, 1.274021393205281, 7.209355561451076, -2.352394808339511, 2.4528266942460277, -5.33498274038206, 2.670385208360752, -2.9092843955297667, 12.53936359491422, -4.36506549610867, 16.388493643262763, 4.08237179636231, -8.897538345639191, 1.6869886244636092, 0.5272666549134826, -0.8677458093006578, 9.121694925662723, 3.322936004105486, 16.098582516601283, 6.464551145461911, -4.350844856751195, 9.989250345476279, -2.253848785720635, 4.580071153944971, -6.101099899711812, 7.017620250179789, 1.1970073059262063, -15.269677814545986, 7.1030289028146285, 1.3592294909719271, -22.952753453975475, 1.5049731974524434, -9.357365749774658, -0.020786725244050785, -7.708783839802834, -0.38094558430687187, -5.567774929845768, -0.605406963223345, -1.4011662229363537, -0.9533801267806226, 5.944024313358824, -1.961372085165067, -12.893999244858264, -8.263097944839842, -3.462791188299576, 2.0918942569877856, -6.443752245319516, 3.0550147424006724, 0.5341122036544732, -5.442494643136115, 2.9488905077687946, -3.1651297627306008, -12.307944439111765, 11.892484241815822, -1.6322019069267242, -6.383876027270219, 4.050291582219245, 4.255404541642463, -10.235192108841858, -3.9158427882509557], 'changes_betw': [0.0, 0.0065069656812774546, 0.000583305017555838, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0005833050175557408, -0.0020104202061388605, 0.0, 0.0, -0.0005833050175557408, 0.00022086306489976283, 0.0, 0.0, 0.00029165250877789123, 0.00029165250877789123, 0.0, 0.0005833050175557825, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0001415788877562052, 0.0005833050175557686, 0.0, -0.0004275682410239401, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0005833050175557131, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0005691471287801592, 0.0, -1.4157888775630234e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.4157888775625897e-05, 1.4157888775626765e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.415788877562503e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (64, 69): {'changes': [-0.5319271594924917, -0.8953065188583764, 0.4290809074814206, -6.116658003053914, 0.8705443517391274, -20.82773512202914, 2.897755958691107, 2.9177773016442643, -0.23994633102839202, -11.32813312473661, 1.900684370636867, -0.552814656280475, 0.7905042194386169, 0.29388754729711764, -0.11389807649995376, 2.630212141936596, -11.86165310802454, -22.366935807930787, 0.3930924969163314, 7.511610258444449, -12.698953433118646, 0.5610256557862385, 5.2429835864106735, -1.3722359950645, -5.367861655936963, -3.225410201937308, 0.6868373056696342, -2.570776586113311, -4.261746558976512, -0.46881378557203846, 1.036448922260199, 16.488725531629086, 3.3095875374189774, -14.77382709724796, -0.8208304641334436, -3.2982498225801464, -1.1106096084753396, -1.4373802311594943, -1.848400355340246, -1.2303308546967884, -1.0446800490958594, -0.6428037943400113, -0.019296775560164292, -3.554377926832501, -13.917097522051705, 1.921444371640007, -5.14654862893893, -4.623197011738284, 1.143796693974494, -2.1382734763717224, 7.378731071755254, 0.2428546311287647, 17.98128270558962, 1.5784738117847041, -12.545200109012853, -10.433376052751186, -1.0924367199556215, 5.234518311935602, -2.7428010121408875, -1.2358278079850749, 0.9912238947888454, 1.5649685331008527, 5.269523151463716, 0.052149473284458736, 3.596299346418377, 1.524127102891967, 8.902510938740534, 7.494628672385659, -6.401002488926991, 2.261628756810282, -11.558572048554822, -19.070612337696105, -4.519396618134834, 0.22400727324844638, -0.791057646425628, -9.645816840254497, 5.995048472068262, -4.2243790291481105, 1.7093136225039416, 2.3300599117189336, -3.8292834921556107, -3.243575752558172, -0.5444294335402162, -3.7678878932647706, 6.799611021397055, 0.5977692467118345, 2.1324829687007707, 2.935648936174914, 2.3042534442629403, 1.2372935354137908, 0.37633699358414674, -1.4289873208286963, 7.6401149109420885, 2.11956053619474, -2.7986487981837342, -3.7518761009573893, -8.148967335423208, -1.7449798126675944, -0.4151332061867947, -7.521789122486325, -3.6934605873396436, -12.48618996332803, 5.549269509935755, 2.1920859392112675, 3.447975275545673, -10.45839722709917, -3.049253489441469, -3.2392339689273513, -5.758754259728903, -16.85056414505499], 'changes_betw': [-9.910522142930756e-05, -0.0002548419979612748, -0.00015573677653189788, 0.0, 0.0, 0.0, 0.0, 0.0, -5.663155510249318e-05, 0.0, -8.494733265375365e-05, -4.2473666326876824e-05, 0.0, 0.0, -0.003412051194925797, 0.0, 0.0, 0.015191414656246462, 0.0002548419979612193, 0.0, 0.0, 0.0, -5.663155510250012e-05, -5.663155510250012e-05, -4.2473666326876824e-05, -0.00011326311020501412, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -5.663155510249318e-05, 0.0, 0.00025484199796119156, 0.0, 5.663155510249318e-05, 0.0002548419979613026, 0.0, 0.0, -8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 5.663155510250706e-05, 0.0, 0.0, 0.0002548419979612193, 0.0, 0.0, 0.0, 0.0, -5.663155510249318e-05, 0.0, 0.0, 0.0002548419979612748, 0.0, 0.0, 0.0, -5.663155510254869e-05, -5.6631555102534814e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (64, 68): {'changes': [0.4176913926848105, 0.5367468662762569, -4.240042894019567, -1.8631571663868698, 0.5185975444149022, -4.452192507255489, -0.34439400333388903, -6.781371033640738, 1.5405187331282661, -19.465270854001346, -1.619644818634626, 3.116225968987834, 2.938657616641592, -7.253160978076977, 1.2794394843950556, 4.910806605974329, -54.83328116757045, 2.8538171397115946, -1.2464000663072987, -4.569246068118304, 2.0338626863829603, 13.36908206277809, 0.7112065030061956, -4.289021822132099, -4.031194149117084, -5.719427310809721, -0.14560556848436335, 2.1332780335778807, -11.777488623559805, 2.6850198480452434, 4.0955456108486885, -12.5133098426125, 1.9203909261295564, -8.949167097245464, 0.08657123069655626, 8.308223745531393, 0.7795678618043205, -2.0183504728740758, 1.4528967438055815, -3.748206245900427, 1.0015377516955368, 0.7334009726911574, -1.0472284067458162, -6.686495890579835, -1.3736337797008673, -18.512964318910377, -4.776666132573524, -0.8682295575556367, 0.709013163654717, 9.433008060323516, 5.505854810765641, -0.5270841429495476, -6.9256090514310245, -6.124021156214724, -12.317692074021352, 2.02392918518143, -0.4366349839272168, 11.553540540366924, 3.3972458659223435, 1.8724234205291168, 7.356566929807201, 3.6295672732683073, -4.401400030995603, 0.045685529567954575, -3.400460448217544, 0.048049091481274786, 5.314822607185846, 9.745848491011031, -0.5599434198444442, 1.5580580722153883, 2.95055447209495, -3.777940090786899, 0.04733039309127207, -5.669155882601444, -7.643074138317587, 0.06099561374583118, 1.4081136616180743, 12.037131081980903, -5.81684069068092, 11.942346961179538, -5.987040659702586, 16.09157252770757, 4.625940207318138, 6.798666556318011, -1.9646440775986775, 0.0678208071045674, -5.1916487840840375, -11.12351100329542, 6.96602672546922, 0.8277138931682231, 1.2202221350118805, 9.603091790071517, 1.8625124507017858, 6.156644077549629, -9.32480855339088, -9.19363373789335, 2.470950866787348, 4.004965776776359, -4.05935168664152, -15.800795490952595, 11.06625266919383, -2.3399311471671638, 12.039481833246015, 0.36972545422310077, -0.2867249865082613, 1.0392924467614932, 1.9570401529466608, 6.891250315376794, -11.232016629745885, 10.603209768365105], 'changes_betw': [-0.00018688413183814978, -0.0004303998187790148, -0.00024351568694076786, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00016989466530753505, 0.0, -3.397893306150701e-05, -1.6989466530750036e-05, 0.0, 0.0, -0.0004303998187790148, 0.0, 0.005674481821270814, 0.0, 0.006104881640049797, 0.0, 0.0, 0.0, -0.00011326311020500024, -0.00011326311020500024, -1.6989466530750903e-05, -0.00022652622041001436, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00043039981877898703, 0.0, 0.0001698946653075073, 0.0004303998187790148, 0.0, 3.397893306150701e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0001698946653075073, 0.0, 0.0, 0.0004303998187789593, 0.0, 0.0, 0.0, 0.0, -0.0001698946653075073, 0.0, 0.0, 0.00043039981877902866, 0.0, 0.0, 0.0, -0.00016989466530753505, -0.00016989466530752118, 0.0001698946653075073, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (64, 77): {'changes': [1.880644096981671, 2.1758484285749304, -2.0887151741843297, 5.738165191270085, -0.47027534872122345, -2.768518302122345, 14.642003320080875, 6.772308378410941, 5.22214531974582, 18.93693643130075, -6.4070986573587305, 1.4785623002464376, 2.8846561258236534, 5.329709888455568, 6.0711438885159374, 7.924343874163611, 9.61044870392945, -182.9431065916212, 3.1124768667835845, 10.07531555622569, 11.844747444124636, 11.290859533982143, -0.9378712552100126, -11.093452880495008, 6.318292521463491, -5.120361562499781, -0.3639953133828442, 7.232253324994929, 1.3596165265884679, 8.30333466898044, 3.280746428752323, -2.9985352194473194, 3.46871598447909, 10.581839398221717, -0.6851535277276426, 11.65379144880228, 0.28983049107498005, -7.8398723447171506, 0.4928322525761928, -0.302984482091631, 3.101500859926233, -3.7323208573768483, -6.595923766765296, 9.291310855594844, 8.500553680220776, 0.45765964901289635, 3.2493348604657655, 11.542001707630831, -0.0038823486320183065, 7.070060176310051, -0.37257559919038385, 4.823793325010904, -6.878976605665201, -2.1629812194967144, 2.9352590696175014, 5.0891283937563685, -0.04833285617068839, -9.324880077379618, 10.122380390739352, 4.775150133105342, 2.6580321393565214, 7.619130504162598, -2.15424341596713, -0.19291559082449794, 2.4646585382377353, 1.1044867006151406, -0.45777433422398417, 5.198233150517808, 6.793330908220327, -5.213927247264834, 18.61455572093564, 4.363244074986028, 20.99579348526305, -15.461762862920438, -11.701101363179419, 11.032283842550385, 6.907050103234113, 6.102543546159836, 10.38193698731041, 15.423076665991573, 13.967270527825917, 9.38051753696891, 9.85065744249846, 1.3182231947796765, -6.192302495119208, -0.3243263973266295, 12.404216294404591, -3.3523580048712063, 1.963399305576587, 2.6030404029258634, 6.048493217391183, 15.113113258606433, -1.3914008176948016, -6.258918991992914, 3.0941575894300684, -4.70342894663213, 12.525928137609412, -1.3674712761301322, 4.9315033895105955, 0.533058215593428, 7.900920678717284, 4.985647030647954, -3.9481942649771327, 7.78527770411452, -1.3707868226655364, -1.1918998095719928, 0.7977599233292949, 5.578870609188186, 5.493169282476231, -0.7132797262345321], 'changes_betw': [-0.010012863453230436, -0.018122906655016896, 0.004790220539456033, 0.0, -0.0018647962073037871, 0.0, 0.0, 0.0, -0.007676811805252204, 0.0, 0.006605666391599027, 0.0007228613497726778, 0.0, 0.0, -0.025935229681406963, -0.0002265262204100122, 0.0, 0.17449597915958776, -0.02263644159668607, 0.00022086306489976283, -0.0009222853259550634, -8.673617379884035e-19, 0.0039031276798860842, 0.0039031276798860842, -0.000977298836626052, 0.009590149345501023, -0.0012175784347037977, 0.0, 0.0, -0.001033525880620681, -0.001864796207303815, 0.0, 0.0006472177726000033, 0.0, 0.004004659967962787, 0.0, 0.022194715466886533, 0.010128149118974805, 0.004967396404705338, 0.012925747941038374, 0.0, 0.00909826383832496, 0.008645211397504918, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004967396404705282, 0.0, 0.0, -0.016024303027361186, -0.000254032975745516, 0.0, 0.0, 0.0, -0.0033691730174910117, 0.00016989466530750947, 0.0, -0.0024351568694077202, -0.0012034205459281917, -0.003121612219471541, 0.0, 5.258644402383883e-05, 0.002079187094477658, 0.0050806595149102685, 0.00023502095367538807, 0.0, 0.0, 0.004307638787761109, 0.0, 0.0, -0.0008110447712894216, 0.003121612219471548, 0.00207554649450673, 0.0, 0.0008494733265375504, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0008494733265375504, -0.002886591265796167, 0.0, 0.0, 0.0008494733265375642, 0.00018405255408313753, 0.0, 0.0008494733265375469, 0.0, 0.0, 0.0, -0.0001840525540831306, -0.001033525880620688, -0.001033525880620681, 0.0, 0.0, 0.0001840525540831358, 0.0, 0.00018405255408313753, 0.00018405255408313406, 0.0, 0.0, 0.0, 0.0, 0.0]}, (70, 60): {'changes': [1.389629724069371, -2.656585484310412, -3.8383010086014906, -3.6211430050483813, -4.19087327902561, -8.80411925922661, 6.893192605359275, 3.0580778421448827, -0.22623868339049125, 0.7602891016950366, -4.805769996307063, -0.6871849206115428, -11.228879159850663, 0.28244200373387685, 0.30849996715186023, 32.28237676379791, -6.036359407961072, 5.606943506742908, -0.9708961021679983, 11.908174858809758, 4.221463844131733, 12.200214546002371, -0.7369251390818476, -10.43835425703736, -4.827896294564084, -1.8895167021863415, -3.8121357633949344, 9.057770812810702, 3.665560412312459, -2.407287499778164, -5.350255296602207, 8.563202781653217, 4.148786329030287, -6.957021840256857, -0.46719794205344556, -9.6291542895205, -1.128884575327607, 2.112510496302562, 0.013677729414759199, 0.12856621818309577, -1.113645219268335, -6.05821374180357, -0.03593741347256696, 11.899633670160256, 4.463610120384885, -1.7402029776565087, -2.4118115226414574, 3.252832232738484, 3.394937811460238, 3.871530370020139, 5.396995176818621, -1.248192488015448, -2.3697279561085907, 2.3888611719091273, 2.7770586592899917, 7.312173453895525, -0.19487426896534998, -6.9890154050575575, 3.08013239702359, -3.747428551194993, 11.84354050156486, 0.7094098929626256, 4.420120336650655, -0.0955868622694922, 1.6556898989611781, -1.1087161276627455, 17.919970252899617, -0.004254181375017652, -1.301788268735379, 0.23805768271637362, -2.20349248349288, -13.407839779669416, 11.960406386255073, 1.205486602394302, 6.090741801196259, -2.4685611832737493, -1.111402463331416, -9.606489716094359, -5.885721806824392, -3.4534464676738708, 3.443892446510688, 5.287229532036349, 5.641171064388317, 2.7718274562907936, -3.5593839142077, -3.4283691460790777, 4.410369400120999, -1.2201177765861644, 2.8569249737786606, -0.16162600524828719, 2.114412711657792, 9.302951343751147, 4.744661219960449, -7.047075285591859, 0.9560697243019263, -8.123124679642302, 1.1118206020479704, 10.900318155522797, 8.844433051356106, 1.0558596542763894, 4.428385952977834, 4.710073613103532, 4.989465544176653, -4.150052785038582, 2.2509153703561253, 6.791144155986757, 6.104404903774906, -1.0035217174896047, -10.833839886564675, 8.467359055765996], 'changes_betw': [0.00011326311020504187, 0.0034347038169667, -0.00026616830898168464, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4157888775623295e-05, 0.0, -1.4157888775623295e-05, 9.910522142938041e-05, 0.0, 0.0, 0.00015290519877686481, -0.0020104202061388605, 0.0, 0.0, 0.00015290519877686481, -0.0016904519198097175, 0.0, 0.0, -7.645259938838384e-05, -7.645259938838384e-05, -0.00011326311020500631, -0.00015290519877676767, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0001415788877562052, -0.00015290519877676767, 0.0, 0.0003086419753085823, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00015290519877683706, 0.0, 0.0, 0.0, 0.0, 1.415788877565105e-05, 0.0, 0.0, 0.00016706308755234933, 0.0, -1.4157888775630234e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.4157888775625897e-05, 1.4157888775626765e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.415788877562503e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (69, 68): {'changes': [-0.6341425276350083, -0.11625154038960872, 1.596072468267053, -0.7516200433714033, 1.895355730934618, -10.56124057283327, -0.1184039390459759, 10.765805018616419, -1.1883402808032741, 13.745897229497842, 0.9154552241391656, 4.832259517976638, 8.982516332975933, 0.9141873641402185, -0.3755668647666326, 12.826306023123578, -6.496088113300061, 38.00231639588242, 0.23007035420969402, -5.710434474093887, 1.3326497874422643, -7.50091828799404, 8.264919339306857, 9.927106234576172, -2.719495421225332, 4.360890786723147, 2.2080289054164055, 9.073263330410015, -2.6446415193478856, -0.24739849595738406, 1.6344485563134583, 14.306405747993153, 5.358707944438805, -8.775052096544982, 0.01316335593726592, -8.860875460545913, -1.1128722871101573, 5.863874650613866, -2.470932453774637, 0.2529661801423728, 7.66213119330061, 1.999727476749129, -1.8454957513986017, 2.407860778142833, 0.18264860849217257, -8.477585745606888, -1.9273751609113532, -1.244325074595352, 5.266102724471544, 0.3060396844407478, 2.290902188089433, -2.647157673925882, -0.842182611171296, -8.052624146415667, 1.9527521713531542, -9.164205990140147, 1.2841605925112738, 1.0014001428075403, -8.235064778283174, 0.76101737287118, 4.182713105084417, -4.349389353390237, -10.624072290961209, 1.1165259910539902, 0.7399945303065181, -1.7545899688521374, -3.727491061648692, 6.147350314350945, -1.7077966382198042, 0.5369673391766696, 2.5816362871491947, -14.159493964694548, 9.200867365259342, -12.462924266751742, 1.1840446163743081, 2.897006385213899, -5.287795168923751, -7.44766964614422, 0.30656927508897525, 9.244264585685357, 3.411860710027213, 6.068437300358767, 2.9056144960734116, 8.23513534081917, -11.300456933994468, 4.374944732352688, -4.908147118324223, 11.357493865965864, -10.527426854544245, -6.513069920835463, -1.9562033391447642, -3.679055411564349, -5.155996303504622, -2.028839235164952, -2.7756451996587828, 2.8996237179344178, 10.775715226285115, 7.983204737665801, 6.548454876071958, 1.8212621397102566, 6.3904886765411675, -9.610887151392177, 2.7188707317431806, 5.524375836162591, -5.744193721984146, -0.012641423121607431, 5.930466281016379, -4.989157644284376, 1.2023871240283484, -7.076421963906881], 'changes_betw': [4.2473666326869886e-05, 5.663155510249318e-05, 1.4157888775637173e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8315777551274346e-05, 0.0, 2.831577755124659e-05, 1.4157888775626765e-05, 0.0, 0.0, 0.0033271038622720572, 0.0, 0.0, -0.0031572091969645486, -5.663155510254869e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 1.415788877562503e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.831577755124659e-05, 0.0, -5.6631555102520936e-05, 0.0, -2.8315777551274346e-05, -5.663155510249318e-05, 0.0, 0.0, 2.831577755124659e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -2.831577755124659e-05, 0.0, 0.0, -5.663155510254869e-05, 0.0, 0.0, 0.0, 0.0, 2.8315777551274346e-05, 0.0, 0.0, -5.663155510249318e-05, 0.0, 0.0, 0.0, 2.831577755119108e-05, 2.8315777551218835e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (68, 77): {'changes': [0.7369564997397262, -1.6465089050369919, -1.869075294465496, -2.228789244643451, -1.4622045938680799, -4.648703873086333, -4.58117044215939, -6.734560461034874, 0.652045695663908, 5.235655332628994, 4.740472137706334, -8.646779048915505, 1.4680328327461325, -11.085287470624706, -1.5335515199090075, 4.909789690240956, -3.631813585458474, 39.97273425160128, -1.3073819215725173, -12.0696627189337, 0.9709276645046288, 8.265885084351765, 4.3398654999048745, -6.109120836355714, -12.448051813059891, -1.1120247957563691, -0.2890231262407994, 0.45294406162122414, -3.4782928894277916, 3.4570634906112048, 1.8562434650699018, -19.458271779714664, -4.28594788703893, -3.387546405254966, 2.0330713246774152, -5.808079514812903, -0.14575583847323514, -4.34279506861175, 1.181797474228759, 0.1611375082360027, 6.172172820264279, -5.841978527144647, 7.805923390064365, -1.697131949648508, -3.2420155546438423, -2.7853239715718985, 5.059969198197038, -0.4200288975859223, -0.023761142023367654, -1.1485782540050877, -4.754566142251747, -1.5076741340525217, -4.757807227049938, -2.0519699222413124, 0.5274235817258273, 0.19686266181122392, 0.06413833092738486, -2.7680981125218693, -6.83101348431677, -0.05295765275830888, -7.137743714232016, -0.7674196507990843, 3.7007668046035747, -1.3317907573668037, 2.326390909562093, 2.12968928447777, 7.3094991067366095, 0.15408478380061297, -3.1883648280028467, 2.8970627493366266, 3.4789771665244302, 16.36666314965754, 0.5757099755829245, 11.314971901692786, 9.95120400573856, 8.197347641520537, 0.32935709661718704, 0.7737945330501361, 1.666898965267535, -3.376251733267196, 11.995245460261884, 0.34338681045920794, -0.8340788969341872, 5.7435637052061566, -6.434350192375376, -0.7194339200969608, -2.0627176091494164, 9.614530814532003, 8.63763795627196, 3.2395947183874725, -0.22703926488836146, -5.7057924405824, -0.9619335596260044, 13.772897123788567, 2.6398467807663053, 7.21238066719539, -2.0476922158655384, 2.947501114613587, 2.689032311090074, -5.366104184462188, 1.7130100652768192, -0.5529260396891686, -0.7592547632287108, 5.459836527574339, 12.566124385816664, 14.076519212298166, 7.103629254431638, -3.672547329069687, 14.364216174655326, -8.660105825045832], 'changes_betw': [0.00018405255408315835, 0.0003681051081663167, 0.0001840525540831306, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00011326311020501412, 0.0, 0.00014157888775624683, 7.078944387812688e-05, 0.0, 0.0, 0.009726469588854869, 0.0, 0.0, -0.0031572091969645486, -0.00036810510816628894, 0.0, 0.0, 0.0, 5.663155510250706e-05, 5.663155510250706e-05, 7.078944387812862e-05, 0.00011326311020501412, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00011326311020501412, 0.0, -0.0003681051081663167, 0.0, -0.00011326311020501412, -0.0003681051081662612, 0.0, 0.0, 0.0001415788877562607, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00011326311020500024, 0.0, 0.0, -0.00036810510816628894, 0.0, 0.0, 0.0, 0.0, 0.00011326311020501412, 0.0, 0.0, -0.0003681051081662473, 0.0, 0.0, 0.0, 0.00011326311020493085, 0.00011326311020497248, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (77, 63): {'changes': [0.5388543435064417, -0.9432020732469155, 1.7470905489790312, -2.782608265841432, 2.8968665516205903, 1.662820338842522, -8.619496436364471, 2.951849913978947, -1.801460825544197, 3.6856122821780275, -2.2125356792989095, -7.310957670643205, -0.8524289915292229, -7.493376662424367, 2.6826744819442006, -2.5469296333702403, 7.694481441833943, 19.319207761393955, -0.9024086647094833, 9.75118516100332, 9.088169610515266, -2.281779778112792, 12.827679962962918, 5.194597056665231, 7.439566206187095, 1.393723496063103, -1.7223090834068842, 4.044220230232554, 4.514563862097589, 2.2241076577311816, -0.27925784275912946, -1.0629649039446747, -2.32759760104868, -8.604627723455138, -2.337013053522867, -5.783431765778943, 0.9593493230201133, 6.355832004713079, -1.173584856758076, 2.735565388657662, -7.362711785012351, 2.2633774347797413, 1.2829754318446192, -4.58519856075236, -0.07031881844977761, -3.354612594082255, -5.052134570418957, 4.030764685702906, -3.093982448303457, -1.1741882448998808, -61.90388321782805, -0.5549944296090956, -4.8191615280916835, 7.311684180578311, 3.688345880536815, -5.423537877189119, 0.40240295852485275, 13.25566739956949, -0.8787595765579113, 1.669177405606149, 10.895033832384343, -1.873599425955021, -19.515056337101413, -0.7671638545894233, -2.169660283366916, -0.006569721540685691, -4.1539330434858925, -5.591730801723713, -4.0868185750409225, -1.5605792747040823, -7.045071136168588, -9.900803871208353, -13.06563600053687, -1.8666721047646604, -0.5633712325300166, -3.7920164151764766, 1.9966844058270254, -8.022097655997186, 6.173356143662659, 0.4317399470241128, 15.155620651738076, -8.453184093792913, -5.501283991915869, 8.625447971085094, -5.449895687013708, 2.5796801091969286, -3.831566174824246, -5.442311240020899, 22.670845710234147, -2.470675244412355, -1.0527741700593651, -5.411971865968667, -14.585244845046418, -3.341514692410499, 3.2826434892060945, 6.26769386392948, 3.6852605636428564, 3.0853555731166864, -4.720950092695375, -6.316848625862747, -7.451654102718095, -4.109609501899882, 13.929035323021964, 5.736458776032606, 3.537267982045222, 2.24698019393162, 4.7605940442876715, 2.0247743054028433, -3.6091067152169387, -9.62415696604748], 'changes_betw': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00019821044285878164, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018207044965454753, -5.663155510254869e-05, 5.663155510250272e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (77, 72): {'changes': [-0.10729187007751939, -0.9279119841240409, -0.8795245286836746, -5.742451156423854, -0.574603278235756, -3.1383018932699827, 2.7169939098094744, -8.748581158054122, 0.6161997821256335, 9.644598407978663, -2.5684970529995184, -4.627993613951958, 0.7332409297891331, -7.421485797237949, 1.6351852624973802, -11.231842593584616, -1.7131300497657662, 7.297993473962606, 1.3377276875636461, -17.596891291490834, -3.356755045345281, 1.5322736256096334, 0.23056960554686157, 5.157526037453465, 1.757081636435288, -8.503490504135826, -0.6848819989641726, 2.83998350133146, -1.641661494318953, -0.46624046009431197, -0.814861801901813, -2.6564409437233962, -2.671836549838602, 11.011966223062103, 1.1897310144447815, 13.144063507457304, -1.1767277808107686, -3.793082079991052, 3.3491025152990233, -0.9096358931712061, -6.780410995221473, 5.132242897414756, -1.5067373525576357, 0.8224843229627368, -9.878694559231064, 5.717742468306909, 5.653995298811182, 8.950860120506022, 1.0105653884213481, -58.680381309847235, -2.973353775005137, -2.324025824516262, -52.38475455273513, 5.800518146914214, -4.262928259329669, 5.943941254450124, -1.3071998633417579, -26.803631856874915, -0.6247599535233803, 1.5587230591824905, -8.579882269564848, -3.5605022411781277, 11.178065102873319, -0.6147818549386876, 3.236701444624522, 4.502552649980046, -5.592894809957102, -1.3344393249797122, 13.097553674013398, -3.9339054861617626, -1.7545759934687908, -1.1523401626548093, 8.331251219581901, -6.018133007742705, -5.129458315630131, 4.512622558293316, 2.4017795809646714, -1.5842954504788054, -1.6192188554614972, -0.4532821425060831, 4.40934466814619, 13.541615342048658, -3.4274279320663084, -7.451449612468224, 9.697865392549232, -0.6750937534239796, 5.7008919550537485, -6.325515839234129, 6.868195664320069, -3.471008383708174, 4.800015192014058, 7.199637059812062, -7.114304035717396, 0.2603712264470346, -3.8361442725548898, -2.1898829114215914, -3.607373159155273, -3.06256061799661, 3.6315490746825105, 7.7145019599732905, 11.029740590109327, 6.562424894364369, -2.090006091385902, 9.014934192458952, -7.315253077284865, 6.953186574744507, 11.303371369279574, 8.542285578669464, -11.590936129580086, -4.674913216118227], 'changes_betw': [-5.9058621749652485e-05, -6.714884390723874e-05, -8.090222157489113e-06, 0.0, 0.0, 0.0, 0.0, 0.0, -5.0968399592260516e-05, 0.0, -1.618044431501986e-05, -8.090222157499521e-06, 0.0, 0.0, -6.714884390723874e-05, 0.0, 0.0, 0.0, -0.0019384172289370616, 0.0, 0.0, 0.0, 0.0, 0.0, -8.090222157500389e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.714884390723874e-05, 0.0, 5.0968399592260516e-05, 6.714884390723874e-05, 0.0, 1.6180444315005982e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0968399592260516e-05, 0.008035208646829434, 0.0, 6.714884390723874e-05, 0.009973625875766532, 0.0, 0.0, 0.0, -5.096839959223276e-05, 0.0019384172289371102, 0.0, 6.714884390725262e-05, 0.0, 0.0, 0.0, -5.096839959228827e-05, -5.096839959224664e-05, 5.096839959224664e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (77, 86): {'changes': [1.6794424091930509, 2.0890081273965926, -4.792399288166408, 4.50743145473686, 2.2541705386980837, -1.526053554744891, -3.3562703865863455, 1.9387671172014223, 1.9550346153177074, 1.2554321118307712, -0.7116815271251653, -7.346527090102569, 0.5981557567293976, 5.314926395058592, 3.8189597064696947, 15.66364196083211, 19.51683188408697, 0.5444966745813247, 3.1126698500139085, -9.991611140651685, 2.440026163811993, 11.590365738474446, -8.248591853700106, -13.171354878852114, 9.137973852670598, -5.176278614281998, -0.42850557845392956, -11.445697369676765, 4.5787085708469135, 5.484757780071675, 0.3667898797956539, -5.403254690019594, -9.64157465840269, -2.4768823185174256, -5.79253440166535, -1.5191694694345301, -1.4220734578029735, -8.085561042899371, -3.8185428760724633, 4.263096530054931, 2.9767998452469158, -3.3915874928897836, -3.9605770107509244, -9.575303715144031, 7.247612986213028, 0.5621952142890905, 1.1820678645189844, -5.454535357974507, -2.1680757931500523, -16.35669320168435, -5.3714346602994, 8.801233224799745, -14.306947860353603, -180.6666185606403, 1.3624456348036915, -174.98377875633122, 0.03356757053532533, -19.1481960767033, 7.873131712478397, 11.658593448732766, 3.5474320453980965, 2.258214035770038, 9.504319569491855, 1.3577618718882505, -4.086073848653385, -6.244253563994242, 4.1589786023630495, 7.905393774794845, 6.7189667042024155, -9.03664923764542, 14.08808113630343, -6.955541419764302, 7.2520694881818315, 0.38562961941315166, -10.342063239172063, 4.716727272287471, 2.8937746791739016, 0.29012671152793246, -6.264576252124073, 6.920833924650481, -0.9959156322346701, -7.475340280327941, 0.6828240718146219, 9.736243226069888, -12.916066854323788, -5.294475923167283, 2.289233115521398, 3.0422653190460096, 5.561489682062017, -3.1553597841830765, -5.074060037805026, 0.8691026585412658, 5.603173636772937, -2.4488522412388534, -3.103135194244288, -8.151211741511332, 8.125467690254112, 9.110912634493872, 8.552198266565682, 8.525120451487453, 2.127597051502562, -4.725228112237261, -12.192820975913065, -1.4469121764599038, 3.4414458958063108, 5.346522877439099, -0.8399503470755576, -5.42328203527785, -1.5957288008336263, 1.3353494130391823], 'changes_betw': [-0.006374286037894472, -0.011601783084963624, 0.0069931880329433815, 0.0, -0.001638269986893759, 0.0, 0.0, 0.0, -0.004916023494005162, 0.0, 0.007356034496707187, 0.0013741242334514535, 0.0, 0.0, -0.0157896865847936, -0.0002265262204100122, 0.0, -0.000467210329595652, -0.025564292995485433, 0.00022086306489976283, -0.0009222853259550634, -8.673617379884035e-19, 0.004395822209277853, 0.004395822209277853, -0.00032603595294726853, 0.009584486189990735, -0.0009910522142937972, 0.0, 0.0, -0.0008069996602106805, -0.0016382699868937728, 0.0, 0.0006472177726000033, 0.0, 0.0037781337475528143, 0.0, 0.015475381453974618, 0.008825623351617226, 0.002164134427131398, 0.0063479928158826915, 0.0, 0.0077532644046404975, 0.008234632623011773, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002164134427131398, 0.0, 2.8315777551251578e-05, -0.02633974078928203, 0.007388395385337282, 0.05807282817986181, 0.0, 0.05807282817986181, -0.0007216478164491225, 0.004584324385547621, 0.0, -0.008120965001698982, -0.001160946879601314, -0.0029517175541640336, 0.0, 0.002516059090982792, 0.0044010808536803225, 0.002418976425092631, 0.00020670517612413715, 0.0, 0.0, 0.004194375677556088, 0.0, 0.0, -0.0007827289937381698, 0.0029517175541640406, 0.001962283384301728, 0.0, 0.0006229471061275221, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0006229471061275221, -0.002745012378039908, 0.0, 0.0, 0.0006229471061275083, 0.00018405255408313753, 0.0, 0.0006229471061275325, 0.0, 0.0, 0.0, -0.0001840525540831306, -0.0008069996602106735, -0.0008069996602106666, 0.0, 0.0, 0.0001840525540831358, 0.0, 0.00018405255408313753, 0.00018405255408313406, 0.0, 0.0, 0.0, 0.0, 0.0]}, (77, 76): {'changes': [0.46960860305200214, -0.6569090582435173, 1.8795602462373466, -2.014447204570928, -2.692332127738993, -6.504373248882217, 2.371578440434263, 8.093206067705836, -0.771618795063624, 6.740685322351851, 3.022183862136373, -6.2054802962116185, -9.377557199976053, 1.2071065882517473, -1.2693160906104204, -0.19726651593552447, -4.904829226975664, 5.88560216887052, -0.16015146612112474, -12.073299776660946, -8.129401277743042, -10.19491895901001, -3.2868378622721366, -8.65058672907827, 8.654683275897781, -10.356549837715718, -1.5965827287842913, 0.8225781261615452, -4.427915497125241, -3.5842524572758947, -3.919419997416483, -24.125097052743456, 6.646079261128463, -8.005009740247374, -1.6474716233837938, -3.8880995217868417, -0.00915876212171085, 0.21595074949863147, -0.5701760172232824, 0.8964519561406448, -8.93676312102383, 6.283165293984027, -0.21693079965288575, -5.002005465882007, -0.29172729537191344, 2.391228627937778, -3.309030046961354, -3.538543913312253, 2.955031745311288, -9.583679676629686, -65.22984517501354, -0.9839013253080608, 32.78412922064214, -2.568199497788214, 2.6436708999370353, -0.3318679253997914, -0.6871728825270047, -22.235831736543247, -8.087581752299457, -1.361182361261939, 6.934831133819841, 1.011188554478153, -21.694432400509555, 0.62894360924005, -0.42725759251329265, 2.668417312568195, -14.094538026170653, 1.6100069058631448, 0.6343264829621944, -3.3155287226412185, -8.330518585440501, 14.0588028083653, -10.307600424981274, -6.248276106919008, -13.371760737172792, -9.953503176969065, -0.3590486500341399, -2.559163294680161, -6.896395759476434, -5.750961388272174, 2.008364545075864, -8.68636709167805, -8.982941600853508, -1.5266451264358523, -12.84476494399297, -0.9651071695083999, 0.1984147297936829, -6.652924375856571, -10.179463339367658, -1.9414560194106727, -4.892511212452774, -4.447597574716099, 16.16497513982253, 12.840794284119625, 4.245126008773866, 5.610452929198317, -3.660871762224332, -5.458068435012308, -2.408085381183213, -3.0363720995501353, -7.97241768336869, -7.379764412666617, 2.0314432823049913, 2.6145766555872, -2.3441257159296356, -8.456329400590363, -1.0395706254273023, -6.4228797041844246, -0.841618406530813, -2.1934863564801503], 'changes_betw': [-0.00026414575344224644, -0.00038064495251038943, -0.00011649919906800421, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0002042781094768742, 0.0, -7.887966603563334e-05, -5.986764396549715e-05, 0.0, 0.0, -0.0003806449525103617, 0.0, 0.0, 0.0, -0.0035944857045774325, 0.0, 0.0, 0.0, -2.831577755125353e-05, -2.831577755125353e-05, -5.9867643965503224e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.831577755124659e-05, 0.0, 0.00038064495251038943, 5.663155510250706e-05, 0.00019821044285878164, 0.0003806449525104172, 0.0, 0.00012580295454914237, 4.085562189537484e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0001982104428587539, 0.0, 0.011779363461320648, 0.006143714706405734, -0.005662346488034563, 0.0, 0.0, 0.0, -0.00020427810947690195, 0.0034120511949258117, 0.0, 0.00038064495251038943, 0.0, 0.0, 0.0, -0.00019821044285883715, -0.00016989466530752118, 0.00019821044285874, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (77, 79): {'changes': [0.2541304831442446, -0.05385538577349536, 2.885383779772269, -5.042534064448233, -1.8265097107287502, -6.402307941688036, 0.795133861448079, 3.354144727239259, -0.08823851702606333, -12.896355508314457, -2.5010880094636576, 3.6292802759747644, 18.82388238060409, 2.6589211995260484, 0.12083054196919463, -1.6427128551320607, -9.402164464403398, 6.347791921169261, 0.19059959324901143, 6.92533019910789, 4.666414277099449, 11.637593179360948, 1.9219037626341162, 4.866830148895133, 1.0720146113194176, -1.284532247668892, -0.06787607836064069, 7.596012148204153, 2.507832869483366, -1.104931487644535, -0.8436679889869012, 4.920473608592971, -8.30461053235679, 4.707243487948858, -0.3432214552594033, -8.386274963861297, -0.11958552813689849, 4.776233004073788, -1.1509234853397814, 3.326824154979178, 11.76075376922472, -5.174383910980211, -0.6028361767683492, 14.187298013643215, 2.730003372399324, 11.189761929394308, -2.011975735077854, 1.6779008155520074, -1.657227214730483, 2.4602801586427745, -1.719304878527737, 2.4578145135792866, 1.2350546746282305, 5.80462759779175, -5.280349662246749, -57.88095433019808, 1.0689463135371042, 12.977725969583332, 5.612188007325955, 1.8593391360847846, -16.49795876269326, 2.648895811476905, 2.27773524724779, -0.3243587735824356, -0.22481504274049513, 1.3460266775524374, 1.966015231578325, -9.552108927965548, -5.773646851348104, -1.5230761045591237, 9.995690334831068, -1.7120483619547144, 9.017894067338233, 1.9915962696542806, 9.061611384111387, 4.48269157842131, 0.8041195787872368, -6.72796176973759, -8.199659049418187, -2.6729884981396026, 6.359904419926181, -6.380945074380691, 5.459844839274979, -1.443692480713139, 0.4302411351868898, -9.769819578680497, 3.8665505657560004, -7.43917038480447, -6.553310615491711, -0.2413815427177468, 1.2991605902531376, -1.8706716367186118, -11.30356030415038, 12.269395508131566, 3.047872238669868, 12.365453319682587, -7.3282323367570825, 2.1085338957130233, 0.9579992167368232, -2.371880917639146, 1.1607688814500534, -8.92424360786572, -6.268376049910103, -2.313963089163252, -2.849054886769892, 6.99367678378664, 1.7568659311135661, -0.7541959699036056, -2.2093992664659936, -15.113762846195073], 'changes_betw': [-8.98014659481694e-05, -0.00022814426484152328, -0.00013834279889324286, 0.0, 0.0, 0.0, 0.0, 0.0, -6.472177725999617e-05, 0.0, 8.090222157502991e-06, -2.5079688688249557e-05, 0.0, 0.0, -0.00022814426484149553, 0.0, 0.0, 0.0, -0.00028477581994401646, 0.0, 0.0, 0.0, -5.663155510250012e-05, -5.663155510250012e-05, -2.5079688688253027e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.6631555102520936e-05, 0.0, 0.00022814426484149553, 0.00011326311020501412, 5.663155510249318e-05, 0.00022814426484152328, 0.0, 5.824959953401598e-05, 5.8249599534002106e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 5.663155510250706e-05, 0.0, 0.0, 0.006145332750837257, 0.0, 0.0, 0.0, 0.005860556930893323, -6.472177725999617e-05, 5.663155510250272e-05, 0.0, 0.0002281442648415094, 0.0, 0.0, 0.0, -5.663155510254869e-05, 0.0, 5.663155510249318e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (77, 81): {'changes': [-0.1297334423012213, 1.154627792045659, -4.465461668163783, 2.3262437151826276, -1.0755819061764598, 1.750303689971986, -4.840022273503877, -5.639581334998013, -3.3762411732479265, 3.2184809838244064, -1.1265002230446868, -7.271847782891257, -0.9535887860054686, 3.8473500694441327, -0.16025353379048468, -0.6035060984771974, -9.509718957298162, -5.088327948105132, -0.5978164930111056, -9.820642642253205, 7.422808629351579, 11.493802023922825, -15.588907419545748, -3.014314915639204, -5.139601727602923, -3.036846337610285, -0.07452976154189628, 7.4997942703028, 15.31632917426424, -2.7487031861693936, -2.2844598681925845, -1.0197672514274814, 2.2629249344918776, 5.817710737467621, 4.006590668219928, 14.1483185312687, -0.05785735025058614, -4.39844294425103, 3.374891715770154, 1.5549481106821332, 4.143489139549899, -0.8554184447439752, 2.998342727127664, 2.666761557183918, 9.220451416250569, 5.152038042229037, -5.222669959513482, -4.590874199417826, 5.132752325614042, -6.288525090091184, -6.570802390492787, 0.59756993368255, -5.872670039871593, -59.78653296151688, 3.4563036991067975, 5.580191972293733, -0.6649719917655723, 13.642889399419147, -7.20627920940376, 0.7618098493372116, 20.21950024695255, -0.07450622512418192, 7.812511503556095, -0.5146602042188384, 0.6100721930165491, -0.7147872001136761, 9.287065534504222, 10.636273549857265, -14.1029869599829, 0.591188501436406, -0.9904783266030677, -4.083902675427197, -1.3136394119172223, -3.974331919444495, -4.212507994536907, -0.6261600909175513, -3.0710311404469195, 9.589643907836944, -5.28439570098999, -1.195497298256356, -10.714921899540968, 0.08588502443126345, 2.6760372133605586, -5.93982864685583, 0.2800553169360853, -6.0399511417153775, 1.3928807188573558, 5.92418659190696, 15.588859670863457, -1.9264836158409935, 10.839154890945252, -3.93516186037499, -0.09248675754326996, 9.735552807211434, 3.672353471289995, 4.864065527855388, 0.5067917695736242, -2.093343531154531, -3.8614298051238904, 2.7438284219672084, -8.194669900193674, -9.384188341244595, 8.92971708196697, -7.828556895707692, -3.8004300568149745, -4.115795707181746, -5.45758166666775, 0.6037278452309351, -4.004427595612242, -4.656962064103141], 'changes_betw': [-8.98014659481694e-05, -0.00022814426484152328, -0.00013834279889324286, 0.0, 0.0, 0.0, 0.0, 0.0, -6.472177725999617e-05, 0.0, 8.090222157502991e-06, -2.5079688688249557e-05, 0.0, 0.0, -0.00022814426484149553, 0.0, 0.0, 0.0, -0.00028477581994401646, 0.0, 0.0, 0.0, -5.663155510250012e-05, -5.663155510250012e-05, -2.507968868825216e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.6631555102520936e-05, 0.0, 0.00022814426484149553, 0.00011326311020501412, 5.663155510249318e-05, 0.00022814426484152328, 0.0, 5.824959953401598e-05, 5.8249599534002106e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 5.663155510250706e-05, 0.0, 0.0, 0.006145332750837257, 0.0, 0.005860556930893323, 0.0, 0.0, -6.472177725999617e-05, 5.663155510250272e-05, 0.0, 0.0002281442648415094, 0.0, 0.0, 0.0, -5.663155510254869e-05, 0.0, 5.663155510249318e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (65, 66): {'changes': [-0.2927377176444157, -0.46363441863504207, 4.851457876476076, 9.618115182807216, 1.1711584857184505, 0.7013412848607175, 11.09269917388437, 7.757867675021771, 2.014674298337944, -0.871743864777784, -0.5214662144410482, 2.0224704523762114, 1.281201569420091, 5.690441188563227, -2.101496080324331, -0.45986873539692397, 6.710321886691958, 4.850753523394161, 0.029067301607692997, 2.718207382987032, 30.89445018988684, -15.60010061690923, -1.4054133338613326, 1.6000929272061626, -6.123656653494265, -1.3833528523440606, -0.002470243257796767, 7.574051348768961, 10.10846281543229, -3.6829503580407987, 3.6311227357816165, 3.400451669505287, 16.114296030147045, 2.662171865021037, -1.3756198717038473, 2.639133264454898, -0.28557188172270287, 1.0485601968489533, -0.26719038078609714, 2.527473175917443, -1.6023175717567142, 1.5620815484404744, -1.1453997284692576, 4.196497814896475, -2.1832629173178475, -14.013011137401548, 1.249831805975333, 3.1030215293975942, 0.4201296925773832, 12.844710647605837, 5.281783421924189, 0.9140297251679215, -5.258873812764193, 2.3343368827056565, 7.844679935907436, 1.779401801590268, 0.2985635847303172, 13.176116659260657, 2.9497174198463085, 1.4137777112412522, 6.94324203246444, 3.263158581981969, 12.604899250224264, 0.7990892795869975, 2.400964249339772, 0.6468678815089177, -2.7997898277625097, -3.24189383082836, 13.390035201179188, -7.22831639700081, -5.525279796430169, 0.16367992305919188, 0.6148928523515735, -0.5456125977187014, -4.2912936378447455, -4.082633700404415, 1.620208200400981, -1.7894241712691041, 2.9763576924181905, 3.8423815609932035, 13.789523139943697, 13.378158952197595, 0.17947690259336468, 9.95384977887474, -10.218082969969345, 2.3958094029748622, 6.492867110371861, 10.071136130192883, 12.8499715110909, 0.15361394006672668, 6.587788007957386, 6.453754109677334, 1.1145785112713043, 3.7601709888977553, 10.968006848719085, 3.164076733913845, -5.667302771798191, 1.2317561864725377, -7.617544421634108, -4.7805723681644565, 2.630117418104021, 5.79711189854487, 1.5936792143148182, 6.635306388849926, -7.596610450208601, -1.1426659695695776, -0.8222247186833442, -5.912226567895971, 3.2474011275887165, 16.35684511929861], 'changes_betw': [0.0006710839279647218, 0.004420092875750403, 6.512628836788381e-05, -0.003032619775739112, 0.0, -0.00016989466530751077, 0.00016989466530751077, 0.0, -7.078944387811648e-06, 0.0, 7.078944387811648e-06, -5.8047343980061755e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.010747253369577525, 0.011200305810397559, 0.0, 0.0, 6.512628836787687e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -7.078944387783892e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00036810510816627024, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (67, 66): {'changes': [-0.18177149708085238, -1.5160095063122156, 1.2267997177163323, 2.6564243350439654, 0.13453975964836928, 9.023592944279159, -0.4050827934955521, -2.383007007961538, -0.7350044062495869, 0.537712531884381, 1.6575077716109519, -2.7395560896446796, 8.474071756752835, 7.906908800826386, -1.511874925991009, -13.530389729692246, -8.235236408151536, 26.017472505267335, 1.435370646059539, 4.620430932621787, 7.55835627273629, 49.90179516817099, 3.351594919956227, -3.489842400902603, 5.312226360638164, 3.909765638450409, -1.2246709403497675, 7.709729336515693, -0.17788522342743818, -0.18915489543579156, 1.264996243409982, 2.966553060508602, 4.490846881164671, 3.42819226452184, -0.7623497729892392, 1.507652547610121, 0.46039629524643644, -3.4194157211829435, 0.012221110127711654, 0.4067588062029657, -4.370788594660041, 1.8121724971339717, 4.945297156377592, 4.815834470151259, -2.219174374900547, 3.483423573454502, -0.04009138282161473, 2.65020258911359, -0.6497842519516581, -0.010212929349734168, 5.315533807388135, -1.2361876943031902, 5.2646459812873445, 0.8424889185435518, 7.437378088373919, -3.2485461805886757, -0.7694607812054741, -2.215650696666728, 8.388681398417532, 1.8451372988023493, -5.3784329546245715, 7.244882893547086, 17.606598839331895, -0.1513366635008424, -6.508189863241874, 1.3736331114111167, 2.8358672287813818, 11.785756686699528, -7.707771100788733, -1.1753557303943722, 6.564537891832458, 10.68012267606116, -0.773235575674704, 3.4388037856094513, 1.1206095764607085, 3.956761555520245, 4.501872211495126, 8.40232667682244, 0.19794182663741822, 1.1889575471510625, -0.6837824150600795, -7.104767395680142, 0.2719457428975147, 0.8056047506162827, -0.29114915363368254, 0.7081682683923844, -8.806137685879236, 10.585438242785528, 1.2699344374479438, 2.2444041503100536, 4.421537235263088, 17.934930761183665, -2.2561558332098457, -3.058759559450948, 7.505457169441115, -2.930061219249069, 3.510086067839609, 3.9760125205495314, 0.8782387378175542, -2.884661642908327, -12.15530944772803, -1.0623406359224816, -0.43036345234096984, 4.071262876189365, 0.8620758558718933, -6.050209751068024, -3.116039630706439, -0.7942493584071713, 17.935023118396487, 7.843563123429988], 'changes_betw': [0.00018405255408315835, -0.0009145996149053648, -4.2473666326869886e-05, 0.0014525993883791471, 0.0, 0.0, 0.0, 0.0, 4.2473666326814374e-06, 0.0, -4.247366632695315e-06, 3.822629969419192e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007176633820364715, -0.0071483180428134506, 0.0, 0.0, -4.247366632687856e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.247366632709193e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00036810510816627024, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (46, 51): {'changes': [-0.7352382887660838, -0.7101337914091808, -4.3984852475392415, -1.968654113260044, 0.8703626043732626, 1.6913885467981231, -0.5297604318724893, -17.372614078815758, 0.2211797886871878, 4.70003918976181, -1.382100128825499, -11.909893099223837, 18.538957435345964, 4.557672137286431, 0.659098577906871, 0.0750093039301305, -9.557339739128793, 1.283115136048508, -1.1466792438873057, -7.09772942087551, -9.696079797453919, 0.5093595146888958, 128.88865338524525, -45.73152603608705, 11.698335828749293, 4.694628106011621, -0.48097963602307203, 7.3465670147580795, -5.237111759322204, 3.385814953535075, 0.7463981203866581, -2.3134187776732915, 11.46806920873361, 0.29474191205963507, 2.282918328859715, 11.179690432853135, 1.1874645925733525, 5.475456028119723, -0.5860819633448529, 0.18305482439484422, -11.65890058175296, -5.979195465303121, -2.123924115981481, 7.808353842473707, 11.421048158293473, 4.001144923412085, -2.956455607648394, 6.738967709991982, 0.5943676718205211, 9.850421671947998, 7.164959207698132, 0.08067618562681389, -11.352200434092566, 11.797953719484553, -3.8784232975783652, -0.058179214561221215, 0.03030412662862858, -1.9225623619957446, 2.3567421588360844, -0.014715721684453342, 11.975585204428086, -2.5615966352363415, 12.218283955601237, 0.21595920284245906, 1.549068908443381, 3.2896356031779774, 2.8455733182612164, 14.255308103449408, 6.792922480079312, 1.926971432581709, -1.3103265369506971, -4.351501202199188, 10.412858826924662, 0.9301653948413389, 19.96052824173171, -3.262852144511669, -1.9857960696951835, -9.517539574709474, 22.38537796830198, 8.042760421405376, 6.717013901234168, 10.193104986167498, 9.540284018955106, 13.816247022989756, 8.461517948247774, 3.1144221577771987, -5.103423602451869, 8.46153400424555, 9.800199895884276, 1.5207801962893797, 2.7124821507472063, 3.6458880011403494, -1.940416746035794, -3.6127098718067145, 4.65801281889739, -2.0827806367913126, 2.475366935085617, 7.462682786398091, 2.986944571812984, 3.532400565344858, 3.03311765712823, -2.920590366086799, 5.868986655377455, 3.5326738123735026, 1.857688838923707, 15.638846141758506, 5.355602412791086, 6.670134341956128, 17.303393588361985, -2.9765019481222907], 'changes_betw': [-0.0023413102923807716, 0.0019372036956134542, -0.005941054641360327, 0.0, -0.0010577965470931483, 0.0, 0.0, 0.0, -0.0005917997508211037, 0.0, 0.007675598271928458, 0.006174053039496436, 0.0, 0.0, 0.0024327298027605193, 6.795786612300405e-05, 0.0, 0.0, 0.0025459929129655334, -0.00014724204326650747, 0.00021034577609501184, 0.0, -0.03774088636473958, 0.030483148067245898, 0.00016989466530750817, -0.012099736258757723, 8.777891040895325e-05, 0.0, 0.0, -2.8315777551232713e-05, -0.0009133860815818129, 0.0, 0.0010181544585214503, 0.0, 0.0015100399656975683, 0.0, -0.001823940585408551, -0.010627315826092595, 0.0003887351746680312, -0.00044617575198624415, 0.0, 0.007505703606620964, 0.007675598271928485, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00078232448263027, 0.0, 0.0, 0.0026592560231704643, 0.0, 0.0, 0.0, 0.0, 0.0007596718605893116, 0.0, 0.0, 0.0017203857417923824, 8.494733265375538e-05, 0.00034423895280163985, 0.0, 0.0016920699642414272, 0.001736566186107541, 0.001113619079979894, 0.0, 0.0, 0.0, 0.001275019012022073, 0.0, 0.0, 0.00017434428749413516, -0.0003442389528016468, -0.00016989466530750903, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003442389528016468, 0.0, 0.0, 0.0, 2.831577755125353e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -2.831577755123965e-05, -2.8315777551267407e-05, -2.831577755124659e-05, 0.0, 0.0, 2.831577755125353e-05, 0.0, 2.831577755124659e-05, 2.831577755124659e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (87, 51): {'changes': [0.05549130740352304, 0.6674691793136631, -4.775139467493169, 4.274716645251203, 1.9153906069060795, 12.01203073181324, 11.433190453272289, 9.23357787212359, 2.637562024677429, 7.889014085874123, -1.7406955386638288, -8.220445700453155, 4.403083574644995, -12.955558516669669, 0.5530952831234437, -8.092840306613596, -0.6912871847094664, -1.6089417753016733, -0.01209069026698728, -9.910067102912961, 2.398792728276618, 11.138855569231879, -41.63761999155301, 130.84465552366652, -5.580321152254356, 4.62800549873414, -4.099762876130029, 5.772178066671074, 0.6180926711211328, 0.0810580615141987, -0.5552447562348561, 12.52024149733802, -6.139771068141016, 5.890766757413473, -3.3652226186058485, 3.5462984470015044, -0.544989447102413, 5.013899783981699, 1.4641817613651256, 2.076056873501493, -3.4338007319235544, -2.926504017960326, -3.062984398275418, 23.499755751879007, 3.7003582263884596, 21.485824449662488, 3.8411206970657616, 2.436294757312538, -0.02907157223250323, -8.022579311436743, -1.1077194288267833, 0.6361740285701245, -5.115809730284127, 5.454809163651703, -5.600006274659563, 8.191677465031432, 1.2676118452719507, 3.9893505659251787, 0.18522448584968743, -1.6429209508927656, -7.348281079341518, 4.73128837898426, 5.723860275394486, 0.27787511455333913, -0.14981586614234033, -1.200293190419437, 3.5882054404658845, -0.3850000339501207, -0.41547863065267165, -6.174428129920457, -1.8052964921987495, 9.956315171520885, 17.04075547507793, 2.004626290703783, 6.355158549519729, 3.5650234472420266, 0.6081670383547504, 10.215666983422068, 2.0907566983360653, 20.62925794680828, -1.5993929196845897, 3.7598683695269415, 3.1132137047533206, -0.5419228136981076, -1.4990078825044577, 0.17811645022845823, 1.1570251013341135, 10.640952291594715, 7.323300197971747, 0.9050372355587086, 6.587668325936761, 6.5995630059299515, 4.249784866576817, -0.7473132112230019, 8.438923941995995, -2.0970322798821712, -3.7020190520698577, 0.2252395308012325, -3.5072866898892414, 9.614194831212956, 5.027212501057903, 11.682500374656996, -4.167274711256283, 7.31510504261459, 0.6762969843358917, 10.761952559239745, -0.6707111119274316, 18.64342980483798, -0.6571219623306774, -1.2235790188553892], 'changes_betw': [-0.0023413102923807716, 0.0019372036956134542, -0.005941054641360327, 0.0, -0.0010577965470931483, 0.0, 0.0, 0.0, -0.0005917997508211037, 0.0, 0.007675598271928458, 0.006174053039496436, 0.0, 0.0, 0.0024327298027605193, 6.795786612300405e-05, 0.0, 0.0, 0.0025459929129655334, -0.00014724204326650747, 0.00021034577609501184, 0.0, 0.030483148067245898, -0.03774088636473958, 0.00016989466530750817, -0.012099736258757723, 8.777891040895325e-05, 0.0, 0.0, -2.8315777551232713e-05, -0.0009133860815818129, 0.0, 0.0010181544585214503, 0.0, 0.0015100399656975683, 0.0, -0.001823940585408551, -0.010627315826092595, 0.0003887351746680312, -0.00044617575198624415, 0.0, 0.007505703606620964, 0.007675598271928485, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00078232448263027, 0.0, 0.0, 0.0026592560231704643, 0.0, 0.0, 0.0, 0.0, 0.0007596718605893116, 0.0, 0.0, 0.0017203857417923824, 8.494733265375538e-05, 0.00034423895280163985, 0.0, 0.0016920699642414272, 0.001736566186107541, 0.001113619079979894, 0.0, 0.0, 0.0, 0.001275019012022073, 0.0, 0.0, 0.00017434428749413516, -0.0003442389528016468, -0.00016989466530750903, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003442389528016468, 0.0, 0.0, 0.0, 2.831577755125353e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -2.831577755123965e-05, -2.8315777551267407e-05, -2.831577755124659e-05, 0.0, 0.0, 2.831577755125353e-05, 0.0, 2.831577755124659e-05, 2.831577755124659e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (51, 39): {'changes': [1.0158226622983406, 0.8315365110488067, 34.2977176654501, -1.6538719977399836, 0.7644368491309308, 6.682432177708961, 1.150881663003588, 7.249411661208455, -0.6554728347150345, 15.045377154528268, -22.733844594294, -49.54922517430586, 7.3263663455333585, -0.006308109338220902, 0.7078777285295672, 0.14351329558982684, 0.5123502317770772, 7.975232261225557, 0.06485205812000672, 17.948455896212323, 5.129478981767747, 1.8189264534633196, 63.99186783882561, 80.23192902080396, -70.32872150205398, 180.3549871553669, 2.407349207644991, 5.954021062148996, 8.438499915137868, 6.066279070407816, 1.7522592624423936, 13.265234811711878, -1.0172942288184998, 4.444067899220499, -4.770107790497427, 7.89286762190909, -0.9488636872101033, 116.9075304172987, -3.376640598840737, 0.6539545776363198, 8.072094780582205, -31.01640504518256, -29.96380279557991, 2.007079699969495, -0.6762795562545989, -6.772263607486934, -1.2983957848279175, -2.5884288746310347, -3.473996716295204, -0.7128263056785045, 2.8714913431837203, -1.4587720876880468, 1.4672232208731373, -6.549271249518654, 9.286813060306258, -3.3781138661033197, 0.5696214610259531, 7.826663448832903, 6.804547187964658, -1.1566815998017361, 7.8451745297202535, 7.542400522790558, 14.697512303149523, -1.3657971320622728, -2.087789586534342, 1.5497222571065024, -0.48716614206443865, 13.766888192270528, -10.504465005168726, 7.344786962649309, -6.357343726789452, 4.370836265514754, -2.030856141374528, -0.7773122929159513, 4.591406848246862, 11.939108456139053, 6.314965376776655, 16.853791658922603, 18.5122537014972, 18.67262716952996, -1.7720438954407314, -5.518028695559849, 6.190421729971206, 7.887759363637429, -3.5877809658208832, -3.485587347682042, 1.106807307193833, 14.266793835678612, -0.34577563280245727, 3.242843901635524, 14.451317593104761, -1.2610671566577025, 7.532645029185829, 0.6492228018824449, 13.450214420325182, 14.358859338143276, 11.789943784101354, 3.3156054605042584, 15.879840154576158, 18.08256776247495, 21.67661662689173, 9.582245238750659, -0.1225672106024831, 3.816500191563023, 2.530897047546148, -5.318233756469766, 3.878791497086894, 13.034910681253052, 10.186941862876324, 10.101340544304207], 'changes_betw': [-0.005079445981586439, 0.006326149216057175, -0.03884965131142497, 0.0010250311473553042, -0.005786531398152106, 0.0, 0.0, 0.0, -0.0011354626798051604, 0.0, 0.056777988123553794, 0.032737083960325555, 0.0, 0.0, 0.014183777486529853, 0.00019821044285876126, 0.0, 0.00016989466530750947, 0.01548630325388739, -0.0010533469249065573, 0.0007370192385482907, -0.0001868841318382608, -0.02865152177078783, -0.02865152177078783, 0.01801449767810625, -0.07310324741517404, -0.0004785366406161451, 0.0, 0.0, -0.0007928417714350294, -0.005358963157128263, 0.0, 0.004925731760594104, 0.0, 0.006162322217368094, 0.0, 0.0046292251185214295, -0.06166367328446841, 0.0016402925424332249, 0.002011633739462393, 0.0, 0.05592851479701634, 0.05660809345824633, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0029767972428522554, 0.0, 0.0, 0.014155461708978495, 8.494733265375452e-05, 0.0, 0.0, 0.0, 0.004528501852660766, 0.0, 0.0, 0.010005582253288708, 0.00025484199796126355, 0.0012616701454622181, 0.0, 0.008524667087358384, 0.007238321764315631, 0.004435059786741705, -8.494733265375452e-05, 0.0, 0.0, 0.005460090934097023, 0.0, 0.0, 0.0005820914842321577, -0.0012616701454621904, -0.0005946313285762825, 0.0, 0.0006795786612300431, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0006795786612300292, 0.0011767228128084437, 0.0, 0.0, 0.000679578661230057, 0.00011326311020500718, 0.0, 0.0006795786612300396, 0.0, 0.0, 0.0, -0.00011326311020500024, -0.0007928417714350572, -0.0007928417714350364, 0.0, 0.0, 0.00011326311020500891, 0.0, 0.00011326311020500024, 0.00011326311020500024, 0.0, 0.0, 0.0, 0.0, 0.0]}, (55, 21): {'changes': [2.147201052193248, 1.0311314735980055, 9.532101819820028, 6.6577054224407135, 41.89070264040518, 10.632383025892238, 12.293194093992383, 18.638254843273558, -11.100886743567113, 0.6402042551524687, 1.8820563817994582, 8.348074746947844, 20.19253475554291, 14.477515140496962, 0.34606835279062764, 13.558298272481295, 16.45874874658847, 3.5872058445769994, 1.9527782577554262, 8.99903654451981, 8.435596591448956, 16.792415770017925, 19.621277017544088, 22.444645378107097, 20.699306390231044, 10.083992989601995, 45.3680956756022, 7.887551787990674, 14.451753251773994, 65.23570443965303, 49.71636610903477, -0.297312864617993, 18.668980395804255, 6.699870099455211, -0.7985076551381454, 19.461103425175736, -0.21618566725709343, 7.649248229685185, 1.9521258981430307, -1.1142137254624487, 7.777204824193234, -0.46109908642128516, 4.6228421309054255, -4.179578157308242, 8.81702688302289, 17.69392712352453, 1.519759817752842, 10.61806927359278, 1.131866178116681, 9.304119831728372, 17.9930349967604, 2.321476217789211, 16.840657542615816, 20.782243141709387, 13.921540159268204, 22.588524603402618, -6.890977288001238, 25.22003813411189, 11.719674022992251, 4.611117710002667, 23.26271578874548, 13.215253828717579, 12.036066884630316, -5.826383172098836, 1.1914540488819299, 1.9561030328155269, 5.531532175830051, 13.84192439452147, 2.790883904230867, -0.49771368618161205, 10.200578868147147, 10.54241887777357, 7.517610979535874, 1.8523149267531096, 8.790081341907609, 13.735582321001232, -28.531382510225782, 16.10704738045456, 13.72647601546123, 16.794261192196103, 2.036846947688616, 21.5559321405365, 9.10036180378279, 0.3388511687142568, 13.372074629178286, -44.18801522568946, 18.184575964229168, 15.100684892638867, 17.405055430390348, -23.77803999493011, 0.3752687104138488, -4.001528199485676, -116.07238668085665, 22.969539083908558, 1.8968852575140431, 6.481487208052869, -55.68815828704609, -1.3831942016862229, -34.414433929831716, 14.975875931827034, 12.234773669419042, 13.954704738323443, 28.410997677051625, -0.13635106109788353, 14.704180474804843, 16.866086944188027, 11.357084410439143, 20.78802517656615, 22.961161528783492, 10.52002229173462], 'changes_betw': [-0.010162937074251976, -0.004481983075255325, -0.005070951248321284, 0.0, -0.07281159490639609, 0.0, 0.0, 0.0, 0.06103465851172271, 0.0, 0.005462518000744293, -0.0027620018445706496, 0.0, 0.0, -0.004481983075255158, 0.0, 0.0, 0.0, -0.004481983075255158, 0.0, 0.0, 0.0, -0.0019408442955843666, -0.0019408442955843666, 0.0, -0.003881688591168733, -0.1080388492468003, 0.0, 0.0, -0.055173292558613654, -0.0745671731145737, 0.0, -0.02135252334029094, 0.0, 0.004798310761613578, 0.0, 0.005897771952817682, -0.002465899713606154, 0.004798310761613522, 0.0036567804151901584, 0.0, 0.0035653609048104384, 0.0035653609048104384, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0047983107616135084, 0.0, 0.0, -0.004481983075255214, 0.0, 0.0, 0.0, 0.0, 0.06976886235296015, 0.0, 0.0, -0.00047125544067444514, -0.00035394721939064483, -0.0010800446580263232, 0.0, 0.08158948594728427, 0.00479831076161355, 0.004798310761613495, 0.0, 0.0, 0.0, 0.008224519845314936, 0.0, 0.0, -0.0001456239988350096, 0.001080044658026337, 0.000934420659191304, 0.0, 0.10983690112130479, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11119605844376486, -0.001080044658026304, 0.0, 0.0, 0.10677879714576965, 0.004374787631668375, 0.0, 0.10992184845395857, 0.0, 0.0, 0.0, 0.08702854230377166, 0.023742779476724415, 0.053984029901461096, 0.0, 0.0, 0.006979839166383511, 0.001500736210216333, 0.006073734284743462, 0.007178049609242263, 0.0, 0.0, 0.0, 0.0, 0.0]}, (55, 33): {'changes': [0.9863959078623523, -0.2043718406863455, 8.48649790024831, 6.443069461824081, 93.11569887250533, 27.877109373912873, 4.795175299136986, 15.22756184242445, -20.847492381191262, 19.470280893577353, -5.77118504766139, 10.966182791406482, 9.881643829757309, 12.99447312705189, 1.3258856545295252, 15.257149669680615, 16.964325439974687, 10.574235640089569, 1.8379087055664023, 15.887361043151913, 14.882009677870712, 8.137773365744977, 18.93332538453552, 23.502365713569873, 15.49218489789206, 7.94730163337519, 12.198143015550578, 17.813662859478313, 25.94209030529686, 15.259360964213329, 219.7975680838174, -2.9840698665789205, -47.300027537904306, 10.000107628863589, 0.6467417160208058, 13.001862778189434, -0.3697728098790556, 11.313551490150331, 0.9852315820975477, -0.3588232788050689, 11.997121606163489, -0.4881859176176704, -3.4602703608961747, 20.193630069521475, 10.616331520142978, 17.43148550484119, 13.393128420758899, 6.791138146338014, 2.6277451163149337, 15.173413951592693, 3.8416839861877747, 2.1359547123232474, 2.231689311859242, 27.442519450545433, 17.081413496715214, 18.690806327337214, -14.326253288727045, 16.086640323190096, 20.113018772722967, 4.219159153272955, 18.54657746397899, 8.369947294317427, 21.15656540842656, -6.239494059759092, -1.0684648920294322, 0.265669666237315, 0.912561796177215, 8.778510082855462, 11.105698957238815, -11.660835457583246, 17.59442890239103, 13.145021110333886, 25.082767270589272, 0.7195000824727629, 1.2203862903657239, 16.947253279665432, -0.4436987912912542, 12.145905147400526, 6.560549041488457, 11.257344517722686, 14.084455125472232, 17.644402121318194, 17.186479439255777, 8.647229221541949, 10.817343738494117, 1.4483878116908784, 11.564179715807683, 8.440309814695297, 11.835425862385705, 1.8077662812094175, -3.5273119806361706, 10.867059902959966, -22.458979407121603, 16.563813646349786, 5.453633019549272, 7.870200365988524, 19.142323470181594, 39.103504965725136, 80.68953115529831, 26.250955983248105, 18.66379870387891, -2.0018000206522686, 13.970705397496118, -4.480507545252294, 4.777200989819875, 6.301722970419178, 9.22122621328009, 19.659774890423904, 13.29827494233922, 22.45896809663401], 'changes_betw': [-0.0037081533258901955, -0.006941410611135568, -0.0077569050046114585, 0.0, -0.10540143682345512, 0.0, 0.0, 0.0, 0.10545078717861583, 0.0, 0.008734203841237423, -0.004388136498228244, 0.0, 0.0, -0.006941410611135235, 0.0, 0.0, 0.0, -0.006941410611135207, 0.0, 0.0, 0.0, -0.002944031843114439, -0.002944031843114439, 0.0, -0.005888063686228878, -0.03321157548986292, 0.0, 0.0, -0.02144920149507304, -0.12083070401113212, 0.0, 0.08684327621636491, 0.0, 0.0070684270990082065, 0.0, 0.009654062100545119, -0.0037360645923337626, 0.007807468893095831, 0.005566072844360281, 0.0, 0.005761047198356073, 0.005761047198356073, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007246816497581021, 0.0, 0.0, -0.006941410611135318, 0.0, 0.0, 0.0, 0.0, 0.11911112729155532, 0.0, 0.0, -0.0008373379933013236, -0.0005379997734737798, -0.0016584955422876066, 0.0, 0.11918272575764938, 0.007068427099008179, 0.007068427099008137, 0.0, 0.0, 0.0, 0.013122340339465723, 0.0, 0.0, -0.00024270666472501513, 0.001658495542287624, 0.0014157888775625706, 0.0, 0.01707441386340469, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01707441386340469, -0.0016584955422875892, 0.0, 0.0, 0.01707441386340469, 0.004374787631668375, 0.0, 0.01647978253482841, 0.0, 0.0, 0.0, -0.004969418960244644, -0.02144920149507307, -0.021449201495073057, 0.0, 0.0, 0.004176577188809606, -0.00019821044285876083, 0.004374787631668368, 0.0043747876316683615, 0.0, 0.0, 0.0, 0.0, 0.0]}, (55, 53): {'changes': [-1.1572525000505163, -0.47277516309490153, -4.588296325302828, -0.9707415492110982, -3.055910094730102, 12.162318703971664, 17.7057909062583, 7.595212941018929, -7.510017341032537, 18.694659070133923, -0.46277497095130116, -22.080331682899413, 9.928583058335732, 10.467136970888646, 1.4317304431608875, 16.542015650920945, -0.06505967075969465, 12.472456495229125, 0.1226769817580795, 0.8589208001378381, 4.172467406600802, -4.06518229512622, -14.593290758741944, -2.259698013544792, 7.5496718072895135, -7.814205485115437, 5.804102024259315, 8.268242791720184, 0.7922781571572841, 2.781167290490302, -7.240672006593179, 0.735539069499282, 162.51267987008822, 6.9403612419495175, 11.155974859799159, -2.628541950292231, -2.6053422209140322, -5.333169405693823, 1.4539212533548351, 0.09080622212679401, 9.97645604345746, -6.285342543264022, -5.9231071374971975, 7.628934187990183, 2.579603219787259, 4.438265980592405, 1.0710026418359035, 4.056896329032625, 12.06403402252235, 2.27411226983628, -0.5813678784047909, 0.9982031783109662, -5.0577977877788385, 6.830169327185047, 8.395399713957573, 23.151444271781884, -4.09508231982997, -13.845410323168636, 20.667950153783238, 0.9869038575372002, 13.36855005084766, 8.872025477484812, 3.9261435154373885, 1.0297740881470219, 11.547756223665743, 12.510857772700184, 5.0141978325889625, 4.806187281924451, -6.341474147202263, 7.728636665882362, 0.9234181219562743, -4.007154637301824, 8.84691114715261, -2.2111781551584784, 3.8067088065306223, -6.482976693430828, -0.5929224124783019, 11.621680284822332, 11.636299059684802, 3.8246977504641677, 7.568660166924815, 9.317657899621963, -0.014164935293536018, 2.078587056886107, 3.018473342433822, -0.9995373267851306, -0.7772992603806017, 10.078630351460617, 12.317114135524974, 0.8742976412470682, -4.739944809261829, -1.0707320101101345, -22.66849123679995, 6.989245357569644, -5.1592758241121714, 4.37879645756081, -16.006398367695795, -1.2600662454291154, -9.530611288394823, 3.6248252158194987, 2.4580845360137573, -1.8325453750551333, 11.605072425345725, -0.08273802753839732, 2.3637228271209665, 4.784950070787033, 10.076857276314797, 3.7464564093860986, 12.839233942885926, 8.927547411269984], 'changes_betw': [0.0509837710143522, 0.0006472177725999062, 0.013508243936378492, 0.0, 0.0436528162063331, 0.0, 0.0, 0.0, 0.029182240344319876, 0.0, 0.005633626199375394, 0.010123699496788156, 0.0, 0.0, 0.0006472177726002115, 0.0, 0.0, 0.0, 0.0006472177726002115, 0.0, 0.0, 0.0, 0.006754121968189218, 0.006754121968189218, 0.0, 0.013508243936378436, -0.02117737003058104, 0.0, 0.0, 0.00077868388265942, 0.036633334951377786, 0.0, -0.06606192256039352, 0.0, -0.018967930359367627, 0.0, 0.01640737504651857, 0.01356487549148093, -0.0024286846916815874, 1.6180444314978226e-05, 0.0, 0.004812468650389146, 0.004812468650389146, 0.0, 0.0, 0.0, 0.0, 0.0, -0.017770172968949727, 0.0, 0.0, 0.0006472177726001005, 0.0, 0.0, 0.0, 0.0, 0.01741501221623548, 0.0, 0.0, 0.0006775561056906604, -1.4157888775625897e-05, 6.876688833876154e-05, 0.0, -0.03658236655178543, -0.01896793035936764, -0.018967930359367682, 0.0, 0.0, 0.0, -0.004490073297412758, 0.0, 0.0, 9.708266589000467e-05, -6.876688833874073e-05, 2.8315777551251795e-05, 0.0, 0.0031005776418620568, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.010066258919469925, 6.876688833875114e-05, 0.0, 0.0, -0.0013166836561331885, -0.0026050515347151304, 0.0, 0.014313625552157663, 0.0, 0.0, 0.0, 0.017173519084834082, 0.006979839166383506, 0.012968626118473221, 0.0, 0.0, 0.002888209310227657, 0.0032846301959451804, -0.0009061048816400505, 0.0013025257673575721, 0.0, 0.0, 0.0, 0.0, 0.0]}, (21, '21_17_1'): {'changes': [1.5771531309790898, 1.4382876870488985, 2.393954818149382, -1.3907365542403909, 17.592033045763422, 3.7334823301274582, 5.4808682644084, 6.171139859867424, -9.845754704580898, 7.889705264144538, -1.7025184924939651, 16.611298178287598, 16.775894895125475, 1.0799157002448965, 1.6094942987677392, 2.592806227608719, 17.443960627329318, 3.8847402066954544, 1.452701733100664, 19.97868744740029, 1.4294388902700206, -4.345274176787797, 11.295432073633549, 6.113358126493523, 11.042640772549106, 6.971183684434237, 16.12406325471477, 10.385984826209494, -2.017678314114164, 63.71860620147493, 28.015270447010167, 14.573362751382888, -5.113780748903224, 19.525723850763598, 0.39948248395894126, 2.6917785019520863, 1.0538674181068544, 13.838198845316953, 0.06368547221022425, 1.3458169016595818, 8.003003827877592, 1.9292977219896699, 1.1430408699455938, 2.7885966950672696, 11.506543394269812, 14.405802994894344, -0.43655093989701754, 8.880181964155199, -1.6968794733325154, 3.8215696923539326, 11.924121797720375, 3.214676609714644, 7.477976277196234, 11.597160058187853, 14.973757940304495, 3.889140437523622, -6.936583826529432, 2.833696889178981, 13.140684796046173, 2.154209270822257, 16.76890203107493, 7.923104633777143, 1.8432201742264738, -2.8223136731993073, 1.4551391995578769, 0.6361595258301307, 7.441962554241314, 15.072413065859564, -4.3280003154430915, -3.7578827073183163, 16.662529253796635, -4.241692371811212, 6.595132836287604, 8.262289150438136, -5.292396899630887, 18.36529833654302, -20.2356578234129, -13.348445333516139, -6.635035548330563, -3.3932324117512564, -0.5803598963129275, -2.768035844787221, 2.5880914137269997, 5.9323996223789095, 7.507637369716093, -26.097558748483195, 1.1189896523362393, 1.6380675674589043, 18.89248774977952, -11.234666071042916, -4.576783778631665, 2.4481971929396877, -76.9857258034588, 7.132324113482639, 4.485458680745666, 17.17766363023054, -18.926867124391407, 141.80246725806, 70.16429207197271, 6.24133866180091, 15.499162981206524, -5.28431826739282, 13.676167925733665, 5.25074942222642, -3.634137290076694, 5.336812032192199, 5.1048282483250205, 15.267569680100223, 16.23189280854487, 13.887746300301842], 'changes_betw': [-0.0036519262818956455, -0.002135818649580179, -0.0020678607834570956, 0.0, -0.03707870168114813, 0.0, 0.0, 0.0, 0.03249235474006118, 0.0, 0.0025192951798456015, -0.0009206672815235441, 0.0, 0.0, -0.0021358186495800124, 0.0, 0.0, 0.0, -0.00213581864958004, 0.0, 0.0, 0.0, -0.0007790883937672938, -0.0007790883937672938, 0.0, -0.0015581767875345875, -0.04008098312379654, 0.0, 0.0, -0.055173292558613654, -0.03815470122809568, 0.0, 0.005435820267624555, 0.0, 0.0020524893613579065, 0.0, 0.0028720288659125903, -0.0008219665712020374, 0.002052489361357851, 0.0018607510962250773, 0.0, 0.0016415060757568323, 0.0016415060757568184, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0020524893613578926, 0.0, 0.0, -0.002135818649580068, 0.0, 0.0, 0.0, 0.0, 0.0361022118667379, 0.0, 0.0, -9.101499927191559e-05, -0.0001840525540831358, -0.0005299095513162724, 0.0, 0.05333195799556656, 0.0020524893613578787, 0.002052489361357865, 0.0, 0.0, 0.0, 0.0034399624613691976, 0.0, 0.0, -4.854133294500407e-05, 0.0005299095513162932, 0.00048136821837127877, 0.0, 0.05615018688413184, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.052752293577981654, -0.0005299095513162724, 0.0, 0.0, 0.05581039755351683, 0.00709310227658852, 0.0, 0.047400611620795105, 0.0, 0.0, 0.0, 0.01907067618076793, -0.04761297995242951, -0.016012572205232757, 0.0, 0.0, 0.005620681843923435, -0.0012175784347038168, 0.0074328916072035345, 0.007178049609242263, 0.0, 0.0, 0.0, 0.0, 0.0]}, (53, 56): {'changes': [-1.7792840398796397, -1.3733291960788137, -2.8134671539809446, 7.722522936159407, -10.173299928829216, -4.784386201194849, -1.9677349847872279, 9.779065528494016, -2.1082032291100177, 4.407368479987184, -5.377197328022504, -25.81767601005471, 11.141713350808516, 7.8263626873429075, -0.1676188551821376, 16.593471119833623, 2.8831382291513705, 8.751603915127788, -0.45691993101238637, 7.189524036432829, 5.738274099550921, 11.12580044597621, -18.856034053923224, -13.540451628695095, -5.210588928659803, -15.256504214584155, -1.4696177870091347, 4.51958827613214, 23.520031631580053, -5.995430136126501, -6.6363081725301, 19.086830283192853, 170.8817019367758, -1.9272321775623027, 13.317332520701378, 5.201144138477105, -0.9732800595387054, -11.023532669725668, 0.5788083329953366, 4.487680046991613, 22.6202650487958, -0.603951558273792, -4.408573252168722, 13.35114086684706, 5.552917139153919, 3.877043362460597, -2.8615001905343007, 3.9951340648634925, 16.467187645581973, 1.4696890038943025, -3.4374652862686617, 1.838914282166705, -0.6423731514512667, 12.910358633262632, 13.2440030294145, 19.36413645875416, 2.253840258998217, 13.473963611229323, 7.803808852275125, 1.3654445319995716, 20.770208146234978, -0.8653715730626885, 10.09142208221536, 8.57456522284448, 14.280688182486116, 13.156450808057876, 5.468722346346311, 34.94668988691433, -4.260463863044208, 19.303768746663934, -1.2267605667533417, -0.6095941869120907, -2.730508938712177, -2.180356596480891, 2.1876124276137148, 7.3401559922865545, -0.7011889998627936, -13.539664422545911, 13.32529897301356, 9.888020102304381, 1.2274884016632086, 10.786438625690607, -0.24539926322231054, 7.203471002762683, 1.3544719580367541, -5.3935978058251095, -11.255835794467174, 15.681783719783652, -10.246275737776102, 0.7961044224325491, 4.453838953626075, 10.134455031659456, -38.93372114277636, 9.1720507668187, 2.578931433353546, -2.8052883492005662, -27.981831279069, -26.5357569702919, -34.25201365515359, 4.519813399085422, -3.1926010541483834, 4.198949183548592, -12.62847020423709, 13.925709319724007, 6.168493754365784, 5.837820613373879, -5.385334656010031, -4.759071977048222, -2.731029404987055, 13.870548429630759], 'changes_betw': [0.05599566364092384, 0.0013098069672991541, 0.015699885118845444, 0.0, 0.05614007410643504, 0.0, 0.0, 0.0, 0.022768716728961386, 0.0, 0.0054977104671294075, 0.011644256751290358, 0.0, 0.0, 0.001309806967299515, 0.0, 0.0, 0.0, 0.0013098069672994594, 0.0, 0.0, 0.0, 0.00776499522676892, 0.00776499522676892, 0.0, 0.01552999045353784, -0.0040180088345225795, 0.0, 0.0, 0.003412051194925811, 0.04963027684740225, 0.0, -0.06606192256039352, 0.0, -0.021686245004287785, 0.0, 0.01740125883856755, 0.01541672734333284, -0.0033631053508728914, -0.0004425351520153109, 0.0, 0.004846447583450625, 0.004846447583450611, 0.0, 0.0, 0.0, 0.0, 0.0, -0.020361066614889248, 0.0, 0.0, 0.0013098069672993762, 0.0, 0.0, 0.0, 0.0, 0.008495542287591251, 0.0, 0.0, 0.0008389560377327909, 2.8315777551251795e-05, 0.00021600893160527573, 0.0, -0.053274517418248235, -0.0216862450042878, -0.02168624500428784, 0.0, 0.0, 0.0, -0.006146546284160982, 0.0, 0.0, 0.00013106159895150734, -0.00021600893160524798, -8.494733265375538e-05, 0.0, 0.001401630988786956, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009556574923547403, 0.00021600893160525839, 0.0, 0.0, -0.003355419639823304, -0.003199682863291413, 0.0, 0.014823309548080192, 0.0, 0.0, 0.0, 0.018787518405255423, 0.00944331181334239, 0.01509230943481709, 0.0, 0.0, 0.002973156642881414, 0.003709366859213954, -0.0014157888775625724, 0.0010476837693963077, 0.0, 0.0, 0.0, 0.0, 0.0]}, (16, 28): {'changes': [-0.8111581675969539, 0.6992381365690683, -0.8755644711776398, 3.0383090992761623, -3.3722612248068486, -14.022240788347546, -7.447628394568085, -1.6804439014882746, 4.464073560417745, -6.667139761839394, -0.5431384774068988, 0.3430947175510539, 3.6888297414843123, -6.929414524799, 0.6354273485080384, 6.5722657449887265, 9.325161029858293, -0.578174336069651, -0.7174233248617128, 10.7690256128937, 0.027233726407388303, -7.354813699333931, -6.599805930072989, -1.442004738737353, -10.18217809243015, -0.219628424254239, 2.115792427880379, -4.495443169381872, -16.881899651421833, 5.898859841625921, 1.024751763402577, -3.1583722756622876, -2.2095638962015016, -0.8857691344734349, 2.3806966056572207, -74.13040930354998, 0.5071770313770685, -3.6855870886742252, 1.2035544194437975, 2.6125081079599184, 12.384435587921871, -0.584534740029909, -1.8186706789791316, 3.189684683466396, -8.567307285595746, 5.005155232038476, -1.06554035948146, 5.4723630153252, 0.6873558574626557, 5.311454704353878, 7.465336383660571, -0.4219316421906285, 5.05861160746548, 8.453272809158136, -4.492832187723025, 2.6418884663832785, 2.0767839092522706, -11.055739708579267, 1.5282332995843433, 2.565613032294145, 0.30574118630104863, 2.9017606481924076, -4.812205188681389, 1.794220073076108, -0.6826652000462801, 1.1865333404344724, 2.318164897064605, -1.952748646522764, -4.980411814616957, 1.1121188828816884, -3.007156837335856, 4.2208371099285955, -4.831975273687476, 2.21328512316461, 3.4529291848967603, -2.872307286534692, 2.451736150913561, 15.706580592163334, -1.6260110582558696, 3.7013035798700855, -9.444992595966141, -9.783936225622426, 2.3008086093549878, -1.7282196638648486, 2.02777878617502, -2.8800384490675697, 0.2043634498186293, 6.202779747415633, -10.939311164985781, 2.0242199870154494, -3.2659509827410886, 4.711743586211469, -1.4425728118341112, -8.162205226999447, 2.1418010327829506, -5.364275917191804, 3.0750272666466287, 6.1374094275366815, 4.6247472723890155, -5.664909912397036, -10.267556697708244, 15.672768142072272, -2.1833387903563732, 3.736236298680211, 9.308240457010697, -6.916534455816844, 4.079697195254994, -8.819238337149585, 6.143477966613062, 11.981409007201421], 'changes_betw': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01834862385321101, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (16, 20): {'changes': [0.19656479461640508, -0.869115127533238, 0.22355469461534483, 1.5803209125979123, -1.3646026630669539, 20.35421593439989, 0.4101048260837672, -0.6091849722104712, -1.1690906625913016, -6.66924002519238, 7.4340068438317815, 5.507975504948348, 8.287732019980012, -9.287199708880735, -1.4783451921191073, -4.5394683546143995, -0.6791909462040167, -6.928266192336622, 0.36669753390927795, 5.588326094502776, 1.9218835098001534, -0.5160695243729947, 4.104530458755818, 5.112369101908683, -8.136187539368876, -9.876981741615694, 0.5238084250052069, 12.960215633475002, -6.938837295730451, 0.3525952663399323, 0.10201894810438716, 3.5987712010937116, 0.15591720967253764, -4.376139364881112, -1.3979180420827362, -1.5167914093475474, -0.38956430785241025, -1.8716331835882727, -0.33804109826903783, 0.022484026338489116, 2.4161014332557897, 3.0058704639502167, 0.7578992689520874, 1.6216313684078045, -6.537398987951093, 1.9037235135767787, 3.307516885265258, 7.8450079926317215, 2.5427775034627587, -3.626421991810929, -3.5200482524432175, 0.919766073625091, -6.957708618172603, 1.323628554652032, -17.931943378099135, -0.8635208435832737, 0.25280219633085466, 2.6998712374305853, -4.77845696560928, 3.583490186730735, -13.166809098356566, -1.791948079771828, 7.977596257425716, -0.5332630490635175, -0.7744555803399038, 1.8149990135451333, 5.100587978019206, -12.456548134596773, -8.53184595122076, 0.8694300213680748, 4.662263504487839, 2.8625499083193517, -0.5606657994250384, 2.2000553822121276, 0.3884879570497617, 1.9586277148113993, -4.740971875676252, 3.511797979296148, -16.447810188964866, -8.5893546324983, 1.4469415141737727, -3.6754034960225113, 3.010176491799058, -3.446684603003831, 0.44566448887016463, 2.287037926825235, -3.7440613460302643, -0.8288368845700802, 8.585829587051649, 1.7141557688388787, 1.5347734098758679, 2.5436319266321163, -0.662121602115775, -6.720426468082678, 0.4562688926746432, -13.672106847745823, 1.8174266113425688, 2.6940384041925682, 1.746056581162577, 14.750738619385686, 2.5536504512076306, 2.6082841884673655, -2.0188679330888135, 6.740174161017677, 1.6416407361494976, -3.7220825614789135, 9.703053297692378, -11.670993670352004, -13.600074014557293, -7.147122914139231], 'changes_betw': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0001698946653075073, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (28, 20): {'changes': [-0.17916271514918947, -0.7078281888144318, 3.2390772398757406, -1.7091522813275475, 1.3737288004987178, 3.608585473150981, -4.662914832764983, 3.669140755093764, -1.6013169638374336, -16.802121943462197, -1.1428287119623093, -12.24918775771934, -1.1178767209839293, 5.550988938580929, -2.2431236985138128, 0.8120820387825063, -5.920333863358451, -5.670924671601284, 0.35871382038058286, 16.93274338273207, -0.6802264643346092, -7.072303738772888, 0.34803919670025607, 8.850409363453963, -10.83690077467324, 1.9014623182211494, 3.964865632853787, 4.854112593654293, 1.4765008457676458, -0.8134813343768457, 1.9637683833910273, 3.5797463192733687, 2.4912980991030906, -67.58377146584814, 2.654880522473185, 1.6775857033316015, 0.19627167507945842, 4.055878730412417, 4.470681977208052, 0.3931437499130297, -0.8654611949904165, -6.262179377102385, 1.4688410681581416, -7.357667719424114, 7.398521162404478, -5.5084817643093515, 1.8465467811220861, 2.727706624191697, 4.420379430778233, 1.4339137294094257, 5.623872517320763, -1.0730357909996187, -7.423995631800437, 6.256170382374705, -8.65644619379583, 12.715595039487255, 1.167248242852665, 18.241396759157908, -4.727054997302616, -0.8651562878880128, -7.669650776612286, -3.8230890642850994, -0.967099473289295, 0.7325831123271449, 1.6567948273780715, 6.627291653116977, -4.653306609597934, 20.230703335604574, 4.3352574040494005, 1.8412304507392747, -10.518259411733595, -7.147331867314961, -3.607235065495445, 1.5524627134447258, -5.049885795275912, -1.5872414679051303, 2.1889151381968617, 7.033925066281057, -17.35647913984883, -9.423231115968576, 4.438316751428658, 4.8391707262977945, -5.0540454534210255, 5.753913536580654, -2.9690789622285365, 4.416302140206497, -6.472784635366139, 5.268555042300989, 3.7692430954882923, 3.0976864494572, -2.9796496497831413, -7.074941693301753, -5.1490354392774975, 1.260553324520231, 4.274677661230754, -4.914871019425874, -1.0210233878827921, 1.396241841553504, 11.676891725528947, -0.6760691086843167, 8.611436066149565, -8.421351175342267, 0.5887353597688616, -9.518674321249819, -3.081908939561856, 3.525679625440489, 7.328495911325518, -13.91750311584434, -1.5261364373621973, 6.11603173905479], 'changes_betw': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01834862385321101, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (28, '28_56_1'): {'changes': [0.00881127985760699, 0.819207955288924, -11.44443763895476, 11.002442857150825, -2.12353350087033, 9.306532012982387, 6.6238081631992145, 15.138355928729652, 5.965756844372272, 32.280071621234924, -35.10817495827512, -28.178718684511296, 8.20253700876441, 11.673714330769144, 3.0813519698106226, 9.545215594950577, 13.008374899792159, 12.64556491637319, 2.0500902908185488, 17.96085653569574, 6.287591079993717, 10.779784330009477, -33.88376727467748, -23.148027468957764, 27.579213690350684, -23.06881460789924, 5.386983597649767, 18.18783749955628, 10.548543861251403, 4.625833249776278, 0.18045953296112316, 20.300766994803894, 25.99373509910943, 15.41831565559653, 94.56647020724246, 27.077668094954845, -6.958740774389579, -14.312505997103017, 1.6189058502812301, 1.615083865588396, 10.23766719358531, -42.02605185912416, -39.71352503215263, 17.88848786027654, 16.666635141531742, 4.483262017271102, 4.951342080532228, 8.938801094877917, 55.98313333329095, 27.96946859427618, 11.140258310189154, -0.534931207247709, 13.844248879794975, 11.520588450059392, 11.167868029516967, 16.709730938665473, -6.896841744038216, 28.695373334744914, 15.183957798882489, 0.8569598793342621, 6.078045036549042, 9.766979562525364, 19.3773387771933, 5.609736288856592, 221.5197908975167, 63.75044742358379, 19.578914804655682, 2.682432997514752, 23.708251322120276, -66.64585276709357, 4.6195052140141115, 11.718554860384813, 10.852871882834194, 14.85415954393224, 22.974722336734544, 21.906300406802984, 6.850520417543905, 18.651256846681463, 27.379198624397304, 13.48088812819526, 17.891423925122467, 29.344322359724913, 27.19452649691121, 10.987891580607794, 15.600875026939747, 7.244059969229511, 6.056278129364074, 14.43769836037518, 24.946638328230193, 7.85519077277889, 17.640303375593476, 28.51350495608861, -0.47760001237338656, 18.259812405021535, 8.475398345978647, 18.098948161897397, 13.722605023807944, 12.969060037606965, 6.787870103229125, 8.621902887966769, 8.748394652658135, 17.929478777539572, 20.044350567182562, 23.6419670488188, 24.891832744419645, 18.423627819159577, 9.019543041265536, 14.588121517676512, 12.831697483288167, 15.246366978448123], 'changes_betw': [0.0196353736873619, 0.0003272494862712194, 0.025635891461579505, -0.000566315551025029, 0.022115835800851108, 0.0, 0.0, 0.0, -0.012784573564390084, 0.0, 0.09267915796967784, 0.010653204536996575, 0.0, 0.0, 0.001479701632606828, 0.0, 0.0, 0.0002831577755125158, 0.005047489604064426, 0.000271831464492014, -0.00011326311020500718, 0.00011326311020500631, 0.013504603336407635, 0.013504603336407635, 0.0, 0.02746225911363527, -0.0018263676520557104, 0.0, 0.0, 7.64525993883769e-05, 0.020136562950018602, 0.0, -0.022710467129427377, 0.0, -0.09446628804426968, 0.0, 0.10173697069721524, 0.026182385968318705, 0.02340946232383545, -6.310373282861215e-05, 0.0, 0.09460463084316305, 0.09443473617785547, 0.0, 0.0, 0.0, 0.0, 0.0, -0.033777082018672244, 0.0, 0.0, 0.005783699820397115, 2.8315777551250927e-05, 0.0, 0.0, 0.0, 0.05990283643188837, 8.494733265375452e-05, 0.0, 0.0023906606475414677, 0.000305810397553518, 0.0006917139944662906, 0.0, -0.042789184991019746, -0.12300859181593127, -0.05049188550717605, -5.6631555102503155e-05, 0.0, 0.0, 0.07944921767551741, 0.0, 0.0, -1.618044431500338e-05, -0.0006917139944662976, -0.0006512628836787913, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0006350824393637679, 0.0, 0.0, 0.0, -7.645259938836996e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 7.645259938838384e-05, 7.645259938836302e-05, 7.64525993883769e-05, 0.0, 0.0, -7.645259938838037e-05, 0.0, -7.64525993883769e-05, -7.645259938838037e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (28, 9): {'changes': [-0.07231464067305104, -0.2577946304375871, -9.903680613296586, 4.139452639915177, -0.4160801573680928, 11.894024596806503, -3.289688641855207, 11.515298646027475, 0.5260924205240016, 15.418749878212225, -30.708200815510118, -0.7129140217500094, 29.807151744942757, 8.329890470605903, -0.7003203382027401, 1.049891162795916, 10.11663539546879, 12.659463391929279, -1.3643933621522528, 15.35047384463499, 12.839168574234094, 24.244972933837488, -25.319545322778154, -26.268391043692674, 25.457556780166982, -15.457478680908835, 4.641101859271089, 20.98407719852375, -0.15852072797514438, 7.105158562802231, 0.4165080515320838, 6.194103015457472, 20.574330082433335, 8.168417536228674, 98.77311317793966, 9.48927499935968, -2.587300316146461, -13.853118144842014, 12.592442273480017, 1.7398891479153278, 5.690089037252221, -38.1742390284211, -39.90026212627988, -5.449782003149721, 10.68118132041343, 16.63429033149066, 11.856186608585801, 10.325685547666524, 143.08206978252966, 17.521651830872486, 15.598207102735671, 0.8304714776063875, 7.847598281041911, 17.74234785607527, 1.236473373846536, 10.66381404038691, -10.890316635264806, 5.866295340676118, 17.125623677523663, 3.47706278973358, 8.747557951392992, 5.317237574243535, 16.891031133621595, 1.2236593340486124, 83.18055236993045, 216.23592544339562, 21.368287856837668, 28.960484889281616, 16.72566063245202, -62.206074612568415, 10.689891044349451, 27.310789111259766, 18.384606584349655, 17.77934203987371, 9.781822440793405, 8.434207667859937, 3.4229397187432653, 6.849560260182159, -1.1973651432439283, 10.278525016443524, 14.144272252244662, 29.260107143281317, 12.223990198336224, 7.622260017026008, -0.5369692964357284, 5.133150880528831, 1.8096094051497857, 12.856604846995822, 25.239138400548654, 3.4132806593878726, 0.390814486769159, 27.06686676542867, 13.373642048308284, 28.019847382340004, 17.102888376432247, 13.858691070044927, 8.425167928696993, 12.22564353862937, 13.533290148053283, 18.65061819030859, 16.663543355855893, 13.178468184865835, 36.609090855296586, 12.10472179522219, 10.977393118705805, 9.106985547168023, 6.066190778931343, -0.004966241748149969, 9.117718330409872, 27.76869780560824], 'changes_betw': [0.03021131660275428, 0.010232108473699042, 0.017582884326003537, -0.0001415788877562607, 0.01778352183550963, 0.0, 0.0, 0.0, 0.009131838260278635, 0.0, 0.08647800268595375, 0.006873048233904496, 0.0, 0.0, 0.010382182094720221, 0.0, 0.0, 2.8315777551251795e-05, 0.00936281410287515, 6.79578661230034e-05, -2.831577755125006e-05, 2.8315777551250927e-05, 0.008586152775755237, 0.008586152775755237, 0.0, 0.01609630600456291, -0.0008069996602106111, 0.0, 0.0, 7.64525993883769e-05, 0.016823616976522168, 0.0, -0.017868469168163363, 0.0, -0.09446628804426968, 0.0, 0.06427519699690945, 0.013627170202093725, -0.02450083329288219, -0.0099339837871949, 0.0, 0.08560021358186502, 0.08644968690840255, 0.0, 0.0, 0.0, 0.0, 0.0, -0.08678421759461516, 0.0, 0.0, 0.0019440803844474608, 0.00028315777551251534, 0.0, 0.0, 0.0, 0.08398540523922776, -0.00016989466530750947, 0.0, -0.006290956749672283, 0.000475705062861027, 0.0012013979903888056, 0.0, -0.0007402553274112322, -0.0689820882481433, -0.10451838907496402, -0.00031147355306376714, 0.0, 0.0, 0.0790669546785755, 0.0, 0.0, -0.00027102244227626693, -0.0012013979903888299, -0.00116094687960132, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0008899244373250324, 0.0, 0.0, 0.0, -7.645259938836996e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 7.645259938838384e-05, 7.645259938836302e-05, 7.64525993883769e-05, 0.0, 0.0, -7.645259938838037e-05, 0.0, -7.64525993883769e-05, -7.645259938838037e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (52, 39): {'changes': [-0.2522707048520232, -1.4557783427790056, 13.580943499941341, 3.8160926724544737, 2.84074466411019, 7.18105223884794, 1.4101458094434633, 10.031544285343188, -0.4583182704424402, 8.566324802773948, -15.751555001505743, -46.37507313222498, 7.9118750989447335, 14.311789403506907, -1.665932689142899, -10.493058285478611, 16.128678971143728, 14.087962426878136, -1.2897136167443009, -0.418843127730014, -6.728911633420097, 17.721211555844377, 9.086772357288595, -0.9628060456336129, -70.10772950092837, 74.41189553519156, 0.684173702786012, 3.349263354872676, 14.5878156813788, 0.6679776605264749, 2.4197742929938, -2.0394463559330234, 1.9229372280469619, -0.040117775373005315, 3.710431861767759, -3.7384538680245782, 4.578582176181847, 106.94961140321985, 4.416736910837912, 3.0991242326758695, 1.4948086561715286, -26.25075587818933, -22.49005783031958, -4.610167129109755, -0.7729223388371338, 14.031395427672521, 6.291253447986392, 4.133331867024381, 3.622244840585452, -0.6553985314590136, 10.010542384253426, -1.4488669808464607, -8.102821090129254, 16.162735177852767, 4.350379864749584, 2.2582980891166926, 1.6915404998628887, 11.319470337862185, 13.710335943954306, -1.5609619889844453, -2.5846106321157265, -8.183866354869366, 4.09097391502587, -0.13220291504287118, -2.185941900591601, 2.160510839529877, 3.0780131758696996, 3.7324972178477083, -0.8267955269185165, 14.890194093435497, -3.9220648736992416, 8.758933972427315, -6.249317270011147, 2.273251177758027, 11.083144771510632, -2.6106364850622867, 6.7279863391328405, -0.6977825527977473, 17.750727980331646, 10.537360106726538, 23.08428919294238, 17.551830662210307, 11.742743411588663, 9.506877077535279, 1.8798091814641396, 6.35019185013897, -1.4946274406946713, -5.7893291616213105, -3.051636043108317, -1.016231509170268, -6.845229656134251, 1.8106715036088303, -0.11891431045702916, 4.274278652513601, 6.398895917467939, 0.05891899459535921, 5.357741040297498, 11.15379659041703, 3.536377723895157, -2.037616455017144, -1.0207654186539799, 2.3308735819981337, 2.242515915877391, 0.8804415809030957, 4.843033586359809, -3.2789719854636985, -4.06953987796544, 0.7725357724228274, 10.96853162973224, 16.7734041321865], 'changes_betw': [0.006667556591104018, 0.014671213371518943, -0.005279274468876882, 0.0008915424817565465, -0.002501901202206974, 0.0, 0.0, 0.0, 0.0073592705855703155, 0.0, 0.047559179975082064, 0.0261354626798052, 0.0, 0.0, 0.02096742876559396, 0.00019821044285876126, 0.0, 0.0, 0.019211850557416366, -0.000883452259599048, 0.0007127485720757887, -0.00016261346536575964, -0.010982476578806855, -0.010982476578806855, 0.022476255197967733, -0.03674578903936704, 0.0005408313512288987, 0.0, 0.0, 5.6631555102520936e-05, -0.0020743329611831035, 0.0, 0.00266046955649403, 0.0, 0.0016317978091678065, 0.0, -0.026857919518470302, -0.06166367328446841, -0.004589178518842135, -0.005985550863226746, 0.0, 0.045010759995469524, 0.046369917317929596, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0032526738184230908, 0.0, 0.0, 0.012444379722667182, 0.0002548419979612644, 0.0, 0.0, 0.0, 0.007303448052683487, -0.00016989466530750947, 0.0, 0.004273659854699588, 0.0004247366632687726, 0.001884617251589761, 0.0, 0.006938983544488242, 0.004066954678575457, -0.001454621943918613, -0.000254841997961264, 0.0, 0.0, -0.0002596961312557808, 0.0, 0.0, 0.0005254599291296541, -0.0018846172515897264, -0.0011043153244988113, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0016297752536284706, 0.0, 0.0, 0.0, -5.663155510250012e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 5.6631555102514e-05, 5.663155510248624e-05, 5.663155510250706e-05, 0.0, 0.0, -5.663155510250012e-05, 0.0, -5.663155510250706e-05, -5.663155510250706e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (52, 27): {'changes': [-0.1198620903589358, -0.11603719231730025, 0.1952121098104982, 6.133396547572318, -0.8481196474174553, -6.448899271195984, 6.861135720123002, 9.511572417437634, -1.2948569173347977, 3.6777063140019095, -0.23291994886760392, 10.425890885881756, 4.561330629382553, 6.063444619165978, -1.0044123908461344, 6.120226633871283, 5.605323809675838, 7.057180756700319, -0.441029697098422, 11.753754793333172, 7.362881152674845, -4.189726608256905, 3.047279917620898, 11.71444830821332, 1.5810759202572342, 0.8978260548255861, 1.008718544744525, 6.223992657951101, 14.648145240285174, 0.614982818480641, -2.201069729100091, 10.48365360445132, -4.396126553143176, 6.307571193204211, 14.128396159706249, -8.707029120836722, 3.4699580440234037, 3.5290196811302508, 10.701651441530565, 3.784569140302203, -204.4440507299479, -3.528558266148778, -1.67026928872464, 4.094544359508575, 4.667156973352405, 13.122561789529072, 0.6804004570619355, 2.1779394243946513, 20.944500940271503, 13.929860935562857, 1.5129020584877821, 0.520112250624905, -7.074229202613225, 11.687822998928624, -2.440016005388941, 12.337736726930842, -4.314564402502356, 13.862951405575217, 2.4496382216215693, 2.2866761874170507, 2.0203613117232067, 4.61365693785892, -14.040301965378205, -0.21224287590406377, 11.461968131985714, 25.030642748900355, 5.974422674352439, 12.64233567919598, 10.013540306740367, -22.294555504264167, 4.5128823532205615, 10.808066161696502, -1.3432762028014906, 10.896635155893051, 13.576087142836599, 2.261380752652258, -2.059308621534143, 11.375605052787279, 14.223501872532722, 10.2484154926982, -7.174778750902419, -7.358876495428547, -0.2900715419839912, -2.971713988931242, -0.3731450631697726, -1.5688362059385952, 7.517877920171202, 0.6781184178954618, 6.62355011526904, -7.887626580659763, 8.89410882690656, 8.000910763719958, 2.203789268018255, 10.936381493788616, -8.640061491736532, -3.1241566636938103, -5.701737484487069, 4.893099862412868, -3.064947487339566, 10.11004750132409, 5.5678682587454205, 7.686729490892645, -7.241120859098061, 6.730533859580191, 10.386964175762813, 3.9567457874088063, 6.8189973230701355, 5.61223402380665, 4.634022039464611, 3.6353702110068298], 'changes_betw': [0.006343138682588623, 0.005102503114735868, -0.00034585699723321817, 6.674433279936498e-05, 0.0006241606394511434, 0.0, 0.0, 0.0, 0.008241913822953584, 0.0, 0.011327929064932032, -0.0007863695937090735, 0.0, 0.0, 0.004962542271410686, 0.0, 0.0, -5.663155510250272e-05, 0.003348542950989325, -1.1326311020500674e-05, 1.2135333236254486e-05, -1.2135333236251017e-05, -0.0005056388848437635, -0.0005056388848437635, 0.0, -0.0015775933207125559, 0.0014016309887870115, 0.0, 0.0, 7.64525993883769e-05, 0.0018728864294613318, 0.0, -0.00036931864148989635, 0.0, -0.012532563144183925, 0.0, -0.014090739931718665, -0.002630940245619176, -0.025224908175978472, -0.004848470138990119, 0.1170129281750077, 0.010535087293497059, 0.010874876624112073, 0.0, 0.0, 0.0, 0.0, 0.0, -0.018571509473650147, 0.0, 0.0, 0.0002621231979030858, 2.8315777551250927e-05, 0.0, 0.0, 0.0, 0.024269452939177594, -0.00016989466530750947, 0.0, -0.0027061793116838068, 0.0002491788424510144, 0.00060676666181253, 0.0, 0.008988236816983075, -0.009134669838033738, -0.015873824895231634, -5.6631555102503155e-05, 0.0, 0.0, 0.012425772211704933, 0.0, 0.0, -1.618044431500338e-05, -0.0006067666618125404, -0.0005663155510250376, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0005501351067100126, 0.0, 0.0, 0.0, -7.645259938836996e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 7.645259938838384e-05, 7.645259938836302e-05, 7.64525993883769e-05, 0.0, 0.0, -7.645259938838037e-05, 0.0, -7.64525993883769e-05, -7.645259938838037e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (52, 36): {'changes': [-0.8217109061731467, -8.221738185681268, -20.818297551451813, 8.215487979393657, 1.2558865158016488, 20.10209666699498, 20.77508384883359, 20.02446556344927, -6.895515345589466, 18.227812184080932, 21.492208686035212, 4.2237530139001365, 23.73492119183885, 15.140097800420904, -10.216269295597296, 1.1664246943090575, 13.376737358160426, 27.43073368889418, -7.426367094058694, 19.81911592206842, 4.571298659495284, 7.559460108710169, -50.520923356119766, -56.564736896749295, -11.578849761342155, -36.96537702428518, 4.785700351737425, 22.847510191816923, 14.755012011084943, 4.988191615637959, 4.890125887015586, 13.388128359566963, 14.905462923191635, 26.879538487802904, 10.510611720503526, 14.46522814758498, 21.454614459520222, -28.490843744507877, 16.17775896833699, 59.04674454289857, 35.160635966406346, 41.75893761396814, 34.31018849574386, 18.978208709011994, 19.274346260245068, 26.554783069965907, 4.261957419940941, 6.004085749559479, 31.20620357714504, 38.0183871256678, 30.67346295986306, 0.8809264399547772, -2.4897135783641033, 21.559110909380138, 14.408157887640584, 23.939921837990994, -0.15829847139333708, 35.363020755196004, 24.132892171052788, 13.157030505123437, 8.302377107345478, 0.9479585918081881, 29.49198003272386, 1.4830118480978989, 9.751391074120797, 32.053918220213724, 34.39237843765338, 12.336468320827635, 15.676743126860629, 55.87318023943001, 22.098910822911677, 29.2460709933778, 2.194163104856443, 39.603385856443055, 53.359291269845386, 16.707632337411894, 12.72510881809714, 12.924685767232575, 37.17740159062129, 18.10426764680267, 8.693909718524594, 7.44565126540391, 24.006504851793636, 13.470622761370322, 13.737822603694411, 7.22413669625773, -11.652442369332789, 20.474549291495464, 29.706675007780916, 9.129409719653573, 7.600072712561911, 20.341430096315776, 26.874892977867802, 12.09638716430959, 12.08534788731464, 13.02421554403432, 12.880354146629045, 11.898066428407589, 2.675373135883518, 23.38391008787596, 24.121323681714244, 7.56280649225485, 1.9977395111870635, 2.7842802269820197, 10.624838335657472, 4.2105373877028, 14.037254778866838, 21.7524746804512, 24.543040041893107, 14.384095364907637], 'changes_betw': [0.053740109703412364, 0.14653333980551136, 0.0694917722440656, 0.0, 0.005610569066226617, 0.0, 0.0, 0.0, 0.042612009125770484, 0.0, -0.016754850088183545, 0.0012183874569195279, 0.0, 0.0, 0.1425153309709886, 0.00019821044285876126, 0.0, -0.0012034205459281909, 0.11677628917690097, 0.00022086306489976283, 0.001201397990388823, 0.0, 0.0295147484749931, 0.0295147484749931, 0.005393751112405547, 0.05313981521932583, 0.0028598935326763775, 0.0, 0.0, 0.0022794200928757807, 0.005610569066226603, 0.0, -0.0027506755335501842, 0.0, -0.011455754575020571, 0.0, -0.0890862903095321, 0.042379819749850264, -0.03054058864456416, -0.09192757633124611, 0.0, -0.030911929841593452, -0.023323301457858048, 0.0, 0.0, 0.0, 0.0, 0.0, -0.030540588644564187, 0.0, 0.0, 0.059040823261006736, 0.0019262818957008557, 0.0, 0.0, 0.0, 0.02538549908580484, -0.0016564729867482163, 0.0, -0.004912787405142199, 0.003171367085740174, 0.014830590748021948, 0.0, 0.011350581686973282, 0.0012297137679401354, -0.026264906234325217, -0.002001925472873485, 0.0, 0.0, -0.017226510039965726, 0.0, 0.0, 0.0038327427471158346, -0.014830590748021944, -0.008995922528032626, 0.0, -0.0016989466530751007, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0016989466530751007, 0.012828665275148449, 0.0, 0.0, -0.001698946653075073, -0.0005804734398006522, 0.0, -0.0016989466530750938, 0.0, 0.0, 0.0, 0.0005804734398006661, 0.002279420092875739, 0.002279420092875753, 0.0, 0.0, -0.0005804734398006557, 0.0, -0.0005804734398006592, -0.0005804734398006592, 0.0, 0.0, 0.0, 0.0, 0.0]}, (52, 24): {'changes': [-0.04662847293436201, 0.13032672025876124, 1.2909978837664084, 4.571090797934403, -1.3595590502831527, 7.965367445388438, -1.8836796448324264, -5.1775050136233745, 3.039063407285184, 5.527154253832805, 0.8683673872563418, 2.0497921315115377, -4.542274913791374, 5.773211468652022, -0.7542577524230758, -4.662161714108578, -0.6397461397872348, 5.735732449519077, -1.107825343293758, -7.594378704814318, -1.8532533600284893, 2.584192280275488, 3.4229350387752504, -3.634739589696352, -4.017502776977693, -0.7978605226612814, 0.7024687403586753, 8.637471921136637, -2.908852541730198, 1.3215577593143166, 1.8420220618114413, -8.709652209234605, 5.181862028714974, 9.177017498483394, 0.909199545016449, -4.989689713277812, 0.5870097635379068, -1.7004137309121177, -0.649030637127737, 2.756739789467723, 12.573155733045155, 8.246494778535208, 5.849856281434839, -2.52636899890868, -13.701813400873533, 1.7124432615757428, 6.684793220793722, 1.659408535868394, -0.7156498252433252, 1.079020697130602, -5.368566557663684, -1.0451730769077017, 0.8859150180135771, 0.011952965056877929, -4.0652167392636045, 13.387597796422426, -1.0111859414399014, 6.783060054603851, -2.2616206038020152, -1.281009150778388, -6.466963206257702, -0.9381874309707712, 1.0031605316945331, 1.8283467961803268, 0.006785061927708114, -1.276825637912303, 12.610061183236752, -4.862391431974174, -10.221578242095404, 5.73628313846109, -7.50749110030128, -5.309985592779327, 8.696881726020166, -0.6547391499403261, -7.0601864696715495, 2.23423901883217, 2.29679903446889, -5.28177249985913, -7.315308470580476, -2.64683043955273, 4.919870002671587, 3.3714757016037993, -5.345440840325864, 6.853990102733405, -0.2030886969901644, 5.471477793782171, 7.210845848361885, -12.053560101786246, -0.6620430913376651, 0.5020656384226143, 0.2672753561263619, 8.464385231459488, 3.2075484275383133, 7.425226184597761, 4.918401283984991, 8.385722323455184, 5.6808457528199625, 8.352147743007535, -1.6553445485612315, -2.2891198553418235, 15.219933022847556, -2.1374199438560026, -6.9095707756061415, 1.8082138710975357, -8.310906470412931, 18.74837643087082, 5.4998242921430744, 6.2997111468081926, 9.109121814195134, 9.253178700308126], 'changes_betw': [-6.795786612295851e-05, 0.0, -4.5305244082000096e-05, 0.0, -4.2473666326869886e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0002859893532676516, -2.2652622041000048e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -2.2652622041000048e-05, -2.2652622041000048e-05, 0.0, -4.5305244082000096e-05, 4.2473666326869886e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 4.2473666326869886e-05, 0.0, 0.0003312945973496517, 0.0, -0.0003312945973496517, -4.5305244082000096e-05, 0.010873258579680606, 0.0, 0.0, -0.00028598935326762387, -0.00028598935326763775, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003312945973496517, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.548419979611638e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003312945973495962, 0.00033129459734962396, 0.00033129459734963784, 0.0, 0.0, 0.0, -0.00026333673122663076, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (52, '52_37_1'): {'changes': [0.040267468266593553, -2.3577530048212196, -16.970737942656513, 5.8141834778415955, 0.5038747697340114, 2.061363150631678, 14.127375658167239, -0.803868021257756, 3.395435860779614, 15.320586466619943, 53.14669377225533, 23.326790508809125, -2.4155818778186813, 5.425934526951778, -1.332446438026416, -7.203013133257599, 27.921107117265137, 3.2393339473794356, -1.7345971021056634, -3.02840673549278, -7.914414319069692, 7.570384115065622, -33.40412278529901, -40.60688501540524, -61.604583203413455, -24.82574762819185, 1.1594594130618816, 14.937874140912072, 11.342194501999472, 5.586953937955769, 0.6260757481980193, 9.91006804837275, -0.8549853974896564, 1.4337649680590516, -5.7529312710333755, 8.5728854906356, 3.61175702623612, -21.60666014589311, -4.461883207489414, 3.630830882784288, 2.997487820317332, 187.30256397236656, 118.87702476436583, 1.9430605922606787, -4.960691874810664, -3.612226244942576, 2.736219336571253, 7.0724651258093445, -13.789201583212218, 2.258131729348122, 15.296104290854714, -2.8410316981110952, -9.65300837385388, 8.91669360431382, 3.2268335900155307, 1.0221302296408794, 3.7714842122974446, 6.398751175916146, -5.843148371128223, -1.9842446505319984, -7.092331464023346, 0.28653167806369595, -3.878427135658228, 1.199031038695047, -10.413294154532046, -6.75249618715884, -3.9331714964168896, 0.01144700232600826, -3.833974420003642, 61.69993005880207, 12.472564237558487, 8.774110871210326, 5.240750964136794, 13.184211247930847, 12.532140035068664, 11.642445189635708, 2.4492506044936704, 1.0072759465409717, -10.339143491112168, 6.32055084947288, 15.012769191132008, -1.2040316953625734, -2.4360170707140867, -1.6215920942217394, 7.098473167302643, 6.895582732282165, 9.150646407114294, -1.1187696129794062, 16.06054967029843, -4.560993149495104, 2.999644147601458, 15.65473750388989, -11.47929472523262, -6.11215551208619, 9.291898387130914, 16.364418230731076, 1.7436849269749075, 4.998835943513939, 5.421896824465648, 17.214564488784845, -4.561978116637704, -0.10680696332875073, -11.256446864592363, -2.163624417521362, 0.3755715437002607, -1.7401061898296177, 3.7009578990571868, 4.013201381019144, 16.505917788068302, 18.48797069702323], 'changes_betw': [0.015533631053508523, 0.014581816416679105, 0.03948837435075954, 0.0, 0.000963949970066158, 0.0, 0.0, 0.0, 0.007341876607931619, 0.0, -0.03687321003834769, -0.011373638820121993, 0.0, 0.0, 0.01448271119524941, 0.0, 0.0, -4.247366632687726e-05, 0.012925343429930541, 0.0, 0.00021034577609501184, 0.0, 0.017500364059997074, 0.017500364059997074, 0.015842273028817386, 0.03432114945876409, 0.0002973156642882002, 0.0, 0.0, 7.64525993883769e-05, 0.0011083604355775212, 0.0, -0.0007940553047586785, 0.0, 0.02218905231137644, 0.0, -0.025075643577172846, 0.03117243499506503, 0.014399381907027109, -0.011966652104266923, 0.0, -0.06500412601330034, -0.050449816351957055, 0.0, 0.0, 0.0, 0.0, 0.0, 0.019422600844619164, 0.0, 0.0, 0.00743248709609573, 0.00019901946507451009, 0.0, 0.0, 0.0, -0.01246136918919788, -0.0001274209989806322, 0.0, -0.0031600407747196385, 0.0007305470608222888, 0.0022329013154701385, 0.0, 0.006865362522855023, 0.023038525637913976, 0.0208582107664676, -0.00024634726469588853, 0.0, 0.0, -0.025432422374318413, 0.0, 0.0, 0.00012620746565700572, -0.0022329013154701177, -0.0018603465851172347, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001986554050774223, 0.0, 0.0, 0.0, -7.645259938836996e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 7.645259938838384e-05, 7.645259938836302e-05, 7.64525993883769e-05, 0.0, 0.0, -7.645259938838037e-05, 0.0, -7.64525993883769e-05, -7.645259938838037e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (27, 24): {'changes': [-0.849284604656054, -1.4925334705938056, 3.6812227077592468, -1.464062083134067, 1.1860813104614039, 3.7343143931942393, -2.7366302238697244, -13.76143559067043, 1.0464834446902813, 14.583418792171642, -2.59583350977897, -12.568710965013281, 9.297008066905732, -1.4867716313920027, -0.4352733300629055, 8.382657175158926, -0.18106586605767916, 2.7096041291597714, -0.8255007737488889, -8.376349741868836, -1.4615291877661747, 16.569229135162686, -1.9264321199521817, -2.5791824213764585, 10.630834234814017, -2.3377279233838095, -0.8073041729429491, 5.122791018054329, -6.651993084901193, -0.622307917395986, -3.141131270367161, 15.484038036442314, -5.5343597526119765, 3.212503529264893, 2.3210722385054012, 6.98918710556768, -0.0772267584385915, -4.686370711875242, 3.5831983088845902, 0.7463271259020843, -4.002274292985248, -7.163109582966612, -3.172024254532758, 7.254490293853735, 14.683509801776552, 3.2779129873890156, -0.2860725690034229, 10.403773697769708, 2.0895704428839963, 1.4624451784321764, -1.8080349137349003, 0.009024333067166879, -5.267084196497905, -8.475815250108496, -0.6846081459545417, -7.221815182491866, 0.137779317677186, 5.789125566428822, -0.9816098848862111, -0.19794312985641227, -7.569606299997702, -4.720874555654547, 13.756506353953142, 1.160772972983068, 1.6899816549118114, 1.304967658847218, -1.5430936847679106, 0.48486956828918437, -1.1809593204799853, -4.89797606586184, 1.0551945683531017, -1.837998715264746, 0.341623576586386, 5.635262590378886, 7.6075026601624245, 9.115333817674468, 3.5304426664431077, 5.098099888672408, 2.119316571324191, 8.013135589955539, 1.9543644672463643, 8.638998427868017, -1.6294068765871828, -4.414108908474276, 1.4923934498908125, 2.342663807383687, -0.7032030317645024, 2.0124923895485836, 4.975162304509126, 0.2810683097228974, -0.27220374898016075, -0.09033624171593146, -4.39361280231887, -4.382925544103273, 3.9345300643399526, 1.5562630697947668, -3.6020430034175206, 4.592028920579054, -3.0952969256339316, -0.2197177342750365, 4.011164440238417, -1.5991233986264604, -8.580494059323428, 2.301839787995675, -5.063382359605782, 10.989774492897368, 3.2193358189781236, -8.04004091085551, 1.4778097562836479, -12.467112193324766], 'changes_betw': [0.0010193679918453769, 0.0, 0.0006795786612299598, 0.0, 0.001019367991845127, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002378525314305116, 0.00033978933061501806, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003397893306150285, 0.0003397893306150285, 0.0, 0.0006795786612300431, 0.0, 0.0, 0.0, 0.0, 0.0010193679918450993, 0.0, -0.0010193679918450577, 0.0, -0.0030581039755351314, 0.0, 0.007645259938837884, 0.0006795786612300431, -0.003058103975535159, 0.0, 0.0, 0.00237852531430513, 0.002378525314305116, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0030581039755352146, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0020387359836900043, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0010193679918450438, -0.003058103975535159, -0.0030581039755352007, 0.0, 0.0, 0.0, 0.0020387359836900876, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (27, 15): {'changes': [0.10150180863507785, 0.48799536653353215, -3.286682079678542, 4.27321632734845, 0.7406099264810493, 7.866477130871118, 16.144331596901765, 14.297596536039407, -2.3062031217743666, 6.6203037972585435, -36.978088254184804, 4.384804067732887, 15.502177556495155, 5.1782009486826155, 0.8569889032347326, 0.8210906395039785, 28.04833575246093, 26.582921479083723, 1.535295705252878, 13.821182837682699, 3.26204549323036, 11.379571017138204, -4.552084396926148, -7.6376997647111295, 18.966312609166522, -4.356341938798337, 0.6482549835776013, 4.252071205877314, 16.4528765926313, 6.526757291594421, 1.5302674202294284, 14.739879929312792, 19.25137406920352, 6.919674026044788, 49.29253891406123, 21.082539016801377, -1.2538461675582013, -5.290111046600799, 24.54615376788137, 4.9504630183144585, 21.178870612130424, -38.79642299861025, -40.349596858005896, 22.31878423481669, 7.261487512213421, 9.633553055742652, 1.4152650165461722, 5.937891977843876, 220.46834797971806, 14.992260260087079, 1.9972459963307756, 4.664597830284528, 11.732803371687368, 10.062987808770629, 10.243351072382438, 8.466319083642816, -11.20325504459198, 17.35651324854382, 23.48662884687849, 7.820776081281437, 20.2010789927742, 4.73371235058913, 13.11846498944493, -0.5234899800190576, 37.65128353526252, 140.08694759899137, 31.53672622445157, 11.190668093815475, 7.516305161280997, -72.25617904879469, 13.51926604233688, 28.76234285946265, 19.78749322438395, 14.793102345928588, 15.493413643279098, 20.505380120609345, 8.130434964161083, 2.1451084911716407, 21.980357347160748, -0.08620014035915347, 8.546298026518457, 12.325199295822244, 13.15904160513, 12.051976178600455, 2.6246896178268457, 8.168052355814211, 13.439411748553937, 6.3284187147322655, 9.288254443267135, 6.821712531189668, 2.7751801655128077, 23.058484573355827, 11.765668232173311, 22.649584819305602, 12.082751790930615, 16.40976526613639, 7.148567811080397, 16.380576369080913, 11.36801540904031, 10.577066431326045, 12.033686794914672, -0.8592602378241736, 8.230345501865628, 10.266489748749137, 6.567232471952138, 4.445652420665709, 3.3515625671508644, 12.683126821263897, 4.377793094054425, 21.482085529148662], 'changes_betw': [0.0372619452130159, 0.016835347798650868, 0.012214212902286234, 0.00014157888775623295, 0.014895312525281978, 0.0, 0.0, 0.0, 0.023742779476724457, 0.0, 0.08064495251039593, 0.004352944031843092, 0.0, 0.0, 0.01631716906946243, 0.0, 0.0, -0.00014157888775625767, 0.012239697102082281, -6.79578661230034e-05, 2.8315777551257e-05, -2.8315777551252662e-05, 0.005307185735320304, 0.005307185735320304, 0.0, 0.008519003931847985, -0.00012742099898055415, 0.0, 0.0, 7.64525993883769e-05, 0.014614986327524546, 0.0, -0.014640470527320683, 0.0, -0.05878840832969272, 0.0, 0.0376017345436305, 0.005257026357943742, -0.05813997702376905, -0.016514570490105768, 0.0, 0.07789832208792456, 0.07942737407569214, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10513284144782617, 0.0, 0.0, -0.0006156659061856695, 0.0004530524408200244, 0.0, 0.0, 0.0, 0.09834150445771234, -0.00033978933061501893, 0.0, -0.012078701681148163, 0.0005889681730660316, 0.001541187321003827, 0.0, 0.02559341779525276, -0.034663365856026404, -0.08616976522175301, -0.0004813682183712766, 0.0, 0.0, 0.07711316602753912, 0.0, 0.0, -0.00044091710758377683, -0.0015411873210038375, -0.0015007362102163382, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0010598191026325414, 0.0, 0.0, 0.0, -7.645259938836996e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 7.645259938838384e-05, 7.645259938836302e-05, 7.64525993883769e-05, 0.0, 0.0, -7.645259938838037e-05, 0.0, -7.64525993883769e-05, -7.645259938838037e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (36, 45): {'changes': [0.6342691813991586, -2.1932860225925666, -5.723741433136588, 8.105941894969106, 2.2872216023276906, -6.396760098869578, 13.48041074860933, -12.456759574531844, -0.7189186638784619, 10.149660229983056, 1.2732676730555994, -7.78905942339523, 17.05857006683405, 6.969583538389145, -1.299623472646772, 21.897435837197037, -0.07624083148868976, 13.39340328546524, 1.80321673778095, 3.099188854592569, 13.38640571544832, 6.671279683357028, -30.211042352005336, -39.52001601555183, 6.672319373694421, -18.725105443991723, 0.2818242747420854, -6.219997735563425, 5.934575272998529, -1.2879129649458605, -1.418871796488105, 10.130653097624474, 0.15157366261745153, 2.980944270546587, 6.643464131137861, -10.65289767474377, 5.856900633303102, -18.704941353996233, 5.169616656230318, 37.64785943681218, 3.8231619900153646, 6.819218560392855, 1.790542239738258, 8.734084189997276, 6.116543995946586, 5.308836517335521, 0.6989500473794266, -1.308039888514088, 12.782016259126806, 8.968708058944031, 8.233084238187473, 10.983209464530162, 10.20291160790697, 7.743509195754541, 13.374270984536793, 6.398936516480944, -1.0371066361125685, 12.972534779910433, -1.9497579735151476, 89.07884224628009, -22.56027813298499, -39.823721693834614, 13.58299115114329, -0.0549660361098816, 5.8944189841317325, 13.747681969610305, 17.397106637456233, 13.598306276895926, 0.3670497321580797, 8.512525376902829, 3.3347306398870273, -2.536409627851242, -122.35310832366774, -70.85248655528, -105.24095432351817, -0.9265925589388075, 6.487526209888813, 8.500020652246178, 0.06667867646126524, 0.5308176314399589, -4.415885702426351, 13.28178702803956, 17.991616646521237, 4.877505031612941, 11.807236771636866, 6.788971923003032, -91.21556899536351, -6.693572618007124, 2.4152432516519298, 0.9903851156504686, -2.162148379030725, 11.180695841012891, 6.293282847931238, -1.4433243170907417, -2.800501646553073, 3.3054178490789354, -1.9746929355343923, 1.6702949755997594, 0.06975003999352225, 7.06341408902631, 11.537903869390504, 0.22618068176487327, -5.948705086480146, -1.4712217193280424, -5.422358644102843, 8.670855733240359, 11.33946501506776, 2.004411725104262, 1.3043183799369444, 10.88531257904583], 'changes_betw': [0.01937041891170327, 0.04742528679837538, 0.03959314272769941, 0.0, -0.0003074284419849749, 0.0, 0.0, 0.0, 0.015839441451062114, 0.0, 0.0019675420287040835, 0.004293480898985524, 0.0, 0.0, 0.03857377473585458, -0.00018405255408313493, 0.0, -0.001458262543889455, 0.01088094429073061, 0.0005436629289840302, -0.0007523906606475492, 0.0, 0.01772405870265198, 0.01772405870265198, 0.002899131110140285, 0.03278643431548632, 0.00022652622041000048, 0.0, 0.0, 0.00011326311020501412, -0.00030742844198500263, 0.0, 0.0005339546623950309, 0.0, -0.0013187062116726822, 0.0, -0.020001456239988558, 0.026273805478698453, -0.012135333236250795, -0.056623464880345656, 0.0, -0.005897771952817772, -0.0015144895878840409, 0.0, 0.0, 0.0, 0.0, 0.0, -0.012135333236250628, 0.0, 0.0, -0.02463229940294162, 0.0007766613271200425, 0.0, 0.0, 0.0, 0.012748167564681195, -0.0016564729867482163, 0.0, -0.08176180767923892, 0.0036244195265601993, 0.028552416549358445, 0.0, 0.009991424364513057, 0.0061849748394089965, -0.00995501836480428, -0.0023983463585910067, 0.0, 0.0, -0.0030912738863809255, 0.0, 0.0, 0.04026382214455608, 0.032145284209504355, 0.037637331521123536, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03831569664902998, 0.0, 0.0, 0.0, -0.00011326311020500024, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00011326311020501412, 0.00011326311020500024, 0.00011326311020500718, 0.0, 0.0, -0.00011326311020500544, 0.0, -0.00011326311020500718, -0.00011326311020500718, 0.0, 0.0, 0.0, 0.0, 0.0]}, (36, '36_31_1'): {'changes': [-0.7844116071715455, -1.0111497655713535, 4.092881109492538, 1.166578401348076, -0.1434824959976666, 7.27440107307757, 12.247785993238551, 6.417327482642179, -1.61772511657351, -3.215834006718751, 9.022853278466059, -0.16588257109020788, 12.54604663418229, 12.013040902235304, -1.89818742432778, 0.5415481137708014, 7.844411469703232, -2.210118395680439, -1.3203835915876176, -9.663164042449921, -1.9584348712062365, 1.3769518500823779, 1.6236808793358364, 12.06782967598707, 13.330714913166304, 8.95266786320633, -0.14165425877096993, 21.952488914152354, 4.453206477460753, 1.8001181264048824, 2.3144024680494013, 5.856162267045818, 9.31584344770809, 8.583038092618438, 3.224028425300375, 6.150672616138536, 5.554368074298765, 12.850877739551592, 1.2375287175167458, 8.094681198948443, 14.015398246651557, 11.247717271290483, 14.396944979806563, -1.720243572954928, 9.041967862727745, 13.61201572307894, 3.1009154091680244, 6.196242532383167, 5.495516408224688, 20.058783684006528, 3.4169384148012227, -1.6471204850207144, 4.285185235614506, -2.819438887206843, 1.971407392499657, -3.240578437071747, 0.4988091940880892, -17.912675322642656, 11.023585006522893, -3.593651924201069, 17.958986629720954, -22.305563701958306, 14.478017950341751, 0.3694794043003782, 4.90758437141254, 5.687895325136466, -0.18214696239584782, 8.645786666218669, -4.840026008425838, 4.540029963666385, -10.014521477100516, 6.037399466854538, -53.59237801002425, 91.50267200247987, -13.560895821234027, 17.116787758139253, 6.0255793617052404, 2.4496794424155155, 12.835876377978337, 9.812479422422399, 5.718822580997028, 1.6483663818042942, -3.520622766320173, -3.221720973598849, -5.786616824412022, 2.3689898561172953, -56.18938223887366, 9.247785498414487, 8.07861179228098, 1.1254345431505541, -3.6238779801151964, 14.13660481399262, 13.536702183241346, -0.9080220147586147, 6.504534844138448, 10.668407033153585, 3.085379015355784, 10.48716386211764, -0.675980221992063, -11.040498511639555, 7.812181081714442, 8.128322394655328, 2.685533720685754, 2.4892044690706143, -6.6688188163547295, 18.666024050822642, 5.873375891795263, 20.408218684220117, -0.6259607149584667, 1.9431196581978725], 'changes_betw': [0.011893031082633676, 0.019206996424122047, -0.0015124670323447553, 0.0, 0.0021560442049739226, 0.0, 0.0, 0.0, 0.009366859213954054, 0.0, -0.008804184262899958, -0.0012475122566865705, 0.0, 0.0, 0.01927778586799983, 7.078944387812905e-05, 0.0, 0.0, 0.019277785867999775, 0.0, 0.00043687199650503314, 0.0, -0.0018081646522013495, -0.0018081646522013495, 0.0003199682863291404, -0.004635697296247757, 0.0011043153244988113, 0.0, 0.0, 0.0008919469928644203, 0.002156044204973895, 0.0, -0.0010517288804750974, 0.0, -0.005197967736193881, 0.0, -0.01927778586799997, -0.0046356972962477705, -0.005197967736193909, -0.027564195912819917, 0.0, -0.009444120835558213, -0.009444120835558213, 0.0, 0.0, 0.0, 0.0, 0.0, -0.005197967736194076, 0.0, 0.0, 0.03596427357895232, 0.0, 0.0, 0.0, 0.0, 0.003115544552853483, 0.0, 0.0, 0.008230183000825134, 0.00019821044285875996, 0.04286321052376099, 0.0, -0.0018728864294612624, -0.004914809960681457, -0.00519796773619409, -0.00014157888775625767, 0.0, 0.0, -0.006251314661100606, 0.0, 0.0, 0.03078936297590732, -0.025194165331780014, 0.007851965114962051, 0.0, -0.0006795786612300431, 0.0, 0.0, 0.0, 0.0, 0.0, -1.734723475976807e-18, 0.0, -1.734723475976807e-18, -0.0006795786612300431, 0.04077633771823373, 0.0, 0.0, -0.0006795786612300292, -0.00021236833163439106, 0.0, -0.0006795786612300361, 0.0, 0.0, 0.0, 0.00021236833163439106, 0.0008919469928644203, 0.0008919469928644272, 0.0, 0.0, -0.0002123683316343876, 0.0, -0.00021236833163439106, -0.00021236833163439106, 0.0, 0.0, 0.0, 0.0, 0.0]}, ('52_37_1', '52_37_2'): {'changes': [0.15260778903403427, -0.1332712656489825, -20.538306648793686, 4.431125017999406, 1.8446066351611705, 8.04463360327793, 3.012488473948622, 12.80443256017125, 4.3882313865525475, 6.088065862268252, 78.39945039001246, 44.548985156551566, 2.4340412997195813, -2.2321337867156217, -0.6657258997788809, 4.232130196670255, 9.267722008511214, 9.257272396095885, -0.4745776787676874, 4.0016470365955, 8.057435396524639, 2.154009478785696, -57.94529464580465, -40.85705303235255, -60.74398954762799, -29.71993274925125, 4.963682582858468, 12.398968415094032, 9.56134824728224, 3.868623220447688, 2.611364516108665, 21.11507255685322, -4.903697538065771, 7.734875919275851, -9.788996503518007, 1.5697138966338002, 1.4971707475941614, -28.27183313450805, -6.693839066823919, 2.671334048536174, 14.320562454462902, 177.92281463537552, 178.1303136792417, 14.994742843602126, 7.078603840695109, 12.583455173523134, 4.659048638990939, 3.1246729886503317, -11.362849022103418, 3.3875057699975173, 4.684515894197489, 0.8639830648376616, 8.736514139622443, 12.527924519512936, 13.810179677966971, 20.860027061632763, 6.263231634244306, 19.68927468519169, 12.63179263602666, -2.5400562733576493, -2.2162372453806256, -0.35448260200048765, 23.660990817451477, 0.5105043947413783, -6.607757838304607, -13.198312678014126, 20.63260744670498, 11.89988145496227, 13.351113569429089, 79.32587017122972, 6.633165272389817, 14.042348579813563, -6.228889461081337, 4.561301909502703, 20.38101624989335, 11.412882654193936, 3.108653066611268, 2.9181054611421473, 15.080807311381818, 9.358532449053087, 14.751468892854746, 9.896270421561411, 18.38476624904979, 4.819956900638772, 1.5832488895108554, 6.625144485068375, -1.2544104300142749, 9.688403041791332, 13.87709572397597, 1.2468370872810226, 2.3176803234582906, 8.95865054147606, 6.460864856298286, 5.578199828167101, -1.1169914209332603, 5.401178941944636, 4.263341196451435, 4.547323226410086, 10.264356323894503, 28.97120765281329, 12.08308784950367, 7.472396191989816, 6.593216738554247, 4.422289266265665, 6.754256681207494, -0.5202392643847418, 6.3391049941588165, 12.068936816771384, 6.668744561022663, 10.471867068656763], 'changes_betw': [0.014185800042069152, 0.011059333689303397, 0.03924485866381884, -9.061048816402795e-05, 0.0012471077455786794, 0.0, 0.0, 0.0, 0.007681665938546661, 0.0, -0.05471214989563619, -0.016912204909146814, 0.0, 0.0, 0.011203744154814482, 0.0, 0.0, 4.247366632687726e-05, 0.011175428377263208, 1.1326311020500458e-05, 0.0001933563095642618, 1.6989466530750903e-05, 0.018814216138375146, 0.018814216138375146, 0.010082843874892822, 0.03745853761144277, 0.0002973156642882002, 0.0, 0.0, 7.64525993883769e-05, 0.0013915182110900565, 0.0, -0.001077213080271186, 0.0, 0.02615326116855174, 0.0, -0.009445334368882036, 0.034819507143666245, 0.019552853421354932, -0.008364885199747707, 0.0, -0.06500412601330034, -0.06879844020516807, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024406177693639508, 0.0, 0.0, 0.008400886688348541, 0.0001140721324207547, 0.0, 0.0, 0.0, -0.017813051146384484, -4.247366632687726e-05, 0.0, -0.00023785253143045193, 0.0006455997281685343, 0.0019780593175088776, 0.0, 0.005732731420804826, 0.02632315583385919, 0.02550199828487293, -0.00016139993204213401, 0.0, 0.0, -0.031293788327427484, 0.0, 0.0, 0.00021115479831076023, -0.0019780593175088602, -0.0016055045871559703, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001816659385466714, 0.0, 0.0, 0.0, -7.645259938836996e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 7.645259938838384e-05, 7.645259938836302e-05, 7.64525993883769e-05, 0.0, 0.0, -7.645259938838037e-05, 0.0, -7.64525993883769e-05, -7.645259938838037e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (15, 9): {'changes': [-1.2053101467826792, 1.222200064010984, -4.869942912751071, 0.44516557369752263, 2.0618171670744374, 6.468241629131484, 21.437531789729462, 12.494307996591317, -5.564991785040796, 20.177476549161383, -33.85412729513798, -11.900896083593551, 21.583331313583443, 16.72222958478136, 1.4120310013759507, 16.97483137990841, 14.60190352429862, 22.997878953460372, 2.2319811280327926, 12.634847494586552, -0.13560292253987427, 12.540248294179037, -18.657868258704028, -16.2637864765543, 28.245988218652002, -17.132189968911774, 2.3003093616345325, -2.7402181379409853, 23.63537412849479, 6.1968731003426, 1.5723108645437591, 13.655183898286793, 17.20804558113302, 9.964943268424861, 62.54776102276137, 17.07316661269431, -3.209543757259418, -9.792573508422791, 18.883869479621133, 8.061522710383478, 15.688526018778362, -42.086782957511325, -37.49671851798013, 25.46845942907501, 11.20054027468359, 11.5153686088114, 5.122637721607745, 5.476914382636551, 220.1409070337345, 21.267729105359223, 19.914770353811548, 0.7917326668838669, 20.40164023576088, 19.809810671673915, 10.44032930584899, 28.483104377146134, -10.610613462589225, 35.8565251696659, 4.547870095197652, 6.122950926402652, 9.642543295484955, 12.618182071516856, 17.562960086747864, -1.3456933513826144, 52.64228729461656, 225.41252881278888, 12.407032100782885, 9.387840833133566, 3.3371415866526206, -67.7659364903156, 23.98164753041135, 25.493084234930222, 23.240507728048783, 28.075105455429423, 25.749024437419763, 12.700181183221673, 1.2319050885713807, 11.038544241071293, 20.958532249687323, 3.589401616898158, 19.098515828278266, 26.165889926747752, 9.697306283031594, 6.250633621969001, 9.887046459950852, -5.024003895890758, 8.455488074301684, 12.78813445027913, 21.086682864768648, 0.5543359225828013, 10.199304788043719, 26.466401911545347, 10.850289993526332, 22.13822002900406, 10.212491948791353, 14.47197857192674, 1.140402418642168, 11.92639750202514, 8.885604904242399, 10.695498777386604, 23.441299304154, 25.274030388727112, 28.56202906020539, 11.124753710925717, 15.674704057435548, 3.569822893374692, 6.728466660167214, 9.215258947583095, 18.91982162156421, 19.19356966845544], 'changes_betw': [0.03373663090788509, 0.013533728136174983, 0.0148985486141449, 0.0, 0.016339417180395804, 0.0, 0.0, 0.0, 0.016437308868501505, 0.0, 0.08373137226348239, 0.005612996132873801, 0.0, 0.0, 0.013349675582091353, 0.0, 0.0, -5.663155510250272e-05, 0.010801255602478688, 0.0, 0.0, 0.0, 0.006946669255537774, 0.006946669255537774, 0.0, 0.012307654968205461, -0.0004672103295955965, 0.0, 0.0, 7.64525993883769e-05, 0.015719301652023357, 0.0, -0.016254469847742023, 0.0, -0.07645745352167369, 0.0, 0.05110836043557751, 0.00944209828001874, -0.04115051049301813, -0.01322427713865032, 0.0, 0.08191916250020233, 0.08310842515735489, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10513284144782617, 0.0, 0.0, 0.0006642072391308818, 0.00036810510816626986, 0.0, 0.0, 0.0, 0.09133334951377758, -0.000254841997961264, 0.0, -0.009184829215410237, 0.0005323366179635306, 0.0013712926556963129, 0.0, 0.012596475899228299, -0.05165283238677734, -0.10451838907496402, -0.00039642088571752165, 0.0, 0.0, 0.07825995501836483, 0.0, 0.0, -0.0003559697749300223, -0.0013712926556963372, -0.0013308415449088291, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000974871769978786, 0.0, 0.0, 0.0, -7.645259938836996e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 7.645259938838384e-05, 7.645259938836302e-05, 7.64525993883769e-05, 0.0, 0.0, -7.645259938838037e-05, 0.0, -7.64525993883769e-05, -7.645259938838037e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (63, 72): {'changes': [0.08491149189551805, 0.16045244833550143, -1.106302701215668, -2.768643506543185, 2.3097377928186447, -11.84618313707665, 7.616806645060535, 9.43314898683974, 2.1537534485683665, 4.851900910365089, -2.588742627430534, 0.8064225150864388, 1.1045744315001116, -5.382220047736865, -1.2797088635529157, 3.635984519732858, -0.8059088665594913, -6.928043574757311, -1.3008510803806743, 10.205512929298663, -4.766785875690999, -1.4176952383766093, -1.1968240573302467, 1.1862773919519896, 4.615979566938705, 2.6902171556082948, 1.8739621971533964, -6.691440231806951, -10.620326999328768, -1.7214883520101267, 2.7380691149656897, 7.911016107398609, 7.890224486060873, 8.71358661761684, -1.2581419062095378, -7.152162335795168, -1.1856832875195025, 3.667323514365094, 0.824058545859927, -0.7377490456752867, 0.36638668297774757, 3.3456067916029326, 3.508974758341651, -6.393335698965814, 2.439056202929578, -5.622951176200786, 3.984745924601228, 2.4849628689175773, -2.3745873503205246, 4.194569184696888, -0.48849035307785016, 0.8197404870424023, -1.940383172112604, -1.556286911907705, -3.639152436801112, 3.554962987183444, -0.6991725055801012, 13.285068046178651, -0.9688263897988918, 2.596011111156386, 14.479593077472288, -6.921493377403692, 2.877388272061637, -1.0149561545276953, -2.363092652185557, 2.2312076193520625, 2.766666966155384, -3.7168360329800976, 1.8327170860401054, -4.415566992704726, 2.281637117859475, -2.6618739518834786, 1.7875929994018236, 11.374812953515999, -2.1488273219372616, -0.5648347551465349, -1.0363828736691971, -7.368732976227847, 7.071083016706439, 1.967970073041812, 4.983666955028468, 8.007197787451332, 6.551792652986762, 3.2127912849404936, -9.261213257077884, 0.5871854929741573, -9.241804948248245, -5.250174213515891, 1.6865307461841326, -0.0794295178675739, 5.152911088847247, -3.540947351030809, 8.092377579533661, 2.297040234224596, 1.6549875180929519, -6.03633796249693, 2.062879210880652, 6.579343560724752, 2.598059370854685, 7.49086629977279, -5.3459025101615225, -2.5836481910611724, 3.5124873920105415, 0.6222608476258529, -4.634180761171081, 0.5828250430341768, -0.5838892591386013, 3.10238767260995, -11.197902534166076, 3.006024905087372], 'changes_betw': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00031147355306374025, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00014157888775625778, 2.8315777551218835e-05, -2.8315777551252662e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (72, 76): {'changes': [-0.7738301388688953, -1.1621823238186622, -1.682822305583315, 0.927557678767954, -0.3906854605853809, -4.2183426558333394, 4.759978955187648, 3.568712053510012, 1.4891913105624965, -11.74485148412748, -3.937686886612198, 7.68653641181092, 8.578017951674099, 0.031155267340921, 2.556904597898903, 1.2609604819768947, 14.258316258822333, 4.958316158976572, 1.6648560341212253, 2.9408203599470824, 4.198131856308137, 2.9087133620816985, -4.342106099894437, 4.336032481393346, -6.790267223382415, 6.372453136201216, -1.999535995189305, 2.0193132129817, 2.228917682428289, -0.12105356972027437, -3.2987189686575604, -12.684443262750335, 4.251599072196285, -4.484554555216903, 5.248521285674244, 17.30403258917204, 0.5337322336327439, 5.324961137652139, -1.0120427528630316, 0.6035421984265241, -14.2237766204982, 0.8447756127921764, 0.7393171783177763, 4.686848116141817, 2.6188155059574, -13.760325054844373, -5.341848899108172, -2.2153320177137914, 6.174244496344073, -5.074484072077979, 1.3827115826355794, 1.523464194189934, -15.197093788237794, 8.506200962039145, -7.396988906375583, -0.26472569623325626, -0.13510527531947147, -3.276706092097868, 5.816140939100023, -4.0474660920432015, -1.1659391018134215, -1.8046528635944412, -14.025891678275514, 0.4077279929115285, 3.448998475761499, 0.9922425704633397, -2.4416657030681392, 6.160222112594795, -9.873989515746615, -1.4961101340568206, -9.368141809543744, -0.8312081503604531, -6.623563785200474, 9.147117423932883, 9.297441833236917, -7.460263619832062, 1.3236918369532447, 0.608480808080941, -2.875798685993061, -4.98843365546216, -10.208718128460191, 1.0699777507401222, -5.570902390866706, 1.2491071061755576, -2.562540054790418, -0.9344040923512011, 0.4900057603110781, 5.220810540866125, -4.655654287144415, 2.072577686242745, 1.305713270932955, -4.198868041288591, -0.25583981898128627, 5.730669629669478, 0.31682993113690827, -5.098956994439163, 1.8019357294415101, -3.802019592009188, -0.1974346060489438, -9.083643242397414, -20.776790487643495, -3.8939738646795377, 2.4582978753249733, -4.213302820540605, -5.464775084536512, -3.0165128682517093, -2.4739496050439698, 1.1916510713501793, 5.218770984392052, -0.7690699263384886], 'changes_betw': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00048136821837124755, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00014157888775625778, 0.0001132631102049586, -0.00011326311020500718, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (86, 78): {'changes': [-0.4993207280856886, 0.45318618952356005, -2.1515352450253715, 2.1963509953246927, -2.921922842754846, 2.892978990086732, 7.3129259073600394, -8.61942060584883, 0.9098315014034668, -4.266934151824529, 5.71869482845355, 8.175626218503083, 8.761025485564574, 3.478473735186469, 0.5946794934365407, -10.327711256110291, -2.406294378399082, -6.953880591177239, -1.1763549102102253, 2.182238831435143, -0.3716752955394611, -2.0661609271298573, -1.1922474486306385, 7.032334208456632, -7.613985101359816, 0.1730885949898635, 0.0778557244708864, 3.702510086907921, 4.310936838974982, 2.93316206769218, -0.812236059453106, 12.43978136044609, -1.0983127450810741, -3.0671534416617305, 0.8299630007990757, 5.803323681514826, 1.5468643612437631, -6.27057446340153, -0.7438836757032306, -2.6170457518623564, 1.4055159284379215, -5.612170701820688, 1.9698026592761124, -9.342819731423162, 10.400265560555852, 2.9971832502120606, 12.17305821309337, 4.7032344332484115, 1.3650698012390734, -2.276248210585834, -0.9523639276407607, -0.5701114706192527, -27.716141736341257, -13.876982088300679, 0.5663243993889182, -3.1147564032870036, 1.2319131592726436, 26.24842488227742, 8.643449634607094, -0.8680841442180665, -0.5141357046640564, 0.08802181260354303, 3.044216440195669, -1.0284120475244478, -0.9110081219333352, -0.9493602734020001, -6.259825703637091, -6.428525144943848, 2.3517221175838188, 3.3221036262141865, -6.009847393472967, -8.193729536326884, 7.03779455674146, 0.7156892935830967, -1.2435333548238532, 9.122994046228143, 0.5011132205338811, -4.10788843244643, 0.8991929684802926, 2.2338474567110325, 10.069656637046165, -7.453892145809277, 9.720962258868553, 3.7321835924302604, 0.08260497797513722, -3.9718089598479622, 0.24030702881350408, -1.54480455304747, 1.988867478353768, 2.3007339776299816, -4.784530811241325, -15.910120358788845, 2.0140627369263484, -10.240996222665217, -1.6878419763262116, -3.1192468474743293, -0.24059050220998301, 2.6765547843860134, 1.8212677846715337, -4.739319463592238, 0.28123391182202795, -7.595328652293091, -2.4578015180625528, 9.447948636814601, 7.275228636239746, 5.887930301373615, -2.4443194711089404, 10.88268395601898, -3.5365687807413337, -9.128825261756703], 'changes_betw': [0.000500380240441467, 0.000766144038315375, 0.0002657637978738664, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00040451110787501077, 0.0, 0.0001334886655987716, 9.586913256637644e-05, 0.0, 0.0, 0.0007661440383153195, 0.0, 0.0, 0.0, 0.009643140300632652, 0.0, 0.0, 0.0, 8.494733265376059e-05, 8.494733265376059e-05, 9.586913256637991e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -5.663155510249318e-05, 0.0, -0.0007661440383152918, -0.0001698946653075073, -0.0003964208857175078, -0.000766144038315264, 0.0, -0.00019982848729024894, -5.8249599534002106e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0003964208857175078, 0.0, 2.8315777551251578e-05, -0.006796595634516095, 0.012431435367215183, 0.0, 0.0, 0.0, 0.00040451110787501077, -0.0029306829765545364, 0.0, -0.0007661440383152918, 0.0, 0.0, 0.0, 0.00039642088571745226, 0.0003397893306150146, -0.0003964208857175494, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (86, 79): {'changes': [0.03347298841666202, 2.3743356079194733, -0.8528138428201828, 6.706887134276499, 2.385292423317125, 2.4194617960185667, -8.069282136901734, -9.74560614667763, -3.329602730518438, -0.5283140291340374, 1.2360050457378549, -3.1252005112796724, -0.5017106847107584, 2.962978977510147, -0.1773799806247851, -0.5031727703169793, 1.187814572476384, -5.354502579501769, -0.4200551841504776, -11.072122974345746, -1.546424262889758, 3.0649939804733606, 2.545816345367058, 3.1002692247900825, -5.758341606840787, -0.3750329666447385, 1.7000461639410887, -7.664042073788153, -10.571637614551264, -4.755542042238062, 2.4404751160637943, 13.46525216054647, 2.695780142264823, 3.0753156081339625, -0.8309273298714928, -1.2674094575637298, 0.11910848306029109, 0.6512251905789412, -1.325665631128956, -1.1249674029111247, -2.246753066533927, -0.040887008388480695, -0.39394422397461426, -6.9608254876760896, -4.7101475345815516, -6.336490206008591, -8.270810843658253, -3.235869840449979, -0.6133127811164414, -11.688705273189356, 4.519172006394456, -0.44950586497215284, 14.379170126551628, -8.239712552801706, -1.9733336685318363, -38.48980290278632, 0.5068803515254885, 7.544161891413324, 1.8249082792481772, 2.5835636437486045, -3.3139814938874963, 3.664830545305378, 8.107523870762236, 0.4801938650816133, -2.483672746266322, -2.0751206747550626, 3.486486512692153, -6.213793479061962, 3.037229109587315, -1.0899732443233887, 1.8177204559668496, 6.5506718599608575, -0.10311417575854875, -2.952343607222275, 13.140804240784604, -4.003969828401438, 0.9150964184769066, -2.194587914506684, -3.9948780861162163, -1.435786462178271, -3.371734541384569, 4.471570933023827, 1.3205317539168675, -1.6306407661147375, 8.825329570237841, 5.138323383767755, -1.4566396454660264, 9.192625941160799, 12.014650427339205, 4.02242473669218, 4.1856824345297525, -14.109147629121964, -6.071461613382411, -6.220496016524919, -10.33797025330361, -13.330804900082882, 1.4439505256368363, -9.358932937103532, -10.551734106773267, 3.9682810359851146, -3.2559095793181996, 1.6892672805303732, -9.324451034672478, 3.07618943670667, 5.177255961897629, -9.132875091196809, -2.223090114182952, 6.79688748188812, -15.072344062044209, -3.495098022145328], 'changes_betw': [0.00021236833163446045, 0.0003114735530637125, 9.910522142939082e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00016989466530747954, 0.0, 8.494733265375365e-05, 4.2473666326876824e-05, 0.0, 0.0, 0.000311473553063768, 0.0, 0.0, 0.0, 0.003355419639823304, 0.0, 0.0, 0.0, 2.8315777551260468e-05, 2.8315777551260468e-05, 4.247366632687769e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00031147355306374025, -5.663155510250706e-05, -0.0001698946653075073, -0.000311473553063768, 0.0, -8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0001698946653075073, 0.0, 0.0, -0.00036810510816628894, 5.663155510250272e-05, 0.0, 0.0, 0.002987314531657039, 0.00016989466530747954, 0.0, 0.0, -0.00031147355306375413, 0.0, 0.0, 0.0, 0.00016989466530742403, 0.00016989466530747954, -0.0001698946653075073, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (86, 45): {'changes': [0.2528204477068865, 0.3012140651916404, -3.6475655358563515, -3.7270397267618165, 3.697924559719411, -12.102190735853867, -15.832482690427497, 0.7498738817424737, -0.7613509510692111, -3.610414668740475, -3.0382969569836433, -14.167768860147333, 9.688364228884438, 1.824842843896704, 3.8933994567240653, 0.7737697104332995, 7.1672249792144385, 15.734481944646006, 6.94060176473338, -4.18673839415095, -4.516758200413818, 2.321575906078891, -27.653508482739355, -23.03501903342115, -7.433325394158231, -13.35804687631736, 3.362978449476458, -4.860362198718121, 12.28562351670837, 3.2395413601452248, -0.8129462916969885, 2.9420922362900512, -5.303854290068969, -6.158082413932959, -2.028472395683437, -9.873664956959999, -0.5779286408457978, -12.327617498243384, 4.701923079175167, 4.383735108091784, -2.9601853221215038, -3.6205101558629877, -2.4124946227159967, 4.855808250948542, -8.391315101160615, 2.9615551158023834, 2.575962704711614, 3.277686046221234, -1.1929126720399097, 8.658472543401103, -2.008828102114933, 15.37733276167716, -2.019122693229207, -10.576943649682505, 2.612927664197457, -0.9519565336657365, -1.5934567618788158, 12.11842940398634, 1.7959138221427224, 28.238730951758377, -123.91295740385567, 0.1895484706062831, 9.004837482004575, -1.2661662375527847, -3.7572057822777793, 3.195638900396226, -152.63250318055202, -13.189325559881212, 2.001719162871325, -5.089704653163068, -5.4594600170003105, 5.883579170649625, -31.494667174280266, -5.930293862418864, -16.86278010059658, -6.605323371874306, -3.026459392297511, -4.208576978325709, -6.840039459290665, -0.8551106190709845, 9.799145011787346, -1.7245266789604443, -3.1589043989919503, 8.887477910060852, -9.320541196504848, 1.2847982897308867, -24.67943076722537, -1.9673559587668592, -12.988454082502642, 1.4095334618123943, -5.372961880743446, -6.297599803306241, 0.49487582975220334, 6.675764360794915, -1.1555627604066103, -5.494418443817381, -2.3032417558441836, 9.189230528163108, 6.7467404676415015, -11.27924125281234, -12.720260754685626, -9.10541895235724, -9.125705362839454, -12.229741467498656, -8.552326298676803, 4.47624781205829, -10.893034233306366, 2.1194315692870305, -15.192511628049488, 16.650353608461756], 'changes_betw': [0.0014776790770676396, 0.0008656537708526812, 0.020661213856932578, 0.0, -0.002544374868533872, 0.0, 0.0, 0.0, 0.0018089736744171525, 0.0, 0.008686876041616054, 0.004389754542659743, 0.0, 0.0, -0.0071193954986004115, -0.00026899988673688966, 0.0, -0.0012034205459281909, -0.02078175816707914, 0.0004813682183712766, -0.001262074656570078, 0.0, 0.009971198809119307, 0.009971198809119307, 0.0012568160121676948, 0.01897966118149605, -0.0008777891040888108, 0.0, 0.0, -0.0007786838826594061, -0.0025443748685338857, 0.0, 0.001666585764445061, 0.0, 0.0028153973108102637, 0.0, 0.009013721016779191, 0.015751662540653355, -0.002281442648414983, -0.008421516754850256, 0.0, 0.004989644515638325, 0.00702271734381818, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0022814426484151357, 0.0, 0.0, -0.04234948141675968, 0.00038024044140251914, 0.0, 0.0, 0.0, 0.005015128715434525, -0.00045305244082002524, 0.0, -0.03910975195378874, 0.05295616717635074, 0.0003952073523938865, 0.0, 0.007329741274695489, 0.006836237723087973, -0.0015169166545313528, 0.05071355759429157, 0.0, 0.0, 0.0032061550410174416, 0.0, 0.0, 0.009275035192466384, 0.004560053719075148, 0.0076679125608789065, 0.0, 0.0006795786612300431, 0.0, 0.0, 0.0, 0.0, 0.0, -1.734723475976807e-18, 0.0, -1.734723475976807e-18, 0.0006795786612300292, 0.006732278368363993, 0.0, 0.0, 0.000679578661230057, 9.910522142938388e-05, 0.0, 0.0006795786612300396, 0.0, 0.0, 0.0, -9.910522142937694e-05, -0.000778683882659427, -0.0007786838826594131, 0.0, 0.0, 9.910522142938215e-05, 0.0, 9.910522142938388e-05, 9.910522142937694e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (86, 43): {'changes': [0.4335783075038293, -0.48387134550530675, -3.6315764917939504, 0.4731037915908587, -3.83638957074249, 3.895439230118228, 6.068172556188642, -1.3768978294526732, 3.2532806173909634, 7.075230704644554, -0.13783101521703145, -5.599273893831025, 4.537947198902714, 3.7861621371934575, -0.3016281439025761, 3.9674876728960555, 4.369019978078313, 17.584697486001033, 0.2705575394280224, 2.5901661968536303, 4.001194098027824, -3.645806468972239, 6.799844677622957, 8.057595997726622, -5.6649987981376455, 3.974798435054808, -2.310377412900472, 0.4394048070067811, 10.67237176381252, 4.364755075753152, -0.938096379312988, -9.004593954508323, 3.572007985601914, -10.523784626920417, 1.7240347858447151, -2.4262811225855785, 0.984072249558146, 1.1531642505603372, 2.039335134886919, -2.80964975005125, -0.5388480718171991, -5.250368766597717, -6.334565402303085, 7.9944017122298305, -1.5811170335109068, 11.46546578427359, 3.0878293530604424, -0.4113826353771515, 2.2999046078895446, 11.347792444863785, -7.525999263265078, 2.237642024927055, -5.527980517194607, -10.987678029056383, 0.002071529906345404, 5.4372069138244115, 0.8189220161552271, 4.844946412487701, 6.886729272072756, 0.5842341626868333, 46.11511815019128, -1.5755886640834404, -2.916757724577792, 1.1430742569859582, -0.5306045434124016, 1.014269087402404, -45.59575825002469, 7.504662887264715, -3.0138955342402767, 2.5736833179009153, 9.007807065076065, -4.513475333473593, 2.5068907112353713, 6.7307466674647, 2.9222908580133264, 0.5397176123453846, -1.4362571059816247, -3.7258232187612066, -10.16165743794295, 6.73144354788019, -6.190919162343221, -0.5089692406272093, -7.599609148034801, 0.10435788767503595, 7.518979176861109, 2.6256895367967275, 4.4197444486353845, 1.7923672172934175, -10.17394377504462, 3.4054413760810505, 1.174317995229444, 5.697257065918393, 0.7421638268156698, 13.767827209311008, 0.6343210964813011, -4.313888167710985, 13.532877951658392, 7.19601444993549, 15.946605123635464, 1.555749750302141, 14.901023735684419, 0.6233979603820217, 10.049324926236949, 0.5026166688578542, -2.2679072730363146, 9.294998229530677, 4.918079034850308, -1.3135788188754987, -10.989228644310344, -0.30570026940694106], 'changes_betw': [-0.003111903952882633, -0.0036414089930906224, -0.0005295050402083779, 0.0, -0.00011326311020498636, 0.0, 0.0, 0.0, -0.002808925133084089, 0.0, 0.0018461886963415525, -7.645259938837343e-05, 0.0, 0.0, -0.0036414089930910387, 0.0, 0.0, 0.0, -0.0036414089930910942, 0.0, 0.0, 0.0, 0.0, 0.0, -7.64525993883795e-05, 0.0003397893306150146, -0.00011326311020504187, 0.0, 0.0, -5.663155510249318e-05, -0.00011326311020500024, 0.0, 0.0, 0.0, 0.0013025257673576485, 0.0, 0.0036414089930911497, 0.0003397893306150146, 0.0013025257673576207, 0.0038113036583983795, 0.0, 0.001999093895118334, 0.00199909389511832, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0013025257673575652, 0.0, 0.0, -0.0063795446822969, 0.0, 0.0, 0.0, 0.0, -0.001223241590213947, 0.0, 0.0, 0.01859213954015175, -0.005379997734737797, -0.0001698946653075073, 0.0, 5.663155510249318e-05, 0.001245894212255086, 0.0013025257673575513, 0.015780382829312493, 0.0, 0.0, 0.0015856835428700936, 0.0, 0.0, -5.663155510250359e-05, 0.0001698946653075073, 0.00011326311020500718, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.734723475976807e-18, 0.0, -1.734723475976807e-18, 0.0, -0.0001698946653075073, 0.0, 0.0, 0.0, 5.663155510250706e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -5.663155510250012e-05, -5.6631555102514e-05, -5.663155510250012e-05, 0.0, 0.0, 5.663155510250359e-05, 0.0, 5.663155510250706e-05, 5.663155510250012e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (86, 83): {'changes': [2.37759146750715, 5.209566157014379, -7.2298424258430956, 2.370522193904833, -0.9974863875997642, 4.707764556545641, 4.929068012693648, 0.693474639393969, 6.511085135766997, 11.289279080435733, -7.196213418561953, 6.415597338533644, 3.1620477922054135, 9.313847331510885, 6.300268809947202, -4.423044214907861, -4.31183090950185, 5.855114158745266, 6.812717306949143, 5.370184000504679, 13.995127410185532, 19.90313666704253, -11.491235526762807, -14.313705301969506, 0.7526539860366483, -14.093606588857568, -2.530142787836432, -12.03534602430176, 1.2719899794377625, 6.437818701425499, -1.752306285541124, 4.1581508447162605, -2.514657994020567, 15.845159250654405, -0.9225809892778472, 7.943894149384619, -2.5204682045895836, -15.204621678330867, -1.517396966681133, -9.258896289701603, 5.920637039632965, -12.451524586281877, -6.494442336203761, 10.122525129806036, 6.666238080238969, 5.569244425878765, 3.166473810638493, 4.994612425749395, 0.9796720988566463, 18.762839274361994, 15.498925833367537, 9.448409935928872, 1.1324002974996858, 6.148063192832126, 8.612654084424321, 0.07829394269833756, 1.5787366678421968, -0.33441204537786007, 23.53580882694075, -5.60243595614736, 3.320361732282464, 69.0475620044256, 3.307770525501894, 1.2331863524193878, -1.5515347596481206, 1.3252524987659626, -10.1100062558709, 3.912123286948656, 0.5304754483040313, -7.079612980300183, 8.449495491291827, -1.5607444114389182, -86.6765463140849, -57.20987798329148, -95.12245222190157, 6.595519830123294, 1.6129801881257322, -3.3852516917831394, 7.717577532592145, 6.144384601506857, 11.128462382916837, 10.884905222341729, 12.52301728333157, 7.977236293690879, 4.382229543615267, 1.1492067187337796, -31.298896826683745, 15.846827471549886, 8.098460950857088, 2.8516562861797397, -0.9478832919780018, 8.556240643060192, 9.06823164183973, 17.839565791005214, -1.2991798793012208, 2.6397235895026085, 1.2029670469201932, 2.0790307093295866, 10.865247946971152, 11.209507141120781, -0.635452356988992, -9.593669549446219, -3.5189865577050057, 10.387060900800293, -2.796041635308711, 3.6278312653033993, 5.772271172686658, 6.3155550364851365, 19.61708299045185, -3.983506712539736], 'changes_betw': [-0.016003672960859583, -0.030793003575877997, 0.005776014109347416, 0.0, -0.004299953076711438, 0.0, 0.0, 0.0, -0.011785026616830813, 0.0, 0.013859764089121845, 0.003373622639677689, 0.0, 0.0, -0.030977056129961267, -0.00018405255408313493, 0.0, 0.0, -0.030977056129961267, 0.0, -0.0010921799912625672, 0.0, 0.004172127566622975, 0.004172127566622975, -0.00027466304224713946, 0.009363623125091008, -0.0019537886510363756, 0.0, 0.0, -0.001656472986748217, -0.00429995307671148, 0.0, 0.00234616442567509, 0.0, 0.007204342831254318, 0.0, 0.03097705612996135, 0.009363623125090995, 0.007204342831254318, 0.0518615601184409, 0.0, 0.014409090173616093, 0.01440909017361608, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007204342831254165, 0.0, 0.0, -0.04444687151109167, 0.0, 0.0, 0.0, 0.0, -0.002881737132501644, 0.0, 0.0, 0.008264161933886613, -0.00031147355306376714, -0.027812970244162905, 0.0, 0.004073426856301521, 0.006977816610844234, 0.0072043428312541374, 0.0003681051081662703, 0.0, 0.0, 0.008903289484329217, 0.0, 0.0, 0.04845840816788829, 0.06315106062812485, 0.060859100690904964, 0.0, 0.0013591573224600723, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0013591573224600723, 0.02310729252625277, 0.0, 0.0, 0.0013591573224600861, 0.00029731566428813777, 0.0, 0.0013591573224600757, 0.0, 0.0, 0.0, -0.00029731566428813777, -0.001656472986748217, -0.001656472986748217, 0.0, 0.0, 0.00029731566428814124, 0.0, 0.00029731566428813777, 0.00029731566428814124, 0.0, 0.0, 0.0, 0.0, 0.0]}, (86, 81): {'changes': [0.8186646978696341, -1.7790639136167563, -0.933046706211826, 7.264394874200377, -1.272068128913702, 14.554424647971814, 4.4817351037140725, -3.602608238370152, -0.2641020038169586, -4.748703437934694, 0.38480790852638336, 0.7988482507223296, 5.970793426082281, -1.9091163661910855, 0.2852156929090839, 4.91176620311478, 4.158221892559027, 9.923461725705124, 1.8370194461357343, 7.053081059604551, 1.5274637947806866, -4.199596210124895, -2.4888687331349786, 7.210020805714436, 13.601182517724482, -2.368900412818249, 1.2626741980607719, -11.734036614463776, 8.859139371422827, -4.688874798790295, -1.5134471300937236, 9.564017847375794, -0.580043096516178, 2.060265358267941, -0.9809595955937738, -7.584834165401645, 0.054667228341955365, 1.717476977731181, -2.7074248887572807, 0.022479516991438686, -2.3359384348831895, -2.9844886656903356, -2.958559831401473, 1.3925109040341965, -0.2234858128082351, 9.214313871420103, 1.7002955180264365, -2.2196381182535276, 0.5776222621636862, -0.06413724104731955, 2.2007095395636327, -0.9137404919247771, -3.5888357960656663, -24.54965073709414, 6.404947144654784, -10.609782422639682, 1.9033503212940879, 3.2411541059403532, -6.984487017141362, 1.6873632349634136, 19.7888999094464, -8.17502598021899, 6.085077584171415, 0.009485289934413998, -0.5972109168707078, 1.399581784076183, 16.761671769764064, 4.56542772045583, 1.8117861563498252, 5.055266302578005, 9.00893412860458, -1.8917563599403024, 2.9212279239784493, 0.4843973505288943, 2.2840488259895153, 3.2958987166347242, 1.4580131922640192, -3.9264132143806023, -3.1958099770889135, 6.753941128758157, 8.926104806678552, -16.049911632280043, 1.1953172847902067, -2.0638851699883105, 5.291953760249896, -0.6379763813336154, 3.9501667842719144, -3.335784830042968, -8.143532384198068, 2.1006674211520746, 6.241908816218711, -0.7598919099213219, -19.92959679183579, -9.654831665693393, -7.1670666949040935, -1.301229295092412, 1.90691492113217, -4.561434495540269, -7.322480333961835, -7.5041179416413115, 1.1935697633876998, -6.703849063152433, 11.656348903292525, 2.184579835371636, 12.270143380817956, 4.896475255310747, -4.1810669066896935, 4.520360342497355, 11.567563080861078, -0.41154330995382793], 'changes_betw': [0.00021236833163446045, 0.0003114735530637125, 9.910522142939082e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00016989466530747954, 0.0, 8.494733265375365e-05, 4.2473666326876824e-05, 0.0, 0.0, 0.000311473553063768, 0.0, 0.0, 0.0, 0.003355419639823304, 0.0, 0.0, 0.0, 2.8315777551260468e-05, 2.8315777551260468e-05, 4.247366632687769e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00031147355306374025, -5.663155510250706e-05, -0.0001698946653075073, -0.000311473553063768, 0.0, -8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0001698946653075073, 0.0, 0.0, -0.00036810510816628894, 5.663155510250272e-05, 0.002987314531657039, 0.0, 0.0, 0.00016989466530747954, 0.0, 0.0, -0.00031147355306375413, 0.0, 0.0, 0.0, 0.00016989466530742403, 0.00016989466530747954, -0.0001698946653075073, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (76, 78): {'changes': [-0.24757687828892472, -1.291712209580897, -0.8979556533327298, 3.800277918530611, 2.3370096751183667, -1.1809152178229283, 0.7813525928081901, -2.958101356415881, 0.1785385125364627, -9.75179134317176, 5.0440739022349135, 7.671318742098379, 7.815259522636154, 10.100622305525121, -1.2203630900663676, -1.9826149068431391, 6.100661197757063, -0.017385902546493526, -1.644795227691091, 2.0277995203030343, -9.022156741122288, 1.560893925965246, 12.251235232799218, 7.5591227492108715, 1.6521694188346885, 3.227618380803733, -1.1684028692280393, 11.656835104887136, 10.263803030315614, 4.856438425915883, 5.834266206930799, 16.413005184713683, 2.820979390386313, -5.0924475310208095, 2.5506007090364875, -0.4353629224270321, 1.0867521952701864, 6.339880894814314, 1.2141858504948075, -0.5217454029810256, -6.068222334844791, 8.776842581551193, 6.439245456339847, 6.592654920038115, 8.500056543855663, 12.823973164263009, 2.0784603782325775, -1.282344265570785, 0.6513788352602532, -0.2230135940669129, -5.9208901568117085, 0.1624059903916617, 40.56165234219708, 4.03530298722967, 16.491748762675513, -23.310410288446434, -1.5469059379339711, 36.508880619483506, 24.984775106937747, -3.46414076683579, 4.752220420564214, -2.950273391714944, -8.53884740401108, 1.777486550750556, 1.0080581548610894, 2.8464305983038116, -4.089775888651616, 1.1869431436875857, -4.086308019148987, 8.025787570823482, -11.436605706074658, 4.773660688255973, 4.181957713005886, 0.7462579114127266, 3.7435668128372583, -0.9945445436275122, 8.088421973063802, -5.630352018265, 3.0175020849405314, 8.077507456264442, -4.245541489789389, 3.6731105071735897, 7.068354689734065, 7.430838899632363, 2.384959046705063, 0.9822433778373352, 6.902673766294498, -4.66842338581705, 16.939988515411045, 3.0636216405130696, 4.226801275838426, 3.715869874634791, 12.747036262215744, -7.975299665673106, -2.9391978911259287, 2.951732509174178, 5.005849394414469, 4.179808570247687, 3.4268348989431843, 4.101835866515927, 8.1669303226505, 1.9586169291668796, 12.575642718038154, -7.838118493386844, -4.941904084245351, -5.680049824190888, 4.644308690280127, 6.784066437050285, -1.4758313606590718, 8.454239241099515], 'changes_betw': [4.732779962146605e-05, 4.4091710758420444e-05, -3.236088862990094e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0563888484372876e-05, 0.0, 3.438344416938077e-05, -3.236088862997033e-06, 0.0, 0.0, 4.409171075839269e-05, 0.0, 0.0, 0.0, 0.003257932462825408, 0.0, 0.0, 0.0, 0.0, 0.0, -3.23608886300137e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.831577755124659e-05, 0.0, -4.409171075839269e-05, 0.0, -5.663155510249318e-05, -4.409171075836493e-05, 0.0, 1.2539844344128248e-05, 4.085562189537484e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -5.663155510250706e-05, 0.0, -5.66315551025031e-05, 0.005973820041098227, -0.005917188485995827, 0.0, 0.0, 0.0, 5.0563888484372876e-05, -0.0029306829765545364, 0.0, -4.409171075837881e-05, 0.0, 0.0, 0.0, 5.663155510243767e-05, 8.494733265375365e-05, -5.6631555102520936e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (79, 81): {'changes': [0.029544852579254766, -0.40520702988036916, 3.5445385232620907, -3.99827309553109, 1.456604275727237, -3.413024407294813, 3.9785697982871397, 5.419723692084403, -0.49748397440971814, 1.6942594466105447, 0.1445001722908188, -0.21541556846491972, -6.078453731518209, -2.8323589219017435, -1.6514044743542478, -11.855370152538654, 3.7727866654697095, 2.415419059874665, -1.0096045387118764, -5.990046961414436, -4.987584396645289, 10.106650435477718, 10.154388261507307, 4.822116615900086, 10.405683415294561, 5.190534151603387, -2.650603766206224, -14.289589755880485, -0.6647109731727028, 0.47889101909471776, -1.389917727287873, -5.598243974571346, -8.743655576231461, 0.5243217719207678, 2.2494141330841657, 3.685856028438991, 0.18261482240297155, 5.358797773075807, 0.5440295138327116, 0.785196757635795, -1.9245404686033112, 0.3739172974438816, -3.098418998794642, -0.45518467979906063, -7.7939646497463855, 7.246087488758121, 4.709540943838036, 1.1144794693012585, -0.07523472724312796, -1.0722613942013481, -9.982312189484105, -2.7542029353279744, 1.606559712135379, -0.8080728522850222, -12.941289813125309, -1.3886225893436404, 0.43420898000115216, 7.298018176938939, 15.494325504956294, -1.4992537694342047, -0.18354383884499725, -1.2301393435966759, -2.0751803926198136, 0.7628093642378886, 1.4875156038171369, 0.6763460551768716, -5.023034093480334, -11.131506348853236, 1.6716406563445219, -6.072004014013416, -5.757936916182416, -4.9474678127910465, -12.186205021139813, -1.421170905585484, 5.645725977570919, 10.037585473351442, -4.781959305978873, -1.0936716527402837, -0.4651055272050826, 0.9418208474544087, -5.6007121619051645, -4.327609302359804, -2.7426896619384706, -5.78220842847486, -9.481818972564753, -5.276027217502943, 1.1983660831542977, -4.642360455607445, -2.713497392520111, -2.556262997707506, 3.451691387651465, -5.25910617013659, -8.785568384566346, -10.940193408603761, -1.4408339631999638, 8.52787499409365, -2.5124674855117064, 4.776644585601105, 1.9084803806321133, -7.552695765241452, 6.432279297160392, 0.09304586844237406, -1.0863412024247907, 4.455088473783775, 5.9580066179552205, 7.0723766092935705, -1.8639426324799047, 6.668490038588942, 1.4343040633388568, -6.195144840573846], 'changes_betw': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.494733265373977e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.494733265373977e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (58, 56): {'changes': [0.3959076140072071, 4.76911785857364, 8.793616351558377, 9.00929953451896, -26.38680376637421, 12.388003865105134, 4.207771359486713, -10.904832236879827, 97.82084582396126, 15.927922188995353, 26.05990696020862, -18.015915334218477, 20.38208658785797, 12.64828262849926, 4.564480678713245, 18.783162584311384, 5.825533199099482, 5.036035226542253, 1.1040859775571121, 35.34148611288083, 4.853652922787546, 7.206961628713628, 7.573680540073923, 23.544967232129068, 9.198329633977266, 9.241083190968695, -23.0641930198995, 3.6988505682320465, 24.643703905445193, -4.932832459941665, -29.36619092188546, 10.187745972016955, -57.29527349711196, -2.8733195074569835, -12.037233259972666, 13.482638091424121, 0.7707758912111267, 15.086397705633715, -7.728898308391962, -1.6456216939346433, 5.538817569147625, 21.19490949204915, 22.96630061969617, 17.936869085971864, 17.352240211113042, 20.66966613536715, 2.4065814383314716, 2.4681719377899043, -15.498028788290789, 18.800576672216863, 11.702027856407824, -1.0909607941716075, 12.845791235909815, 5.012460018170259, 11.738663965278079, 1.3710801188618689, 112.01256380950834, 15.618458145383158, 13.170944264650359, -1.4886384272086914, 19.542982354019642, 11.646209537366232, 18.841250923559357, 5.8331953762474775, -7.396291959120063, -13.16976769518159, 9.96237024886986, 11.906991593359749, 1.8808389246670174, 92.76849493097895, 22.26796763216143, 15.30226195798349, 2.084175570982069, -7.686702777601965, 3.293720580144196, 10.056353235213066, 4.12494751763866, 6.731567937918783, 17.01561394100804, 16.664715936280373, 10.3181979205076, 25.04474122289696, 13.189768158443172, 7.302423728686449, 7.501183187485793, 7.872637423555702, 17.669106677176615, 0.6521312970714916, 14.861843593659955, 2.919511632759601, 14.401767087794155, 15.728094029164595, 23.34764470237056, 15.93260158401057, 6.69649065768013, 16.633454924415418, -22.28490482703117, -20.722047139192426, -26.207225565684993, 18.503294748961082, 8.586786940287311, 37.261749210145325, -0.2680897741906847, 22.036102013950142, 30.28372808192185, -1.4957760869648382, 15.594795288953236, 3.1913975205060297, 16.397508142996287, 7.775321181948527], 'changes_betw': [0.01292736598547023, -0.016846269598563057, 0.002419785447308559, -0.0001415788877562607, 0.15797210491400104, 0.0, 0.0, 0.0, -0.12436006342734175, 0.0, -0.01979960519715876, 0.012513551122113786, 0.0, 0.0, -0.016144038315292042, 0.0, 0.0, 0.00019821044285876126, -0.012689513454039292, 6.79578661230034e-05, -2.831577755125006e-05, 2.8315777551250927e-05, 0.00013389317670661066, 0.00013389317670661066, 0.0, 0.0018874488293448566, 0.14889568467550127, 0.0, 0.0, 0.019574697021180226, 0.16101888257851565, 0.0, 0.11251152856657447, 0.0, 0.033070805624322464, 0.0, 0.03622841933239476, 0.005545847288966607, 0.05144774525508472, 0.01701697328608648, 0.0, -0.009577609501156902, -0.014221397019562168, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04756565215280814, 0.0, 0.0, -0.0017652864747665364, -0.00047004190735077615, 0.0, 0.0, 0.0, -0.1916941734220022, 0.00014157888775625767, 0.0, 0.013062877206608037, -0.000679578661230037, -0.0017292849861657023, 0.0, -0.05232310729252615, 0.02483091435690829, 0.04277745416889148, 0.0008636312153131758, 0.0, 0.0, -0.03477662896623143, 0.0, 0.0, 0.0009040823261006747, 0.0017292849861657023, 0.001769736096953219, 0.0, -0.010788311247026838, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.010788311247026852, -0.0008656537708525616, 0.0, 0.0, -0.010788311247026838, -0.008786385774153353, 0.0, -0.0028882093102276588, 0.0, 0.0, 0.0, 0.016686487710952547, 0.019574697021180185, 0.0195746970211802, 0.0, 0.0, -0.004997734737795901, 0.0037886510363574592, -0.00878638577415336, -0.00878638577415336, 0.0, 0.0, 0.0, 0.0, 0.0]}, (58, '58_37_1'): {'changes': [-1.1056415854396704, -0.1859107303632861, -1.1450171478659996, 2.532422243273402, 2.682975190706678, 0.9727953104386131, 2.608332744663045, -0.15553798083283255, -2.946508668643567, 11.533727363426522, 28.161995214042093, -41.18110885671004, 4.598457896787977, 2.906655735976358, -0.5081431591163295, 9.512023963814102, 8.411803882549464, 6.591811058619044, 0.5949947934311659, 11.601701644496131, 7.208859044155091, 5.776593865107003, 6.336220142761562, 1.539871960932743, -9.973471803355096, 5.914218559393433, 2.0712477202978548, 11.145375893925518, 7.029537251772069, 0.05003242250896278, 2.7941009886750834, 12.947636825082157, 6.884989316076528, 3.0487311637492667, -6.963080071598824, 3.0046009966332576, -0.42761430701467873, 0.9890923810943661, -4.390129887237123, 1.9953661143970365, 0.2710479151147638, 40.956735539152334, 37.26192100411281, -7.2558563898646184, -9.92526107838313, 0.00453716424550521, -2.450334282390287, -4.662597307406827, -8.066476163729199, -4.5568546615808145, 4.064946996667459, 0.8305857588081551, 1.8745438397737644, -7.97753245453589, -10.437550033582255, -6.455199618990434, 5.538013684236208, 2.1510333949847507, 7.512828677792754, 0.21171824410153306, 6.545182386763116, -5.210690967845835, 4.034904347909077, 0.5468104683010147, -7.634746406962925, -6.362623353961155, 7.163077784242233, 12.039430881711638, 20.417606318691952, 158.03232702505903, 2.680214208818313, 13.127821994421083, -4.607451948522055, -7.897653513907613, 8.55481113113359, 14.331937175620652, -1.71045320258402, -6.496690529438922, 10.31799405737479, 1.4057963177058923, 2.8394450955559876, 3.2751687741632054, 4.826642479806935, 5.2404994480700395, -10.656807322828115, -1.6073904884575967, -4.920866082097746, -4.8906999917680025, -3.225102014903598, -2.494047991083555, -2.787824181150228, -4.24825285737262, 15.731994252489386, -4.331293230984954, 0.15226869646787122, 4.107237680292485, -2.0490392085789892, 4.2595214964625825, 6.840676863762923, 19.44487157743714, -7.114504299783874, -6.0068741183908685, 9.4594118405588, 3.3628379468763683, -5.31239027914674, -3.602683411281987, 10.492633303234669, -16.592706659052624, 8.957702527221443, -11.582840458009883], 'changes_betw': [0.03498171609792439, 0.0037975502807310324, 0.00894899923951907, -1.9821044285883715e-05, 0.008453473132372241, 0.0, 0.0, 0.0, 0.023891032797760614, 0.0, -0.025370329919259596, 0.022855484361600575, 0.0, 0.0, 0.0037918871252203834, 0.0, 0.0, 0.0, 0.0037918871252203556, 1.4157888775625464e-05, -4.247366632684907e-06, 4.247366632687509e-06, 0.002011229228354623, 0.002011229228354623, 0.0005092794848146542, 0.004022458456709246, 0.0024747989579793905, 0.0, 0.0, 0.0020302412504247386, 0.008396841577269679, 0.0, -0.005922042619290316, 0.0, 0.019303674578904, 0.0, -0.005148212869925506, 0.0026859537562901187, 0.01860427487338806, -0.0029925731760595264, 0.0, -0.02462623173632352, -0.026523388832257383, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02088935812177406, 0.0, 0.0, 0.003880070546737252, -4.854133294500754e-06, 0.0, 0.0, 0.0, -0.029047335889843567, 0.0, 0.0, 0.0005570117955439879, 0.0003341261751047689, 0.0008870928595699074, 0.0, -0.006213695128068075, 0.01836925391971267, 0.020436305680954003, 8.494733265375452e-06, 0.0, 0.0, -0.04979814895717038, 0.0, 0.0, 0.00013955633221688322, -0.0008870928595699316, -0.0007560312606184234, 0.0, -0.0015290519877675796, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0015290519877675934, 0.0008955875928352841, 0.0, 0.0, -0.0015290519877675657, -0.0005011892626571451, 0.0, -0.001529051987767583, 0.0, 0.0, 0.0, 0.000501189262657159, 0.0020302412504247247, 0.0020302412504247455, 0.0, 0.0, -0.0005011892626571538, 0.0, -0.0005011892626571521, -0.0005011892626571555, 0.0, 0.0, 0.0, 0.0, 0.0]}, (45, 41): {'changes': [-0.26052038925933374, 0.02008675421233974, 2.6973113295542674, -0.2861531022751933, -1.8532046524405885, -4.43352261771409, -5.851388721964042, 4.814398078677925, -0.442453872046066, 4.663763795479554, 2.590529415518631, 9.51908343107408, -0.8131718843022213, 2.1652635387499686, -0.4196993169312506, 7.00151255210622, -3.436856023341875, 5.31883325486757, 1.427755172149638, 1.893472501112626, 3.919748124651818, 3.6580862497392843, 9.09050713949597, 13.417811243291823, 2.767069759146068, -1.0806826241532974, 0.24018425424725365, 6.2704717982873035, 10.510884069580413, -3.5351691789533817, -1.0925755032056799, -0.5676945764927268, 0.2437197628698442, -11.867179950796185, 1.5344418898620091, 7.917619632808595, 1.2871277476904197, 0.02254338545463952, -1.0022177583588032, 2.304833638807601, 7.354051777854579, 2.778984450334818, 3.5262571365151842, 1.4072759986984238, 12.250179178238056, 5.736565649328156, 0.1358267578393395, -0.9363321668500078, -2.0515440940545986, 6.022825654838698, 11.008558242671626, 0.8351128054503931, 17.933007224354753, 3.7993863440767086, 0.3363969916850351, -2.814385101679761, -0.9463381649636133, 6.909172517068612, -0.5799922874614936, 8.538199043963274, -20.92834017617818, 2.9597336986697798, -2.357054147940687, 0.46666386384460523, -2.9817959855857623, -5.220197100211877, 12.725604142418092, -5.97689708535691, 4.957824769433131, -3.4839478435255984, 9.041758432198662, 12.520999954346081, 2.808504965403017, 7.0262913492440475, 15.156962269216109, -3.9914627256134168, 0.34952516880895246, 4.9608435926191135, 0.0999333658161845, 3.675349226317735, -9.476993550403733, 9.062744521178445, -5.749279448254839, 1.9829290718587345, -0.39166166753193465, -2.981755121737109, 15.829911035409992, 2.1708307700335467, -0.20133588935519242, -0.722107791656974, -5.343286555161853, -7.024366296574726, 6.937736070743171, -4.422504079546002, -1.521445001055696, 0.48925468643076897, -5.810301769331673, -5.037735606115774, -8.23032548824196, -0.011778356918227928, -0.1447683213306732, 3.5475791694969416, -1.4327785547120868, -3.3719479993614243, -1.752468069793565, 6.851090022696837, -0.6440205484486796, 1.8409770622307349, -0.20005744238318357, 3.2275929881614047], 'changes_betw': [0.001984936006342808, 0.0025031147355306094, 0.0005181787291879264, 0.0, 5.6631555102520936e-05, 0.0, 0.0, 0.0, 0.0017499150526673335, 0.0, -0.001070336391437332, 0.00012175784347038046, 0.0, 0.0, 0.0025031147355306094, 0.0, 0.0, 0.0, 0.0025031147355305816, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0001217578434703796, -0.0003397893306150146, 5.6631555102465425e-05, 0.0, 0.0, 2.8315777551260468e-05, 5.663155510250706e-05, 0.0, 0.0, 0.0, -0.0008494733265375365, 0.0, -0.002503114735530637, -0.0003397893306150146, -0.0008494733265375365, -0.0026730094008381167, 0.0, -0.0013138520783780444, -0.0013138520783780583, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0008494733265375365, 0.0, 0.0, 0.010156869407633773, 0.0, 0.0, 0.0, 0.0, 0.0008438101710273038, 0.0, 0.0, -0.008053007135575968, 0.012968626118473214, 0.00016989466530750036, 0.0, 5.663155510249318e-05, -0.0007362102163325501, -0.0008494733265375642, -0.002568241023898516, 0.0, 0.0, -0.0009061048816400435, 0.0, 0.0, 0.00011326311020500371, -0.00016989466530751077, -5.6631555102501854e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00016989466530751077, 0.0, 0.0, 0.0, -2.831577755124659e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8315777551260468e-05, 2.831577755123965e-05, 2.831577755125353e-05, 0.0, 0.0, -2.831577755125006e-05, 0.0, -2.831577755125353e-05, -2.831577755125353e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (43, 41): {'changes': [1.308690740042488, 0.35950897423680317, -1.4207148268831702, -4.03995173535256, -1.5711482490781847, 0.49091312198785886, 0.9326194780346384, -6.7009970288963245, 1.1227316292369949, 4.525789666481842, -4.927703120729831, 3.1191917828379587, 3.943245033431822, 15.499714451806994, 0.13670393801727698, 13.65307305095817, -8.015117373162298, 1.9876471975353525, -1.1784152446393747, -3.1137772297676065, -2.170490829103585, -0.34757618425800274, 3.935472108129943, -3.5105527710479407, 9.802473653489812, 3.650413833543496, 0.16823553069023944, -2.8242215073542525, -5.192613053593334, 4.700253876914871, 0.9353225263184655, -11.461105723005346, 1.016365313904572, 2.942100019759664, -1.8290334774886574, 3.7351047149537067, -1.6078976291480238, -0.018932083217436002, 0.23846591589211386, -2.0157370283807126, -1.2516287115242335, 0.034524062904282005, -3.560118348409489, -16.775032275796264, 3.0773123745088924, -3.2798350605438316, -4.002958928729782, -0.8673779189585389, 0.6624318097558799, 4.640825039318543, 0.7958645512911176, -1.1288452271513378, 4.389433453230595, -4.072142229729991, 5.713619963551423, -12.151441285051277, 0.8430750385289585, -1.5284648145721746, 4.677455162749254, 0.06534884862233525, 44.417518880356425, 11.07365727792515, -1.693202042189455, -0.7695586099530587, 2.6072781301157875, 1.5557972388866403, 31.716034740878115, 7.355340092925303, 2.693755497433017, -1.6024139885051056, -10.369615307217146, -6.2813443163457805, -4.652531171488306, 5.347186302940429, -4.977903097305671, -5.446077403635968, -1.0887774049894148, 0.45910961868764844, 3.089095838438027, 10.202636049943806, 6.7624681664649415, -11.323573362189393, -5.395896379138151, -3.9472422214383016, -0.40034207200656624, -2.607128001552468, 3.3168428145091013, 3.2170021164504874, 5.70195304583882, 1.2295162113943618, 6.620784820134432, 1.4244767752151404, -11.218521319269286, 0.5892595775290488, -5.4402205409864735, -6.917245677053955, 3.2648446282515238, 2.5729740328342245, 4.141532456162537, -6.042199652840452, 7.067870752117727, 2.0567208355720368, 7.313035762779492, 13.897001742523514, 8.970036101902423, -13.995553813372595, -3.891330573797404, 3.3431536943511446, 12.5003538141207, 5.81453516167997], 'changes_betw': [-0.0005634839732698294, -0.0005691471287800343, -5.6631555102465425e-06, 0.0, -2.831577755124659e-05, 0.0, 0.0, 0.0, -0.0005295050402085166, 0.0, 0.00038792615245218653, 2.2652622041003517e-05, 0.0, 0.0, -0.0005691471287800898, 0.0, 0.0, 0.0, -0.0005691471287800343, 0.0, 0.0, 0.0, 0.0, 0.0, 2.2652622041000048e-05, 0.0, -2.8315777551274346e-05, 0.0, 0.0, -1.4157888775623295e-05, -2.831577755124659e-05, 0.0, 0.0, 0.0, 0.00022652622040994497, 0.0, 0.0005691471287800065, 0.0, 0.0002265262204099172, 0.0005691471287802841, 0.0, 0.0003426209083702142, 0.0003426209083702003, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0002265262204100421, 0.0, 0.0, 0.0020585570279759158, 0.0, 0.0, 0.0, 0.0, -0.00018971570959341877, 0.0, 0.0, 0.0054394608675953915, -0.005379997734737797, 0.0, 0.0, 5.663155510249318e-05, 0.0002548419979612193, 0.00022652622041001436, -0.002568241023898516, 0.0, 0.0, 0.00033978933061502153, 0.0, 0.0, 2.8315777551250927e-05, 0.0, 2.8315777551251795e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.734723475976807e-18, 0.0, 0.0, 0.0, 1.4157888775630234e-05, 0.0, 0.0, 0.0, 0.0, 0.0, -1.4157888775616356e-05, -1.4157888775637173e-05, -1.4157888775623295e-05, 0.0, 0.0, 1.4157888775626765e-05, 0.0, 1.4157888775630234e-05, 1.4157888775623295e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (83, '83_31_1'): {'changes': [0.5719527457344054, 2.309630193204079, -1.3932596500845165, -4.430836167346726, -0.20256563242629966, 7.295264142697164, 0.1071576759322852, 1.5613062550240215, 3.6690640151313048, 3.9746350002554323, -7.4053140554597405, -2.9913666082825046, 12.3249030628742, 6.289979199433276, 3.270853425613538, 5.03749252740306, -12.162013055624357, 15.25207454733939, 2.9431919221914598, 17.181472183540023, 0.8173187857223354, 2.8410730729967497, 0.26213616069725276, 9.187036402917443, 4.333508459221662, -0.8994660035808977, 1.1226768289866413, 0.5146960931938906, 13.417089496984744, 2.3344137588834997, 2.828787102295472, 2.012730561825947, 2.0220234846527205, 5.337937607636206, 0.11647263343294867, 12.521769461282247, -0.46207332992789674, -4.294096573839525, 0.42872056262637415, -1.2137321476328964, 2.554183492389086, -2.934191696494011, -6.931357343192346, -1.2507576969541674, -9.429338158214392, 8.454636129650908, 3.119556391245382, 3.2466186743522343, 2.0425585175449754, 13.954637102355434, 16.789330324753905, 4.588088346776786, -2.2271013575058305, 3.484731357749183, 12.963215381750217, 10.315929376827285, 0.555213148241446, 0.7647524092829769, -19.346622359339563, -2.7211840774531453, 2.1370585892819634, 66.90777683526905, 0.9586544324424153, 0.22741790159498976, -1.2472477774623911, 4.073021666657887, 14.244717041203074, 5.004326363757741, 7.443884210251127, -3.34823801488821, 9.236532516591012, 11.151076066818177, -24.106395076364663, -37.0553087578873, -48.153723692878515, -1.2281356434074837, 1.2940056541815181, -2.4345721920138317, 6.090080838374774, -3.7095150775859054, 8.442893240787782, 16.21309735278379, -1.5641941820047123, -5.000402216658557, 9.710800095406483, -3.1939181445142424, 74.83216481544338, 1.7247372845676523, -4.8540654945686015, 6.991102058810725, 6.067854180403117, 9.202957253404861, 6.9352606958423735, 15.432393715042082, -0.19052943986181958, 9.39429064061946, 0.8897887724127713, 7.388090638512267, 9.33287737028408, 9.529156825821644, 2.4657229962949714, -2.9453910673991857, 8.553236987764706, 7.6781467076847605, 2.4058619357344355, 3.2413445108557823, 14.084570368898113, 2.9626671631221484, -7.135454799985553, 12.81557934586803], 'changes_betw': [-0.006704771613028515, -0.014126336909211112, 0.0033465203954501227, 0.0, -0.002147953982816364, 0.0, 0.0, 0.0, -0.00473439800656919, 0.0, 0.006305114638447906, 0.0018332443408895956, 0.0, 0.0, -0.014225442130640947, -9.91052214293802e-05, 0.0, 0.0, -0.01422544213064092, 0.0, -0.0005824959953400384, 0.0, 0.002178696827014867, 0.002178696827014867, -7.64525993883795e-05, 0.004697182984644749, -0.000934420659191304, 0.0, 0.0, -0.0008069996602106805, -0.0021479539828164057, 0.0, 0.0012135333236250462, 0.0, 0.0030702393087715107, 0.0, 0.01422544213064092, 0.004697182984644749, 0.0030702393087715107, 0.02674546543048062, 0.0, 0.006458019837224674, 0.00645801983722466, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0030702393087713997, 0.0, 0.0, -0.01628399915861678, 0.0, 0.0, 0.0, 0.0, -0.0008826432373832682, 0.0, 0.0, 0.009611992945326234, -0.0001415788877562581, -0.027812970244162905, 0.0, 0.0020913224277138154, 0.003013607753668962, 0.0030702393087713997, 0.00019821044285876083, 0.0, 0.0, 0.0038517547691859777, 0.0, 0.0, 0.012780528453311327, 0.02883233823600796, 0.025860799637558045, 0.0, 0.0006795786612300431, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0006795786612300292, -0.013250165849554237, 0.0, 0.0, 0.000679578661230057, 0.00012742099898063047, 0.0, 0.0006795786612300396, 0.0, 0.0, 0.0, -0.00012742099898063047, -0.0008069996602106735, -0.0008069996602106666, 0.0, 0.0, 0.0001274209989806322, 0.0, 0.00012742099898063047, 0.00012742099898063394, 0.0, 0.0, 0.0, 0.0, 0.0]}, (56, '28_56_1'): {'changes': [0.8508082517224498, 2.0886154862483863, -10.114045274422544, 11.966079350494027, 0.5048392854213901, 4.972061854858225, 11.93364308632593, 28.901326066865664, 11.065236894284276, 22.83736036284796, -34.01675197259683, -18.913010469425615, 25.155884380393957, 38.47004405221688, 1.9574508781870925, 15.245904431866876, 27.840135944038792, 7.241886776762669, 0.2793946384012358, 27.550970106765106, 3.70737806014958, 15.93509023221489, -26.006792519206854, -30.111880208945763, 15.541130243623456, -25.07812861231436, 1.4674122005869918, 21.101107100057902, 19.426915248731632, 7.175477596590937, -0.2653998457558444, 20.185556925178787, 37.22832880852421, 18.235244829952705, 60.61403689517696, 14.782548534642103, -9.414847949998244, -19.624853936025076, -4.107075105610569, 1.2590177529657183, 27.84629506352627, -45.205279987081035, -40.36552253862429, 13.672571934788607, 12.367179706041327, 30.423289110113586, 5.479546989268201, 8.718363748750221, 33.89052213426407, 18.68154704537642, 29.52157915326393, 0.14387756761576753, 3.7046625184510162, 33.94399373082703, 30.818908437861182, 14.16125248359549, -6.285755223219887, 34.328566997496694, 10.954810539392724, 4.090312338428532, 26.009968741368112, 7.920475646538151, 15.574217583793086, 6.08099659225012, 232.85821956984591, 44.773655982825595, 9.672030722821944, 34.87417270338781, 20.72948754675798, -63.864844636715524, 27.756210568157712, 25.174569911018835, 21.82682906195936, 11.18279361157002, 33.61116999211464, 15.639404641854071, 7.467392223267666, 25.36996041667294, 5.798880953205071, 24.130378279421137, 4.267321999160515, 10.342470227614967, -4.141520545921708, 3.5370934309480333, 9.889279696637544, 5.8028531603854105, 15.29329211796474, 20.44629169380198, 29.62381474705444, 5.871986841478694, 8.51019237686748, 17.90613104487693, 18.880852064650554, 23.389622574038697, 18.350245276912943, 16.86942963846687, 19.51066729994706, 14.274939071961313, 16.05144252765166, 33.80488539058993, 24.460179508097696, 4.2731281281939175, 5.848957561909145, 8.07821633274844, 4.327008502682048, 20.895830889754222, 11.643061892928841, 20.4042401621914, 25.45759857839994, 8.524689164826384], 'changes_betw': [0.01611005938223109, -0.0029743701762047214, 0.028320227173438198, -0.0007078944387812897, 0.023559940455964934, 0.0, 0.0, 0.0, -0.02009004417261301, 0.0, 0.09406663106968917, 0.011913256638027264, 0.0, 0.0, -0.0014877918547643032, 0.0, 0.0, 0.0003681051081662707, 0.003609048104460888, 0.00033978933061501763, -0.00014157888775625724, 0.0001415788877562581, 0.015144086856625105, 0.015144086856625105, 0.0, 0.03125091014999276, -0.002166156982670725, 0.0, 0.0, 7.64525993883769e-05, 0.021240878274517414, 0.0, -0.024324466449848717, 0.0, -0.07645745352167369, 0.0, 0.11354464993608723, 0.030367457890393718, 0.038699982201511274, 0.003227189618626808, 0.0, 0.09692652460236563, 0.09641684060644307, 0.0, 0.0, 0.0, 0.0, 0.0, -0.016787615487921306, 0.0, 0.0, 0.007063572965713638, -5.663155510250359e-05, 0.0, 0.0, 0.0, 0.05119573483487849, 0.00016989466530750947, 0.0, 0.005284533113279435, 0.0002491788424510144, 0.0005218193291587764, 0.0, -0.057485073540119336, -0.12300859181593127, -0.0331626296458101, 2.831577755125136e-05, 0.0, 0.0, 0.07889706001326796, 0.0, 0.0, 6.876688833875287e-05, -0.0005218193291587868, -0.00048136821837128224, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0005501351067100126, 0.0, 0.0, 0.0, -7.645259938836996e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 7.645259938838384e-05, 7.645259938836302e-05, 7.64525993883769e-05, 0.0, 0.0, -7.645259938838037e-05, 0.0, -7.64525993883769e-05, -7.645259938838037e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (56, 13): {'changes': [0.7414992429332514, 1.4219272655484794, -0.6120694996758616, 5.379464246034786, -19.844061264813707, 4.847406828381452, 5.914315100077772, 15.808440201365045, 54.25246048368613, 12.713936896390294, 11.093172582069045, -7.8594005161763505, 11.93199317429648, 10.04640266147453, 2.9493097917065967, 3.195788927384143, 24.317684541511966, -3.0421062874719382, 3.1515281355667177, 13.914125214530657, 2.4718209412440757, 4.167891373876671, 5.5164318320157975, -9.77768486296199, -8.306713405631143, 6.272173357584165, -22.953218028206543, -4.186289214780004, 7.937764376499956, -42.965401260052914, -22.48746790712174, 25.405881021703635, -5.707497545819834, 20.13700806863568, 13.61729164284938, 3.700622075234662, 3.8101080056488286, 5.09081943642029, 6.51038525028391, 3.490937099729848, 17.47277213967095, 12.399036754638786, 14.903906508556574, 12.865274194853328, 7.883862581867902, 14.985972232524773, 0.4449407067976523, 6.249778976774778, 9.674420503786948, 16.28878032972284, 8.4998443638749, 5.219534984665152, 20.69054751893529, 12.555604992187227, 8.17215702021224, 7.513648606583672, 40.544888996650556, 10.022857436865422, 10.368864926349488, 6.0191425064871495, 5.560066604108101, 7.120347016461523, 23.082293052438445, 18.78191532350789, 11.347473232884, 11.904067113730221, 22.766309376808437, 15.919143768625787, 14.157727827770799, 40.178511176501786, -2.5889626434206434, 12.950472441376519, 12.987452418297948, 20.182610131777494, 19.856296010973125, 14.057008451003583, 58.33199307142057, 12.783541983482223, -7.8075365500861835, 2.1957346868308605, 15.179116683872024, 2.885143402508959, 10.815073964910567, 4.267838119635741, 1.9120322325219092, -17.642151724055665, 4.876266985751272, 14.951919694967643, 20.765464286401084, 141.67151134435886, -22.917787915739055, 19.795690403098604, -120.63251690515264, 12.734684417878157, 8.958465231966045, 6.05556813740202, -76.32858755680371, -84.5119922270508, -89.74466711071364, 4.783244750764254, 21.596156476315457, -80.68135246416932, -98.74287529418967, -42.733657447756755, -54.36159055771664, 17.910405852479926, 5.2735300273105, 17.147505177079438, 9.172893501244971, 4.461313579627642], 'changes_betw': [0.015197482322864608, 0.0057197870653528105, 0.007224568386647964, 0.0, 0.08791239907447879, 0.0, 0.0, 0.0, -0.07110091743119272, 0.0, -0.007341876607931716, 0.0038671261912852396, 0.0, 0.0, 0.0057197870653528105, 0.0, 0.0, 0.0, 0.005719787065352727, 0.0, 0.0, 0.0, 0.002677863534132671, 0.002677863534132671, 0.0, 0.005355727068265342, 0.11688348462048778, 0.0, 0.0, 0.12877611119201338, 0.08944145106224631, 0.0, 0.016568774978560893, 0.0, -0.0064519521706065674, 0.0, -0.007418733718427911, 0.0036567804151902417, -0.00645195217060654, -0.004473892853097711, 0.0, -0.004623561963011551, -0.004623561963011544, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0064519521706065674, 0.0, 0.0, 0.005719787065352699, 0.0, 0.0, 0.0, 0.0, -0.0829894988916397, 0.0, 0.0, 0.0008211575489862621, 0.0004247366632687726, 0.0013591573224600514, 0.0, -0.10762018025014977, -0.006451952170606595, -0.0064519521706065674, 0.0, 0.0, 0.0, -0.01120900279921689, 0.0, 0.0, 0.00022652622041001262, -0.0013591573224600757, -0.0011326311020500666, 0.0, -0.06804281345565749, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03950050968399592, 0.0013591573224600723, 0.0, 0.0, -0.13260278627251104, 0.026252770901088948, 0.0, 0.12878015630309209, 0.0, 0.0, 0.0, 0.15383961943594976, 0.1318342151675485, 0.13217400449816355, 0.0, 0.0, 0.03270876818277429, 0.03304855751338931, 0.029480769541931635, 0.03168940019092922, 0.0, 0.0, 0.0, 0.0, 0.0]}, (56, 23): {'changes': [0.6924278194013009, 2.585304628147025, 7.3274722046913325, 1.5900770268760596, -7.527232830361115, 13.531816517117846, -3.4808923437208534, 5.19924285986508, 11.18111350030967, 9.513963381273754, 5.57057121399265, 17.777671582679545, 4.352551100533219, 19.367371381136024, 1.1132338051664163, 9.032383735945814, 21.339712946585166, 6.738691084348545, 0.3388682172908446, 15.87982038031447, -0.7233070034411782, -0.31763365088301043, 4.977813104219905, 12.35463880957738, 22.311360916540565, 4.265299788898332, -5.325126701771353, -2.4333715293649334, 10.39328315063375, -13.219671010680763, -8.400847433926884, 24.725951332064028, 3.7495261473315367, 3.030744107241162, 3.018902814397677, 9.444971826414871, 1.1120364479250746, 1.6655626739573393, 1.4735118883682503, 1.4546286316027945, 14.771957895949754, 4.3264988709779715, 4.772838555359257, 1.1986485280355055, 10.036239819797117, 10.26802022351535, 5.34007664232881, -0.5633808689196798, 3.3737125639802343, 8.675445780216478, 21.899883478146933, 2.8024821904441097, -1.438344638066127, 12.341960501433732, 8.732441061139866, -2.8898109288765568, 9.165280810788659, 7.461425118285007, 9.768184969846175, 2.3375377640319854, 7.41033326475943, -2.7432635224463695, 9.582066880830382, 4.41776654676044, 1.5781192648768823, 0.18550179227393926, -0.25341481626202267, 1.776469883557013, 14.409804409945139, 2.3788192741074567, 13.612425102052725, 20.352330638741705, 0.4720554736857423, -2.916568485328412, 13.491853053201027, 4.197632397273367, -12.45112360897452, 20.771081504209974, 14.962193914074646, 15.179535497527638, 21.295812473853886, 7.122388793170273, -1.5507304402046884, 0.777696736809645, -0.05875886615936565, -18.5077444944356, 7.10317872289275, -2.2541876638071585, 1.3922279619102937, -10.635884715569468, 139.60225164383144, 8.02566861299573, -52.16915205597053, 9.491359297285555, 12.268197770371842, 16.398073295084373, -42.222838755510494, -27.739410668402257, -47.49586610696717, 21.16899566760469, 17.38758183048469, -70.52863708051251, -105.60071791786984, 56.085418087431634, -2.8485250729863765, -6.795023697817612, 15.8890184801933, 7.444322688121417, 21.98648428612981, 10.36838808425739], 'changes_betw': [0.002321893759202609, 0.002010420206138841, 0.0012175784347038116, 0.0, 0.034984143164571324, 0.0, 0.0, 0.0, -0.03198267074413866, 0.0, -0.00222278853777326, 0.0, 0.0, 0.0, 0.0020104202061388965, 0.0, 0.0, 0.0, 0.0020104202061388687, 0.0, 0.0, 0.0, 0.0003964208857175286, 0.0003964208857175286, 0.0, 0.0007928417714350433, 0.03433288028089257, 0.0, 0.0, 0.03962793068297657, 0.035607090270698874, 0.0, -0.005634839732699057, 0.0, -0.001401630988786956, 0.0, -0.00280326197757394, 0.0, -0.001401630988786956, -0.002010420206138841, 0.0, -0.0014016309887869421, -0.0014016309887869421, 0.0, 0.0, 0.0, 0.0, 0.0, -0.001401630988786956, 0.0, 0.0, 0.0020104202061388687, 0.0, 0.0, 0.0, 0.0, -0.03420545928191182, 0.0, 0.0, -0.00019821044285878164, 0.00019821044285876256, 0.0005379997734737754, 0.0, -0.046253822629969465, -0.001401630988786956, -0.001401630988786956, 0.0, 0.0, 0.0, -0.002222788537773239, 0.0, 0.0, 0.0, -0.0005379997734737824, -0.0005379997734737806, 0.0, 0.027947672443085297, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.030665987088005428, 0.0005379997734737772, 0.0, 0.0, 0.025569147128780167, -0.05729697587495752, 0.0, 0.03253482840638804, 0.0, 0.0, 0.0, 0.07471117906897726, 0.041581719334012904, 0.04192150866462794, 0.0, 0.0, 0.04044908823196285, 0.06487144636991732, -0.022808358817533132, 0.010066258919469929, 0.0, 0.0, 0.0, 0.0, 0.0]}, (31, '36_31_2'): {'changes': [0.33815255344752515, 0.5158521326099539, -2.6662142105590547, 6.008696168271598, 1.3961406259070017, 7.3198835283407675, 3.992600002575614, 11.873495434891197, -0.497823417820662, 10.091132637993184, 2.7261302515640153, 1.8785110004936314, 0.5577159256191635, 0.9119708590288838, -0.8254393864452823, 3.637200119162145, 0.6798153948199115, 0.06582051569171199, -0.14286619189019234, -9.480419143915242, 2.1911092405132564, -6.4340638436352435, -9.53585785295678, -9.598689243146126, 1.337678385230845, -4.195526127671371, 1.2432208858834812, -0.35133133310256426, -0.6938388113789244, 5.689473566554028, 0.7655592061015639, 11.174878873023516, -1.2537769211317595, 5.153827297405314, -0.2764976055354609, 2.7379247230557553, 1.288052668153096, -0.4970958283418838, -0.012694518235697672, -1.5847773327751682, 12.600638441529867, 4.328061249298543, 0.5829866750336521, 8.470247601364235, 18.103412941961153, -2.515806204176613, -1.424258017618925, 0.29684910728092007, 4.009066059409143, 17.25576609650969, 4.797108420715915, -1.8005009642885668, 1.9526218344586823, 2.8356314616318627, -3.8786125544646666, 13.341733280615443, -1.2320102155998534, 7.217938950656844, 4.110076052120121, -7.2289339853023975, 1.6793372381225424, -5.2012564428857075, 15.813304556905734, 0.692416453680071, 1.9364163669719474, 1.7802303580370449, 2.0920667188009077, -4.925413121887061, -0.41117720409093295, -3.9630711559651814, 6.471715380142655, 14.118000528379952, 47.750811872618556, 19.551200678743328, 42.804931606945104, -13.489941508849881, 2.3037323253780073, -1.0343337158390113, 12.980250593549044, -6.324032947413684, 1.6710315394568056, 14.437819935303821, 8.623912182236353, 9.774928795523039, 14.918083140843265, -1.0349346161054171, -5.79691374259059, 0.38462165180487773, 7.4986468079704025, -1.155643761561798, 12.52299788492482, 3.743507955267262, -3.5636253088156025, -2.8732828794320824, -6.616700844694748, 1.8492022744767382, 0.07304182697120609, -1.20383072012433, 3.818399826926168, -7.685391679299471, 1.6365877564986135, 15.168256392115353, 3.162001495609502, 5.94277530150535, 7.561072376315224, -2.502686488483562, -1.814171053236123, 9.022401820178231, 16.261878702448314, 7.835233508258881], 'changes_betw': [0.0025941297348027748, 0.0025403297574555506, 0.0009170266815526767, 0.0, 4.045111078765373e-06, 0.0, 0.0, 0.0, 0.0023162306036924596, 0.0, -0.0012495348122260053, 0.00029286604210151945, 0.0, 0.0, 0.0025261718686795664, -1.4157888775625464e-05, 0.0, 0.0, 0.002526171868679511, 0.0, -7.281199941750263e-05, 0.0, 0.00018526608740675882, 0.00018526608740675882, 0.0001217578434703796, 3.074284419850304e-05, 8.494733265373977e-05, 0.0, 0.0, 4.2473666326869886e-05, 4.04511107872374e-06, 0.0, 8.090222157500215e-05, 0.0, -0.0010638642137111853, 0.0, -0.002526171868679511, 3.074284419850304e-05, -0.0010638642137111576, 0.0002702134200603945, 0.0, -0.0014930504991667454, -0.0014930504991667454, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0010638642137113241, 0.0, 0.0, 0.0105197158713978, 0.0, 0.0, 0.0, 0.0, 0.0011164506577349964, 0.0, 0.0, 0.009600666634305671, 2.8315777551251795e-05, 0.00820469880102908, 0.0, 0.0001092179991262765, -0.0009506011035062406, -0.001063864213711338, 2.831577755125136e-05, 0.0, 0.0, -0.0011997799459573036, 0.0, 0.0, -0.005568095399899683, -0.006845541478569004, -0.01049665873824896, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005098458003656773, 0.0, 0.0, 0.0, -4.2473666326876824e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 4.2473666326876824e-05, 4.2473666326876824e-05, 4.2473666326876824e-05, 0.0, 0.0, -4.247366632687856e-05, 0.0, -4.247366632688376e-05, -4.2473666326876824e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, (31, '83_31_1'): {'changes': [-0.4465225030210256, 0.7929293832285929, 2.0997714378951926, 0.6310917031752297, 0.5278909972049632, 0.03511665404172959, 12.181296641310468, 9.53294029634884, 1.802425889221709, 5.4522700672085875, 2.465373938013741, 5.649757892385139, 15.10933216953481, 1.4938949852780752, 2.5579455094404935, -10.270899915770286, -1.3851080073465027, 2.8201755217930895, 0.8393969240793702, -0.7881675739293144, -2.4767118895984197, 3.5226175002968034, 9.612504385549585, 6.3469580861093675, 10.340223983758136, -2.2464383074816396, -1.0306347252679586, 1.4992972074035151, 7.648552720145574, 2.3840982875736927, 2.899596633533335, -0.25619208134634164, -1.5021580509447858, 13.657540454333912, -0.8442461811129078, 9.526248816661962, -0.524950865922662, 1.7174196825399406, 1.4655630315387427, -2.1150882871444523, 7.562018411954227, 1.4967911895702457, -0.22785830773781868, 4.744144624478793, -2.4478076858988516, 3.359938494581286, -3.767124984168092, 1.4533595040865208, -1.5109985660851493, 3.7591825133497423, 13.14190643684293, -2.5088845214679054, 9.677900737163128, -2.9289519866058527, 7.228649909827141, 6.157192030417264, -2.2183787606551917, 4.81919522861071, 1.2054367479443044, -6.526392904082449, 4.164880566357283, 15.263989480750539, 2.2603079918342814, 0.772227163013099, 0.965134314487841, -2.0593274258862593, -1.2762671534951266, 15.686950253880525, 7.769839853930449, -6.6296544979180965, 9.625937355116719, 1.7489059751657123, 43.47335278560698, -21.522719995158297, -14.997221313670536, 6.530318791944865, 6.142101083790919, 8.653460611914227, -7.861439733103396, 6.842430460827188, 7.3524055411787685, 16.030011397942246, 3.330975463112054, 0.4685090478456573, 7.756138855560209, 1.0326063808832373, 66.84042536269715, 3.4380804682963344, 9.439098662175752, 4.171743870335014, -5.3370275885573335, 2.1724379341223994, 4.292095379651556, -4.742737763625769, -9.39239781038279, -13.599619417537298, 9.465368778656028, 5.060746621668443, -0.8778568612650588, 11.389500592012894, 12.725838876037471, -4.112277955428226, -0.33327583991851384, 0.07450337091839287, -3.3915122896860055, 7.67065168913598, 10.566289464362455, -4.1853412655355555, 16.251613853088372, 9.177916821421434], 'changes_betw': [-0.002055320939112981, -0.00579300357587792, 0.0021317735385013303, 0.0, -0.001071954435868716, 0.0, 0.0, 0.0, -0.0012090837014383515, 0.0, 0.0025277899131109505, 0.0010630551914955523, 0.0, 0.0, -0.005849635130980663, -5.663155510250272e-05, 0.0, 0.0, -0.0058496351309806904, 0.0, -0.0003276539973787705, 0.0, 0.0011819814572108095, 0.0011819814572108095, 2.2652622041000048e-05, 0.002363962914421605, -0.0004247366632687821, 0.0, 0.0, -0.00038226299694189836, -0.0010719544358687993, 0.0, 0.0006472177726000033, 0.0, 0.0010031875475302043, 0.0, 0.005849635130980718, 0.002363962914421605, 0.0010031875475302043, 0.013677734090578014, 0.0, 0.0024824846690289504, 0.0024824846690289504, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0010031875475300378, 0.0, 0.0, -0.002712246978301941, 0.0, 0.0, 0.0, 0.0, 0.0001169037101759196, 0.0, 0.0, 0.009776224455123467, -5.663155510250272e-05, -0.009634241056259402, 0.0, 0.0011002702134200737, 0.0010315033250813954, 0.001003187547530024, 0.00011326311020500631, 0.0, 0.0, 0.0013259874116143128, 0.0, 0.0, -0.005568095399899683, 0.011163293044026992, 0.007851965114962051, 0.0, 0.0003397893306150146, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003397893306150146, -0.013250165849554237, 0.0, 0.0, 0.00033978933061504235, 4.2473666326876824e-05, 0.0, 0.00033978933061502153, 0.0, 0.0, 0.0, -4.2473666326876824e-05, -0.00038226299694189836, -0.00038226299694189836, 0.0, 0.0, 4.2473666326876824e-05, 0.0, 4.2473666326869886e-05, 4.2473666326876824e-05, 0.0, 0.0, 0.0, 0.0, 0.0]}, ('36_31_1', '36_31_2'): {'changes': [-0.587320111118613, -0.45424969757817735, 2.016804517224557, 3.5637266254176723, -3.0018940290261185, 2.20623834368007, 5.108281200567177, 5.639912146464553, -1.9258769619943905, -2.4346189539079433, 1.5160304175775536, 14.110601512879896, -0.5477918016156309, 16.87893793415293, -1.2744175483962437, -0.36535253438037785, 7.412396942386749, 5.391556295532695, -2.1167852570633485, 10.58694912514946, -4.199399157954048, 12.92205079509364, 5.428157498453601, 14.316461042417785, 12.998912090487352, 6.422699909650618, 1.097893075170127, -0.5684265112660682, 6.368039062956655, -2.638092846594219, -0.4853795258595426, 11.063750244091409, 6.657776162433947, 5.432705219216643, 2.4088076305918946, 27.540189215072814, 1.357703769057757, 2.689768753850757, -0.08877217738498899, 1.5408055381579686, 3.4446711320312033, 3.375028503001758, 4.307857831097806, -8.064886878432844, 5.525289397972472, 2.3243556602922695, 6.905335845600817, 4.6053393579895925, 0.8724369529915066, -17.2105650996628, -4.283965794064812, -3.3387347918769663, 8.8790450197057, 9.416168844535804, 2.1106600281572128, 1.5248833315490629, -2.4240973666585433, 12.184893871636064, -10.801094733249101, -4.5218753420315, 3.1599378054201566, -22.55953071240566, 3.0839956617118673, 0.3042706600850593, 5.978867816938596, 5.46113636796391, 12.44882409247748, -2.712496326078167, 4.016977062866232, -2.815929892122284, 12.025565467723084, 14.688296975318593, -23.8551624989866, 91.97950295686348, 51.34357514247131, -4.8714791670481645, 8.930048862112685, 9.825316541608629, 4.28308841643792, 7.427455432379787, -1.2645419021035877, -8.856326646667725, -10.055328800486706, 7.497528981903997, 13.806325932010623, 0.349489462439422, -39.791826507562945, 7.3294314491679415, 0.7478344539396744, 4.326897950103792, 4.306062172833862, 8.62633970530203, 4.8101106439949035, 10.622818772462438, 3.6189502020843634, 3.472925777131053, 1.5021351642524223, 17.012452344768263, 14.742871802106293, -0.20137095699362817, 2.607284433503196, 5.382145631184301, -3.969899519788413, -0.08601763504566406, -2.5146370253118846, 9.522546924041876, 3.1779855526914105, 11.375351760187755, 11.954801286216764, -3.1256813578562515], 'changes_betw': [0.007243580408718142, 0.010873663090788854, -0.0002977201753959907, 0.0, 0.0010800446580263023, 0.0, 0.0, 0.0, 0.005841544908823271, 0.0, -0.0050268595375630165, -0.0004773231072925238, 0.0, 0.0, 0.010901978868339546, 2.8315777551251795e-05, 0.0, 0.0, 0.010901978868339518, 0.0, 0.00018202999854376872, 0.0, -0.0008114492823972919, -0.0008114492823972919, 0.00022086306489976087, -0.002302477226024627, 0.0005946313285762617, 0.0, 0.0, 0.00046721032959563813, 0.0010800446580262746, 0.0, -0.00048541332945005455, 0.0, -0.0031309159749526028, 0.0, -0.010901978868339657, -0.002302477226024627, -0.003130915974952575, -0.01347709658107224, 0.0, -0.005468585667362497, -0.0054685856673625105, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0031309159749526583, 0.0, 0.0, 0.023411889390482526, 0.0, 0.0, 0.0, 0.0, 0.0021159976052942675, 0.0, 0.0, 0.009085319482872917, 0.00011326311020500544, 0.025703849327702546, 0.0, -0.0008818342151676317, -0.0029327055320938905, -0.003130915974952672, -5.6631555102503155e-05, 0.0, 0.0, -0.0037255473035289477, 0.0, 0.0, 0.012780528453311327, -0.025194165331780014, -0.01049665873824896, 0.0, -0.0003397893306150146, 0.0, 0.0, 0.0, 0.0, 0.0, -1.734723475976807e-18, 0.0, -1.734723475976807e-18, -0.0003397893306150146, 0.02310729252625277, 0.0, 0.0, -0.0003397893306150146, -0.00012742099898063047, 0.0, -0.00033978933061501806, 0.0, 0.0, 0.0, 0.0001274209989806374, 0.00046721032959564507, 0.000467210329595652, 0.0, 0.0, -0.0001274209989806322, 0.0, -0.0001274209989806374, -0.0001274209989806374, 0.0, 0.0, 0.0, 0.0, 0.0]}, (18, 8): {'changes': [0.8435495982901067, 1.9509106924346753, -1.7951230795034974, 6.2375926790699765, -14.38427662982749, 3.6723647287932124, 13.69208622628176, 2.570370350790597, 20.06930499365474, 18.270777754377747, 1.1285842187140531, 1.970067714100196, 4.510863414620189, -0.304909177683669, 0.5169911746681848, -1.7868858791981665, 21.244814220477934, 10.139751048070082, -0.4666063883327318, -3.093139975742986, 6.470236850659504, -4.423992832806562, 1.0330333735320778, -0.9579697200511248, 6.661480498879627, -4.984518644570315, -16.60702005219876, 5.55624165170957, -7.450008780236118, -32.47724610541231, -13.797014698784821, 8.524436154848729, -13.60284000792798, -1.4150893921356555, 7.480627450046967, 0.44359899446158124, 0.6043938244207361, -0.872965915060874, 3.1425431805086816, -2.5681360614444912, -8.912851476557876, 5.0879730077958385, 0.7126157833054947, -6.261516893505302, 7.716382831971771, 13.380287032356023, 5.367348796200332, 6.389368287051369, 6.234862377131151, 8.086998665271722, -2.5885950641528552, 0.1808003765390751, 3.5669644363731834, -14.511164481647512, 9.961893201731016, 1.376346465136976, 15.578137738315199, 2.336887216183868, 5.704651407929475, -3.2122590628221133, -1.0310820843661475, -1.245518723731621, -6.3484100006156154, 9.239652614265637, 3.7532787989689425, 6.810053058488222, -10.31283120170599, 2.4618850991774934, 0.32513571073914704, 9.191711792914006, 4.236846147978099, -1.4866511136582403, -1.6979398086531035, -12.318468749124492, 4.768190635978016, 5.800552574252322, 130.3824396373371, 6.830624872997504, 16.24767773229297, 1.8477413493712618, 12.939793035374606, 4.4132819551252, 0.9753435818405194, 3.1961635880295773, 0.9160279585526894, 39.416360495832365, -10.088737687802364, 6.782750698294649, -1.0445964836243888, 66.37191759064888, -19.648586154542485, -6.423470930391403, -68.22841261432531, -18.522304849086794, 8.723300609023596, -6.222227605514149, -58.80088473290779, -61.06369756168728, -69.91843616257145, 2.845646659838451, 10.07168233115405, -61.696333705235276, -81.85081572970361, -31.691876387185673, -49.236942827020115, 2.4346762123661847, 10.979708213136888, 3.800671745463916, 5.810602592783994, 6.759073188049456], 'changes_betw': [0.009933444439051031, 0.0035300336013893707, 0.004646484259124478, 0.0, 0.05342647875777348, 0.0, 0.0, 0.0, -0.04255861365953112, 0.0, -0.004583110852224043, 0.0025780841275234896, 0.0, 0.0, 0.003530033601389343, 0.0, 0.0, 0.0, 0.0035300336013892875, 0.0, 0.0, 0.0, 0.0017286108009859427, 0.0017286108009859427, 0.0, 0.0034572216019718716, 0.07823379663338895, 0.0, 0.0, 0.08333063659261422, 0.05427595208431092, 0.0, 0.017841636598007637, 0.0, -0.004103091004212273, 0.0, -0.0045494015932344145, 0.0024378536101268278, -0.004103091004212273, -0.0026994374598859616, 0.0, -0.002884164199148935, -0.002884164199148942, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0041030910042122865, 0.0, 0.0, 0.0035300336013892875, 0.0, 0.0, 0.0, 0.0, -0.05017286108009869, 0.0, 0.0, 0.0005757541435420926, 0.00025484199796126355, 0.0008305961415033675, 0.0, -0.05478563608023351, -0.004103091004212314, -0.0041030910042122865, 0.0, 0.0, 0.0, -0.007161194979747494, 0.0, 0.0, 0.00015101748027334088, -0.0008305961415033744, -0.0006795786612300396, 0.0, -0.10406048250084948, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.029136935100237862, 0.0008305961415033761, 0.0, 0.0, -0.09658511722731905, 0.022089003230695375, 0.0, 0.06422018348623854, 0.0, 0.0, 0.0, 0.09131838260278627, 0.08299084726199914, 0.08197147927015411, 0.0, 0.0, 0.024467528545000505, 0.023448160553155455, 0.023957844549077963, 0.02480731787561551, 0.0, 0.0, 0.0, 0.0, 0.0]}, (18, 13): {'changes': [-0.39020012718490094, 0.37138350494720385, 0.06257090085846073, 5.327867643010464, -16.112632381843973, 10.09551031065368, 10.606536822981752, 16.698401238122898, 25.365931842928454, -1.836760887471769, 9.415150997713354, 4.603999671669754, 7.6457158893578026, 2.175070797967635, 1.0750566005079847, -3.3312228836642532, 14.370477218360236, 0.9551992089759835, 0.8585764743565889, 12.333449208544295, -1.7577318679210663, 5.038471627256399, 0.4199989853584327, 9.48879292999274, 2.385735086273911, 1.6201067152585296, -18.5219566000064, 11.740228409277393, 3.5154035604286946, -37.052092858074126, -21.35878102443027, 1.827729934108163, -5.50021547601078, 21.170417590298285, 4.764333632558383, 3.0367757947451537, 3.3693803976805476, 2.9429399231414948, 0.48508955515386987, 4.168586254656596, 11.322420515069695, 10.07966651879768, 10.500547545657668, 0.67314608863677, 6.548852904597879, -0.6453977968174058, 8.989173849535632, 3.846469129993764, 9.832394149309295, 2.507004579071463, 2.0361732434701025, 2.764756984588338, 10.265798692724246, 9.33769690491738, 16.142906634308446, 12.678361548963437, 21.5566615613899, 24.42369138350287, 20.18435767169825, 3.0880241116405287, 16.55737249625244, 5.339885779799317, 13.491148128104982, 13.094668794013376, 3.9112211195767017, 8.990829404056818, 5.876565085378928, 0.12045102151972742, 4.31897590482086, 25.499119912782888, 17.83345767179651, 3.1919682628576425, 19.578772295166544, 14.1102973626827, 24.029392777321817, 9.804548747900185, 131.5041183015787, 7.221670910205091, 2.292665093384471, 4.555253986282992, 20.037478999916345, 5.037151012427074, 10.417482607293124, 2.3088358570415437, 3.3696087875021874, -0.5398262271947658, 1.4546212737324993, 5.487952409571221, 18.7233956339675, 134.79506497703227, -29.564361559891708, 7.582061781556035, -104.32238743054855, 5.232481426331049, -1.4308665779936973, 16.707901374467156, -71.69796201681804, -66.15867225663284, -74.19350997310059, 2.291099621633691, 5.424161962597623, -89.24008678245043, -93.13813850069508, -43.270402766125315, -63.2192611954103, -2.477148382644316, 9.259486884032754, 4.225038468432331, 3.51845259935547, 5.186526600003617], 'changes_betw': [0.01256546338095782, 0.004624910333371091, 0.005935526322886228, 0.0, 0.0706694389161262, 0.0, 0.0, 0.0, -0.05682976554536191, 0.0, -0.005962493730077886, 0.0032226051594043577, 0.0, 0.0, 0.004624910333371091, 0.0, 0.0, 0.0, 0.004624910333371035, 0.0, 0.0, 0.0, 0.0022032371675593104, 0.0022032371675593104, 0.0, 0.004406474335118621, 0.0978984299575534, 0.0, 0.0, 0.1063931632229288, 0.07185870157327866, 0.0, 0.017544995118899273, 0.0, -0.005277521587409462, 0.0, -0.005984067655831177, 0.0030473170126585347, -0.005277521587409378, -0.0035866651564918084, 0.0, -0.003753863081080229, -0.003753863081080236, 0.0, 0.0, 0.0, 0.0, 0.0, -0.005277521587409434, 0.0, 0.0, 0.0046249103333709796, 0.0, 0.0, 0.0, 0.0, -0.06658117998586921, 0.0, 0.0, 0.0006984558462641705, 0.00033978933061501806, 0.001094876731981713, 0.0, -0.08052332950396163, -0.005277521587409462, -0.005277521587409448, 0.0, 0.0, 0.0, -0.009185098889482192, 0.0, 0.0, 0.00018877185034167632, -0.0010948767319817268, -0.0009061048816400522, 0.0, -0.10406048250084948, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005861365953109071, 0.0010948767319817233, 0.0, 0.0, -0.13260278627251104, 0.024510676396507197, 0.0, 0.09717974855589535, 0.0, 0.0, 0.0, 0.12325857968059803, 0.10775232054538887, 0.10741253121477387, 0.0, 0.0, 0.02892793769450242, 0.028588148363887407, 0.0270590963761198, 0.028588148363887383, 0.0, 0.0, 0.0, 0.0, 0.0]}, (1, 3): {'changes': [-0.021611699108884608, -1.2593925239156185, 0.33409495234970166, 3.4093915297903266, 0.2632761738085492, 9.459051101219075, 3.3677025717198887, 11.154038136747403, 1.5321881845187377, 7.631959596109226, -1.2343941632147448, 4.393852094594678, 3.7108963228453433, 9.167616831410726, -1.0194224651468993, -12.947581426720546, 4.2610595244262015, 6.320080157695543, -1.7923313114020232, 10.352845745462929, 0.9005736084227323, 16.788511517347786, -4.358927778011235, 1.7444671758385937, 4.128062106911955, -1.8708511810692272, -1.9565377194345501, 0.8665340638993939, -1.7654172789492009, -2.5425276781195265, -2.2130740644463955, 4.782246023410494, 4.062291967232682, -1.5391370690353483, 2.480277663631469, 4.980834395183081, -0.9056734748529216, 0.5382961569172124, 1.6086680637171256, -0.9731776552290512, 11.571281534601269, -1.7493998806115485, 0.12523746155038396, -0.7573312093344384, -10.218730027085542, 2.4485119055901237, -5.668720140776472, -1.9031058573519175, 2.486615291534804, 4.261778358775416, -4.2459275530634955, -1.0534397487395921, 3.229646215103145, 2.8234607220193766, -7.522574162631969, 9.259455862764355, 0.23782557674851645, -5.8897108630698085, 7.461644283620785, -0.7316610167571298, 9.646700743927, 2.473820674019265, -1.415696849636845, 0.0013298257487441845, -1.8205666368479712, 0.3476951876457477, 5.54108522006797, -4.023088418566488, 6.316757827282231, 7.234223111541922, 4.40203675054903, -1.9749924481340315, 1.3172705755315235, 5.214977193758926, -4.984310954815811, -3.6489981813555232, -3.1128052838565026, 1.4737217782831635, 7.0683302985748355, 3.777937333476615, 2.9883012728514586, -2.758349049189121, 64.95521744629337, -18.86101339137484, 11.382429836777476, -2.9756112704348823, 0.6780551888133743, -2.6348212539802773, 3.618459069989086, -2.496211133651741, 5.546052283825475, 9.71879327493238, 4.923649824467844, 1.570849130972647, -4.663518136934897, -2.401135429205482, 6.196053876527557, 18.93119352301845, 13.113195890879496, 0.35687031926255486, -6.841065082534783, 6.60555471101236, 3.7080838658191624, 5.243599316191762, 1.8388653227035832, -4.551396720888221, -1.7938653344221507, 5.198139015075071, 4.476517400029621, 15.374440012853114], 'changes_betw': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -8.494733265375468e-05, -0.009089364593951751, 0.00025484199796126095, 0.009259259259259259, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (1, '1_2_1'): {'changes': [-0.12500479994277924, 0.7203824348704835, -1.6212758077653575, 6.838447324418965, 3.576805757606053, 6.9221063658618505, 1.8554597275443143, -11.499286891307435, -4.833696068723157, 19.97290543031096, 2.462957539173445, 18.978812185415023, 4.58459705658646, 9.15309129383246, 1.383130646179712, 9.428141872157369, 5.247613393989866, 17.511926480825053, -0.44655511796137404, -3.45212251883612, 8.312827126969609, -6.6743798941082275, 4.210264580099221, 0.06356177483110059, -1.879215178624122, 2.8386937314241294, 1.5176532164683536, 2.546402482441806, 3.007014475479025, 3.9052678433585157, 1.2973200137791352, 21.223428752351964, -0.012481031103604323, -1.5742513488134477, -1.0742011422602005, 4.028536504418867, -0.5928320799448485, 0.6114513095560312, -0.0255503554018901, -2.9378873955945863, 13.500830941588958, 2.9546933902305597, 3.4734682908538588, -2.5023068099758348, 0.18945176541643605, -11.926104306305831, 4.3572025878158485, 3.850581502287639, 0.3182356068476082, -1.157093302781874, -4.143887271516974, -1.2339547851120898, 10.365681959173855, 1.8874159292083164, 3.606407619362926, 1.2428140671307233, -2.198059347817896, 7.347629564114101, -7.682588149066646, 0.9983727840819299, 5.460954938156192, 4.476074492389429, 18.737214324664734, -0.012903645355944349, -1.7173330907151225, -3.0310004564138353, 5.116517006171819, -4.585709786889993, 10.031235060615103, -0.9946485960649909, 2.530931877076455, -7.908499858066705, -8.210072177126563, -6.971888811081982, 9.391502031992616, 5.422789984827091, -5.027797447695249, 8.130473453064837, -4.1155758500065645, 4.5828153128667, 5.397419279694361, 7.818408416919112, 8.407084922063916, -13.823816124397183, 72.21054907175318, -6.752518696872002, 4.342536560233896, 8.817101142762738, -1.6243319288946054, -0.4872957752146192, 0.5275576824253108, -8.66668613060429, -3.6788192541167177, -0.07358021371356926, 3.5129957558633578, 2.7486964184599287, -1.7144127496589192, -2.4619979279218853, 2.2128260071549164, -3.1047184417835183, -0.3104137466957013, -5.018220835401195, 16.499954511693403, 3.532867140382365, -5.346961717055336, 15.219790688015422, 12.640037854356393, 7.985543072017379, 3.375702447099286, 5.638092866943396], 'changes_betw': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -8.494733265375468e-05, 0.009259259259259259, 0.00025484199796126095, -0.009089364593951751, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (3, 2): {'changes': [-0.49647209082133514, -0.47551265370678664, 3.3134811220662, -4.278392244116603, 1.252193705865352, -8.141457215380598, 2.176792923399489, 2.8823140295551752, 0.7705840588251078, 1.4287477305057905, 1.050645913169717, 16.062078402712473, -3.760395782124192, 13.945461748501572, -1.3901021065634964, -2.0979024071271226, 1.9332535159494455, 5.853551179902297, -1.1054192071768334, 0.9804687234552603, 0.978543738095695, -4.820391420530058, -3.6472239917113995, 0.27543701160249157, 4.266807186094667, 1.7378575999252064, -2.629680804251578, 9.695466982795693, 1.3035614210319864, -0.47451132472946256, 0.4210473081381423, -11.517343639149402, 0.6381554581967492, -2.71806542608698, 1.142265193240135, 6.385584411425953, 1.6308294352383257, -1.5851885959751826, 2.2338301910518794, 0.19216070940778707, 11.151827494564401, 2.8152093777696052, 2.2335745801998996, -3.2251446034728133, 7.913494078548496, -8.350192186418838, -7.083687069102837, 1.97319390037471, 4.284015335072546, -5.231040731165734, 8.358871276496046, -0.21001484460515485, 13.960377948965771, 9.511148547778703, 8.742441958499398, -5.317727235818666, -0.09378703346367345, 19.208686976308428, -9.382772478803986, -2.6824819153236206, 9.699677374680306, 5.193789380039959, -4.445001513383204, -1.3311406049357934, 1.9049132961106636, 1.9820440263045924, 8.468307941817017, 5.407154647967786, -4.048707398600271, -0.2401735579938844, -0.605323214830662, -8.317006714699374, -6.489786547522328, -3.4441908299030786, -1.6534418513984122, 7.603923773874385, -3.2738684289386555, 4.464171465684387, 6.474635655508507, 2.883222826501452, -0.8136291525489696, -69.57957379093742, 59.84058205689763, -2.630156847660146, -42.077217767137654, 6.880379688796694, 0.5563411166134813, -5.731077653215976, 8.306935862998614, 2.8756306797019633, -1.0528537240901699, 3.8340488996465467, 4.005032832170485, -3.3121828979979, 1.2030824130291933, 1.1795400337481965, -6.103281595149255, 3.6624733441839226, -0.7702206735285699, -3.3792258922206884, -2.7424950520637026, 8.689738820116162, 1.5591790224740976, -5.7183929725940175, 1.9821427651328634, -1.6091440783465032, 2.472580898630298, 1.2743224717575004, 12.670951669451341, 0.17315435644013633], 'changes_betw': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018263676520557257, -0.009089364593951751, -8.494733265376059e-05, 0.027268093781855254, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (2, '1_2_1'): {'changes': [0.30382225033927845, -0.7349964780350859, 2.756718611344816, -1.558552859425788, -2.234335165001717, -7.746455588644096, -12.426458218506127, -7.289334547396209, 2.330157894045165, -14.694562480386509, 1.9219302858664378, 5.8400467440846455, -13.426425960634617, -7.980798938554244, -1.7247855454592198, -1.2942881977605225, -18.509798111203253, 2.9729544395387393, -0.7316130844119968, 2.1404244010533944, -5.2232605060844435, 2.397675621805348, 2.0026035555233364, 5.004379913214535, 9.896342018046681, -3.518497832139346, 0.8944190160177499, -12.498361595442361, -5.934797039007947, 2.070029360102069, -0.4343507440853358, -18.421214075699197, -1.194502610352572, -9.65123237770257, 0.9226472198942659, 16.69695943592498, -1.1260657935883032, -5.535174030968818, -1.58865901721542, -0.05906208631915888, 6.80182326498425, 4.617494295210761, 3.642522143594931, 4.9055272591849075, -2.784362921545039, 5.320868458040877, -5.758219508348745, -6.9774690447475365, 2.281547954425619, -10.329559011770101, -20.1957908318023, -0.5178354202271578, -4.404878519842214, -5.092776790228868, -11.145759598227727, -9.882615475893232, 0.200125243784413, 0.47907101479509606, -3.346710041722531, -2.920206408106111, -0.3196220987091465, 1.962102322557925, -8.476705801353546, 2.821362371840678, 1.8452230879748939, 1.5082364007831472, -12.416735877066714, -1.8706747048870795, 2.0605265246016415, 10.373674581403819, -5.726023901786732, -26.975775570693003, -1.4047315717485276, -1.7720204437770803, 6.573244936280872, -3.9190032379621016, 5.916389105572094, -8.02606295317537, 11.684921448087096, 0.06527853693054908, -5.878221523566225, -64.40715079024929, -36.277016156768184, 1.9853419717004215, 52.61173131174439, 3.4197080928768884, 9.34193508048142, -6.930711505949034, -10.448493227049141, -0.24409380938978842, 7.670370730871127, -2.7197611885511037, 6.641114861890884, -9.929691740562532, -3.46056024681252, -4.180066951498475, -2.7607218369319355, 2.447281096919454, -5.3815559574852045, 6.840546837773047, 10.964849262302295, 6.5958805540029175, 1.490729374179267, 14.78376221948352, 6.828015339995659, -6.856653900979637, 4.9416976823735865, -5.159164211751431, -0.7251065137106707, 6.608360929678696], 'changes_betw': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018263676520557257, 0.027268093781855254, -8.494733265376059e-05, -0.009089364593951751, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, (8, 10): {'changes': [-0.26771352566951556, 0.28407087118393193, 2.8009938076331906, 0.4224854928947934, -7.446569209886825, 13.577322137912546, 7.087907728468082, -3.294365794252826, 4.975326470453197, -5.531451540526348, -1.1624419045697607, 12.026863888172102, 1.2141518658910968, 4.26464089378203, -0.35783051940487454, -4.060893780131323, 9.532376596787486, 2.593284644706813, 0.21517845990684137, -1.082721286636513, 6.763595633099044, -1.904749823354848, 10.81921784839679, 7.218734728917099, -7.523911622321151, 3.4382657423951457, -6.7834136112394745, -0.4917549859371775, -3.9713959323399877, -7.447501961808712, -6.710405410311523, -2.387610123026775, -15.490854541616443, -0.6975206310245312, 3.7798366892767916, -7.901528915203443, 2.240795397318788, -2.240935740782973, 2.9614301290327774, -1.8192087778968613, 9.660812832882641, 1.5892583985275337, 0.5954250931289096, 11.457784319016298, -3.8893889962823494, -3.664896206294202, 2.3777068003223008, 1.0024872291201916, 2.547277863201657, -2.001803768023194, -10.771282656196718, 0.18761789303982113, 2.919569241493889, -7.020436537945216, -7.003786054869863, 1.0291220483582038, 0.338783229290911, -4.180179515169726, -6.33959970007254, 2.5332870806899592, 0.6775859722734197, -9.330313834627972, -8.963603050387093, -0.02167860607690031, 2.6291830880890856, 2.6162593525116833, -1.7289245112984304, 4.00416478050559, 10.630787478067617, 4.465244427938018, -6.3425544651689165, -6.036634709389148, -11.919663704185098, 4.667344489824927, 0.4051332661340723, -11.867700806455986, 14.287179242535615, 6.907832901563893, -10.728076542606004, 3.290423532525608, 3.3396834931797628, -16.530000212076686, -9.016673085487554, -4.0735209049284435, -12.109301652057013, 35.187768871800515, -7.215818860986104, -1.2065977428882775, -2.3716590204185195, 5.999553548410631, -9.656099713871853, 1.1183564018795664, 90.0326027284678, -13.573299097919431, 5.378889639353787, -0.6611274374578784, -9.428150622282146, -18.873135905260312, -29.2439255235726, 4.4777162458168505, -8.084938888978343, -41.08565867384041, -57.120213103609785, -22.524814237223183, -26.70410456918364, -0.455671426521576, 10.742886637820106, -5.5416784393501075, 12.50362220698861, 1.687237576797429], 'changes_betw': [0.003353397084283949, 0.0007928417714350711, 0.00142387909972011, 0.0, 0.010319078361891887, 0.0, 0.0, 0.0, -0.006880733944954115, 0.0, -0.0011346536575894822, 0.0009667815478213125, 0.0, 0.0, 0.0007928417714350711, 0.0, 0.0, 0.0, 0.0007928417714350433, 0.0, 0.0, 0.0, 0.0005420448845525408, 0.0005420448845525408, 0.0, 0.0010840897691050816, 0.02609905668009649, 0.0, 0.0, 0.02270116337394626, 0.010319078361891817, 0.0, 0.015610083652897094, 0.0, -0.0011670145462194248, 0.0, -0.0009627364367426061, 0.0009141951037975604, -0.0011670145462193693, -0.0004813682183712753, 0.0, -0.0007099169943206585, -0.0007099169943206723, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0011670145462194387, 0.0, 0.0, 0.0007928417714350433, 0.0, 0.0, 0.0, 0.0, -0.009152063815672323, 0.0, 0.0, 0.00026899988673687036, 4.247366632687769e-05, 0.00016989466530750036, 0.0, 0.003612284193323878, -0.0011670145462194526, -0.0011670145462194526, 0.0, 0.0, 0.0, -0.0021014352054107358, 0.0, 0.0, 5.663155510250359e-05, -0.0001698946653075073, -0.00011326311020500544, 0.0, -0.01656472986748217, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.029136935100237862, 0.00016989466530751077, 0.0, 0.0, -0.012487257900101939, 0.013061663673284471, 0.0, -0.024125042473666328, 0.0, 0.0, 0.0, 0.0055215766224940704, 0.018114007410643493, 0.015395692765723362, 0.0, 0.0, 0.010343349028364325, 0.00762503438344417, 0.013231558338591964, 0.012382085012054424, 0.0, 0.0, 0.0, 0.0, 0.0]}, (23, 30): {'changes': [0.5002864575785253, 1.0895868115433984, -0.2622737069238781, 2.1476431020597033, -9.540820354114864, 10.395448773549816, 13.422325887369709, 9.884219062375024, 9.614899432487015, 7.133657837675457, 6.800873582460028, 11.627629459633823, -0.2830823691573414, 6.281045372814447, 1.9455885738702712, -0.011279957619819925, 11.560438164412915, -3.271596114125714, 0.7035726905831297, 11.501382755956513, -2.7735849580169827, 4.411432918898242, 4.0944131975808205, -1.043368885931244, 9.486990783313132, 1.5490752695161802, -6.7797268160624, 22.601360699019665, 14.48299751905651, -16.15109359888524, -4.853856724751459, 12.701368958839453, 4.377741071196851, 2.1270220644615847, 3.2018746659884485, 7.061493768702462, 1.2254938976649825, 5.856049009662016, 2.409796182508785, 1.3913554563418913, 15.423400810993826, 3.6954406912970654, 7.528706702544284, 14.444243222464934, -3.5166232364517214, -8.480139410127578, 2.4428225846737917, 6.184246708384578, 3.0457846727510045, 2.896566737394039, 11.476626555270911, 2.025091782471641, 9.304220624418718, 16.441859179600442, 1.4769167629577282, 7.833399160896761, 7.3190136070116765, 6.964513336544854, -1.6766648152030825, -0.4759554488941262, -1.4703229078394884, 3.7371987784957383, 15.744260201075576, 4.3570081334597965, 1.2098287335716336, 4.718703288286591, 2.85217492167169, -0.20166715532963053, 7.274272577806329, 7.5798982537688175, -1.0797919717788886, 3.593495747809186, -0.4745716387711525, 5.7461577557118915, 11.349076299121691, 2.3765363719857078, -8.586988603869308, 5.2886647880010855, 10.426253093701064, 15.49174132244039, 1.4955910986740264, -3.3609564901679505, 6.071682273060361, 1.7358515791927118, 6.145822708769231, -17.287708749811884, 3.657902234783478, 9.093190828669009, 9.1972280374126, -2.8811792113167485, 142.3747958069231, 1.79430791813229, -51.2621620339408, 0.5694962618518957, 2.41372757258992, 18.608566656260763, -38.80855097835119, -31.95528443365525, -33.46439305901261, -1.7637416793626244, 10.607193497435503, -34.427820085342546, -98.79791515346906, 113.80363236262104, 23.273904408346027, -0.3145151670525479, 7.602608030848046, -1.5074993524059437, 15.285343674192006, 14.298158679023174], 'changes_betw': [0.0018971570959339101, 0.0015856835428700866, 0.0009627364367425506, 0.0, 0.027253935893079673, 0.0, 0.0, 0.0, -0.02484709480122327, 0.0, -0.001755578208177594, 0.0, 0.0, 0.0, 0.0015856835428701144, 0.0, 0.0, 0.0, 0.0015856835428700589, 0.0, 0.0, 0.0, 0.00031147355306377494, 0.00031147355306377494, 0.0, 0.000622947106127536, 0.02728225167063092, 0.0, 0.0, 0.030878355419639847, 0.02770698833389966, 0.0, -0.0035961037490089484, 0.0, -0.0011043153244987836, 0.0, -0.0022086306489976504, 0.0, -0.0011043153244988113, -0.0015856835428700589, 0.0, -0.0011043153244987974, -0.0011043153244987974, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0011043153244988113, 0.0, 0.0, 0.0015856835428701144, 0.0, 0.0, 0.0, 0.0, -0.02660267300940078, 0.0, 0.0, -0.00015573677653189788, 0.00015573677653188487, 0.00042473666326875437, 0.0, -0.033426775399252484, -0.0011043153244988113, -0.0011043153244988113, 0.0, 0.0, 0.0, -0.001755578208177594, 0.0, 0.0, 0.0, -0.00042473666326877865, -0.0004247366632687752, 0.0, 0.023955147808358807, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.025484199796126414, 0.00042473666326877, 0.0, 0.0, 0.022256201155283734, -0.05729697587495752, 0.0, 0.02633367312266395, 0.0, 0.0, 0.0, 0.05789160720353382, 0.03198267074413863, 0.03198267074413864, 0.0, 0.0, 0.022780043039981886, 0.047542190508551366, -0.04115698267074414, -0.007942575603126067, 0.0, 0.0, 0.0, 0.0, 0.0]}, (10, 17): {'changes': [0.5009426738292113, 1.2862488869606992, -6.537463326463296, -4.284286123933185, 2.9527399959192095, -5.360083994826738, 8.02866399774831, 3.687437505854348, 1.9496530433356511, -3.2044127198178103, 6.344503376661109, -2.6893934824971666, 1.1881626393510487, 3.77777408536474, 0.7800413854478272, 9.962223329200071, 2.3529645980148075, 4.670385009912792, 0.29777948180586833, -4.2431752968260525, 4.820778944945147, -9.621857524333109, 8.05556546128713, -2.118054705736199, 18.6718159454511, -4.449447578821491, -1.3336118473973855, -4.7830197544523685, -6.687181681887637, 0.3606105397851991, 0.4011598212864982, -2.84786667508007, -15.22786627006866, 4.20374951478027, 0.4616105908911763, 9.181580392310764, 0.42298705744738996, 0.4913312953894575, 2.2917217121454954, 0.33109551567412865, 7.066911633900844, 1.044760070852277, 4.46853209202915, 5.02239771087244, 7.607466017040451, 3.318157976235682, 1.225575402358757, -2.344917476044529, 0.7272964924347249, 6.073828333825645, 13.404341868334285, 1.7854091646652108, 9.116195287244352, -1.1256228684704865, -8.072141787447947, 4.516900537110416, 0.2643616944867446, 8.98998923915562, 7.278159145326526, -1.2819691837276608, 12.070507049669374, -1.8849051381399136, -0.48453065157031006, -0.4085574065296669, 0.04038294784692198, 4.159939253669961, 11.556596758224089, 13.736256557194679, 11.357298134233616, 5.4947711251666504, -7.939428440859501, -4.662632734870215, -1.0675590760413627, 6.263436506625624, 7.993173190992167, 4.214251723574989, 1.5472474933466316, -5.297483241462714, 5.334011861280317, -7.737697043915716, -3.564163568473248, -0.7649854696574039, -4.539910061750163, -7.6994637170891735, -8.506248953386631, 3.383636945532956, -3.8365079250851295, -6.093316285849056, -6.204110642702602, 0.030845832224407843, -6.352000269410496, 6.2328253902115875, 82.77633166703404, -5.081146790440755, 11.149454389235814, 1.2664337902112095, 27.300029379720485, 4.958009525405188, 4.5997356087068795, 9.667666539613123, 8.028441414559921, -20.095990014459886, -29.584428875287983, -19.681778819808727, -17.494939459147105, 1.0711292728818762, 7.266815213086488, -1.3130323237058974, 10.999774044636865, -3.5561422409576835], 'changes_betw': [0.0020373876133305546, 0.00024540340544421113, 0.0007793580678392281, 0.0, 0.00169759828271554, 0.0, 0.0, 0.0, 0.0002548419979612748, 0.0, -0.0004449622186625396, 0.0006445210318808785, 0.0, 0.0, 0.00024540340544421113, 0.0, 0.0, 0.0, 0.0002454034054441556, 0.0, 0.0, 0.0, 0.00030473170126585347, 0.00030473170126585347, 0.0, 0.0006094634025317069, 0.015162424693515447, 0.0, 0.0, 0.010065584734290145, 0.0015277036174080189, 0.0, 0.014654089067952472, 0.0, -0.0005797992546208441, 0.0, -0.0002454034054441556, 0.0006094634025317069, -0.0005797992546208441, -3.775437006833804e-05, 0.0, -0.00027506755335500455, -0.0002750675533550184, 0.0, 0.0, 0.0, 0.0, 0.0, -0.000579799254620858, 0.0, 0.0, 0.0002454034054441556, 0.0, 0.0, 0.0, 0.0, -0.0009479043627871053, 0.0, 0.0, 0.00020764903537583146, 0.0, 3.775437006833804e-05, 0.0, 0.014272500256190257, -0.0005797992546208719, -0.0005797992546208719, 0.0, 0.0, 0.0, -0.0010894832505433868, 0.0, 0.0, 3.775437006833544e-05, -3.775437006833457e-05, 0.0, 0.0, -8.494733265375365e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.011467889908256881, 3.7754370068336304e-05, 0.0, 0.0, 0.003312945973496434, 0.010746511765879763, 0.0, -0.024125042473666328, 0.0, 0.0, 0.0, -0.012657152565409443, 0.004628955444449835, 0.0015708514689146622, 0.0, 0.0, 0.007008829129114552, 0.003950725153579384, 0.010576617100572241, 0.009387354443419687, 0.0, 0.0, 0.0, 0.0, 0.0]}, (17, '21_17_2'): {'changes': [-0.4619678992130023, 2.3447126366418694, 3.685010117499438, 3.617964408475885, 5.681686478309189, 3.294652287984576, -7.405503364221488, 6.334155255263113, -4.230254135794958, 9.48189866278608, 1.5135613660847156, 3.0734409805284884, 3.7135780964577236, 8.199586075562479, 1.050387994076619, 20.1048397923108, -1.8807083189544755, 15.606001441694502, 2.1676977107750623, 0.5199017322900659, -3.5086183239616275, 14.080096769509026, 0.30573368465502426, 4.061479892165892, 1.973085200278632, 2.331136507783313, 2.1491001005945662, 1.3772331874454835, 4.551521688767764, 15.304669068817034, 5.4755827453972685, 14.448675957312787, -24.03725217285634, 4.509214984904759, 3.7191529669427013, 13.525112567845497, 0.20767844790554335, -0.8140394669084685, 4.17107539319607, -0.04684634123704967, 5.279511889851733, 4.637335003812126, 5.5188648556776485, 14.79202920545751, 8.937869494657093, 4.8566725044341865, 14.130997080959844, -0.822731708744783, -1.6390929799133716, -5.309026000895585, 17.190006655908604, 1.2560990020638982, 15.38162962058422, 0.4636294583274321, 6.6391907342026, 12.732091748391554, -2.198147909191942, 11.553145306131142, -3.462832756450325, 4.788704942669561, 12.825859193864943, 0.22834422047650094, -0.6049358803650193, -4.060631397799, 3.8304204172096306, 0.6694081405958414, 3.719095579801234, 4.653848188831944, 7.848206617474176, 13.681221291381576, 4.0550676950339835, 3.440887958055839, 13.073901011506194, -4.935363745917925, 8.000987719695445, 7.903305158786168, -4.243323471366303, 12.361613778644795, -2.016705895327675, 3.08330781095907, 7.4179824147054205, 10.67529424183681, 2.6670454550345823, 0.23900243215871342, -7.125000040378183, -11.671581852907366, 1.392019663177365, 2.803875685821822, 10.034996076752861, -4.688544629104712, -1.8135750064840863, 13.553923295997322, -25.951720802237332, 7.626356841423444, 5.006650886773571, 6.108945126787944, 59.71926863605438, 71.25631206715178, 131.6526432287894, 14.92340923943459, 11.67795642655392, 2.2742901443658354, 22.501195225767816, -8.822847899598742, 0.19330293114211372, -0.8187533724933189, 5.673029742031503, 15.402409293033998, 3.4400664134885233, 14.261314529552578], 'changes_betw': [-0.00039642088571745226, -0.0009627364367426061, -0.000566315551025029, 0.0, -0.019212255068524184, 0.0, 0.0, 0.0, 0.0182212028542304, 0.0, 0.0010476837693963043, 0.0, 0.0, 0.0, -0.0009627364367425228, 0.0, 0.0, 0.0, -0.0009627364367425506, 0.0, 0.0, 0.0, -0.0001982104428587539, -0.0001982104428587539, 0.0, -0.00039642088571752165, -0.00814078604598481, 0.0, 0.0, -0.019835202174651706, -0.01994846528485672, 0.0, 0.016791256087892184, 0.0, 0.000679578661230057, 0.0, 0.0013591573224600306, 0.0, 0.0006795786612300292, 0.0009627364367425506, 0.0, 0.000679578661230057, 0.0006795786612300431, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0006795786612300292, 0.0, 0.0, -0.0009627364367425506, 0.0, 0.0, 0.0, 0.0, 0.01926888662362669, 0.0, 0.0, 9.910522142936307e-05, -9.910522142938128e-05, -0.00025484199796126095, 0.0, 0.0371644580360177, 0.0006795786612300292, 0.0006795786612300431, 0.0, 0.0, 0.0, 0.0010476837693963043, 0.0, 0.0, 0.0, 0.0002548419979612679, 0.0002548419979612644, 0.0, 0.02726809378185524, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021491675161399937, -0.0002548419979612627, 0.0, 0.0, 0.028287461773700312, 0.008452259599048592, 0.0, 0.01410125722052328, 0.0, 0.0, 0.0, -0.01694699286442405, -0.0292643560992185, -0.03436119605844377, 0.0, 0.0, 0.004941103182693399, -0.0025767357571638916, 0.00811247026843357, 0.007178049609242263, 0.0, 0.0, 0.0, 0.0, 0.0]}, (17, '17_14_1'): {'changes': [-0.2845235668060333, 0.2877028631431955, 0.3512800218090746, 3.092273747654218, 1.5488184579063713, 1.9236409257625553, 4.7463355736227015, -2.004190799741423, -0.602839278265705, 12.281614541492246, -2.9911310740693438, 11.038827261777215, 1.4717721878974999, 8.986308663667046, -1.1923261097335072, -5.966677539416736, 3.7627763771337186, 8.138673232811612, -0.39009175856786626, 9.67584734030666, 1.4569429481882707, -4.436670389057497, 3.453481105470388, 8.550791981068329, -0.43545680054978675, 2.9662278626634873, 0.9230752162709166, 5.535900763785378, -8.434786384431789, 10.489224344107171, 6.099662741247769, -3.009579270073175, -7.495739065919665, -10.112812142470403, -0.34358714069697527, -3.3575380949241094, -1.7572590160692023, 9.018470654153703, 3.6576912594117132, -0.5661231430012492, 10.692520301141002, -3.3547203477175884, 2.028866794264431, -11.553627086262622, 6.465932858912254, -15.199759274445569, 1.0166465064323376, -2.8153165266765683, -0.2503695021662722, -5.833830317206832, 2.045353645728852, 0.4535565213765409, -15.419876525112869, 2.75781376887673, 4.151880186223366, 9.136915588149066, -0.6421621828014494, 5.979079073265694, 2.5914017162434675, 1.593970178907533, -3.7422818290745, 1.2534543100813664, -8.132262238235256, -0.1691316644039773, 2.3652187105899216, 0.9905999386924833, 8.444023224581088, 1.704529883987135, 7.3433652401619725, -2.798047756232293, -1.2973067968140413, 0.6582152600946074, 10.720396259233695, 3.805224021635155, 13.852082874647948, -3.467263670135651, -5.122477497874556, 3.0229202914196094, 3.6532369487474625, -8.01913699558392, -11.916922273259217, 4.308611604162593, -5.024185820308787, -6.9055665485361715, -2.6528533688231164, -3.665329021012937, 15.667262781856294, 15.365298060794714, 9.323315690341644, -0.8506200550528433, -5.819255899449814, 12.435186347112051, 8.831474447337797, -12.528720080391281, 2.409465762008921, -1.6327762824030287, 34.93314284348344, 18.524474653095282, 18.435726891423514, 0.223124805016937, -4.86135990699097, -10.358100273035348, 30.20069917052811, -3.04912883036107, -13.1726498383849, -4.571930023579313, 1.9574432270261184, 3.1279899138698966, 11.104507825129929, -2.441231064545036], 'changes_betw': [0.00019821044285872613, -0.00011326311020498636, -5.663155510250706e-05, 0.0, -0.003666893192887044, 0.0, 0.0, 0.0, 0.0036952089704383184, 0.0, 0.00011326311020498636, 0.0, 0.0, 0.0, -0.0001132631102049586, 0.0, 0.0, 0.0, -0.00011326311020498636, 0.0, 0.0, 0.0, -2.831577755124659e-05, -2.831577755124659e-05, 0.0, -5.663155510250706e-05, -0.0009202627704156252, 0.0, 0.0, -0.004119945633707087, -0.003893419413297086, 0.0, 0.004558840185751506, 0.0, 8.494733265376753e-05, 0.0, 0.0001698946653074518, 0.0, 8.494733265376753e-05, 0.00011326311020501412, 0.0, 8.494733265376753e-05, 8.494733265376753e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 8.49473326537259e-05, 0.0, 0.0, -0.0001132631102049586, 0.0, 0.0, 0.0, 0.0, 0.0038084720806433603, 0.0, 0.0, 1.4157888775609417e-05, -1.4157888775625897e-05, -2.831577755125353e-05, 0.0, 0.014483520217465129, 8.49473326537259e-05, 8.494733265373977e-05, 0.0, 0.0, 0.0, 0.00011326311020501412, 0.0, 0.0, 0.0, 2.831577755125353e-05, 2.831577755125353e-05, 0.0, 0.0045871559633027525, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0013591573224600723, -2.831577755125006e-05, 0.0, 0.0, 0.005606523955147824, 0.014058783554196402, 0.0, -0.001868841318382601, 0.0, 0.0, 0.0, -0.01278457356439007, -0.006413523615358491, -0.0077726809378185525, 0.0, 0.0, 0.004771208517385888, -0.006484313059236607, 0.01286952089704383, 0.010066258919469929, 0.0, 0.0, 0.0, 0.0, 0.0]}, ('21_17_1', '21_17_2'): {'changes': [1.467548688351794, -0.14041101627423913, 6.480022684048649, -2.8968894832358387, 10.756923802800756, -1.1165657115249132, -5.808061624575743, 3.2891253792099064, -3.0895876960970767, 6.758307788942716, 3.3390452563188404, 15.240260054489795, 4.9212917800757054, 8.40611810829671, 0.4167949386163521, -3.9452273281932264, 10.800093804968185, 3.4192295121140717, 0.9583040371826925, 8.663801142624834, 6.074772740387303, 10.215627456190049, 10.705774661934925, 10.43577677424787, 11.23738380027035, 8.092045524179298, 7.671107629887807, -1.3233232792515537, 6.8086700549516195, 34.907863703465026, 7.801676941278359, 10.874089362848679, -18.25818375681274, 6.51778833960492, -0.22910636753950087, -5.938321044436407, 1.3057110318021046, 4.442690173677832, 0.7590795175814051, 1.4986008923648342, 16.682417288435715, -1.1509804100619476, 4.559977264716466, 1.8033408716693202, 15.140342349013906, 9.821863068769687, 4.99502004025706, 0.7441552020909654, -2.413345086652768, 4.781431900471546, 4.74155721724668, -0.3268780976793835, 2.1757027229978974, 22.631226356683555, 10.65330913502686, -1.6432479157260218, -6.255690484482415, 6.9249635520936295, 21.532411750306863, 1.3677676434854433, 6.136695961456098, 7.522675326252056, 8.50828631014599, -3.587038158688511, -4.589544449601917, -2.0186351789894914, 4.389748643191979, -0.2203169944427259, 14.513583195704712, 4.492558851825336, 12.888890982317491, 1.2833096314257375, 13.152915354946572, 3.208527177979903, 2.0194055637330166, 6.732831129915439, -12.94818568034588, -14.50522104685723, 2.6936655104505576, -10.848939334952576, 8.319819549114868, 3.2054950465843035, 27.8002749243052, 19.999644762766366, 18.2747913731896, -21.079735501816877, 10.470257381545537, 13.473533769340406, 1.5758307499179978, -9.748293162475193, 0.5747063981199005, 7.506913078846424, -62.88406523841935, 13.864221366986897, 3.2014354500513207, 7.1754352096440925, 8.645999985292605, 152.9622077549824, 129.23380019489795, 12.363604988916137, 12.387791028104687, -4.199504078023665, 17.71501647183095, -4.077003716540361, -6.196340279566698, 14.836351757714624, 10.651355655059035, 1.1090393447442466, 7.076398988780113, -1.1577380156503523], 'changes_betw': [-0.002024173583806521, -0.0015492775431614758, -0.0013170881672410623, 0.0, -0.028145478374836122, 0.0, 0.0, 0.0, 0.02535677879714579, 0.0, 0.001783489474620939, -0.00046033364076177724, 0.0, 0.0, -0.001549277543161226, 0.0, 0.0, 0.0, -0.0015492775431612538, 0.0, 0.0, 0.0, -0.0004886494183130308, -0.0004886494183130308, 0.0, -0.0009772988366260754, -0.023940989919583183, 0.0, 0.0, -0.037334352701325166, -0.029051583256476193, 0.0, 0.011283432843065877, 0.0, 0.0013660340112939817, 0.0, 0.002115593094186269, -0.00041098328560104647, 0.0013660340112939817, 0.0014117437664838417, 0.0, 0.0011605423684934446, 0.0011605423684934446, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00136603401129394, 0.0, 0.0, -0.0015492775431612815, 0.0, 0.0, 0.0, 0.0, 0.02768554924518224, 0.0, 0.0, 4.045111078737618e-06, -0.0001415788877562581, -0.0003923757746387563, 0.0, 0.04541810268109969, 0.00136603401129394, 0.0013660340112939262, 0.0, 0.0, 0.0, 0.002243823115382751, 0.0, 0.0, -2.42706664725029e-05, 0.0003923757746387771, 0.00036810510816627333, 0.0, 0.04187903499830106, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.037291879034998296, -0.0003923757746387684, 0.0, 0.0, 0.04221882432891608, 0.007772680937818563, 0.0, 0.030920829085966703, 0.0, 0.0, 0.0, 0.0012317363234794487, -0.04761297995242951, -0.03436119605844377, 0.0, 0.0, 0.005280892513308417, -0.0018971570959338546, 0.007772680937818556, 0.007178049609242263, 0.0, 0.0, 0.0, 0.0, 0.0]}, (14, '17_14_1'): {'changes': [0.16108472766836712, 2.221126107704176, -1.0776314569788212, 1.4716436127409196, 1.2988179159299236, 8.312316658891177, 6.0329428277269415, 13.498547055895045, -0.8971698538456891, 0.05824864596712587, 4.608909842633722, 4.376425297508831, 9.91728805763023, 2.090831499364782, -0.6170088146757529, 8.07612859055746, 4.743728809909726, 5.741197402638647, -0.19118412486452385, 1.5555058105612716, -0.8868351576082603, 0.25567651352412213, 8.923634014341332, 5.685148703850189, 31.148930941749256, 7.345429947610114, -0.4732400753544468, 23.838770012193066, 5.18569183313906, 0.043749466017914074, -2.546973487042962, 17.420236502127835, -0.536163398685801, 14.549619901532651, 0.04349154567865554, 8.353107580755932, -0.20826446927742737, 0.6540247846235445, 1.4321183416303427, 1.4513932884296707, 8.146552584106189, 7.71433198995976, 9.011907063909788, -11.07092782350702, -15.107511064706102, -0.8032917730985787, 9.624304470691087, 5.456202127936052, 1.9735290680459698, 2.2678737817881256, 15.456683251690635, -2.8871533128452285, 6.343225511298414, 7.200551719246278, 6.06602866082568, 16.91750547722063, 0.11994401100717056, 7.867155784555081, 21.972661097031448, 0.25655491696986843, 1.4829994321262632, 5.414193850327635, 8.256754169114174, -0.9316792240783336, -2.363687304191707, 1.3318824044842472, -6.355881144581019, -10.147594008133865, 5.359109105314019, 13.200267382130278, 12.751864128532475, -0.1257352288768061, 1.6508313560054546, 2.1134762169313603, 8.952873629691055, 15.334380823856208, -6.257824471977038, 25.091457372834583, -2.4023694712792576, -6.7210718484502365, 9.503055202570295, 6.0989557829366845, 13.58126184421559, -0.4389168142842266, -0.18295058309078627, -10.208715312396336, 5.3119630895527905, 5.776659328224326, 10.952521136301073, -6.373240053934083, 10.489602279451333, 5.9136372487346875, -27.878876167705812, 13.862051963236524, 8.889746067107211, 8.922510946871796, -9.51090835322222, -5.997088528533382, -1.2113357983793094, 6.8605505122062596, 12.073937425249937, 57.56920074756178, 9.4005158374394, 20.93716715353972, 19.06946248267988, -0.826332627200884, 7.709605630908641, 13.04623786712807, 13.50902164598915, 13.715158623447508], 'changes_betw': [0.0006229471061275915, 0.000311473553063768, 0.0001982104428587539, 0.0, 0.004063314078604635, 0.0, 0.0, 0.0, -0.0034403669724770714, 0.0, -0.00035394721939065177, 0.0, 0.0, 0.0, 0.00031147355306379576, 0.0, 0.0, 0.0, 0.00031147355306374025, 0.0, 0.0, 0.0, 5.663155510250706e-05, 5.663155510250706e-05, 0.0, 0.00011326311020500024, 0.006130365839846025, 0.0, 0.0, 0.004629629629629636, 0.0040066825235021, 0.0, 0.0025201042020613906, 0.0, -0.00021236833163434943, 0.0, -0.0004247366632688099, 0.0, -0.00021236833163437718, -0.00031147355306374025, 0.0, -0.00021236833163437718, -0.00021236833163437718, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00021236833163439106, 0.0, 0.0, 0.000311473553063768, 0.0, 0.0, 0.0, 0.0, -0.0037943141918677092, 0.0, 0.0, -2.8315777551274346e-05, 2.8315777551251795e-05, 8.494733265376059e-05, 0.0, 0.0030156303092082615, -0.00021236833163439106, -0.00021236833163439106, 0.0, 0.0, 0.0, -0.0003539472193906379, 0.0, 0.0, 0.0, -8.494733265375712e-05, -8.494733265375538e-05, 0.0, 0.009938837920489302, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007900101936799186, 8.494733265375365e-05, 0.0, 0.0, 0.010278627251104316, -0.00327047230716955, 0.0, 0.005691471287801564, 0.0, 0.0, 0.0, 0.005394155623513433, 0.0031855249745157896, 0.002166156982670746, 0.0, 0.0, -0.013577415335825122, -0.006484313059236607, -0.004799524294937137, -0.007942575603126067, 0.0, 0.0, 0.0, 0.0, 0.0]}, (14, '14_30_1'): {'changes': [1.0629928361903378, -0.320865900159383, -0.6155392950889791, 7.742081946871167, -2.4134971231284794, 0.9901754677396468, -2.387587280735403, -8.819952308813015, 3.229041809755664, 6.731335009312716, 4.684677051808109, 10.96804483658184, -5.7171875429206125, -2.8525181184211874, -0.09258351237006579, 7.975461031922407, -1.520133047674335, 7.722828549430687, 0.12957843111933443, 9.275976870192494, 1.7092115820513811, -1.1419131300865502, 13.387806475327096, 9.408794139300454, 6.96900275091005, 11.589027077205898, -2.9414792261965843, 11.80874898681816, 1.1528881523448717, -5.376224255382425, -4.956420327665292, 3.3887032690714136, 4.921357575440837, 2.425443987234928, 0.6745653159092626, 10.982390412263783, 3.501457837032092, 7.425138067722614, 4.493586167188077, 3.4670717252792187, 6.66297674750831, 3.6377677701403712, 5.404867276833855, -0.7032607349073032, 3.120272726263522, 7.486081159485934, 7.907234194124115, 1.878625879166563, 1.8234416251926433, 2.376799386972664, 10.136970282372005, 1.336769718092036, -12.35915077391067, 16.84462138261415, -7.706154870330636, 0.6026291730696585, 4.52610334056639, 2.152240594398336, 9.485681653956192, 3.5685720856804934, 8.173054644178706, 5.884525096469815, 13.185085808765677, -0.04322295290752365, -0.6224598463573727, 1.3616084481737403, -1.3856084833803948, 1.5909908330987719, 19.61590238591947, 5.901003006645283, 6.7846382279140585, 8.232513625184424, 12.796188756910738, -3.7445072278247267, 3.043125152447658, 1.6138025639417606, -7.367976422909159, 12.119055290573215, 0.011682419434976055, -0.4995393679296001, -0.5154601971147486, 15.42672826342178, 11.548082569833866, -4.984128847705904, 3.419338540276158, -8.86500592842485, 8.399151628834375, -0.2860431924538034, 12.842525406693937, -7.986626512079084, 34.76116949197933, -0.26095900744633127, -40.499115401775725, -7.674450752466726, -3.8712256374168135, -3.2239209439412377, -15.172244121982956, -9.00752993625916, -18.022993330649, -6.5467939726140685, -2.678960479612499, 62.563764237714736, -32.46573347511983, 54.65605027472208, 89.11783128388637, 7.431559235858799, 5.506527071029893, 4.727427285204726, -6.118350995536048, 2.5319736845947887], 'changes_betw': [0.0010476837693962904, 0.0007362102163325224, 0.0004530524408200287, 0.0, 0.011793521350096314, 0.0, 0.0, 0.0, -0.010575942915392461, 0.0, -0.0008211575489863038, 0.0, 0.0, 0.0, 0.0007362102163325779, 0.0, 0.0, 0.0, 0.0007362102163325501, 0.0, 0.0, 0.0, 0.0001415788877562607, 0.0001415788877562607, 0.0, 0.00028315777551250754, 0.013180994450107647, 0.0, 0.0, 0.013379204892966373, 0.011906784460301287, 0.0, 0.0004813682183712753, 0.0, -0.0005096839959225219, 0.0, -0.0010193679918450993, 0.0, -0.0005096839959225219, -0.0007362102163325224, 0.0, -0.000509683995922508, -0.0005096839959225219, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0005096839959225358, 0.0, 0.0, 0.0007362102163326056, 0.0, 0.0, 0.0, 0.0, -0.011397100464378751, 0.0, 0.0, -7.078944387814423e-05, 7.078944387812775e-05, 0.00019821044285876777, 0.0, -0.008792048929663676, -0.0005096839959225358, -0.0005096839959225219, 0.0, 0.0, 0.0, -0.0008211575489862899, 0.0, 0.0, 0.0, -0.00019821044285876083, -0.00019821044285876083, 0.0, 0.014950730547060836, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014101257220523272, 0.00019821044285876083, 0.0, 0.0, 0.014610941216445822, -0.020939517499150517, 0.0, 0.012911994563370714, 0.0, 0.0, 0.0, 0.023233095480801914, 0.01278457356439007, 0.01210499490316004, 0.0, 0.0, -0.013577415335825122, 0.011864310793974403, -0.022808358817533132, -0.026291199456337077, 0.0, 0.0, 0.0, 0.0, 0.0]}, (30, '14_30_1'): {'changes': [-0.11396021450310556, 0.8087164005351468, -2.7193105622729234, 3.9897385880782252, -8.432712080209711, -2.5524944805961525, 16.07904751410976, 14.284233254775103, 5.739573782889444, 7.527012406888673, 0.7234922312605647, -4.921857353639126, 9.244532033406813, 1.7429921491849996, 1.9203246882068967, 17.85958300155528, -0.74331586759331, -4.3438908642812635, 1.1801385270847256, 2.8494058374812994, 5.802847343549857, 9.949057644243567, -0.7447597693396517, -4.033371112202474, 2.207635494520929, -0.5226435355659049, -3.462925876436792, 10.526973600795856, 6.551994987266539, -10.289369856422923, -8.785024147384647, -15.620617389632713, -2.265849781500677, -13.332179673513906, -0.8811028466951001, 7.839260855029181, -0.5851173235305609, 0.43478992473610845, 3.580473797879229, 1.4085183985342553, 3.89372106149051, -5.228665466086426, -2.0966726501181085, -7.309028866118126, 4.41182579069789, 3.7501767478747183, 1.1014926671530816, 3.0860516015440282, -0.4015045749982491, 1.1604679136903542, -0.9425896962968068, -1.3905132764392079, 2.598396955247722, 5.477157742054658, -4.2779809105612685, 16.845784991481253, 5.492597086601716, 8.752596775029076, 20.830082086569234, 2.2471755294255544, 5.577178259158757, -0.12147098677641566, -8.160377624618718, 3.325594340287406, 0.4923209219994362, -3.316923474902552, 0.7807099111115576, 21.916244716510136, 12.89458353228548, 0.4449388184185352, 0.6318314024654796, -1.1490672399255573, 6.912038341267277, 7.330319803568983, 13.915654915694574, -5.856668258961008, -5.218693260263514, -12.319584314102713, 2.450093148633812, 7.800391139219812, -0.7331619577269635, 13.754590033267789, 10.923786896302516, 7.640211487067688, 8.800252070126618, -5.49113997985765, 3.2974109094609787, -2.568971793925357, 3.2183192855892457, -4.757030813977671, 66.4896932277558, -0.5406581694493298, -43.76359041200462, -9.593519518951666, -1.1192701656765678, 0.14184851661417497, -31.81964861522424, -23.131463616245426, -22.53437858854167, 12.89134369128817, 0.2432049785554682, -13.197516057909837, -75.87356736834568, 133.54933842740877, 93.88113935076854, 17.725586619386235, 16.82801876622122, 1.9406681841791737, 10.653281618102028, -1.503704380459567], 'changes_betw': [0.0014724204326650447, 0.0011609468796013322, 0.0007078944387812897, 0.0, 0.019523728621587993, 0.0, 0.0, 0.0, -0.01771151885830788, 0.0, -0.0012883678785819697, 0.0, 0.0, 0.0, 0.00116094687960136, 0.0, 0.0, 0.0, 0.0011609468796013322, 0.0, 0.0, 0.0, 0.00022652622041001436, 0.00022652622041001436, 0.0, 0.00045305244082001483, 0.020231623060369297, 0.0, 0.0, 0.022128780156303096, 0.019806886397100446, 0.0, -0.00155736776531884, 0.0, -0.0008069996602106666, 0.0, -0.0016139993204214165, 0.0, -0.0008069996602106666, -0.0011609468796012767, 0.0, -0.0008069996602106527, -0.0008069996602106666, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0008069996602106944, 0.0, 0.0, 0.00116094687960136, 0.0, 0.0, 0.0, 0.0, -0.018999886736889793, 0.0, 0.0, -0.000113263110205028, 0.00011326311020500544, 0.00031147355306375413, 0.0, -0.020939517499150573, -0.0008069996602106944, -0.0008069996602106805, 0.0, 0.0, 0.0, -0.001288367878581942, 0.0, 0.0, 0.0, -0.0003114735530637715, -0.00031147355306376627, 0.0, 0.01962283384301733, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.019962623173632343, 0.00031147355306376627, 0.0, 0.0, 0.018603465851172285, -0.03894835202174651, 0.0, 0.019792728508324843, 0.0, 0.0, 0.0, 0.040732246007475374, 0.022383622154264343, 0.022043832823649342, 0.0, 0.0, 0.004771208517385888, 0.0298731453165704, -0.04115698267074414, -0.026291199456337077, 0.0, 0.0, 0.0, 0.0, 0.0]}}
#distances={}
for edge in local_edges.keys():
    distances=[]
    for node in G.nodes():
        if edge[0]==node:
            l1=0
        else:
            l1=nx.shortest_path_length(G,source=edge[0],target=node)
        if edge[1]==node:
            l2=0
        else:
            l2=nx.shortest_path_length(G,source=edge[1],target=node)
        l=min(l1,l2)
        distances.append(l)
    local_edges[edge]['distance']=distances
print(distances) 

for edge in local_edges.keys():
    plt.scatter(local_edges[edge]['distance'], local_edges[edge]['changes'], color='BLue')
plt.xlabel('Distance of Specific Node from the Failure')
plt.ylabel('Change in Wait Time of Specific Node')